In [ ]:
using ChaosTools
using StaticArrays
function lorenz_rule(u, p, t)
    σ = p[1]; ρ = p[2]; β = p[3]
    vector = [σ*(u[2]-u[1])]
    append!(vector,u[1]*(ρ-u[3]) - u[2])
    append!(vector,u[1]*u[2] - β*u[3])
    dx = @SVector [vector[k] for k in 1:3]
    return dx
end

lor = CoupledODEs(lorenz_rule, fill(10.0, 3), [10, 32, 8/3])
λλ = lyapunovspectrum(lor, 10000; Δt = 0.1)

In [ ]:
using ChaosTools
using StaticArrays

function Gene_ODE(u, Parameters, time)
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    Epi_Factors = Parameters[3]
    test = InvSqr*ParameterMatrix[1,:]'*u[:]
    vector = [tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1]]
    for c=2:M
        append!(vector,tanh(40*(InvSqr*ParameterMatrix[c,:]'*u[:]+Epi_Factors[c]+CArray[c])) - u[c])
    end
    dx = @SVector [vector[k] for k in 1:M]

    return dx
end  


t = 7
lor = CoupledODEs(Gene_ODE, u[1,t,:], [ParameterMatrix,CArray,u[2,t,:]])
λλ = lyapunovspectrum(lor, 1000; Δt = 0.1)
sort(λλ,rev=true)

In [ ]:
M = 40
u = rand(Float64,(2,7,M)).-0.5
InvSqr = 1/sqrt(M)
CArray = rand(M) .- 0.5
ParameterMatrix = rand(Float64,(M,M)).-0.5
1+1



In [ ]:
t = 7
lor = CoupledODEs(Gene_ODE, u[1,t,:], [ParameterMatrix,CArray,u[2,t,:]])
λλ = lyapunovspectrum(lor, 1000; Δt = 0.1)
sort(λλ,rev=true)

In [ ]:
print(vector)

In [ ]:
0.7376477483585008*0.54333 +0.08181141180196017*0.427269 + 0.4849197255046691*0.142589

In [ ]:
ParameterMatrix[1,:]'*u[1,1,:]

In [ ]:
tanh(40*(InvSqrt*ParameterMatrix[1,:]'*u[1,t,:].+CArray[1])) - u[1,t,1]

In [ ]:
InvSqrt*ParameterMatrix[1,:]'*u[1,t,:]

In [ ]:
#### SMALL PERT IS SET TO 1 WE ONLY HAVE 1 SAMPLE NOW
### WE MESSED WITH SOME DATA_TO_ANALYSE and the loading ones

#Data_To_Analyse = "21_12_NoSplit_HighODE_SplitTime=0.1_2000Gen_Noise01"
#Data_To_Analyse = ARGS[1]
#Noise = parse(Float64, ARGS[2])
#Data_To_Analyse = "21_12_NoSplit_HighODE_SplitTime=0.1_2000Gen_Noise005_Mut=2"
#Noise = 0.005

Noise = 0.01
Data_To_Analyse = "07_01_NoSplit_HighODE_SplitTime=0.1_2000Gen_Noise0.01_Sample4"

Load_Generations = [900]

Epi_Limiter = 1
a::Float32 = 1.0*Epi_Limiter   #Ratio of gene to epigenetic factor
v::Float32 = 6*10^-4/Epi_Limiter  #Speed at which epigenetic factor changes
#v::Float32 = 0  #Speed at which epigenetic factor changes

Load_Data_String = Data_To_Analyse*"Epi0.1"



println("OUTPUTTING MY INPUT")
println(Data_To_Analyse)
println(Noise)

using Random
using Plots
using Distributions
using DifferentialEquations
using Statistics
using ProgressMeter
using TickTock
using LinearAlgebra
using StatsBase
using LaTeXStrings
using MultivariateStats
using Dates
using Distances
using JLD
using KernelDensity
using Plots
using StatsPlots




function Setting_Up_Randomly(M,Sparse,rng=MersenneTwister(123))
    "Function used to randomly with seed define our c_i's, J_ij's and the initial positions
    Input:
    M      = Amount of Genes in our system
    Sparse = Bool to determine if we have a sparse graph
    rng    = Random Julia seed to make sure we can recreate the same initial conditions
    Output:
    CArray          = C_i's used, can only be between -0.5 and 0.5 now
    PositionArray   = Random Initial position, between -1 and 1 for all genes, epifactors = 0
    ParameterMatrix = J_ij's Currently a dense network with values between -0.5 and 0.5
    "
    ParameterMatrix = zeros(M,M)
    CArray = zeros(1,M)
    PositionArray = zeros(2,1,M)
    
    CArray = rand(rng,Float32,M).-0.5; #Random c_j between -0,5 and 0.5
    PositionArray[1,1,1:M] = 2*(rand(rng,Float32,M).-0.5); #Random Initial position 
    ParameterMatrix = 2 * bitrand(rng,M,M).-1; #J_jk's
    if Sparse
       for i=1:M
            for j=1:M
                if rand(rng) > 5/M #Average of 5 connections per gene

                    ParameterMatrix[i,j] = 0 
                end
            end
        end
    end
    return CArray, PositionArray, ParameterMatrix
end







function Noisy_Splitting(solution,Total_Splits,M,Seed)
    " Once our cell decides to split, we decide to introduce some random noise in how these genes get distributed.
    This function takes the cells, doubles them and then gives the daughters cell the same gene parameters +- noise
    one daughter gets + the other gets -. For the epigeneitc factors they are simply inherited and do not see any noise
    Input:
    solution     = These are the initial conditions of the parent cells that we need to split 
    (First genes of all cells, then all epigenetic factors)
    Total_Splits = Amount of times the cell has split (including this current split)
    M            = Amount of Genes
    Seed         = Random Noise seed for reproducing results
    Output:
    SplittedArray = New array of the genes and epigenetic factors of the daughter cells
    How is this structured? Suppose we take M=3 for convenience, we would have [g1,g2,g3,e1,e2,e3] for the first parent cell
    The daughters become: [g11,g12,g13,g21,g22,g23,e11,e12,e13,e21,e22,e23]
    Where we have genes of daughter 1, then genes of daughter 2, then epi of daughter 1 and epi of daughter 2.
    When the Daughters split we get the genes of Children of cell 1, then genes of Children of cell 2 and same for epi.
    "
    Random.seed!(Seed)
    s = Total_Splits #For shortening
    SplittedArray = zeros(Float32,2,2^(s),M) #2^(s+1), 2M is the normal length, 4M after 1 split, 8M after 2 splits etc.
    for g=1:2^(s-1) #looping over the length of the parent
        for t=1:M
            Gaussian = Normal{Float32}(0.0f0,0.0f0)
            Displace = rand(Gaussian) #Noise for split
            #index = Int32.(floor((g-1)/M)*M +g) #example M=10: Index 1-> 1 and 11, index 2->2 and 12 ... 
            #index 10->10 and 20, index 11->21 and 30 and so on. This is old method

            #Genes
            SplittedArray[1,2*g-1,t] = solution[1,g,t] + Displace
            SplittedArray[1,2*g,t] = solution[1,g,t] - Displace
            
        end
        #Epigenetic
        SplittedArray[2,2*g-1,:] = solution[2, g,:]
        SplittedArray[2,2*g,:] = solution[2, g,:]   
    end

    return SplittedArray  
end

function Differentiaton_Detection_Parameter(M,Epigenetic_Plot)
    "Function used to convert epigenetic factors into cell types and bits
    Input:
    M               = Amount of Genes 
    Epigenetic_Plot = Array of epigenetic factors which have to be converted to 0 and 1 representing on or off
    Output:
    b        = Array of scalar values representing cell types
    BitGenes = Array of vectors representing the genes of the cell types
    "
    b = Int128[]
    Counter = Int[]
    Epi_Bitwise = signbit.(-Epigenetic_Plot)

    
    BitGenes = [] #We will put the bit notation of the gene in here
    #Later rewrite to let this replace Bit_Represent
    
    for i=1:length(Epi_Bitwise[:,1])
        Bit_Represent::Int128 = 0 #CAREFUL WITH M>127
        for j=1:M
            Bit_Represent += Int128(Epi_Bitwise[i,j]*2)^(j-1)
        end
        #print(Bit_Represent)
        #print("     ")
        if !(Bit_Represent in b)

            push!(BitGenes,collect(Epi_Bitwise[i,:]))
            push!(Counter,1)
        end
        push!(b,Bit_Represent)

        
    end
    
    #print(b)
    Counter = sort(Counter,rev=true)
    #print(Counter)
    Score = 0
    #Make here the formula to turn b and the counter into a score


    return b,BitGenes
end

function Entropy_Score(M,Epigenetic_Plot)
    "Function used to convert epigenetic factors into cell types and bits to then calculate the entropic score.
    See previous function for alternative, this is currently not in use.
    Input:
    M               = Amount of Genes 
    Epigenetic_Plot = Array of epigenetic factors which have to be converted to 0 and 1 representing on or off
    Output:
    Score = Entropy score of the cell types
    "
    b = Int[]
    Counter = Int[]
    Epi_Bitwise = signbit.(-Epigenetic_Plot)
    
    for i=1:length(Epi_Bitwise[:,1])
        Bit_Represent::Int128 = 0 #CAREFUL WITH M>127
        for j=1:M
            Bit_Represent += Int128(Epi_Bitwise[i,j]*2)^(j-1)
        end
        #print(Bit_Represent)
        #print("     ")
        if !(Bit_Represent in b)
            push!(b,Bit_Represent)
            push!(Counter,1)
        else 
            Counter[findfirst(==(Bit_Represent), b)] += 1
        end
        
    end
    
    #print(b)
    Counter = sort(Counter,rev=true)
    #print(Counter)
    Score = 0
    #Make here the formula to turn b and the counter into a score
    for i=1:length(b)
       prob = Counter[i]/M
       Score -= prob*log(prob)
    end
    
    return Score
end



function Mutate_Parameters(ParameterMatrix_Mut,M,Mutations,Seed)
    "Function used to mutate the network (we turn a 1 or -1 to 0 and we turn a 0 to 1 or -1)
    Input:
    ParameterMatrix_Mut = Original Network that we want to mutate
    M                   = Amount of Genes 
    Mutations           = How many connections will we mutate
    Seed                = Randomness seed
    Output:
    ParameterMatrix_Mut = Mutated Network
    "
    Zero_Indices = findall(iszero,ParameterMatrix_Mut)
    One_Indices = findall(!iszero,ParameterMatrix_Mut)
    Random.seed!(Seed)
   
    for j=1:Mutations 
        t_zero = rand(1:length(Zero_Indices))
        t_one = rand(1:length(One_Indices))
        i_zero = Zero_Indices[t_zero][1]
        j_zero = Zero_Indices[t_zero][2]
        i_Value = One_Indices[t_one][1]
        j_Value = One_Indices[t_one][2]



        ParameterMatrix_Mut[i_zero,j_zero] = rand(0:1)*2-1 #turn a zero into 1 or -1
        ParameterMatrix_Mut[i_Value,j_Value] = 0 #turn 1 or -1 into 0




        
    end
    return ParameterMatrix_Mut
end


#Some Julia functions I played around with, may reconsider using more later
MultiplyBroad_(a,b) = a .* b

SummingDim2_(a) = sum(a,dims=2) 

HyperTanh_(a) = tanh.(40*a)



function Epigenetic_Evolution3(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    
    #⊗_(Wi, bi) = Wi .* bi
       
    for t=1:2^Splits

        du[2,t,:] = v*(a*u[1,t,:].-u[2,t,:])    
        du[1,t,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,t,:].+u[2,t,:].+CArray[:])) .- u[1,t,:] 
            #Epigenetic Evolution, no need to worry about boundaries here so simple loop      
    end  
    
end



function Gradient(u,Splits,t)
    "Function used to calculate the gradient of a cells neighbours, takes into accounts if there even are neighbours.
    Input:
    u      = Gene concentrations over which we take the gradient
    Splits = How many times have the cells split already (determines where neighbours are)
    t      = What location is our cell (boundaries only have 1 neighbour)
    Output:
    Multiple different version depending on the splits, this will be the gradient we want
    "
    if Splits == 0
        return 0
    elseif Splits ==1
        if t==1
            return u[t+1,:] - u[t,:]
        else
            return u[t-1,:] - u[t,:]
        end
    else
        if t==1
            return u[t+1,:] - u[t,:]
        elseif t==2^Splits
            return u[t-1,:] - u[t,:]
        else
            return u[t-1,:] - 2*u[t,:] + u[t+1,:]
        end
    end
    
end

function Epigenetic_Evolution4(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    
    #⊗_(Wi, bi) = Wi .* bi
       
    for t=1:2^Splits

        du[2,t,:] = v*(a*u[1,t,:].-u[2,t,:])    
        du[1,t,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,t,:].+u[2,t,:].+CArray[:])) .- u[1,t,:] +KArray.*Gradient(u[1,:,:],Splits,t)
            #Epigenetic Evolution, no need to worry about boundaries here so simple loop      
    end  
    
end

function σ_Genes(du, u,Parameters,Time)
    du[1,:,:] .= Noise
    
end



function Epigenetic_Evolution2(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    Cells::Int32 = 2^Splits
    #⊗_(Wi, bi) = Wi .* bi
       
    for t=1:Cells  

       du[2,t,:] = v*(a*u[1,t,:].-u[2,t,:])    

       du[1,t,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,t,:].+u[2,t,:].+CArray[:])) .- u[1,t,:] .+ KArray.*(vec(sum(u[1,:,:],dims=1))/Cells - u[1,t,:])

    end  
    
end

#Defining it all in function form


function Mutating_Network(N,M,K,ParameterMatrix_Collection,PositionArray_Collection, Indices, Counter, Genetics)
    "Function used to make mutated copies of the best performing networks. Currently not in use
    Input:
    N                          = How many copies of the network do we make
    M                          = Amount of Genes
    K                          = Supposed to work as a fraction of N, telling us how many mutant versions
    ParameterMatrix_Collection = The original networks
    PositionArray_Collection   = Original Positions
    Indices                    = Sorted indices giving the best networks
    Counter                    = Randomiser seed
    Genetics                   = Used to track heritance of networks
    Output:
    ParameterMatrix_Collection = Updated network
    PositionArray_Collection   = Updated initial positions
    Counter                    = Updated seed
    Updated_Genes              = Updated inheritance
    "
    Updated_Genes = zeros(N)
    Mutations = 1
    Seed_Mutations = 1234
    Indexation_Mutate = 1
    Temporary_Matrix = zeros(Float32,N,M,M)
    Temporary_Vector = zeros(Float32,N,2,1,M)
    for p = 1:K
        for q=1:N/(2*K)
            Temporary_Matrix[Indexation_Mutate,:,:] = Mutate_Parameters(ParameterMatrix_Collection[Indices[p],:,:],M,Mutations,Seed_Mutations+Counter)
            Temporary_Vector[Indexation_Mutate,:,:,:] = PositionArray_Collection[Indices[p],:,:,:]
            Updated_Genes[Indexation_Mutate] = Genetics[p]
            Temporary_Matrix[Indexation_Mutate+1,:,:] = Temporary_Matrix[Indexation_Mutate,:,:]
            Temporary_Vector[Indexation_Mutate+1,:,:,:] = Temporary_Vector[Indexation_Mutate,:,:,:]
            Updated_Genes[Indexation_Mutate+1] = Genetics[p]
            Counter += 1
            Indexation_Mutate +=2
        end
        Temporary_Matrix[Indexation_Mutate-1,:,:] = ParameterMatrix_Collection[Indices[p],:,:] #We overwrite it to add the original back
        Temporary_Matrix[Indexation_Mutate-2,:,:] = ParameterMatrix_Collection[Indices[p],:,:]
    end
    ParameterMatrix_Collection = deepcopy(Temporary_Matrix);
    PositionArray_Collection = deepcopy(Temporary_Vector);
    return ParameterMatrix_Collection, PositionArray_Collection, Counter, Updated_Genes
end



function Running_Network(CArray,ParameterMatrix,PositionArray,Random_Noise)
    "Function used to run our simulation
    Input:
    CArray          = Constants used in our ODE solver  
    ParameterMatrix = Collection of Networks
    PositionArray   = Original Positions
    Random_Noise    = Randomiser seed
    Output:
    PositionArray[2,:,:] = Epigenetic factors from our simulation
    Random_Noise         = Updated seed
    "
    #CArray = CArray_Collection[x,:]
    #ParameterMatrix = ParameterMatrix_Collection[x,:,:]
    #PositionArray = PositionArray_Collection[x,:]
    Orbit_Collection = zeros(Float32,2^Splits*M+1,0)
    Epi_Collection = zeros(Float32,2^Splits*M+1,0)
    
    
    
    
    for s=1:Splits #Looping over the splits
        Parameters = (ParameterMatrix, CArray,KArray,Int32(s-1)) #We have to update the splits every rerun, rest stays same
        tspan = (Float32(0.0),SplitTime) #How long ODE will run
        if Mean_Field
            prob = ODEProblem(Epigenetic_Evolution2,PositionArray,tspan,Parameters) #ODEdefining
        elseif Diffusing_System
            prob = ODEProblem(Epigenetic_Evolution4,PositionArray,tspan,Parameters) #ODEdefining
        elseif Stochastic_Noise
            prob = SDEProblem(Epigenetic_Evolution4, σ_Genes,PositionArray,tspan,Parameters)

        else
            prob = ODEProblem(Epigenetic_Evolution3,PositionArray,tspan,Parameters) #ODEdefining
        end    #Solving
        if !Stochastic_Noise
            sol = solve(prob,BS3())
        else
            sol = solve(prob,SRA3())
            #sol = solve(prob,LambaEulerHeun())
        end
        Times = sol.t #Save the time, doesn't get used yet though
        StepAmount = length(Times)  #Amount of data points per gene
        Genes = sol 
        PositionArray = Genes[StepAmount] #Saving the genes and epi factors
        
        OrbitalCollection = Times' .+ (s-1)*SplitTime
        EpiCollected = Times' .+ (s-1)*SplitTime
        for i=1:2^(Splits-s+1)  #s=1 this gives all 2^Splits, this is to say how much we repeat same path
            for j=1:2^(s-1) #Together multiply, always 2^Splits orbits, these j represent different paths
                OrbitalCollection = vcat(OrbitalCollection,sol[1,j,:,:])   
                EpiCollected = vcat(EpiCollected,sol[2,j,:,:])   
            end
        end
        Orbit_Collection = hcat(Orbit_Collection,OrbitalCollection)
        Epi_Collection = hcat(Epi_Collection,EpiCollected)
        #Orbit1 = [Times,Genes[1,1,1:M,1:StepAmount]]
        #Orbit2 = [Times,Genes[1,1,1:M,1:StepAmount]]
        #EpiOrbit1 = [Times,Genes[2,1,1:M,1:StepAmount]]
        #EpiOrbit2 = [Times,Genes[2,1,1:M,1:StepAmount]]
        #Now we must take all the values from the updated PositionArray and give appropriate terms to the daughter cells
        PositionArray = Noisy_Splitting(PositionArray,s,M,Random_Noise)  #Splitting function
        Random_Noise += 1
    end

    Final_Run_Time::Int32 = 20000 #Long evolving of the cells after splitting to get the final epigenetic state
    Parameters = (ParameterMatrix, CArray,KArray,Int32(Splits)) 
    tspan = (Int32(0.0),Final_Run_Time)

    if Mean_Field
        prob = ODEProblem(Epigenetic_Evolution2,PositionArray,tspan,Parameters) #ODEdefining
    elseif Diffusing_System
        prob = ODEProblem(Epigenetic_Evolution4,PositionArray,tspan,Parameters) #ODEdefining
    elseif Stochastic_Noise
        prob = SDEProblem(Epigenetic_Evolution4, σ_Genes,PositionArray,tspan,Parameters)

    else
        prob = ODEProblem(Epigenetic_Evolution3,PositionArray,tspan,Parameters) #ODEdefining
    end    #Solving
    if !Stochastic_Noise
        sol = solve(prob,BS3())
    else
        #,dtmin=0.1,force_dtmin=true
        sol = solve(prob,SRA3(),dtmin=0.1,force_dtmin=true)
    end
    #sol = solve(prob,alg_hints = [:nonstiff]) #Solving
    #sol = solve(prob)
    Times = sol.t
    StepAmount = length(Times)
    GenesEpi = sol
    PositionArray = GenesEpi[StepAmount]

    OrbitalCollection = Times' .+ Splits*SplitTime 
    EpiCollected = Times' .+ Splits*SplitTime 
    for j=1:2^(Splits) #Together multiply, always 2^Splits orbits, these j represent different paths
        OrbitalCollection = vcat(OrbitalCollection,sol[1,j,:,:])  
        EpiCollected = vcat(EpiCollected,sol[2,j,:,:])   

    end
    Orbit_Collection = hcat(Orbit_Collection,OrbitalCollection)
    Epi_Collection = hcat(Epi_Collection,EpiCollected)
    #Orbit1 = [cat(Orbit1[1],Times.+SplitTime,dims=1),cat(Orbit1[2],GenesEpi[1,1,1:M,1:StepAmount],dims=2)]
    #Orbit2 = [cat(Orbit2[1],Times.+SplitTime,dims=1),cat(Orbit2[2],GenesEpi[1,2,1:M,1:StepAmount],dims=2)]
    #EpiOrbit1 = [cat(EpiOrbit1[1],Times.+SplitTime,dims=1),cat(EpiOrbit1[2],GenesEpi[2,1,1:M,1:StepAmount],dims=2)]
    #EpiOrbit2 = [cat(EpiOrbit2[1],Times.+SplitTime,dims=1),cat(EpiOrbit2[2],GenesEpi[2,2,1:M,1:StepAmount],dims=2)]

    return PositionArray[2,:,:],Random_Noise, Orbit_Collection, Epi_Collection
end


function Scoring_Indexes(N,Final_Epigenetic_Factors,Cell_Bit)
    "Function used to calculate the score of all the networks, distribution robustness and the cell type robustness. Note
    that we are still selecting on amount of different cells, not entropy.
    Input:
    N                       = How many copies of the same networks we have 
    Final_Epigentic_Factors = Array of epigenetic factors after running the simulations
    Cell_Bit                = Cell type tracker
    Output:
    Indices                   = Indices of networks sorted by score, we only select these
    AllIndices                = Longer copy of Indices, contains also those we will no longer use
    Evolution_Indices         = Gives us the score corresponding with the Indices we selected on
    [Cell_Types,Cell_Amounts] = Vector of the cell types and how many of these cell types there are
    Cell_Bit                  = Updated cell type tracker
    KLD_Distance              = Kullback-Lieber divergence score for the distribution robustness
    Bit_Distances             = Cell Type robustness measure
    "
    Scoring = zeros(2,N)
    Cell_Types = []
    Cell_Amounts = []
    Cell_Distribution = Array{Vector{Float64}}(undef, N) 
    Cell_Bit_Array = Array{Vector{Any}}(undef, N)

    for x=1:N
        Differentiation, Bit_Genes = Differentiaton_Detection_Parameter(M,Final_Epigenetic_Factors[x,:,:]) 
     
        #Here we save some cell types and bits to use for our robustness calculation after the for loop
        #Cell types
        alpha = countmap(Differentiation)
        Temp_Array = Vector{Float64}()
        for i in keys(alpha)
            Temp_Array = push!(Temp_Array,alpha[i])
        end
        #print(Temp_Array)
        Cell_Distribution[x] = sort(Temp_Array,rev=true)/2^Splits #We sort the distribution of our cells and normalise
        #Bits
        Cell_Bit_Array[x] = Bit_Genes
        
        
        
        
        #This part here is where we track the cell types we had before as well
        for tau in Bit_Genes
            if !(tau in Cell_Bit)

                push!(Cell_Bit,tau)


            end
        end
        for tau in Differentiation 
            if !(tau in Cell_Types)
                push!(Cell_Types,tau)
                push!(Cell_Amounts,1)
            else
                Cell_Amounts[findfirst(==(tau), Cell_Types)] += 1
            end
        end

        Scoring[1:2,x] = [length(countmap(Differentiation)),x]

    end

    #Robustness calculation
    KLD_Distance = 0
    
    KLD_Adder = 0
    for i=1:N
        for j=1:N #Loop over all and calculate KL for all distributions
            if i!=j #If the same we get no difference anyways
                Length_Differ = length(Cell_Distribution[i])-length(Cell_Distribution[j])
                if Length_Differ > 0 #i is longer so we need to add 0 to j
                    KLD_Adder = kldivergence(Cell_Distribution[i],vcat(Cell_Distribution[j],zeros(Length_Differ)))
                    if KLD_Adder != Inf #There are zeros in the second distribution (divide by 0 error)
                        KLD_Distance += kldivergence(Cell_Distribution[i],vcat(Cell_Distribution[j],zeros(Length_Differ)))
                    end
                else
                    KLD_Adder = kldivergence(vcat(Cell_Distribution[i],zeros(-Length_Differ)),Cell_Distribution[j])

                    KLD_Distance += kldivergence(vcat(Cell_Distribution[i],zeros(-Length_Differ)),Cell_Distribution[j])
                end
            end
        end
    end
    KLD_Distance = KLD_Distance/(N*(N-1))
    
    #Now we see the closest type in the other groups
    Bit_Distances = 0
    for x =1:N
        for Bit_Inside in Cell_Bit_Array[x] #These are all the cell types in bit form
            for y =1:N #We loop over the others
                if x!=y #Must look at different otherwise distance 0 again
                    Bit_Distan_Tem = M
                    for Bit_Outside in Cell_Bit_Array[y] #We loop over all the cell types in this other one
                        Bit_Distan_Temp = sum(abs.(Bit_Inside .- Bit_Outside)) 
                        if Bit_Distan_Temp < Bit_Distan_Tem #Distance must be smaller if it is the best fit
                            Bit_Distan_Tem = Bit_Distan_Temp
                        end
                    end
                    Bit_Distances += sqrt(Bit_Distan_Tem)/(N*(N-1)) #Sqrt to make euclidean distance, N*(N-1) for normalisation
                end
            end
        end
    end
    
    
    #Here we mess a bit with the indices to just have the ones with the highest score later
    
    Indices = sortslices(Scoring,dims=2,rev=true)
    Evolution_Indices = Indices[1,1:K]

    AllIndices = deepcopy(Indices[2,:])
    Indices = Indices[2,1:K]
    Indices = Int.(Indices)
    #print(Indices[2,1:K])
    return Indices, AllIndices, Evolution_Indices, [Cell_Types,Cell_Amounts],Cell_Bit, KLD_Distance, Bit_Distances
end

function Epi_Plotter(Indices,K,Final_Epigenetic_Factors)
    "Function used to plot the epigenetic factors of different cells
    Input:
    Indices                 = Indices of highest scoring runs  
    k                       = 4 or 8 plots
    Final_Epigentic_Factors = Epigenetic factors that we will plot
    Output:
    k = Plot element, simply use display on the output to see the requested plot
    "
    pa = plot(Final_Epigenetic_Factors[Indices[1],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pb = plot(Final_Epigenetic_Factors[Indices[2],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pc = plot(Final_Epigenetic_Factors[Indices[3],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pd = plot(Final_Epigenetic_Factors[Indices[4],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    if K==8
        pe = plot(Final_Epigenetic_Factors[Indices[5],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        pf = plot(Final_Epigenetic_Factors[Indices[6],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        pg = plot(Final_Epigenetic_Factors[Indices[7],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        ph = plot(Final_Epigenetic_Factors[Indices[8],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        k = plot(pa,pb,pc,pd,pe,pf,pg,ph,layout=(4,2),legend=false)
    else
        k = plot(pa,pb,pc,pd,layout=(2,2),legend=false)
    end
    return k
end

function Gene_Variance(Final_Epigenetic_Factors,M,N)
    "Function used to calcualte the variance in expressed genes
    Input:
    Final_epigenetic_Factors = Final epigenetic values which represent gene expression
    M                        = Amount of Genes
    N                        = How many copies of same network do we have
    Output:
    Deviation = Variance in these genes (normalised by 2^Splits*N)
    "
    BitterMatrix = signbit.(-Final_Epigenetic_Factors[:,:,:])
    Average_Bits = zeros(M)
    for i=1:N
        for j=1:2^Splits
            Average_Bits += BitterMatrix[i,j,:]
        end
    end
    Average_Bits /= N*2^Splits
    Deviation = 0

    for i=1:N
        for j=1:2^Splits
            Deviation += sum((Average_Bits .- BitterMatrix[i,j,:]).^2)/(2^Splits*N-1)
        end
    end
    
    return Deviation
end

function PCA_Distribution_Creator(PCA_Dimensions,Transformed,Dimensions_Distribution,Cell_Distribution)
    
    
    PCA_Dimensional_Values = zeros(PCA_Dimensions,findmax(Dimensions_Distribution)[1]) 
    #Here we will save all the values and use these for indexes later
    B = Array{Vector{Tuple{Float64, Int64}}}(undef, PCA_Dimensions)
    for j=1:PCA_Dimensions
        B[j] = [(i, count(==(i), Transformed)) for i in unique(Transformed[j,:])] #Here we find what the arguments are for the
        #different dimensions
        for k = 1:length(B[j])
            PCA_Dimensional_Values[j,k] = B[j][k][1] #Here we save all the individual arguments in the corresponding dimension
        end
    end
    #print(PCA_Dimensional_Values)   #If we wanna see them

    Components_Length = length(Transformed[1,:]) 
    for k in 1:Components_Length
        Index_PCA = zeros(PCA_Dimensions)

        for l=1:PCA_Dimensions
            Index_PCA[l] = (findfirst(==(Transformed[l,k]), PCA_Dimensional_Values[l,:]))
        end 
        Index_PCA = Int.(Index_PCA)
        
        
        if PCA_Dimensions==2 #I really tried but I couldn't fix it so just throwing in this hack
            Cell_Distribution[Index_PCA[1],Index_PCA[2]] += 1/Components_Length
        elseif PCA_Dimensions==1
            Cell_Distribution[Index_PCA[1]] += 1/Components_Length   
        elseif PCA_Dimensions==3
            Cell_Distribution[Index_PCA[1],Index_PCA[2],Index_PCA[3]] += 1/Components_Length
        elseif PCA_Dimensions==4
            Cell_Distribution[Index_PCA[1],Index_PCA[2],Index_PCA[3],Index_PCA[4]] += 1/Components_Length
        elseif PCA_Dimensions==5
            Cell_Distribution[Index_PCA[1],Index_PCA[2],Index_PCA[3],Index_PCA[4],Index_PCA[5]] += 1/Components_Length
        else
        print("ADD MORE HERE THERE IS A MISTAKE, DO NOT GO OVER 5 PCA DIMENSIONS YET")
        end
    end
    

    
    return Cell_Distribution
end



function Earth_Movers_Distance_1D(Vector1,Vector2)

    #Takes 2 distribution vectors containing bins with a position and their %
    Length_V = length(Vector1[1,:]) #Both should be same length anyways
    EMD = zeros(Length_V+1) #We add an extra one because of the following loop
    
    for i =1:Length_V
        EMD[i+1] = Vector1[2,i] - Vector2[2,i] + EMD[i]
    end
    EMD_Distance = 0
    
    for i =1:Length_V-1
        EMD_Distance += abs(EMD[i+1])*(Vector1[1,i+1] - Vector1[1,i]) #We take moved distribution times moved distance

    end

    
    return EMD_Distance
end


function Robustness_Distribution_Scoring()
    
    
    #Here we create a very general robustness measure
    #This is done by taking the PCA components over all the different generations
    #Then we categorise all cell types and measure how similar their distribution
    #is in a given network

    #!!!!!!! This robustness does not consider cell type distances
    
    Robustness_Score = zeros(Generations,P)
    for Genere=1:Generations  #We want to calculate this for all generations
        for i = 1:P #We have to repeat this for all different networks in a generation
            #This is used to find the maximum of different cell types along every dimension in this network
            Dimensions_Distribution = zeros(Int32,PCA_Dimensions)
            for j=1:N
                Transformed = transform(model,reshape(signbit.(-History_Final_Epigenetic_Factors[Genere,i,j,:,:]),2^Splits,M)')

                for k=1:PCA_Dimensions
                    if Int.(length(countmap(Transformed[k,:]))) >  Dimensions_Distribution[k]
                        Dimensions_Distribution[k] = Int.(length(countmap(Transformed[k,:])))
                    end
                end   
            end

            #Now we have the max dimensions, now we want to make all distributions
            #Of this specific network

            Dimensions_Distribution = (Dimensions_Distribution...,) #Magically makes a tuple!
            #print(Dimensions_Distribution) #if we wanna visualise

            Cell_Distribution_Collection = Array{Array{Float64, PCA_Dimensions}}(undef, N)
            Vectorised_Cell_Distribution_Collection = Array{Vector{Float64}}(undef, N)

            for j=1:N #Now we collect all the distributions
                Transformed = transform(model,reshape(signbit.(-History_Final_Epigenetic_Factors[Genere,i,j,:,:]),2^Splits,M)')

                Cell_Distribution = zeros(Dimensions_Distribution) #This is where we need it to be a tuple!
                Cell_Distribution_Collection[j] = PCA_Distribution_Creator(PCA_Dimensions,Transformed,Dimensions_Distribution,Cell_Distribution)

                #Now we want to convert all of these into vectors to apply Jensen-Shannon
                Vectorised_Cell_Distribution_Collection[j] = sort(vec(Cell_Distribution_Collection[j]),rev=true)
            end

            #JS-divergence
            for tau = 1:N-1
                for pau = tau:N
                    Robustness_Score[Genere,i] += js_divergence(Vectorised_Cell_Distribution_Collection[tau],Vectorised_Cell_Distribution_Collection[pau])
                end
            end


        end
    end
    return Robustness_Score
end



function EMD_Generational_Calculator()
    
    #Here we will create a Robustness measure that takes into account how far
    #away different cell types are for the same network.
    #we will do this by creating a PCA analysis for EVERY single network
    #for all the different generations
    #(Alternatively we see if we can use the same PCA analysis within one generation)
    #We will create a one dimensional distribution and use our simplified
    #EMD algorithm
    
    Vectorised_Cell_Network_Collection = Array{Matrix{Float64}}(undef, N) #Here we save these distributions
    #This loops over copies of same network and makes all our distributions
    EMD_Distances = zeros(Generations)
    for generation_p = 1:Generations
        EMD_Dist = 0
        for P_Valu = 1:P
            FullNetworkBits = reshape(signbit.(-History_Final_Epigenetic_Factors[generation_p,P_Valu,:,:,:]),2^Splits*N,M)'
            Networkmodel= fit(PCA,FullNetworkBits;maxoutdim=1)
            Networkmodel_EigenV = eigvals(Networkmodel::PCA)
            for i=1:N
                #This creates the Distribution of a single individual
                #We start to apply this PCA to one of these individuals
                NetworkBits = reshape(signbit.(-History_Final_Epigenetic_Factors[generation_p,P_Valu,i,:,:]),2^Splits,M)'
                Transformed_Network = transform(Networkmodel,NetworkBits)
                Distribution_Netw = zeros(0)
                X_Values = zeros(0)
                Transformed_Network = round.(Transformed_Network,digits=4) #Otherwise ugly floating point numbers
                Length_Netw_Normal = 1/length(Transformed_Network)
                for i in Transformed_Network

                    if i in X_Values
                        Distribution_Netw[findfirst(==(i), X_Values)] += Length_Netw_Normal
                    else
                        X_Values = vcat(X_Values,i)
                        Distribution_Netw = vcat(Distribution_Netw,Length_Netw_Normal)
                    end
                end



                Combined_Distribution = zeros(2,length(X_Values))
                Combined_Distribution[1,:] = round.(X_Values/Networkmodel_EigenV,digits=4)
                Combined_Distribution[2,:] = Distribution_Netw
                Combined_Distribution = sortslices(Combined_Distribution,dims=2)

                Vectorised_Cell_Network_Collection[i] = Combined_Distribution
            end


            #Now we need to apply EMD to these distributions and save the result

            for tau = 1:N-1   #Preparing the vectors to have the same X values and length
                for pau = tau+1:N
                    Vector1 = Vectorised_Cell_Network_Collection[tau]
                    Vector2 = Vectorised_Cell_Network_Collection[pau]
                    for i in Vector1[1,:]
                        if !(i in Vector2[1,:])
                            Vector2 = hcat(Vector2, [i,0]) #Adding empty position
                        end
                    end
                    for i in Vector2[1,:]
                        if !(i in Vector1[1,:])
                            Vector1 = hcat(Vector1, [i,0]) #Adding empty position
                        end
                    end

                    Vector1 = (sortslices(Vector1,dims=2)) #Sorting them with the added X values
                    Vector2 = (sortslices(Vector2,dims=2))
                    EMD_Dist += Earth_Movers_Distance_1D(Vector1,Vector2) #Calculating EDM for 1D

                end
            end
        end
        EMD_Dist = 2*EMD_Dist/(P*N*(N-1))  #Normalisation
        EMD_Distances[generation_p] = EMD_Dist
    end
    return EMD_Distances
end


function Two_Generational_Plotter(Generation,Cut_Off,Duration,Length_Orbits,Opacity)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1+2*(Generation-1):2+2*(Generation-1)
        for j=1:Amount_Copies
            if Shortest>size(Orbit_Array[i,j])[2]
                Shortest = size(Orbit_Array[i,j])[2] #Used later to check which data set is the shortest when plotting
            end

            Total_Length += size(Orbit_Array[i,j])[2]

        end
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2*8*2^Splits)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1+2*(Generation-1):2+2*(Generation-1)
        for j=1:Amount_Copies
            if Cut_Off
                
                for p=0:2^Splits-1
                    PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i,j][2+p*M:1+(p+1)*M,1:Length_Orbits]
                end
            else
                Length_Orbits =  size(Orbit_Array[i,j])[2]
                for p=0:2^Splits-1
                    PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i,j][2+p*M:1+(p+1)*M,:]
                end
            end

            Collection_Index += Length_Orbits*2^Splits
        end
    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,2,Amount_Copies,2^Splits,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:2
        for j=1:Amount_Copies

            for tau=1:2^Splits
                tester = Orbit_Array[i+2*(Generation-1),j][2+(tau-1)*M:1+tau*M,1:Shortest]
                Transformed = transform(Orbital_Model,tester)
                PCA_Orbits_2Gen[i,j,tau,:,:] = Transformed
            end
        end
    end
    
    
    Colours = [:red,:orange,:maroon,:chocolate4]
    @gif for k=1:Duration
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen[1,1,1,1,1:k],PCA_Orbits_2Gen[1,1,1,2,1:k],PCA_Orbits_2Gen[1,1,1,3,1:k],legend=false,c=:blue,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)


        for j=1:8
            for tau=1:2^Splits
                PCAPlots = plot!(PCA_Orbits_2Gen[i,j,tau,1,1:k],PCA_Orbits_2Gen[i,j,tau,2,1:k],PCA_Orbits_2Gen[i,j,tau,3,1:k],legend=false,c=:blue,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)
            end
        end

        i=2
        for j=1:4
            colour = Colours[j]
            for tau=1:2^Splits
                PCAPlots = plot!(PCA_Orbits_2Gen[i,2*j-1,tau,1,1:k],PCA_Orbits_2Gen[i,2*j-1,tau,2,1:k],PCA_Orbits_2Gen[i,2*j-1,tau,3,1:k],legend=false,c=colour,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                PCAPlots = plot!(PCA_Orbits_2Gen[i,2*j,tau,1,1:k],PCA_Orbits_2Gen[i,2*j,tau,2,1:k],PCA_Orbits_2Gen[i,2*j,tau,3,1:k],legend=false,c=colour,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)

            end
        end


    end
end




function Generational_Tracker(Family_Distance,Original_Tracking,Types)
    Cell_Generational_Tracking = deepcopy(Original_Tracking)
    Original_Tracking = deepcopy(Cell_Generational_Tracking)
    size(Types[1,:])


    TypeFam = [Types[1,:],1]
    TypeFam = hcat(TypeFam,[Types[1,:],2])
    TypeFam = hcat(TypeFam,[Types[1,:],3])
    TypeFam = hcat(TypeFam,[Types[1,:],4])

    print(size(TypeFam[1,:]))

    TypeFam = [Types[1,:],1]
    Distances_Gen = zeros(length(Cell_Generational_Tracking[1,:]))
    Cell_Generational_Tracking[1,1] = 1
    for i=2:length(Cell_Generational_Tracking[1,:])
        NewFam = true
        for j=1:length(TypeFam[1,:]) 
            if sum(abs.(Types[i,:]-TypeFam[1,j])) < Family_Distance + 0.5
                if NewFam
                    NewFam = false
                    Cell_Generational_Tracking[1,i] = j
                    Distances_Gen[j] = sum(abs.(Types[i,:]-TypeFam[1,j]))
                elseif Distances_Gen[j] > sum(abs.(Types[i,:]-TypeFam[1,j]))
                    Cell_Generational_Tracking[1,i] = j
                    Distances_Gen[j] = sum(abs.(Types[i,:]-TypeFam[1,j]))
                end
            end
        end
        if NewFam
            Distances_Gen[i] = sum(abs.(Types[i,:]-TypeFam[1,1]))
            Cell_Generational_Tracking[1,i] = Int(length(TypeFam[1,:])+1)
            TypeFam = hcat(TypeFam,[Types[i,:],Int(length(TypeFam[1,:])+1)])

        end
    end

    Cell_Generational_Tracking = Cell_Generational_Tracking[:,sortperm(Cell_Generational_Tracking[1,:])]



    Amount_Types = Cell_Generational_Tracking[1,end]
    Top_Bound_Index = zeros(Amount_Types)
    for j=1:Amount_Types
        Top_Bound_Index[j] = findfirst(isequal(j),Cell_Generational_Tracking[1,:])
    end
    print(Top_Bound_Index[:])


    #print(Cell_Generational_Tracking)
    Copied_Tracker = deepcopy(Cell_Generational_Tracking[2:end,:])
    Copied_Tracker = Copied_Tracker/sum(Copied_Tracker[1,:])

    Boundaries = zeros(Generations,length(Copied_Tracker[1,:]))
    for beta in 2:length(Copied_Tracker[1,:])
        Copied_Tracker[:,beta] += Copied_Tracker[:,beta-1]
        Boundaries[:,beta] = Copied_Tracker[:,beta] - Copied_Tracker[:,beta-1]
    end

    Cell_Type_Plotter = plot(Copied_Tracker[:,1]; ribbon = (Copied_Tracker[:,1],LinRange(0, 0, Generations)))
    for beta in 2:length(Copied_Tracker[1,:])
        Cell_Type_Plotter = plot!(Copied_Tracker[:,beta]; ribbon = (Boundaries[:,beta],LinRange(0, 0, Generations)),legend=false)
    end
    display(Cell_Type_Plotter)




    

    #Now we do the same but we group them in families


    Family_Boundaries = ones(Generations,Amount_Types)
    Family_Boundaries[:,1] = Copied_Tracker[:,Int(Top_Bound_Index[2]-1)].-0
    for beta in 2:Amount_Types-1

        Family_Boundaries[:,beta] = Copied_Tracker[:,Int(Top_Bound_Index[beta+1]-1)] .- Copied_Tracker[:,Int(Top_Bound_Index[beta]-1)]
    end
    Family_Boundaries[:,end] = 1 .- Copied_Tracker[:,Int(Top_Bound_Index[Amount_Types]-1)]


    Cell_Type_Family_Plotter = plot(Copied_Tracker[:,Int(Top_Bound_Index[2]-1)]; ribbon = (Family_Boundaries[:,1],LinRange(0, 0, Generations)))
    for beta in 2:Amount_Types-1
        Cell_Type_Family_Plotter = plot!(Copied_Tracker[:,Int(Top_Bound_Index[beta+1]-1)]; ribbon = (Family_Boundaries[:,beta],LinRange(0, 0, Generations)),legend=false)
    end
    Cell_Type_Family_Plotter = plot!(ones(Generations); ribbon = (Family_Boundaries[:,Amount_Types],LinRange(0, 0, Generations)),legend=false)

    if false
        Cell_Type_Family_Plotter = plot!(Copied_Tracker[:,1],linestyle=:dash)
        for beta in 2:length(Copied_Tracker[1,:])
            Cell_Type_Family_Plotter = plot!(Copied_Tracker[:,beta],linestyle=:dash)
        end
    end

    display(Cell_Type_Family_Plotter)
end

function Generational_Tracker_Initialisation()
    TypesString = deepcopy(Cell_Generational_Tracking[1,:])
    TypesString = string.(TypesString,base=2,pad=100)
    Types = zeros(Int16,length(Cell_Generational_Tracking[1,:]),M)
    for i in 1:length(Cell_Generational_Tracking[1,:])
        for j=1:M
            Types[i,j] = (parse(Int16,TypesString[i][j]))
        end
    end
    Original_Tracking = deepcopy(Cell_Generational_Tracking)
    return Original_Tracking, Types
end

function Interpolator(Times,Play_Array)
    Vectors = zeros(length(Times),1+M*2^Splits)
    for i=1:length(Times)
        Index_Interp = findfirst(>=(Times[i]), Play_Array[1,:])
        Vectors[i,:] = Play_Array[:,Index_Interp-1] + (Times[i]-Play_Array[1,Index_Interp-1])*(Play_Array[:,Index_Interp] - Play_Array[:,Index_Interp-1])/(Play_Array[1,Index_Interp] - Play_Array[1,Index_Interp-1])
    end
    return Vectors
end


function One_Generational_Plotter(Cut_Off,Duration,Length_Orbits,Opacity,Name)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1:N_Perturbed

            if Shortest>size(Orbit_Array[i])[2]
                Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
            end
            Total_Length += size(Orbit_Array[i])[2]
        
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1:N_Perturbed
        
        if Cut_Off

            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
            end
        else
            Length_Orbits =  size(Orbit_Array[i,j])[2]
            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
            end
        end

        Collection_Index += Length_Orbits*2^Splits

    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,N_Perturbed,2^Splits,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:N_Perturbed
        
        for tau=1:2^Splits
            tester = Orbit_Array[i][2+(tau-1)*M:1+tau*M,1:Shortest]
            Transformed = transform(Orbital_Model,tester)
            PCA_Orbits_2Gen[i,tau,:,:] = Transformed
        end

    end
    
    
    anim = @animate for k=1:Duration
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen[1,1,1,1:k],PCA_Orbits_2Gen[1,1,2,1:k],PCA_Orbits_2Gen[1,1,3,1:k],legend=false,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)


        for i=1:N_Perturbed
            for tau=1:2^Splits
                PCAPlots = plot!(PCA_Orbits_2Gen[i,tau,1,1:k],PCA_Orbits_2Gen[i,tau,2,1:k],PCA_Orbits_2Gen[i,tau,3,1:k],legend=false,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)
            end
        end

        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 5
    gif(anim, Name, fps = 15)
    
end

function adjoint(a)
    return a'
end


function Coarsed_Variance(Variance_Length,Time_Orbital_Collection,Coarsing,Orbit_Length)
    
    
    #Variance[i] = sum((Time_Orbital_Collection[i,:,:] .- mean(Time_Orbital_Collection[i,:,:],dims=1)).^2)/(M-1)

    
    
    Variance = zeros(Variance_Length)
    Average = zeros(Variance_Length,M)
    
    for i = 1:Variance_Length #For all different times
       Average[i,:] =  mean(Time_Orbital_Collection[i,:,:],dims=1)[1,:]
        for k=1:Orbit_Length #Have to do it for all the orbits
            Temp_Var = 1000
            Add_Var = 0
            
            for j=max(1,i-Coarsing):i+Coarsing 
                Add_Var = sum((Time_Orbital_Collection[j,k,:] .- Average[i,:]).^2)/(M-1)

                if Add_Var < Temp_Var
                    Temp_Var = Add_Var               
                end
            end
            Variance[i] += Temp_Var  

        end
    end
    
    return Variance
end


function Multi_One_Generational_Plotter(Cut_Off,Duration,Length_Orbits,Opacity,Name,Amount_Perturbations,Partial,N_Perturbed)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1:N_Perturbed*Amount_Perturbations

            if Shortest>size(Orbit_Array[i])[2]
                Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
            end
            Total_Length += size(Orbit_Array[i])[2]
        
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed*Amount_Perturbations)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1:N_Perturbed*Amount_Perturbations
        
        if Cut_Off

            for p=0:2^Splits-1

                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
            end
        else
            Length_Orbits =  size(Orbit_Array[i,j])[2]
            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
            end
        end

        Collection_Index += Length_Orbits*2^Splits

    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,N_Perturbed*2^Splits,Amount_Perturbations,3,Shortest)
    PCA_Orbits_Partial = zeros(Float32,Int(N_Perturbed/Partial)*2^Splits,Amount_Perturbations,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:N_Perturbed
        for j=0:Amount_Perturbations-1
        for tau=1:2^Splits
            tester = Orbit_Array[i+N_Perturbed*j][2+(tau-1)*M:1+tau*M,1:Shortest]
            Transformed = transform(Orbital_Model,tester)
            PCA_Orbits_2Gen[(i-1)*2^Splits+tau,j+1,:,:] = Transformed
            if mod(i,Partial) == 0
                PCA_Orbits_Partial[(Int(i/Partial)-1)*2^Splits+tau,j+1,:,:] = Transformed
            end
        end
        end

    end
    Colours = [:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
    
    #PCA_Orbits_Partial = reshape(PCA_Orbits_Partial,(Int(2^Splits*N_Perturbed*Amount_Perturbations/Partial),3,Shortest))
    PCA_Orbits_2Gen_Plot = permutedims(PCA_Orbits_Partial,(4,3,2,1))
    Orbit_Range = Int(N_Perturbed/Partial)*2^Splits
    tick()
    anim = @animate for k=1:Int(Duration/5)
        
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,:],PCA_Orbits_2Gen_Plot[1:k,2,1,:],PCA_Orbits_2Gen_Plot[1:k,3,1,:],c=Colours[1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for p=1:Amount_Perturbations
            PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,:],PCA_Orbits_2Gen_Plot[1:k,2,p,:],PCA_Orbits_2Gen_Plot[1:k,3,p,:],c=Colours[p],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)

        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 5
    tock()
    tick()
    animlong = @animate for k=1:Duration
        
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,:],PCA_Orbits_2Gen_Plot[1:k,2,1,:],PCA_Orbits_2Gen_Plot[1:k,3,1,:],c=Colours[1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for p=1:Amount_Perturbations
            PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,:],PCA_Orbits_2Gen_Plot[1:k,2,p,:],PCA_Orbits_2Gen_Plot[1:k,3,p,:],c=Colours[p],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)

        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 50
    tock()
    abra = gif(anim, Name, fps = 10)
    display(abra)

    kadabra = gif(animlong, Name[1:end-4] * "_Long" *Name[end-3:end], fps = 3)
    display(kadabra)



end


function Multi_One_Generational_Plotter2(Cut_Off,Duration,Length_Orbits,Opacity,Name,Amount_Perturbations,Partial,N_Perturbed,Orbit_Array)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1:N_Perturbed*Amount_Perturbations

            if Shortest>size(Orbit_Array[i])[2]
                Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
            end
            Total_Length += size(Orbit_Array[i])[2]
        
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed*Amount_Perturbations)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1:N_Perturbed*Amount_Perturbations
        
        if Cut_Off

            for p=0:2^Splits-1

                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
            end
        else
            Length_Orbits =  size(Orbit_Array[i,j])[2]
            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
            end
        end

        Collection_Index += Length_Orbits*2^Splits

    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,N_Perturbed*2^Splits,Amount_Perturbations,3,Shortest)
    PCA_Orbits_Partial = zeros(Float32,Int(N_Perturbed/Partial)*2^Splits,Amount_Perturbations,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:N_Perturbed
        for j=0:Amount_Perturbations-1
        for tau=1:2^Splits
            tester = Orbit_Array[i+N_Perturbed*j][2+(tau-1)*M:1+tau*M,1:Shortest]
            Transformed = transform(Orbital_Model,tester)
            PCA_Orbits_2Gen[(i-1)*2^Splits+tau,j+1,:,:] = Transformed
            if mod(i,Partial) == 0
                PCA_Orbits_Partial[(Int(i/Partial)-1)*2^Splits+tau,j+1,:,:] = Transformed
            end
        end
        end

    end
    Colours = [:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
    
    #PCA_Orbits_Partial = reshape(PCA_Orbits_Partial,(Int(2^Splits*N_Perturbed*Amount_Perturbations/Partial),3,Shortest))
    PCA_Orbits_2Gen_Plot = permutedims(PCA_Orbits_Partial,(4,3,2,1))
    Orbit_Range = Int(N_Perturbed/Partial)*2^Splits
    tick()
    anim = @animate for k=1:Int(Duration/5)
        
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,:],PCA_Orbits_2Gen_Plot[1:k,2,1,:],PCA_Orbits_2Gen_Plot[1:k,3,1,:],c=Colours[1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for p=1:Amount_Perturbations
            PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,:],PCA_Orbits_2Gen_Plot[1:k,2,p,:],PCA_Orbits_2Gen_Plot[1:k,3,p,:],c=Colours[p],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)

        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 5
    tock()
    tick()
    animlong = @animate for k=1:Duration
        
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,:],PCA_Orbits_2Gen_Plot[1:k,2,1,:],PCA_Orbits_2Gen_Plot[1:k,3,1,:],c=Colours[1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for p=1:Amount_Perturbations
            PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,:],PCA_Orbits_2Gen_Plot[1:k,2,p,:],PCA_Orbits_2Gen_Plot[1:k,3,p,:],c=Colours[p],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)

        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 50
    tock()
    abra = gif(anim, Name, fps = 10)
    display(abra)

    kadabra = gif(animlong, Name[1:end-4] * "_Long" *Name[end-3:end], fps = 3)
    display(kadabra)

    
    PCAPlots_Final = scatter(PCA_Orbits_2Gen_Plot[end,1,1,:],PCA_Orbits_2Gen_Plot[end,2,1,:],PCA_Orbits_2Gen_Plot[end,3,1,:],title="Final Cell States",c=Colours[1],legend=false)
    for p=1:Amount_Perturbations
        PCAPlots_Final = scatter!(PCA_Orbits_2Gen_Plot[end,1,p,:],PCA_Orbits_2Gen_Plot[end,2,p,:],PCA_Orbits_2Gen_Plot[end,3,p,:],c=Colours[p],legend=false)

    end

    display(PCAPlots_Final)
    savefig(Name[1:end-4] * "_Final.png")
end

function Multi_One_Generational_Plotter3(Cut_Off,Duration,Length_Orbits,Opacity,Name,Amount_Perturbations,Partial)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1:N_Perturbed*Amount_Perturbations

            if Shortest>size(Orbit_Array[i])[2]
                Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
            end
            Total_Length += size(Orbit_Array[i])[2]
        
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed*Amount_Perturbations)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1:N_Perturbed*Amount_Perturbations
        
        if Cut_Off

            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
            end
        else
            Length_Orbits =  size(Orbit_Array[i,j])[2]
            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
            end
        end

        Collection_Index += Length_Orbits*2^Splits

    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,N_Perturbed*2^Splits,Amount_Perturbations,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:N_Perturbed
        for j=1:2^Splits
            for tau=1:Amount_Perturbations
                tester = Orbit_Array[i+ (tau-1)*N_Perturbed][2+(j-1)*M:1+j*M,1:Shortest]
                Transformed = transform(Orbital_Model,tester)
                PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,:,:] = Transformed

            end
        end
    end
    Colours = [:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
    
    PCA_Orbits_2Gen_Plot = permutedims(PCA_Orbits_2Gen,(4,3,2,1))
    anim = @animate for k=1:Duration
        
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1],c=Colours[1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for l=1:Amount_Perturbations
            
            PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,l,:],PCA_Orbits_2Gen_Plot[1:k,2,l,:],PCA_Orbits_2Gen_Plot[1:k,3,l,:],c=Colours[l],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
            
        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 15

    abra = gif(anim, Name, fps = 15)
end

function N_Dimensional_Euclid(Position1,Position2,Dimensions)
    Summed = 0
    for i =1:Dimensions
        Summed += (Position2[i]-Position1[i])^2
    end
    Distance = sqrt(Summed)
    return Distance
end



function Distance_Movement_Matrix(Movement_Matrix)
    
    for i=1:N_Sources #we must loop over all the sources to find a good sink for it

        #First we try to find the closest sinks
        Distance_Indices = (hcat(DistanceMatrix[i,:], 1:N_Sinks))
        Distance_Indices = Distance_Indices[sortperm(Distance_Indices[:, 1]), :]
        Distance_Indices = Int.(Distance_Indices[:,2])
        print(Distance_Indices)

        #Then we check if we can still send anything to this sink
        Displacing = true
        D_Index = 1
        while Displacing
            I_Sink = Distance_Indices[D_Index] #This is the index of the sink we are checking now
            Available_Sink = Sinks_Input[I_Sink] - sum(Movement_Matrix[:,I_Sink])
            print(Sinks_Input[I_Sink])
            if Available_Sink > 0 #It can have more added to it
                #We check if we can send everything to the sink or only partial
                Remaining_Flow = Sources_Output[i] - sum(Movement_Matrix[i,:])
                if Remaining_Flow > Available_Sink
                #If partial we fill a bit and go back to the start to find the next closest sink
                    Movement_Matrix[i,I_Sink] = Available_Sink
                    D_Index += 1
                    print("Partial.     ")
                    #BUT WE HAVE TO CHECK IF WE JUST FILLED IT ALL IN!!!! 
                    if Sources_Output[i] - sum(Movement_Matrix[i,:]) < 0.0001
                        Displacing = false
                        print("Finished After Partial")
                    end

                else  
                #If we can send it all we do and go to the next Source i+1
                    Displacing = false
                    Movement_Matrix[i,I_Sink] = Remaining_Flow
                    print("Finished.     ")

                end
            else #Sink is full
                print("Full.     ")

                #Cannot send more in the sink so we have to check the next one
                D_Index += 1
            end
        end
    end
    Movement_Matrix = round.(Movement_Matrix,digits=4)

    return Movement_Matrix
end

function Cost_Reducing_Rewiring(Attempts,Optimised_Indices)
    print("Initial Costs: ")
    print(sum(Cost_Matrix))
    for yota =1:Attempts
        println(" ")
        #We make a big while loop until a certain condition we decide later
        Cost_Matrix = DistanceMatrix .* Movement_Matrix
        print(sum(Cost_Matrix))
        for theta =1:1   #Old loop that can be removed

            if Optimised_Indices != []
                for kappa=1:length(Optimised_Indices[1,:])
                    Cost_Matrix[Optimised_Indices[1,kappa],Optimised_Indices[2,kappa]] = 0
                end
            end

            #We find the connection that has the highest cost, find the source and the corresponding sink
            Source_Index = argmax(Cost_Matrix)[1]
            Sink_Index = argmax(Cost_Matrix)[2]
            Improvement = 0 #Use this to keep track of best
            Displace = 0 #Keep track of best
            Best_Source = 0 #Keep track
            Best_Sink = 0 #Keep track

            #We let this source connect to other sinks instead and try to change another source that was connected to that sink
            for i=1:N_Sinks #We loop over all other sinks
                if i!=Sink_Index #Make sure it is a different sink
                    for j=1:N_Sources #We loop over all other Sources
                        if j!=Source_Index #Make sure it is a different source
                            #Check how much we may change 
                            Displace_Temp = min(Movement_Matrix[Source_Index,Sink_Index],Movement_Matrix[j,i])
                            #Calculate the improvement
                            #-DM[So,Si] + DM[So,i] We remove So -> Si and make So -> i
                            #-DM[j,i] + DM[j,Si] We remove j -> i and make j -> Si
                            Temp_Improvement = Displace_Temp*(-DistanceMatrix[Source_Index,Sink_Index] + DistanceMatrix[Source_Index,i] -DistanceMatrix[j,i] + DistanceMatrix[j,Sink_Index])

                            #Find the best improvement for this initial Source_Sink combo
                            if Temp_Improvement < Improvement #Found a better improvement
                                Improvement = Temp_Improvement
                                Displace = Displace_Temp
                                Best_Source = j
                                Best_Sink = i
                            end

                        end
                    end
                end
            end
            print("Better amount:  ")
            print(Improvement)
            if Improvement==0
                print("No improvement???")
                print(Displace)
                print("    ")
                print(Sink_Index)
                print(Source_Index)
                if Optimised_Indices == []
                    Optimised_Indices = vcat(Optimised_Indices,[Source_Index;Sink_Index])
                else
                    Optimised_Indices = hcat(Optimised_Indices,[Source_Index;Sink_Index])
                end

            else
                print("WE IMPROVED")
                Movement_Matrix[Source_Index,Sink_Index] -= Displace
                Movement_Matrix[Best_Source,Best_Sink] -= Displace
                Movement_Matrix[Best_Source,Sink_Index] += Displace
                Movement_Matrix[Source_Index,Best_Sink] += Displace


            end

            #Switch the best one and repeat
        end
        print("   ")
        Cost_Matrix = DistanceMatrix .* Movement_Matrix
        print(sum(Cost_Matrix))
    end

    println(" ")
    print("Final Costs: ")
    print(sum(Cost_Matrix)) 
    return sum(Cost_Matrix)
end

function Final_Cell_PCAScatterPlot(Orbit_Array,Single_Plot)
    Final_Cell_States_PCA = zeros(Amount_Large_Perturbations*Amount_Small_Perturbations*2^Splits,M)
    Transformed_Data = zeros(Amount_Small_Perturbations*2^Splits,Amount_Large_Perturbations,2)
    Before_Data = zeros(Amount_Small_Perturbations*2^Splits,Amount_Large_Perturbations,M)
    for j=1:Amount_Large_Perturbations*Amount_Small_Perturbations
        for i=1:2^Splits
            Final_Cell_States_PCA[j+Amount_Large_Perturbations*Amount_Small_Perturbations*(i-1),:] = Orbit_Array[j][2+(i-1)*M:1+i*M,end]
        end
    end
    #This is added in in case we only have the same cell types, then our code would crash
    Final_Cell_States_PCA[1,1] += 1
    
    
    for j=1:Amount_Large_Perturbations
        for i=1:Amount_Small_Perturbations
            for tau=1:2^Splits
                Before_Data[i+(tau-1)*Amount_Small_Perturbations,j,:] = Orbit_Array[i+(j-1)*Amount_Small_Perturbations][2+(tau-1)*M:1+tau*M,end]
            end
        end
    end

    Final_Cell_States_PCA[1:5,1:10] .+= 1
    Final_Cell_PCA = fit(PCA,Final_Cell_States_PCA';maxoutdim =2,pratio =1)
    print(size(Final_Cell_PCA))
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Final_Cell_PCA))
    Final_Cell_States_PCA[1:5,1:10] .-= 1
 

    for j=1:Amount_Large_Perturbations

        Transformed_Data[:,j,:] = transform(Final_Cell_PCA,Before_Data[:,j,:]')'

    end
    Plot_Legend = true
    Colours = [:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
    if Single_Plot == false
        j=1
        abrada = scatter((Transformed_Data[1,j,1],Transformed_Data[1,j,2]),xlabel="PCA 1",ylabel="PCA 2",title="Final Cell States in PCA landscape",c=Colours[j],alpha=0.4,legend=Plot_Legend,label="Initial Noise = " * string(Large_Pert_Radius[j]))
        for j=2:Amount_Large_Perturbations
            abrada = scatter!((Transformed_Data[1,j,1],Transformed_Data[1,j,2]),c=Colours[j],alpha=0.4,legend=Plot_Legend,label="Initial Noise = " * string(Large_Pert_Radius[j]))
        end
        for j=1:Amount_Large_Perturbations
            for l=2:length(Transformed_Data[:,j,1])
                for j=1:Amount_Large_Perturbations
                    abrada = scatter!((Transformed_Data[l,j,1],Transformed_Data[l,j,2]),c=Colours[j],alpha=0.4,label=false)
                end
            end
        end
        savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/PCA_Maps/Scatter_PCA"*Copy_String*".png")

        KernelData = reshape(Transformed_Data, (Amount_Small_Perturbations*2^Splits*Amount_Large_Perturbations, 2))
        
        k = kde((KernelData[:,1], KernelData[:,2]))


        display(contourf(k,
             c = :vik,title="Final Cell State PCA Heatmap",
             linewidth = 0))
        savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/PCA_Maps/Heatmap_"*Copy_String*".png")

        
    else
        j = Single_Plot
        abrada = scatter((Transformed_Data[:,j,1],Transformed_Data[:,j,2]),xlabel="PCA 1",ylabel="PCA 2",title="Final Cell States in PCA landscape",c=Colours[j],alpha=0.4,legend=Plot_Legend,label="Initial Noise = " * string(Large_Pert_Radius[j]))
        
    end
    return abrada
end



function Multi_One_Generational_Plotter4(Cut_Off,Duration,Length_Orbits,Opacity,Name,Amount_Perturbations,Partial,N_Perturbed,Orbit_Array,Final_Epigenetic_Factors,Gene_Colour_Collection)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1:N_Perturbed*Amount_Perturbations

            if Shortest>size(Orbit_Array[i])[2]
                Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
            end
            Total_Length += size(Orbit_Array[i])[2]
        
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed*Amount_Perturbations)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1:N_Perturbed*Amount_Perturbations
        
        if Cut_Off

            for p=0:2^Splits-1

                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
            end
        else
            Length_Orbits =  size(Orbit_Array[i,j])[2]
            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
            end
        end

        Collection_Index += Length_Orbits*2^Splits

    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,Amount_Perturbations,N_Perturbed,2^Splits,3,Shortest)
    PCA_Orbits_Partial = zeros(Float32,Amount_Perturbations,Int(N_Perturbed/Partial),2^Splits,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:N_Perturbed
        for j=0:Amount_Perturbations-1
        for tau=1:2^Splits
            tester = Orbit_Array[i+N_Perturbed*j][2+(tau-1)*M:1+tau*M,1:Shortest]
            Transformed = transform(Orbital_Model,tester)
            PCA_Orbits_2Gen[j+1,i,tau,:,:] = Transformed
            if mod(i-1,Partial) == 0
                PCA_Orbits_Partial[j+1,(Int(i/Partial)),tau,:,:] = Transformed
            end
        end
        end

    end
    Colours = [:red,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
     
    #PCA_Orbits_Partial = reshape(PCA_Orbits_Partial,(Int(2^Splits*N_Perturbed*Amount_Perturbations/Partial),3,Shortest))
    PCA_Orbits_2Gen_Plot = permutedims(PCA_Orbits_Partial,(5,4,3,2,1))
    Orbit_Range = Int(N_Perturbed/Partial)*2^Splits
    
    print(size(PCA_Orbits_2Gen_Plot))
    if false
        for Gene_Colour in Gene_Colour_Collection
            tick()
            
            anim = @animate for k=1:Int(Duration/5)
                i=1
                PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1,1],c=Colours[Final_Epigenetic_Factors[1,1,1,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                for q=1:Int(N_Perturbed/Partial)
                    for p=1:2^Splits
                        for t =1:Amount_Perturbations
                        PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,q,t],PCA_Orbits_2Gen_Plot[1:k,2,p,q,t],PCA_Orbits_2Gen_Plot[1:k,3,p,q,t],c=Colours[Final_Epigenetic_Factors[t,q,p,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                        end
                    end
                end
        
                #for i=1:Int(N_Perturbed/5)
                #    for j=1:Amount_Perturbations
                #        for tau=1:2^Splits
                #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                #        end
                #    end
                #end
                
                #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])
        
                title!("Gene = $Gene_Colour Time = $k")
            end every 5
            tock()
            tick()
            animlong = @animate for k=1:Duration
                
                i=1
                PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1,1],c=Colours[Final_Epigenetic_Factors[1,1,1,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                for q=1:Int(N_Perturbed/Partial)
                    for p=1:2^Splits
                        for t =1:Amount_Perturbations
                        PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,q,t],PCA_Orbits_2Gen_Plot[1:k,2,p,q,t],PCA_Orbits_2Gen_Plot[1:k,3,p,q,t],c=Colours[Final_Epigenetic_Factors[t,q,p,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                        end
                    end
                end
                #for i=1:Int(N_Perturbed/5)
                #    for j=1:Amount_Perturbations
                #        for tau=1:2^Splits
                #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                #        end
                #    end
                #end
                
                #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])
        
                title!("Gene = $Gene_Colour Time = $k")
            end every 50
            tock()
    
            abra = gif(anim, Name[1:end-4]*"_Gene:"*string(Gene_Colour)*Name[end-3:end], fps = 10)
            display(abra)
        
            kadabra = gif(animlong, Name[1:end-4] * "_Long"*"_Gene:"*string(Gene_Colour) *Name[end-3:end], fps = 3)
            display(kadabra)
        
            
            Total_Ratio = principalratio(Orbital_Model)
            Eigen_Values = eigvals(Orbital_Model)
            Eigen_Values = Eigen_Values/sum(Eigen_Values)
            Eigen1 = Eigen_Values[1]
            Eigen2 = Eigen_Values[2]
            Eigen3 = Eigen_Values[3]
            PCAPlots_Final = scatter(PCA_Orbits_2Gen_Plot[end,1,1,1,:],PCA_Orbits_2Gen_Plot[end,2,1,1,:],PCA_Orbits_2Gen_Plot[end,3,1,1,:],c=Colours[Final_Epigenetic_Factors[1,1,1,Gene_Colour]+1],legend=true,label="
            Principal 1: $Eigen1
            Principal 2: $Eigen2]
            Principal 3: $Eigen3")
        
            for q=1:Int(N_Perturbed/Partial)
                for p=1:2^Splits
                    for t =1:Amount_Perturbations
                        PCAPlots_Final = scatter!((PCA_Orbits_2Gen_Plot[end,1,p,q,t],PCA_Orbits_2Gen_Plot[end,2,p,q,t],PCA_Orbits_2Gen_Plot[end,3,p,q,t]),c=Colours[Final_Epigenetic_Factors[t,q,p,Gene_Colour]+1],legend=true,label="")
                    end
                end
            end
            
            
            title!("Gene = $Gene_Colour , Ratio: $Total_Ratio")
            display(PCAPlots_Final)
            savefig(Name[1:end-4] * "_Final_Static_$Gene_Colour.png")
            
        end
    end
    Multi_Colours = [:cyan,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:darkmagenta,:lightcyan4,:olive,:gold4,:lightsalmon,:navajowhite,:pink]
    Only3Types = false
    tick()
    anim = @animate for k=1:Int(Duration/5)
        
        i=1
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
        if Only3Types
            Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
        end
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1,1],c=Multi_Colours[Colour_Index],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for q=1:Int(N_Perturbed/Partial)
            for p=1:2^Splits
                for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,q,t],PCA_Orbits_2Gen_Plot[1:k,2,p,q,t],PCA_Orbits_2Gen_Plot[1:k,3,p,q,t],c=Multi_Colours[Colour_Index],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                end
            end
        end
    
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])
    
        title!("Gene = Multiple Time = $k")
    end every 5
    tock()
    tick()
    animlong = @animate for k=1:2*Duration
        
        i=1
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
        if Only3Types
            Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
        end
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1,1],c=Multi_Colours[Colour_Index],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for q=1:Int(N_Perturbed/Partial)
            for p=1:2^Splits
                for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,q,t],PCA_Orbits_2Gen_Plot[1:k,2,p,q,t],PCA_Orbits_2Gen_Plot[1:k,3,p,q,t],c=Multi_Colours[Colour_Index],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                end
            end
        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])
    
        title!("Gene = Multiple Time = $k")
    end every 50
    tock()
    abra = gif(anim, Name[1:end-4]*Name[end-3:end], fps = 10)
    display(abra)
    
    kadabra = gif(animlong, Name[1:end-4]*Name[end-3:end], fps = 3)
    display(kadabra)
    
    
    
    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    PCAPlots_Final = scatter(PCA_Orbits_2Gen_Plot[end,1,1,1,:],PCA_Orbits_2Gen_Plot[end,2,1,1,:],PCA_Orbits_2Gen_Plot[end,3,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Final = scatter!((PCA_Orbits_2Gen_Plot[end,1,p,q,t],PCA_Orbits_2Gen_Plot[end,2,p,q,t],PCA_Orbits_2Gen_Plot[end,3,p,q,t]),c=Multi_Colours[Colour_Index],legend=false)
            end
        end
    end
    Total_Ratio = principalratio(Orbital_Model)
    Eigen_Values = eigvals(Orbital_Model)
    Eigen_Values = Eigen_Values/sum(Eigen_Values)
    title!("Static Multi Colour, Ratio: $Total_Ratio")
    display(PCAPlots_Final)
    savefig(Name[1:end-4] * "Final_Multi_Colour.png")



    
    PcaPlots_Time_Length = 500
    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    
    PCAPlots_Time1 = plot(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,1,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Time1 = plot!(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,1,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA 1")
            end
        end
    end
    display(PCAPlots_Time1)

    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    
    PCAPlots_Time2 = plot(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,2,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Time2 = plot!(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,2,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA 1")
            end
        end
    end
    display(PCAPlots_Time2)

    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    
    PCAPlots_Time3 = plot(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,3,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Time3 = plot!(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,3,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA 1")
            end
        end
    end
    display(PCAPlots_Time3)
    
end

function Evolution_NoEpi(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    
    
    
    #⊗_(Wi, bi) = Wi .* bi
       

    du[1,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,:].+u[2,:].+CArray[:])) .- u[1,:]
    
    #Epigenetic Evolution, no need to worry about boundaries here so simple loop      

    
end




##############################################################################################################################
##############################################################################################################################
##############################################################################################################################








#Load_Generations = [70]
#Load_Generations = vcat(1,Load_Generations) #These are all the generations that we will analyse
Copies = [1] #Can reduce this to just 1,2 or just 1 if we don't want as many copies



#Main variables to change:
Stochastic_Noise = true
N::Int32 = 8 #How many copies of same network we have
#K::Int32 = N  Depricated parameter, do not change, if ever K not defined replace it by N
P = 4 #How many different networks will we use in the evolution run
Generations = 6 #How many generations in the evolution run

#DO NOT CHANGE THESE
Splits::Int32 = 6 #DO NOT CHANGE
SplitTime::Float32 = 0.1 #Time before a splitting event happens, DOES NOT MATTER HERE
M::Int32 = 40 #This is the amount of genes we want in our network
Splitting_Strength = 0.0f0

#CAREFUL WITH M>127 DO NOT INCREASE WITHOUT REWRITING DIFFERENTIATION FUNCTION
#BIT OVERFLOW!!!!!


#Constant and plotting tools


InvSqr::Float32 = 1/sqrt(M)
PCA_Plotting = false 
Orbit_Plotting = true  #Currently not used yet
Epi_Plotting = false #Currently not used yet
Counter = 0 #Used to randomise the mutations, leave to keep same seed
Saved_Generations = [2,4] #These are the generations where we want to save some orbits for


#Diffusion or mean field parameters
Mean_Field = false
Diffusing_System = false
KArray = zeros(Float32,M,1) #Diffusion/Mean Field array is set to zero everywhere
#KArray[1:10] .= 0.1 #Here we can manually adjust which genes will have diffusion















Amount_Large_Perturbations = 1 #How many large perturbations we will have each time
Amount_Small_Perturbations = 1 #How many smaller perturbations we will have around our large perturbation
Large_Pert_Radius = [0,1,5,20]
Small_Pert_Radius = 0
#Perturbations = zeros(32)









Generation_String = string(Load_Generations[1])
Copy_String = string(Copies[1])
Load_Data_String_Load = Data_To_Analyse
#Load_Data_String = Data_To_Analyse
#Copies = [1]
#Load_Generations = [10]
Orbit_Array = 0
Time_Orbital_Collection2 = 0
Orbit_Collection = 0
EpiThroughTime = 0

#To have access after the loop
Interpolated_Orbits = 0
Final_Epigenetic_Factors = 0
Gene_Cells_Colour = 0

@showprogress 1 for zeta in Load_Generations
    GC.gc()
    for iota in Copies
        Generation_String = string(zeta)

        Copy_String = string(iota)
        Current_Network_String = "/Gen_"*Generation_String*"_"*Copy_String*".jld"
        Loaded_Data = load(Load_Data_String_Load*Current_Network_String)
        mkpath(Load_Data_String*"/Processed/Gen:"*Generation_String*"/PCA_Maps")
        mkpath(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Orbits")
        mkpath(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance")


        #Here we load and perturb the networks


        Duration = 1000 #For Animation should be 3000
        Length_Orbits = 3000 #For PCA should be 4000
        Opacity = 75
        Partial = 1

        Variance_Length = 2500
        Times = [i for i=1:Variance_Length+50]
        Times_Orbit = [i for i=1:Length_Orbits]


        Perturb_Seed = 12345
        rng=MersenneTwister(Perturb_Seed)
        #Amount_Large_Perturbations = 1 #How many large perturbations we will have each time
        #Amount_Small_Perturbations = 16 #How many smaller perturbations we will have around our large perturbation
        #Large_Pert_Radius = 10
        #Small_Pert_Radius = 3
        CArray_Collection = zeros(Float32,Amount_Large_Perturbations,M)
        PositionArray_Collection = zeros(Float32,Amount_Large_Perturbations,Amount_Small_Perturbations,2,1,M)
        ParameterMatrix_Collection = zeros(Int32,Amount_Large_Perturbations,M,M)
        Final_Epigenetic_Factors = zeros(Int8,Amount_Large_Perturbations,Amount_Small_Perturbations,2^(Splits),M)
        Epi_Factors = zeros(Float16,2^(Splits),M)
        for j=1:Amount_Large_Perturbations
            CArray_Collection[j,:] = Loaded_Data["CArray"]
            ParameterMatrix_Collection[j,:,:] = Loaded_Data["Network"]

            #Write large pert vector here

            Noise_Vector = randn(rng,M).-0.5
            Norm = sqrt(sum(Noise_Vector.^2))
            Noise_Vector = Large_Pert_Radius[j]*Noise_Vector/Norm
            for i=1:Amount_Small_Perturbations


                #Small pert vector here
                Noise_Vector2 = randn(rng,M).-0.5
                Norm = sqrt(sum(Noise_Vector2.^2))
                Noise_Vector2 = rand()^(1/M)*Small_Pert_Radius*Noise_Vector2/Norm

                PositionArray_Collection[j,i,1,1,:] = Loaded_Data["PositionArray"] + Noise_Vector' + Noise_Vector2'

            end
        end

        Evolutionary_Complexity = []


        Random_Noise = 574 #seed for splitting noise

        #Need to define outside for loop, otherwise it will be destroyed afterwards
        Genes = 0





        KLDistance_temp = 0
        BitDistance_temp =  0
        Evolutionary_Complexity = []
        Indices = 0
        Information = zeros(1)
        Ratio = zeros(1)
        Cell_Type_Tracker = []
        Cell_Collection = []
        Evolutionary_Complexity = zeros(N)
        Temporary = []
        Orbit_Array = Array{Matrix{Float16}}(undef,Amount_Large_Perturbations*Amount_Small_Perturbations)
        Interpolated_Orbits = Array{Matrix{Float16}}(undef,Amount_Large_Perturbations*Amount_Small_Perturbations)

        Time_Orbital_Collection = 0

        for yeta=1:Amount_Large_Perturbations
            for x=1:Amount_Small_Perturbations
                #CArray = CArray_Collection[x,:]
                #ParameterMatrix = ParameterMatrix_Collection[x,:,:]
                #PositionArray = PositionArray_Collection[x,:]
                Epi_Factors,Random_Noise,Temporary, EpiThroughTime = Running_Network(CArray_Collection[yeta,:],ParameterMatrix_Collection[yeta,:,:],PositionArray_Collection[yeta,x,:,:,:],Random_Noise)
                Final_Epigenetic_Factors[yeta,x,:,:] = signbit.(Epi_Factors)
                Orbit_Array[x+(yeta-1)*Amount_Small_Perturbations] = Temporary

            end

        end
        #Indices,AllIndices,Evolutionary_Complexity[:],Cell_Collection, Cell_Type_Tracker,KLDistance_temp,BitDistance_temp = Scoring_Indexes(N,Final_Epigenetic_Factors,Cell_Type_Tracker)


        #HERE WE DO THE VARIANCE DATA ANALYSIS

        Orbit_Collection = Orbit_Array

        #Collect all orbits in function of time,


        #Maybe apply PCA to these orbits or keep them like this
        Time_Orbital_Collection = zeros(Float16,length(Times),Amount_Small_Perturbations*2^Splits*Amount_Large_Perturbations,M)
        Time_Orbital_Collection2 = zeros(Float16,length(Times),Amount_Large_Perturbations,Amount_Small_Perturbations*2^Splits,M)



        Orbit_Counter=0
        Larger_Orbit_Counter = 1
        for Orbits in Orbit_Collection
            #Here we do it for the variance
            Interpolated_Vector = Interpolator(Times,Orbits) #We do linear interpolation to have all the same timesteps
            for i=1:2^Splits #We put the different splitted cells all together with the same networks.
                Time_Orbital_Collection[:,Orbit_Counter*2^Splits+i,:] = Interpolated_Vector[:,2+(i-1)*M:1+i*M]
            end
            for k=1:2^Splits
                Time_Orbital_Collection2[:,Larger_Orbit_Counter,mod(Orbit_Counter,Amount_Small_Perturbations)+k,:] = Interpolated_Vector[:,2+M*(k-1):1+M*k]
            end 
            Orbit_Counter += 1
            if mod(Orbit_Counter,Amount_Small_Perturbations) == 0
                Larger_Orbit_Counter += 1
            end


            #Here we do it for the orbits
            Interpolated_Vector2 = Interpolator(Times_Orbit,Orbits)
            Interpolated_Orbits[Orbit_Counter] = permutedims(Interpolated_Vector2,[2,1])

        end
        #Calculate variance in gene expression in function of time



        Variance = zeros(Variance_Length)
        for i=1:Variance_Length
            Variance[i] = sum((Time_Orbital_Collection[i,:,:] .- mean(Time_Orbital_Collection[i,:,:],dims=1)).^2)/(M-1)
        end



        Spatial_Var = Coarsed_Variance(Variance_Length,Time_Orbital_Collection,10,Amount_Large_Perturbations*Amount_Small_Perturbations*2^Splits)
        aba = plot(Variance,label="Temporal",xlabel="Time",ylabel="Variance",title="Spatial and Temporal Variance of all Orbits")
        aba = plot!(Spatial_Var,label="Spatial")
        aba =vline!(SplitTime*[i for i=1:Splits],colour=:"red",linealpha=:0.3,label=false)
        display(aba)
        
        savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance/Spatial_Temporal_"*Copy_String*".png")
        #savefig("Data_All/Data_Folder/TestFile/TestImage.png")



        aca = plot(Variance[1:250],label="Temporal",xlabel="Time",ylabel="Variance",title="Spatial and Temporal Variance of all Orbits")
        aca = plot!(Spatial_Var[1:250],label="Spatial")
        aca =vline!(SplitTime*[i for i=1:Splits],colour=:"red",linealpha=:0.3,label=false)
        ylims!(0,max(maximum(Variance[3:250]),maximum(Spatial_Var[3:250]))*1.3)

        display(aca)
        
        savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance/Spatial_Temporal_Zoomed_"*Copy_String*".png")


        if false

            Seperate_Variance = zeros(Amount_Large_Perturbations,Variance_Length)
            for j=1:Amount_Large_Perturbations
                for i=1:Variance_Length
                    Seperate_Variance[j,i] = sum((Time_Orbital_Collection2[i,j,:,:] .- mean(Time_Orbital_Collection2[i,j,:,:],dims=1)).^2)/(M-1)
                end

            end

            ada = plot(Seperate_Variance[1,1:250],label=false,xlabel="Time",ylabel="Variance",title="Temporal Variance of Specific I.C.")
            for j=2:Amount_Large_Perturbations
                ada = plot!(Seperate_Variance[j,1:250],label=false)
            end
            display(ada)

            savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance/Temporal_Zoomed_IC_"*Copy_String*".png")


            aea = plot(Seperate_Variance[1,1:2500],label=false,xlabel="Time",ylabel="Variance",title="Temporal Variance of Specific I.C.")
            for j=2:Amount_Large_Perturbations
                aea = plot!(Seperate_Variance[j,1:2500],label=false)
            end
            display(aea)

            savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance/Temporal_IC_"*Copy_String*".png")

            Spatial_Sep = Coarsed_Variance(Variance_Length,Time_Orbital_Collection[:,1:Amount_Small_Perturbations*2^Splits,:],25,Amount_Small_Perturbations*2^Splits)
            afa = plot(Spatial_Sep[1:250],label=false,xlabel="Time",ylabel="Variance",title="Spatial Variance of Specific I.C.")
            for j=1:Amount_Large_Perturbations-1
                Spatial_Sep = Coarsed_Variance(Variance_Length,Time_Orbital_Collection[:,1+j*Amount_Small_Perturbations*2^Splits:(j+1)*Amount_Small_Perturbations*2^Splits,:],25,Amount_Small_Perturbations*2^Splits)

                afa = plot!(Spatial_Sep[1:250],label=false)
            end

            display(afa)

            savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance/Spatial_Zoomed_IC_"*Copy_String*".png")
        end




        #Perturbations = [0 for i=1:32]
        for kappa=1:Amount_Large_Perturbations
            println(kappa)
            display(Final_Cell_PCAScatterPlot(Orbit_Array,kappa)) #Don't really need this one
        end
        
        display(Final_Cell_PCAScatterPlot(Orbit_Array,false));
        
        Gene_Cells_Colour = [1,2,3,4]
        print("Before the Multi Big One")
        GC.gc()
        display(Multi_One_Generational_Plotter4(true,Duration,Length_Orbits,Opacity,Load_Data_String*"/Processed/Gen:"*Generation_String*"/Orbits/Genes_"*Copy_String*".gif",Amount_Large_Perturbations,Partial,Amount_Small_Perturbations,Interpolated_Orbits,Final_Epigenetic_Factors,Gene_Cells_Colour));
    






        Copy_String = string(1)
        Current_Network_String = "/Gen_"*Generation_String*"_"*Copy_String*".jld"
        Loaded_Data = load(Load_Data_String_Load*Current_Network_String)
        
        CArray = Loaded_Data["CArray"]
        ParameterMatrix = Loaded_Data["Network"]
        

        
        TimesWeCheck = [1,5,10,25,40,60,80,100,150,200,300,500,1000,4000,20000]
        TimeChecks = length(TimesWeCheck)
        Cell_Gatherings = zeros(TimeChecks,2,2^Splits,M)
        CycleAmount = zeros(TimeChecks)
        UniqueCycleAmount = zeros(TimeChecks)
        FixPointAmount = zeros(TimeChecks)
        
        Interpolated_Cells = Interpolator(TimesWeCheck,Orbit_Array[1])
        Interpolated_EPIs = Interpolator(TimesWeCheck,EpiThroughTime)
        for i=1:2^Splits
            Cell_Gatherings[:,1,i,:] = Interpolated_Cells[:,2+(i-1)*M:1+(i)*M]
            Cell_Gatherings[:,2,i,:] = Interpolated_EPIs[:,2+(i-1)*M:1+(i)*M]
        end
        
        
        Copy_String = string(1)
        Current_Network_String = "/Gen_"*Generation_String*"_"*Copy_String*".jld"
        Loaded_Data = load(Load_Data_String_Load*Current_Network_String)
        
        CArray = Loaded_Data["CArray"]
        ParameterMatrix = Loaded_Data["Network"]
        FixPoint_Positions = Array{Float64}(undef,0)
        Counter_Cycle = 0
        XPositionTol = 0
        
        
        
        
        for t = 1:TimeChecks
            Counter_Cycle = 0
            FixPoint_Positions = Array{Float64}(undef,0)
            VectorTest = []
            for k = 1:2^Splits
                PositionArray = Cell_Gatherings[t,:,k,:]
                
                Parameters = (ParameterMatrix, CArray) #We have to update the splits every rerun, rest stays same
                tspan = (Float32(0.0),100) #How long ODE will run
                
                prob = ODEProblem(Evolution_NoEpi,PositionArray,tspan,Parameters) #ODEdefining
                
                
                
                
                sol = solve(prob,BS3())
                aba = plot(sol.t,sol[1,1,:])
                for j=2:40
                    aba = plot!(sol.t,sol[1,j,:],legend=false,title=k)
                end
                #if t> 10
                #    display(aba)
                #end
                
                
                
                
                
                
                
                XVector = sol[1,:,end]
                EpiVector = sol[2,:,end]
                SumOfDerivates = sum(abs.(tanh.(40*(InvSqr*ParameterMatrix*XVector.+EpiVector.+CArray[:])) .- XVector))
                DerivativeTol = 0.1
                XPositionTol = 0.1
                
                #if t> 10   
                    #print(SumOfDerivates)
                #end
                
                if SumOfDerivates < DerivativeTol
                    if FixPoint_Positions == [] #Still empty
                        FixPoint_Positions = vcat(FixPoint_Positions,XVector)
                    else #Already has a point in it
                        NewFixPoint = true
                        #print("The distance to the fixpoints are:")
                        for i=1:length(FixPoint_Positions[1,:]) #Here we check over all the existing ones to see if it is not close to one already existing
                            #println(sqrt(sum((FixPoint_Positions[:,i] .- XVector).^2)))
                            if sqrt(sum((FixPoint_Positions[:,i] .- XVector).^2)) < XPositionTol
                                NewFixPoint = false
                            end    
                        end
                        if NewFixPoint #If it is a new fixpoint we add it in now
                            FixPoint_Positions = hcat(FixPoint_Positions,XVector)
                        end
                    end
                else
                    #Exception, we have a cycle? try to plot it
                    
                    CycleToleranceActive = 0.01
                    Active_Gene = zeros(M)
                    
                    prob2 = ODEProblem(Evolution_NoEpi,sol[:,:,end],(Float32(0.0),20),Parameters) #ODEdefining
                    sol2 = solve(prob2,BS3())
                    
                    for j=1:M
                        if abs(maximum(sol2[1,j,:].-mean(sol2[1,j,:]))) > CycleToleranceActive
                            Active_Gene[j] = 1
                        end
                    end
                    if VectorTest == []
                        VectorTest = vcat(VectorTest,Active_Gene)
                    else
                        New_Cycle = true
                        for tau in length(VectorTest[1,:])
                            if Active_Gene == VectorTest[:,tau]
                                New_Cycle = false
                            end
                        end
                        if New_Cycle
                            VectorTest = hcat(VectorTest,Active_Gene)
                        end
                    end
        
        
        
                    
                    Counter_Cycle += 1
                end
            end
            try FixPointAmount[t] = length(FixPoint_Positions[1,:])
        
            catch 
                FixPointAmount[t] = 0
            else
                FixPointAmount[t] = length(FixPoint_Positions[1,:])
            end
            CycleAmount[t] = Counter_Cycle
            try UniqueCycleAmount[t] = length(VectorTest[1,:])
        
            catch
                UniqueCycleAmount[t] = 0
            else
                UniqueCycleAmount[t] = length(VectorTest[1,:])
            end
        end
        
        aba = plot(TimesWeCheck,FixPointAmount,label="Fix Points")
        aba = plot!(TimesWeCheck,CycleAmount,label="Cycles (DoubleCounted)",xaxis=:log,title="PositionalTolerance="*string(XPositionTol))
        plot!(TimesWeCheck,UniqueCycleAmount,label="Cycles (Unique)")
        display(aba)

    end
end
    





function Multi_One_Generational_Plotter4(Cut_Off,Duration,Length_Orbits,Opacity,Name,Amount_Perturbations,Partial,N_Perturbed,Orbit_Array,Final_Epigenetic_Factors,Gene_Colour_Collection)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1:N_Perturbed*Amount_Perturbations

            if Shortest>size(Orbit_Array[i])[2]
                Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
            end
            Total_Length += size(Orbit_Array[i])[2]
        
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed*Amount_Perturbations)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1:N_Perturbed*Amount_Perturbations
        
        if Cut_Off

            for p=0:2^Splits-1

                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
            end
        else
            Length_Orbits =  size(Orbit_Array[i,j])[2]
            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
            end
        end

        Collection_Index += Length_Orbits*2^Splits

    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,Amount_Perturbations,N_Perturbed,2^Splits,3,Shortest)
    PCA_Orbits_Partial = zeros(Float32,Amount_Perturbations,Int(N_Perturbed/Partial),2^Splits,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:N_Perturbed
        for j=0:Amount_Perturbations-1
        for tau=1:2^Splits
            tester = Orbit_Array[i+N_Perturbed*j][2+(tau-1)*M:1+tau*M,1:Shortest]
            Transformed = transform(Orbital_Model,tester)
            PCA_Orbits_2Gen[j+1,i,tau,:,:] = Transformed
            if mod(i-1,Partial) == 0
                PCA_Orbits_Partial[j+1,(Int(i/Partial)),tau,:,:] = Transformed
            end
        end
        end

    end
    Colours = [:red,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
     
    #PCA_Orbits_Partial = reshape(PCA_Orbits_Partial,(Int(2^Splits*N_Perturbed*Amount_Perturbations/Partial),3,Shortest))
    PCA_Orbits_2Gen_Plot = permutedims(PCA_Orbits_Partial,(5,4,3,2,1))
    Orbit_Range = Int(N_Perturbed/Partial)*2^Splits
    
    print(size(PCA_Orbits_2Gen_Plot))
    if false
        for Gene_Colour in Gene_Colour_Collection
            tick()
            
            anim = @animate for k=1:Int(Duration/5)
                i=1
                PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1,1],c=Colours[Final_Epigenetic_Factors[1,1,1,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                for q=1:Int(N_Perturbed/Partial)
                    for p=1:2^Splits
                        for t =1:Amount_Perturbations
                        PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,q,t],PCA_Orbits_2Gen_Plot[1:k,2,p,q,t],PCA_Orbits_2Gen_Plot[1:k,3,p,q,t],c=Colours[Final_Epigenetic_Factors[t,q,p,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                        end
                    end
                end
        
                #for i=1:Int(N_Perturbed/5)
                #    for j=1:Amount_Perturbations
                #        for tau=1:2^Splits
                #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                #        end
                #    end
                #end
                
                #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])
        
                title!("Gene = $Gene_Colour Time = $k")
            end every 5
            tock()
            tick()
            animlong = @animate for k=1:Duration
                
                i=1
                PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1,1],c=Colours[Final_Epigenetic_Factors[1,1,1,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                for q=1:Int(N_Perturbed/Partial)
                    for p=1:2^Splits
                        for t =1:Amount_Perturbations
                        PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,q,t],PCA_Orbits_2Gen_Plot[1:k,2,p,q,t],PCA_Orbits_2Gen_Plot[1:k,3,p,q,t],c=Colours[Final_Epigenetic_Factors[t,q,p,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                        end
                    end
                end
                #for i=1:Int(N_Perturbed/5)
                #    for j=1:Amount_Perturbations
                #        for tau=1:2^Splits
                #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                #        end
                #    end
                #end
                
                #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])
        
                title!("Gene = $Gene_Colour Time = $k")
            end every 50
            tock()
    
            abra = gif(anim, Name[1:end-4]*"_Gene:"*string(Gene_Colour)*Name[end-3:end], fps = 10)
            display(abra)
        
            kadabra = gif(animlong, Name[1:end-4] * "_Long"*"_Gene:"*string(Gene_Colour) *Name[end-3:end], fps = 3)
            display(kadabra)
        
            
            Total_Ratio = principalratio(Orbital_Model)
            Eigen_Values = eigvals(Orbital_Model)
            Eigen_Values = Eigen_Values/sum(Eigen_Values)
            Eigen1 = Eigen_Values[1]
            Eigen2 = Eigen_Values[2]
            Eigen3 = Eigen_Values[3]
            PCAPlots_Final = scatter(PCA_Orbits_2Gen_Plot[end,1,1,1,:],PCA_Orbits_2Gen_Plot[end,2,1,1,:],PCA_Orbits_2Gen_Plot[end,3,1,1,:],c=Colours[Final_Epigenetic_Factors[1,1,1,Gene_Colour]+1],legend=true,label="
            Principal 1: $Eigen1
            Principal 2: $Eigen2]
            Principal 3: $Eigen3")
        
            for q=1:Int(N_Perturbed/Partial)
                for p=1:2^Splits
                    for t =1:Amount_Perturbations
                        PCAPlots_Final = scatter!((PCA_Orbits_2Gen_Plot[end,1,p,q,t],PCA_Orbits_2Gen_Plot[end,2,p,q,t],PCA_Orbits_2Gen_Plot[end,3,p,q,t]),c=Colours[Final_Epigenetic_Factors[t,q,p,Gene_Colour]+1],legend=true,label="")
                    end
                end
            end
            
            
            title!("Gene = $Gene_Colour , Ratio: $Total_Ratio")
            display(PCAPlots_Final)
            savefig(Name[1:end-4] * "_Final_Static_$Gene_Colour.png")
            
        end
    end
    Multi_Colours = [:cyan,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:darkmagenta,:lightcyan4,:olive,:gold4,:lightsalmon,:navajowhite,:pink]
    Only3Types = false


    
    
    
    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    PCAPlots_Final = scatter(PCA_Orbits_2Gen_Plot[end,1,1,1,:],PCA_Orbits_2Gen_Plot[end,2,1,1,:],PCA_Orbits_2Gen_Plot[end,3,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Final = scatter!((PCA_Orbits_2Gen_Plot[end,1,p,q,t],PCA_Orbits_2Gen_Plot[end,2,p,q,t],PCA_Orbits_2Gen_Plot[end,3,p,q,t]),c=Multi_Colours[Colour_Index],legend=false)
            end
        end
    end
    Total_Ratio = principalratio(Orbital_Model)
    Eigen_Values = eigvals(Orbital_Model)
    Eigen_Values = Eigen_Values/sum(Eigen_Values)
    title!("Static Multi Colour, Ratio: $Total_Ratio")
    display(PCAPlots_Final)
    savefig(Name[1:end-4] * "Final_Multi_Colour.png")



    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    
    PCAPlots_Time1 = plot([PcaPlots_Time_Start:PcaPlots_Time_Length],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,1,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Time1 = plot!([PcaPlots_Time_Start:PcaPlots_Time_Length],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,1,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA 1")
            end
        end
    end
    display(PCAPlots_Time1)

    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    
    PCAPlots_Time2 = plot([PcaPlots_Time_Start:PcaPlots_Time_Length],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,2,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Time2 = plot!([PcaPlots_Time_Start:PcaPlots_Time_Length],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,2,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA 2")
            end
        end
    end
    display(PCAPlots_Time2)

    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    
    PCAPlots_Time3 = plot([PcaPlots_Time_Start:PcaPlots_Time_Length],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,3,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Time3 = plot!([PcaPlots_Time_Start:PcaPlots_Time_Length],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,3,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA 3")
            end
        end
    end
    display(PCAPlots_Time3)



    for Colouring =1:16
        Plotting_Something = false
        PCAPlots_TimeFilter = plot(zeros(PcaPlots_Time_Start:PcaPlots_Time_Length))
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
        if Only3Types
            Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
        end

        if Colour_Index == Colouring
            Plotting_Something = true
            PCAPlots_TimeFilter = plot!([PcaPlots_Time_Start:PcaPlots_Time_Length],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,SeperatePCAIndex,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
        end
        for q=1:Int(N_Perturbed/Partial)
            for p=1:2^Splits
                for t =1:Amount_Perturbations
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                    if Only3Types
                        Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                    end
                    if Colour_Index == Colouring
                        Plotting_Something = true
                        PCAPlots_TimeFilter = plot!([PcaPlots_Time_Start:PcaPlots_Time_Length],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,SeperatePCAIndex,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA Colour Index = $(Colouring)")
                    end
                end
            end
        end
        if Plotting_Something
            display(PCAPlots_TimeFilter)
        end
        
    end

    PcaDim1 = 1
    PcaDim2 = 2
    PCAPlots_TimeFilter = plot([0])
    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end

    
    PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,1,1,:],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,1,1,:],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end

                PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,p,q,t],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,p,q,t],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
                
            end
        end
    end
    display(PCAPlots_TimeFilter)

    PcaDim1 = 2
    PcaDim2 = 3
    PCAPlots_TimeFilter = plot([0])
    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end

    
    PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,1,1,:],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,1,1,:],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end

                PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,p,q,t],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,p,q,t],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
                
            end
        end
    end
    display(PCAPlots_TimeFilter)

    PcaDim1 = 1
    PcaDim2 = 3
    PCAPlots_TimeFilter = plot([0])
    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end

    
    PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,1,1,:],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,1,1,:],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end

                PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,p,q,t],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,p,q,t],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
                
            end
        end
    end
    display(PCAPlots_TimeFilter)

    
    PcaDim1 = 1
    PcaDim2 = 2
    
    for Colouring =1:16
        Plotting_Something = false
        PCAPlots_TimeFilter = plot([0])
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
        if Only3Types
            Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
        end

        if Colour_Index == Colouring
            Plotting_Something = true
            PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,1,1,:],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,1,1,:],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
        end
        for q=1:Int(N_Perturbed/Partial)
            for p=1:2^Splits
                for t =1:Amount_Perturbations
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                    if Only3Types
                        Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                    end
                    if Colour_Index == Colouring
                        Plotting_Something = true
                        PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,p,q,t],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA Colour Index = $(Colouring)",opacity=0.1)
                    end
                end
            end
        end
        if Plotting_Something
            display(PCAPlots_TimeFilter)
        end
        
    end

    PcaDim1 = 2
    PcaDim2 = 3
    
    for Colouring =1:16
        Plotting_Something = false
        PCAPlots_TimeFilter = plot([0])
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
        if Only3Types
            Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
        end

        if Colour_Index == Colouring
            Plotting_Something = true
            PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,1,1,:],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,1,1,:],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
        end
        for q=1:Int(N_Perturbed/Partial)
            for p=1:2^Splits
                for t =1:Amount_Perturbations
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                    if Only3Types
                        Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                    end
                    if Colour_Index == Colouring
                        Plotting_Something = true
                        PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,p,q,t],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA Colour Index = $(Colouring)",opacity=0.1)
                    end
                end
            end
        end
        if Plotting_Something
            display(PCAPlots_TimeFilter)
        end
        
    end

    PcaDim1 = 1
    PcaDim2 = 3
    
    for Colouring =1:16
        Plotting_Something = false
        PCAPlots_TimeFilter = plot([0])
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
        if Only3Types
            Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
        end

        if Colour_Index == Colouring
            Plotting_Something = true
            PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,1,1,:],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,1,1,:],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
        end
        for q=1:Int(N_Perturbed/Partial)
            for p=1:2^Splits
                for t =1:Amount_Perturbations
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                    if Only3Types
                        Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                    end
                    if Colour_Index == Colouring
                        Plotting_Something = true
                        PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,p,q,t],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA Colour Index = $(Colouring)",opacity=0.1)
                    end
                end
            end
        end
        if Plotting_Something
            display(PCAPlots_TimeFilter)
        end
        
    end

    return Orbital_Model
    
end


Duration = 1000 #For Animation should be 3000
Length_Orbits = 3000 #For PCA should be 4000
Opacity = 75
Partial = 1

Variance_Length = 2500
Times = [i for i=1:Variance_Length+50]
Times_Orbit = [i for i=1:Length_Orbits]


PcaPlots_Time_Start = 100
PcaPlots_Time_Length = 600
SeperatePCAIndex = 3

Orbital_Model = Multi_One_Generational_Plotter4(true,Duration,Length_Orbits,Opacity,Load_Data_String*"/Processed/Gen:"*Generation_String*"/Orbits/Genes_"*Copy_String*".gif",Amount_Large_Perturbations,Partial,Amount_Small_Perturbations,Interpolated_Orbits,Final_Epigenetic_Factors,Gene_Cells_Colour);


#TimesWeCheck = [1,5,10,25,40,60,80,100,150,200,300,500,1000,4000,20000]
TimesWeCheck = [50,100,150,200,300,400,500]
TimeChecks = length(TimesWeCheck)


Cell_Gatherings = zeros(TimeChecks,2,2^Splits,M)
CycleAmount = zeros(TimeChecks)
UniqueCycleAmount = zeros(TimeChecks)
FixPointAmount = zeros(TimeChecks)

Interpolated_Cells = Interpolator(TimesWeCheck,Orbit_Array[1])
Interpolated_EPIs = Interpolator(TimesWeCheck,EpiThroughTime)
for i=1:2^Splits
    Cell_Gatherings[:,1,i,:] = Interpolated_Cells[:,2+(i-1)*M:1+(i)*M]
    Cell_Gatherings[:,2,i,:] = Interpolated_EPIs[:,2+(i-1)*M:1+(i)*M]
end


Copy_String = string(1)
Current_Network_String = "/Gen_"*Generation_String*"_"*Copy_String*".jld"
Loaded_Data = load(Load_Data_String_Load*Current_Network_String)

CArray = Loaded_Data["CArray"]
ParameterMatrix = Loaded_Data["Network"]
FixPoint_Positions = Array{Float64}(undef,0)
Counter_Cycle = 0
XPositionTol = 0
Full_TimeSpan = 200
sol = 0
Sampling_Amounts = 4
Transformed_Data =  zeros(Float32,TimeChecks,2^Splits,3,Full_TimeSpan,Sampling_Amounts)

for q=1:Sampling_Amounts
    Saved_Orbits_All = zeros(Float16,TimeChecks,2^Splits,Full_TimeSpan,M)
    for t = 1:TimeChecks
        Counter_Cycle = 0
        FixPoint_Positions = Array{Float64}(undef,0)
        VectorTest = []
        for k = 1:2^Splits
            PositionArray = Cell_Gatherings[t,:,k,:]
            
            Parameters = (ParameterMatrix, CArray) #We have to update the splits every rerun, rest stays same
            tspan = (Float32(0.0),Full_TimeSpan) #How long ODE will run
            
            prob = SDEProblem(Evolution_NoEpi,σ_Genes,PositionArray,tspan,Parameters) #ODEdefining
            
            
            
            
            sol = solve(prob,SRA3(),dtmin=0.1,force_dtmin=true)
            if false 
                aba = plot(sol.t,sol[1,1,:])
                for j=2:40
                    aba = plot!(sol.t,sol[1,j,:],legend=false,title=TimesWeCheck[t])
                end
                if t> 1 && (k==1) 
                    
                    display(aba)
                end
            end
            for time=1:Full_TimeSpan
                Saved_Orbits_All[t,k,time,:] = sol(time)[1,:]
            end
            
            
            
            
            XVector = sol[1,:,end]
            EpiVector = sol[2,:,end]
            SumOfDerivates = sum(abs.(tanh.(40*(InvSqr*ParameterMatrix*XVector.+EpiVector.+CArray[:])) .- XVector))
            DerivativeTol = 0.1
            XPositionTol = 0.1
            
        end
            
    
    
    end

    for i=1:TimeChecks  
        for tau=1:2^Splits
            Tested_Transform = transform(Orbital_Model,Saved_Orbits_All[i,tau,:,:]')        
            Transformed_Data[i,tau,:,:,q] = Tested_Transform
        end
    end
    
end
## Save somehow the data/orbits of these frozen versions somewhere


# Plot this for the PCA as their axis


Multi_Colours = [:cyan,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:darkmagenta,:lightcyan4,:olive,:gold4,:lightsalmon,:navajowhite,:pink]


function PCA_Individual_Colour_Plotting(PcaDim1,PcaDim2,Time_Index_To_Plot)
    for Colouring =1:16
        Plotting_Something = false
        PCAPlots_TimeFilter = plot([0])
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    
    
        if Colour_Index == Colouring
            Plotting_Something = true
            PCAPlots_TimeFilter = plot!(Transformed_Data[Time_Index_To_Plot,1,PcaDim1,:,1],Transformed_Data[Time_Index_To_Plot,1,PcaDim2,:,1],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
        end
        for q=1:Sampling_Amounts
            for p=1:2^Splits
                for t =1:1
                    Colour_Index = Final_Epigenetic_Factors[t,1,p,1]+Final_Epigenetic_Factors[t,1,p,2]*2+Final_Epigenetic_Factors[t,1,p,3]*4+Final_Epigenetic_Factors[t,1,p,4]*8+1
    
                    if Colour_Index == Colouring
                        Plotting_Something = true
                        PCAPlots_TimeFilter = plot!(Transformed_Data[Time_Index_To_Plot,p,PcaDim1,:,q],Transformed_Data[Time_Index_To_Plot,p,PcaDim2,:,q],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1,title="Final Type Index $(Colouring) Time = $(TimesWeCheck[Time_Index_To_Plot])")
                    end
                end
            end
        end
        if Plotting_Something
            display(PCAPlots_TimeFilter)
        end
        
    end
end
PCADim1 = 1
PCADim2 = 2
for t in [1,4,5,6,7]
    PCA_Individual_Colour_Plotting(PCADim1,PCADim2,t)
end

#TimesWeCheck = [1,5,10,25,40,60,80,100,150,200,300,500,1000,4000,20000]
TimesWeCheck = [50,100,150,200,300,320,340,360,380,400,500,600,700,800,1000,1500,2000]
TimeChecks = length(TimesWeCheck)


Cell_Gatherings = zeros(TimeChecks,2,2^Splits,M)
CycleAmount = zeros(TimeChecks)
UniqueCycleAmount = zeros(TimeChecks)
FixPointAmount = zeros(TimeChecks)

Interpolated_Cells = Interpolator(TimesWeCheck,Orbit_Array[1])
Interpolated_EPIs = Interpolator(TimesWeCheck,EpiThroughTime)
for i=1:2^Splits
    Cell_Gatherings[:,1,i,:] = Interpolated_Cells[:,2+(i-1)*M:1+(i)*M]
    Cell_Gatherings[:,2,i,:] = Interpolated_EPIs[:,2+(i-1)*M:1+(i)*M]
end


Copy_String = string(1)
Current_Network_String = "/Gen_"*Generation_String*"_"*Copy_String*".jld"
Loaded_Data = load(Load_Data_String_Load*Current_Network_String)

CArray = Loaded_Data["CArray"]
ParameterMatrix = Loaded_Data["Network"]
FixPoint_Positions = Array{Float64}(undef,0)
Counter_Cycle = 0
XPositionTol = 0
Full_TimeSpan = 200
sol = 0
Sampling_Amounts = 8
Transformed_Data =  zeros(Float32,TimeChecks,2^Splits,3,Full_TimeSpan,Sampling_Amounts)

for q=1:Sampling_Amounts
    Saved_Orbits_All = zeros(Float16,TimeChecks,2^Splits,Full_TimeSpan,M)
    for t = 1:TimeChecks
        Counter_Cycle = 0
        FixPoint_Positions = Array{Float64}(undef,0)
        VectorTest = []
        for k = 1:2^Splits
            PositionArray = Cell_Gatherings[t,:,k,:]
            
            Parameters = (ParameterMatrix, CArray) #We have to update the splits every rerun, rest stays same
            tspan = (Float32(0.0),Full_TimeSpan) #How long ODE will run
            
            prob = SDEProblem(Evolution_NoEpi,σ_Genes,PositionArray,tspan,Parameters) #ODEdefining
            
            
            
            
            sol = solve(prob,SRA3(),dtmin=0.1,force_dtmin=true)
            if false 
                aba = plot(sol.t,sol[1,1,:])
                for j=2:40
                    aba = plot!(sol.t,sol[1,j,:],legend=false,title=TimesWeCheck[t])
                end
                if t> 1 && (k==1) 
                    
                    display(aba)
                end
            end
            for time=1:Full_TimeSpan
                Saved_Orbits_All[t,k,time,:] = sol(time)[1,:]
            end
            
            
            
            
            XVector = sol[1,:,end]
            EpiVector = sol[2,:,end]
            SumOfDerivates = sum(abs.(tanh.(40*(InvSqr*ParameterMatrix*XVector.+EpiVector.+CArray[:])) .- XVector))
            DerivativeTol = 0.1
            XPositionTol = 0.1
            
        end
            
    
    
    end

    for i=1:TimeChecks  
        for tau=1:2^Splits
            Tested_Transform = transform(Orbital_Model,Saved_Orbits_All[i,tau,:,:]')        
            Transformed_Data[i,tau,:,:,q] = Tested_Transform
        end
    end
    
end
## Save somehow the data/orbits of these frozen versions somewhere


# Plot this for the PCA as their axis

Multi_Colours = [:cyan,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:darkmagenta,:lightcyan4,:olive,:gold4,:lightsalmon,:navajowhite,:pink]


function PCA_Individual_Colour_Plotting(PcaDim1,PcaDim2,Time_Index_To_Plot)
    for Colouring =1:16
        Plotting_Something = false
        PCAPlots_TimeFilter = plot([0])
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    
    
        if Colour_Index == Colouring
            Plotting_Something = true
            PCAPlots_TimeFilter = plot!(Transformed_Data[Time_Index_To_Plot,1,PcaDim1,:,1],Transformed_Data[Time_Index_To_Plot,1,PcaDim2,:,1],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
            PCAPlots_TimeFilter = scatter!((Transformed_Data[Time_Index_To_Plot,1,PcaDim1,end,1],Transformed_Data[Time_Index_To_Plot,1,PcaDim2,end,1]),c=Multi_Colours[Colour_Index])

        end
        for q=1:Sampling_Amounts
            for p=1:2^Splits
                for t =1:1
                    Colour_Index = Final_Epigenetic_Factors[t,1,p,1]+Final_Epigenetic_Factors[t,1,p,2]*2+Final_Epigenetic_Factors[t,1,p,3]*4+Final_Epigenetic_Factors[t,1,p,4]*8+1
    
                    if Colour_Index == Colouring
                        Plotting_Something = true
                        PCAPlots_TimeFilter = plot!(Transformed_Data[Time_Index_To_Plot,p,PcaDim1,:,q],Transformed_Data[Time_Index_To_Plot,p,PcaDim2,:,q],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1,title="Final Type Index $(Colouring) Time = $(TimesWeCheck[Time_Index_To_Plot])")
                        PCAPlots_TimeFilter = scatter!((Transformed_Data[Time_Index_To_Plot,p,PcaDim1,end,q],Transformed_Data[Time_Index_To_Plot,p,PcaDim2,end,q]),c=Multi_Colours[Colour_Index])

                    end
                end
            end
        end
        if Plotting_Something
            display(PCAPlots_TimeFilter)
        end
        
    end
end
PCADim1 = 1
PCADim2 = 2
for t in [1,4,5,6,7]
    PCA_Individual_Colour_Plotting(PCADim1,PCADim2,t)
end



#TimesWeCheck = [1,5,10,25,40,60,80,100,150,200,300,500,1000,4000,20000]
TimesWeCheck = [50,100,150,200,300,320,340,360,380,400,450,500,550,600,650,700,750,1000,1500,2000]
TimeChecks = length(TimesWeCheck)


Cell_Gatherings = zeros(TimeChecks,2,2^Splits,M)
CycleAmount = zeros(TimeChecks)
UniqueCycleAmount = zeros(TimeChecks)
FixPointAmount = zeros(TimeChecks)

Interpolated_Cells = Interpolator(TimesWeCheck,Orbit_Array[1])
Interpolated_EPIs = Interpolator(TimesWeCheck,EpiThroughTime)
for i=1:2^Splits
    Cell_Gatherings[:,1,i,:] = Interpolated_Cells[:,2+(i-1)*M:1+(i)*M]
    Cell_Gatherings[:,2,i,:] = Interpolated_EPIs[:,2+(i-1)*M:1+(i)*M]
end


Copy_String = string(1)
Current_Network_String = "/Gen_"*Generation_String*"_"*Copy_String*".jld"
Loaded_Data = load(Load_Data_String_Load*Current_Network_String)

CArray = Loaded_Data["CArray"]
ParameterMatrix = Loaded_Data["Network"]
FixPoint_Positions = Array{Float64}(undef,0)
Counter_Cycle = 0
XPositionTol = 0
Full_TimeSpan = 200
sol = 0
Sampling_Amounts = 8
Transformed_Data =  zeros(Float32,TimeChecks,2^Splits,3,Full_TimeSpan,Sampling_Amounts)
Saved_Orbits_All = 0
for q=1:Sampling_Amounts
    Saved_Orbits_All = zeros(Float16,TimeChecks,2^Splits,Full_TimeSpan,M)
    for t = 1:TimeChecks
        Counter_Cycle = 0
        FixPoint_Positions = Array{Float64}(undef,0)
        VectorTest = []
        for k = 1:2^Splits
            PositionArray = Cell_Gatherings[t,:,k,:]
            
            Parameters = (ParameterMatrix, CArray) #We have to update the splits every rerun, rest stays same
            tspan = (Float32(0.0),Full_TimeSpan) #How long ODE will run
            
            prob = SDEProblem(Evolution_NoEpi,σ_Genes,PositionArray,tspan,Parameters) #ODEdefining
            
            
            
            
            sol = solve(prob,SRA3(),dtmin=0.1,force_dtmin=true)
            if false 
                aba = plot(sol.t,sol[1,1,:])
                for j=2:40
                    aba = plot!(sol.t,sol[1,j,:],legend=false,title=TimesWeCheck[t])
                end
                if t> 1 && (k==1) 
                    
                    display(aba)
                end
            end
            for time=1:Full_TimeSpan
                Saved_Orbits_All[t,k,time,:] = sol(time)[1,:]
            end
            
            
            
            
            XVector = sol[1,:,end]
            EpiVector = sol[2,:,end]
            SumOfDerivates = sum(abs.(tanh.(40*(InvSqr*ParameterMatrix*XVector.+EpiVector.+CArray[:])) .- XVector))
            DerivativeTol = 0.1
            XPositionTol = 0.1
            
        end
            
    
    
    end

    for i=1:TimeChecks  
        for tau=1:2^Splits
            Tested_Transform = transform(Orbital_Model,Saved_Orbits_All[i,tau,:,:]')        
            Transformed_Data[i,tau,:,:,q] = Tested_Transform
        end
    end
    
end
## Save somehow the data/orbits of these frozen versions somewhere


# Plot this for the PCA as their axis

Multi_Colours = [:cyan,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:darkmagenta,:lightcyan4,:olive,:gold4,:lightsalmon,:navajowhite,:pink]




PCADim1 = 1
PCADim2 = 2





function PCA_Individual_Colour_Plotting(PcaDim1,PcaDim2,Time_Index_To_Plot,Colour_Index)
    Colour_Counter = 0
    for Colouring =Colour_Index:Colour_Index
        Plotting_Something = false
        plot([0])
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    
    
        if Colour_Index == Colouring
            Colour_Counter = mod(Colour_Counter,16) + 1
            Plotting_Something = true
            PCAPlots_TimeFilter = plot!(Transformed_Data[Time_Index_To_Plot,1,PcaDim1,:,1],Transformed_Data[Time_Index_To_Plot,1,PcaDim2,:,1],c=Multi_Colours[Colour_Counter],legend=false,opacity=0.2)
            PCAPlots_TimeFilter = scatter!((Transformed_Data[Time_Index_To_Plot,1,PcaDim1,end,1],Transformed_Data[Time_Index_To_Plot,1,PcaDim2,end,1]),c=Multi_Colours[Colour_Counter])

        end
        for q=1:Sampling_Amounts
            for p=1:2^Splits
                for t =1:1
                    Colour_Index = Final_Epigenetic_Factors[t,1,p,1]+Final_Epigenetic_Factors[t,1,p,2]*2+Final_Epigenetic_Factors[t,1,p,3]*4+Final_Epigenetic_Factors[t,1,p,4]*8+1
    
                    if Colour_Index == Colouring
                        Colour_Counter = mod(Colour_Counter,16) + 1
                        Plotting_Something = true
                        PCAPlots_TimeFilter = plot!(Transformed_Data[Time_Index_To_Plot,p,PcaDim1,:,q],Transformed_Data[Time_Index_To_Plot,p,PcaDim2,:,q],c=Multi_Colours[Colour_Counter],legend=false,opacity=0.2,title="Final Type Index $(Colouring) Time = $(TimesWeCheck[Time_Index_To_Plot])")
                        #PCAPlots_TimeFilter = scatter!((Transformed_Data[Time_Index_To_Plot,p,PcaDim1,end,q],Transformed_Data[Time_Index_To_Plot,p,PcaDim2,end,q]),c=Multi_Colours[Colour_Index])

                    end
                end
            end
        end
        if Plotting_Something
            display(PCAPlots_TimeFilter)
        end
        
        xlims!((-3,2))
        ylims!((-4,1))

    end
end






for t in [1,4,5,6,7]
    PCA_Individual_Colour_Plotting(PCADim1,PCADim2,t,3)
    PCA_Individual_Colour_Plotting(PCADim1,PCADim2,t,4)

end



In [ ]:
Gene_To_Plot = [1,2,3,4,6,30,32]
Gene_To_Plot = [i for i=1:40]
Cells = [i for i=1:8]

Orbit_Array = Interpolated_Orbits
#Multi_One_Generational_Plotter4(true,Duration,Length_Orbits,Opacity,Load_Data_String*"/Processed/Gen:"*Generation_String*"/Orbits/Genes_"*Copy_String*".gif",Amount_Large_Perturbations,Partial,Amount_Small_Perturbations,Interpolated_Orbits,Final_Epigenetic_Factors,Gene_Cells_Colour));
#Multi_One_Generational_Plotter4(Cut_Off,Duration,Length_Orbits,Opacity,Name,Amount_Perturbations,Partial,N_Perturbed,Orbit_Array,Final_Epigenetic_Factors,Gene_Colour_Collection)
Amount_Perturbations = Amount_Large_Perturbations #1
N_Perturbed = Amount_Small_Perturbations  #1
Cut_Off = true #Remove later times from the PCA dimension
Length_Orbits = 3000
Partial = 1
Total_Length = 0
Shortest = size(Orbit_Array[1,1])[2]




#In this option we take all the lengths and concatenate them, however this
#may create a bias in the PCA analysis towards only final cell states
for i=1:N_Perturbed*Amount_Perturbations

        if Shortest>size(Orbit_Array[i])[2]
            Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
        end
        Total_Length += size(Orbit_Array[i])[2]
    
end
if Cut_Off
    PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed*Amount_Perturbations)
else
    PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
end
Collection_Index=0


for i=1:N_Perturbed*Amount_Perturbations
    
    if Cut_Off

        for p=0:2^Splits-1

            PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
        end
    else
        Length_Orbits =  size(Orbit_Array[i,j])[2]
        for p=0:2^Splits-1
            PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
        end
    end

    Collection_Index += Length_Orbits*2^Splits

end


Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)

PCA_Initial_Orbit_Array = [] #Emptied to save space
PCA_Orbits_2Gen = zeros(Float32,Amount_Perturbations,N_Perturbed,2^Splits,3,Shortest)
PCA_Orbits_Partial = zeros(Float32,Amount_Perturbations,Int(N_Perturbed/Partial),2^Splits,3,Shortest)
print(" ")
print("The Principalratio is: ")
print(principalratio(Orbital_Model))
for i=1:N_Perturbed
    for j=0:Amount_Perturbations-1
    for tau=1:2^Splits
        tester = Orbit_Array[i+N_Perturbed*j][2+(tau-1)*M:1+tau*M,1:Shortest]
        Transformed = transform(Orbital_Model,tester)
        PCA_Orbits_2Gen[j+1,i,tau,:,:] = Transformed
        if mod(i-1,Partial) == 0
            PCA_Orbits_Partial[j+1,(Int(i/Partial)),tau,:,:] = Transformed
        end
    end
    end

end
Colours = [:red,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
 
#PCA_Orbits_Partial = reshape(PCA_Orbits_Partial,(Int(2^Splits*N_Perturbed*Amount_Perturbations/Partial),3,Shortest))
PCA_Orbits_2Gen_Plot = permutedims(PCA_Orbits_Partial,(5,4,3,2,1))
Orbit_Range = Int(N_Perturbed/Partial)*2^Splits

print(size(PCA_Orbits_2Gen_Plot))


function Interpolator(Times,Play_Array)
    Vectors = zeros(length(Times),1+M*2^Splits)
    for i=1:length(Times)
        Index_Interp = findfirst(>=(Times[i]), Play_Array[1,:])
        Vectors[i,:] = Play_Array[:,Index_Interp-1] + (Times[i]-Play_Array[1,Index_Interp-1])*(Play_Array[:,Index_Interp] - Play_Array[:,Index_Interp-1])/(Play_Array[1,Index_Interp] - Play_Array[1,Index_Interp-1])
    end
    return Vectors
end

Interpolated_Epi = Interpolator([i for i=1:Length_Orbits],EpiThroughTime)

Interpolated_Epi_Reshaped = reshape(Interpolated_Epi[:,2:end], (Length_Orbits,M,2^Splits))

Interpolated_Orbits_Reshaped = reshape(Interpolated_Orbits[1][2:end,:]', (Length_Orbits,M,2^Splits));


function Orbits_Final_Cell_Type(Time_Lower,Time_Upper,Dimension_To_Plot,Samples_To_Plot,Orbits_To_Plot,Updating)
    #Orbits[Times,Dimension,Different_Cells]
    Colours = [:red,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:orange,:magenta,:mistyrose3,:darkgreen,:gold4,:salmon1]
    if Updating
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1

        aca =  plot!([Time_Lower:Time_Upper],Orbits_To_Plot[Time_Lower:Time_Upper,Dimension_To_Plot,Samples_To_Plot[1]],legend=false,xscale=:log10,linewidth=2,linestyle=:dot,linecolor=Colours[Colour_Index])
        for j=2:size(Samples_To_Plot)[1]
            Colour_Index = Final_Epigenetic_Factors[1,1,j,1]+Final_Epigenetic_Factors[1,1,j,2]*2+Final_Epigenetic_Factors[1,1,j,3]*4+Final_Epigenetic_Factors[1,1,j,4]*8+1
            aca =  plot!([Time_Lower:Time_Upper],Orbits_To_Plot[Time_Lower:Time_Upper,Dimension_To_Plot,Samples_To_Plot[j]],legend=false,xscale=:log10,linewidth=2,linestyle=:dot,linecolor=Colours[Colour_Index])
        end
    else
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1

        aca =  plot([Time_Lower:Time_Upper],Orbits_To_Plot[Time_Lower:Time_Upper,Dimension_To_Plot,Samples_To_Plot[1]],legend=false,xscale=:log10,linecolor=Colours[Colour_Index],title="Dimension = $Dimension_To_Plot")
        for j=2:size(Samples_To_Plot)[1]
            Colour_Index = Final_Epigenetic_Factors[1,1,j,1]+Final_Epigenetic_Factors[1,1,j,2]*2+Final_Epigenetic_Factors[1,1,j,3]*4+Final_Epigenetic_Factors[1,1,j,4]*8+1
            aca =  plot!([Time_Lower:Time_Upper],Orbits_To_Plot[Time_Lower:Time_Upper,Dimension_To_Plot,Samples_To_Plot[j]],legend=false,xscale=:log10,linecolor=Colours[Colour_Index])
        end
    end
    return aca
end
Time_Lower = 10
#Orbits[Times,Dimension,Different_Cells]
aaa = Orbits_Final_Cell_Type(Time_Lower,3000,1,[1,3,4,6,11,12],PCA_Orbits_2Gen_Plot[:,:,:,1,1],false)
display(aaa)
aba = Orbits_Final_Cell_Type(Time_Lower,3000,2,[1,3,4,6,11,12],PCA_Orbits_2Gen_Plot[:,:,:,1,1],false)
display(aba)
aca = Orbits_Final_Cell_Type(Time_Lower,3000,3,[1,3,4,6,11,12],PCA_Orbits_2Gen_Plot[:,:,:,1,1],false)
display(aca)
for tau in Gene_To_Plot
    ada = Orbits_Final_Cell_Type(Time_Lower,3000,tau,Cells,Interpolated_Orbits_Reshaped,false)
    ada = Orbits_Final_Cell_Type(Time_Lower,3000,tau,Cells,Interpolated_Epi_Reshaped,true)
    display(ada)
end

Individual_Cells = reshape(Interpolated_Orbits[1][2:end,:],(2^Splits,M,Length_Orbits))
1+1

function Active_Counter(Tolerance, TimeFrame,Cell_Orbit)
    Activity = ones(Length_Orbits)*M
    for t=TimeFrame+1:Length_Orbits
        Mean_Single = mean(Cell_Orbit[:,t-TimeFrame:t],dims=2)
        Difference = abs.(Cell_Orbit[:,t-TimeFrame] .- Mean_Single)  
        BitMatrixValue = Difference .> ones(M)*Tolerance
        
        for l=1:TimeFrame
            Difference = abs.(Cell_Orbit[:,t-TimeFrame+l] .- Mean_Single)  
            BitMatrixValue2 = Difference .> ones(M)*Tolerance
            BitMatrixValue = BitMatrixValue .| BitMatrixValue2
        end
        Activity[t] = sum(BitMatrixValue)
    end
    return Activity
end

Test = Active_Counter(0.1, 10,Individual_Cells[1,:,:])
aba = plot(Test[1:end],xscale=:log10)
for j=2:2^Splits
    Test = Active_Counter(0.1, 10,Individual_Cells[j,:,:])
    aba = plot!(Test[1:end],xscale=:log10)
end
display(aba)

In [ ]:
using ChaosTools
using StaticArrays

function Gene_ODE(u, Parameters, time)
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    Epi_Factors = Parameters[3]
    test = InvSqr*ParameterMatrix[1,:]'*u[:]
    #vector = Vector{Float16}([])
    #append!(vector,tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1])
    vector = [tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1]]
    for c=2:M
        append!(vector,tanh(40*(InvSqr*ParameterMatrix[c,:]'*u[:]+Epi_Factors[c]+CArray[c])) - u[c])
    end
    dx = @SVector [vector[k] for k in 1:M]

    return dx
end  

Cell = 7
Times = [10,40,70,100,150,200,250,300,400,500,600,700,800,900,1000,1500,2000,3000]
Lyapunov_In_Time = zeros(length(Times),M)

for l = 1:length(Times)
    Time = Times[l]
    lor = CoupledODEs(Gene_ODE, Float64.(Interpolated_Orbits_Reshaped[Time,:,Cell]), [ParameterMatrix,CArray,Interpolated_Epi_Reshaped[Time,:,Cell]])
    λλ = lyapunovspectrum(lor, 100; Δt = 0.1)
    Lyapunov_In_Time[l,:] = λλ
end


In [ ]:
plot(Times,Lyapunov_In_Time,xscale=:log10,legend=false)

In [ ]:
using ChaosTools
using StaticArrays

function Gene_ODE(u, Parameters, time)
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    Epi_Factors = Parameters[3]
    test = InvSqr*ParameterMatrix[1,:]'*u[:]
    #vector = Vector{Float16}([])
    #append!(vector,tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1])
    vector = [tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1]]
    for c=2:M
        append!(vector,tanh(40*(InvSqr*ParameterMatrix[c,:]'*u[:]+Epi_Factors[c]+CArray[c])) - u[c])
    end
    dx = @SVector [vector[k] for k in 1:M]

    return dx
end  

Cell = 7
Times = [10,40,70,100,150,200,250,300,400,500,600,700,800,900,1000,1500,2000,3000]
Lyapunov_In_Time = zeros(length(Times),M)

for l = 1:length(Times)
    Time = Times[l]
    lor = CoupledODEs(Gene_ODE, Float64.(Interpolated_Orbits_Reshaped[Time,:,Cell]), [ParameterMatrix,CArray,Interpolated_Epi_Reshaped[Time,:,Cell]])
    λλ = lyapunovspectrum(lor, 1000; Δt = 0.1)
    Lyapunov_In_Time[l,:] = λλ
end

plot(Times,Lyapunov_In_Time,xscale=:log10,legend=false)

In [ ]:
using ChaosTools
using StaticArrays

function Gene_ODE(u, Parameters, time)
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    Epi_Factors = Parameters[3]
    test = InvSqr*ParameterMatrix[1,:]'*u[:]
    #vector = Vector{Float16}([])
    #append!(vector,tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1])
    vector = [tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1]]
    for c=2:M
        append!(vector,tanh(40*(InvSqr*ParameterMatrix[c,:]'*u[:]+Epi_Factors[c]+CArray[c])) - u[c])
    end
    dx = @SVector [vector[k] for k in 1:M]

    return dx
end  

Times = [10,40,70,100,150,200,250,300,400,500,600,700,800,900,1000,1500,2000,3000]
Lyapunov_In_Time = zeros(64,length(Times),M)

for Cell=1:64
    
    for l = 1:length(Times)
        Time = Times[l]
        lor = CoupledODEs(Gene_ODE, Float64.(Interpolated_Orbits_Reshaped[Time,:,Cell]), [ParameterMatrix,CArray,Interpolated_Epi_Reshaped[Time,:,Cell]])
        λλ = lyapunovspectrum(lor, 100; Δt = 0.1)
        Lyapunov_In_Time[Cell,l,:] = λλ
    end
    aba = plot(Times,Lyapunov_In_Time[Cell,:,:],xscale=:log10,legend=false)
    display(aba)
end


In [ ]:
#### SMALL PERT IS SET TO 1 WE ONLY HAVE 1 SAMPLE NOW
### WE MESSED WITH SOME DATA_TO_ANALYSE and the loading ones

#Data_To_Analyse = "21_12_NoSplit_HighODE_SplitTime=0.1_2000Gen_Noise01"
#Data_To_Analyse = ARGS[1]
#Noise = parse(Float64, ARGS[2])
#Data_To_Analyse = "21_12_NoSplit_HighODE_SplitTime=0.1_2000Gen_Noise005_Mut=2"
#Noise = 0.005

Noise = 0.1
Data_To_Analyse = "07_01_NoSplit_HighODE_SplitTime=0.1_2000Gen_Noise0.1"

Load_Generations = [325,525]
#Load_Generations = [24]
Load_Generations = [325]

Epi_Limiter = 1
a::Float32 = 1.0*Epi_Limiter   #Ratio of gene to epigenetic factor
v::Float32 = 6*10^-4/Epi_Limiter  #Speed at which epigenetic factor changes
#v::Float32 = 0  #Speed at which epigenetic factor changes

Load_Data_String = Data_To_Analyse*"Epi0.1"



println("OUTPUTTING MY INPUT")
println(Data_To_Analyse)
println(Noise)

using Random
using Plots
using Distributions
using DifferentialEquations
using Statistics
using ProgressMeter
using TickTock
using LinearAlgebra
using StatsBase
using LaTeXStrings
using MultivariateStats
using Dates
using Distances
using JLD
using KernelDensity
using Plots
using StatsPlots




function Setting_Up_Randomly(M,Sparse,rng=MersenneTwister(123))
    "Function used to randomly with seed define our c_i's, J_ij's and the initial positions
    Input:
    M      = Amount of Genes in our system
    Sparse = Bool to determine if we have a sparse graph
    rng    = Random Julia seed to make sure we can recreate the same initial conditions
    Output:
    CArray          = C_i's used, can only be between -0.5 and 0.5 now
    PositionArray   = Random Initial position, between -1 and 1 for all genes, epifactors = 0
    ParameterMatrix = J_ij's Currently a dense network with values between -0.5 and 0.5
    "
    ParameterMatrix = zeros(M,M)
    CArray = zeros(1,M)
    PositionArray = zeros(2,1,M)
    
    CArray = rand(rng,Float32,M).-0.5; #Random c_j between -0,5 and 0.5
    PositionArray[1,1,1:M] = 2*(rand(rng,Float32,M).-0.5); #Random Initial position 
    ParameterMatrix = 2 * bitrand(rng,M,M).-1; #J_jk's
    if Sparse
       for i=1:M
            for j=1:M
                if rand(rng) > 5/M #Average of 5 connections per gene

                    ParameterMatrix[i,j] = 0 
                end
            end
        end
    end
    return CArray, PositionArray, ParameterMatrix
end







function Noisy_Splitting(solution,Total_Splits,M,Seed)
    " Once our cell decides to split, we decide to introduce some random noise in how these genes get distributed.
    This function takes the cells, doubles them and then gives the daughters cell the same gene parameters +- noise
    one daughter gets + the other gets -. For the epigeneitc factors they are simply inherited and do not see any noise
    Input:
    solution     = These are the initial conditions of the parent cells that we need to split 
    (First genes of all cells, then all epigenetic factors)
    Total_Splits = Amount of times the cell has split (including this current split)
    M            = Amount of Genes
    Seed         = Random Noise seed for reproducing results
    Output:
    SplittedArray = New array of the genes and epigenetic factors of the daughter cells
    How is this structured? Suppose we take M=3 for convenience, we would have [g1,g2,g3,e1,e2,e3] for the first parent cell
    The daughters become: [g11,g12,g13,g21,g22,g23,e11,e12,e13,e21,e22,e23]
    Where we have genes of daughter 1, then genes of daughter 2, then epi of daughter 1 and epi of daughter 2.
    When the Daughters split we get the genes of Children of cell 1, then genes of Children of cell 2 and same for epi.
    "
    Random.seed!(Seed)
    s = Total_Splits #For shortening
    SplittedArray = zeros(Float32,2,2^(s),M) #2^(s+1), 2M is the normal length, 4M after 1 split, 8M after 2 splits etc.
    for g=1:2^(s-1) #looping over the length of the parent
        for t=1:M
            Gaussian = Normal{Float32}(0.0f0,0.0f0)
            Displace = rand(Gaussian) #Noise for split
            #index = Int32.(floor((g-1)/M)*M +g) #example M=10: Index 1-> 1 and 11, index 2->2 and 12 ... 
            #index 10->10 and 20, index 11->21 and 30 and so on. This is old method

            #Genes
            SplittedArray[1,2*g-1,t] = solution[1,g,t] + Displace
            SplittedArray[1,2*g,t] = solution[1,g,t] - Displace
            
        end
        #Epigenetic
        SplittedArray[2,2*g-1,:] = solution[2, g,:]
        SplittedArray[2,2*g,:] = solution[2, g,:]   
    end

    return SplittedArray  
end

function Differentiaton_Detection_Parameter(M,Epigenetic_Plot)
    "Function used to convert epigenetic factors into cell types and bits
    Input:
    M               = Amount of Genes 
    Epigenetic_Plot = Array of epigenetic factors which have to be converted to 0 and 1 representing on or off
    Output:
    b        = Array of scalar values representing cell types
    BitGenes = Array of vectors representing the genes of the cell types
    "
    b = Int128[]
    Counter = Int[]
    Epi_Bitwise = signbit.(-Epigenetic_Plot)

    
    BitGenes = [] #We will put the bit notation of the gene in here
    #Later rewrite to let this replace Bit_Represent
    
    for i=1:length(Epi_Bitwise[:,1])
        Bit_Represent::Int128 = 0 #CAREFUL WITH M>127
        for j=1:M
            Bit_Represent += Int128(Epi_Bitwise[i,j]*2)^(j-1)
        end
        #print(Bit_Represent)
        #print("     ")
        if !(Bit_Represent in b)

            push!(BitGenes,collect(Epi_Bitwise[i,:]))
            push!(Counter,1)
        end
        push!(b,Bit_Represent)

        
    end
    
    #print(b)
    Counter = sort(Counter,rev=true)
    #print(Counter)
    Score = 0
    #Make here the formula to turn b and the counter into a score


    return b,BitGenes
end

function Entropy_Score(M,Epigenetic_Plot)
    "Function used to convert epigenetic factors into cell types and bits to then calculate the entropic score.
    See previous function for alternative, this is currently not in use.
    Input:
    M               = Amount of Genes 
    Epigenetic_Plot = Array of epigenetic factors which have to be converted to 0 and 1 representing on or off
    Output:
    Score = Entropy score of the cell types
    "
    b = Int[]
    Counter = Int[]
    Epi_Bitwise = signbit.(-Epigenetic_Plot)
    
    for i=1:length(Epi_Bitwise[:,1])
        Bit_Represent::Int128 = 0 #CAREFUL WITH M>127
        for j=1:M
            Bit_Represent += Int128(Epi_Bitwise[i,j]*2)^(j-1)
        end
        #print(Bit_Represent)
        #print("     ")
        if !(Bit_Represent in b)
            push!(b,Bit_Represent)
            push!(Counter,1)
        else 
            Counter[findfirst(==(Bit_Represent), b)] += 1
        end
        
    end
    
    #print(b)
    Counter = sort(Counter,rev=true)
    #print(Counter)
    Score = 0
    #Make here the formula to turn b and the counter into a score
    for i=1:length(b)
       prob = Counter[i]/M
       Score -= prob*log(prob)
    end
    
    return Score
end



function Mutate_Parameters(ParameterMatrix_Mut,M,Mutations,Seed)
    "Function used to mutate the network (we turn a 1 or -1 to 0 and we turn a 0 to 1 or -1)
    Input:
    ParameterMatrix_Mut = Original Network that we want to mutate
    M                   = Amount of Genes 
    Mutations           = How many connections will we mutate
    Seed                = Randomness seed
    Output:
    ParameterMatrix_Mut = Mutated Network
    "
    Zero_Indices = findall(iszero,ParameterMatrix_Mut)
    One_Indices = findall(!iszero,ParameterMatrix_Mut)
    Random.seed!(Seed)
   
    for j=1:Mutations 
        t_zero = rand(1:length(Zero_Indices))
        t_one = rand(1:length(One_Indices))
        i_zero = Zero_Indices[t_zero][1]
        j_zero = Zero_Indices[t_zero][2]
        i_Value = One_Indices[t_one][1]
        j_Value = One_Indices[t_one][2]



        ParameterMatrix_Mut[i_zero,j_zero] = rand(0:1)*2-1 #turn a zero into 1 or -1
        ParameterMatrix_Mut[i_Value,j_Value] = 0 #turn 1 or -1 into 0




        
    end
    return ParameterMatrix_Mut
end


#Some Julia functions I played around with, may reconsider using more later
MultiplyBroad_(a,b) = a .* b

SummingDim2_(a) = sum(a,dims=2) 

HyperTanh_(a) = tanh.(40*a)



function Epigenetic_Evolution3(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    
    #⊗_(Wi, bi) = Wi .* bi
       
    for t=1:2^Splits

        du[2,t,:] = v*(a*u[1,t,:].-u[2,t,:])    
        du[1,t,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,t,:].+u[2,t,:].+CArray[:])) .- u[1,t,:] 
            #Epigenetic Evolution, no need to worry about boundaries here so simple loop      
    end  
    
end



function Gradient(u,Splits,t)
    "Function used to calculate the gradient of a cells neighbours, takes into accounts if there even are neighbours.
    Input:
    u      = Gene concentrations over which we take the gradient
    Splits = How many times have the cells split already (determines where neighbours are)
    t      = What location is our cell (boundaries only have 1 neighbour)
    Output:
    Multiple different version depending on the splits, this will be the gradient we want
    "
    if Splits == 0
        return 0
    elseif Splits ==1
        if t==1
            return u[t+1,:] - u[t,:]
        else
            return u[t-1,:] - u[t,:]
        end
    else
        if t==1
            return u[t+1,:] - u[t,:]
        elseif t==2^Splits
            return u[t-1,:] - u[t,:]
        else
            return u[t-1,:] - 2*u[t,:] + u[t+1,:]
        end
    end
    
end

function Epigenetic_Evolution4(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    
    #⊗_(Wi, bi) = Wi .* bi
       
    for t=1:2^Splits

        du[2,t,:] = v*(a*u[1,t,:].-u[2,t,:])    
        du[1,t,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,t,:].+u[2,t,:].+CArray[:])) .- u[1,t,:] +KArray.*Gradient(u[1,:,:],Splits,t)
            #Epigenetic Evolution, no need to worry about boundaries here so simple loop      
    end  
    
end

function σ_Genes(du, u,Parameters,Time)
    du[1,:,:] .= Noise
    
end



function Epigenetic_Evolution2(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    Cells::Int32 = 2^Splits
    #⊗_(Wi, bi) = Wi .* bi
       
    for t=1:Cells  

       du[2,t,:] = v*(a*u[1,t,:].-u[2,t,:])    

       du[1,t,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,t,:].+u[2,t,:].+CArray[:])) .- u[1,t,:] .+ KArray.*(vec(sum(u[1,:,:],dims=1))/Cells - u[1,t,:])

    end  
    
end

#Defining it all in function form


function Mutating_Network(N,M,K,ParameterMatrix_Collection,PositionArray_Collection, Indices, Counter, Genetics)
    "Function used to make mutated copies of the best performing networks. Currently not in use
    Input:
    N                          = How many copies of the network do we make
    M                          = Amount of Genes
    K                          = Supposed to work as a fraction of N, telling us how many mutant versions
    ParameterMatrix_Collection = The original networks
    PositionArray_Collection   = Original Positions
    Indices                    = Sorted indices giving the best networks
    Counter                    = Randomiser seed
    Genetics                   = Used to track heritance of networks
    Output:
    ParameterMatrix_Collection = Updated network
    PositionArray_Collection   = Updated initial positions
    Counter                    = Updated seed
    Updated_Genes              = Updated inheritance
    "
    Updated_Genes = zeros(N)
    Mutations = 1
    Seed_Mutations = 1234
    Indexation_Mutate = 1
    Temporary_Matrix = zeros(Float32,N,M,M)
    Temporary_Vector = zeros(Float32,N,2,1,M)
    for p = 1:K
        for q=1:N/(2*K)
            Temporary_Matrix[Indexation_Mutate,:,:] = Mutate_Parameters(ParameterMatrix_Collection[Indices[p],:,:],M,Mutations,Seed_Mutations+Counter)
            Temporary_Vector[Indexation_Mutate,:,:,:] = PositionArray_Collection[Indices[p],:,:,:]
            Updated_Genes[Indexation_Mutate] = Genetics[p]
            Temporary_Matrix[Indexation_Mutate+1,:,:] = Temporary_Matrix[Indexation_Mutate,:,:]
            Temporary_Vector[Indexation_Mutate+1,:,:,:] = Temporary_Vector[Indexation_Mutate,:,:,:]
            Updated_Genes[Indexation_Mutate+1] = Genetics[p]
            Counter += 1
            Indexation_Mutate +=2
        end
        Temporary_Matrix[Indexation_Mutate-1,:,:] = ParameterMatrix_Collection[Indices[p],:,:] #We overwrite it to add the original back
        Temporary_Matrix[Indexation_Mutate-2,:,:] = ParameterMatrix_Collection[Indices[p],:,:]
    end
    ParameterMatrix_Collection = deepcopy(Temporary_Matrix);
    PositionArray_Collection = deepcopy(Temporary_Vector);
    return ParameterMatrix_Collection, PositionArray_Collection, Counter, Updated_Genes
end



function Running_Network(CArray,ParameterMatrix,PositionArray,Random_Noise)
    "Function used to run our simulation
    Input:
    CArray          = Constants used in our ODE solver  
    ParameterMatrix = Collection of Networks
    PositionArray   = Original Positions
    Random_Noise    = Randomiser seed
    Output:
    PositionArray[2,:,:] = Epigenetic factors from our simulation
    Random_Noise         = Updated seed
    "
    #CArray = CArray_Collection[x,:]
    #ParameterMatrix = ParameterMatrix_Collection[x,:,:]
    #PositionArray = PositionArray_Collection[x,:]
    Orbit_Collection = zeros(Float32,2^Splits*M+1,0)
    Epi_Collection = zeros(Float32,2^Splits*M+1,0)
    
    
    
    
    for s=1:Splits #Looping over the splits
        Parameters = (ParameterMatrix, CArray,KArray,Int32(s-1)) #We have to update the splits every rerun, rest stays same
        tspan = (Float32(0.0),SplitTime) #How long ODE will run
        if Mean_Field
            prob = ODEProblem(Epigenetic_Evolution2,PositionArray,tspan,Parameters) #ODEdefining
        elseif Diffusing_System
            prob = ODEProblem(Epigenetic_Evolution4,PositionArray,tspan,Parameters) #ODEdefining
        elseif Stochastic_Noise
            prob = SDEProblem(Epigenetic_Evolution4, σ_Genes,PositionArray,tspan,Parameters)

        else
            prob = ODEProblem(Epigenetic_Evolution3,PositionArray,tspan,Parameters) #ODEdefining
        end    #Solving
        if !Stochastic_Noise
            sol = solve(prob,BS3())
        else
            sol = solve(prob,SRA3())
            #sol = solve(prob,LambaEulerHeun())
        end
        Times = sol.t #Save the time, doesn't get used yet though
        StepAmount = length(Times)  #Amount of data points per gene
        Genes = sol 
        PositionArray = Genes[StepAmount] #Saving the genes and epi factors
        
        OrbitalCollection = Times' .+ (s-1)*SplitTime
        EpiCollected = Times' .+ (s-1)*SplitTime
        for i=1:2^(Splits-s+1)  #s=1 this gives all 2^Splits, this is to say how much we repeat same path
            for j=1:2^(s-1) #Together multiply, always 2^Splits orbits, these j represent different paths
                OrbitalCollection = vcat(OrbitalCollection,sol[1,j,:,:])   
                EpiCollected = vcat(EpiCollected,sol[2,j,:,:])   
            end
        end
        Orbit_Collection = hcat(Orbit_Collection,OrbitalCollection)
        Epi_Collection = hcat(Epi_Collection,EpiCollected)
        #Orbit1 = [Times,Genes[1,1,1:M,1:StepAmount]]
        #Orbit2 = [Times,Genes[1,1,1:M,1:StepAmount]]
        #EpiOrbit1 = [Times,Genes[2,1,1:M,1:StepAmount]]
        #EpiOrbit2 = [Times,Genes[2,1,1:M,1:StepAmount]]
        #Now we must take all the values from the updated PositionArray and give appropriate terms to the daughter cells
        PositionArray = Noisy_Splitting(PositionArray,s,M,Random_Noise)  #Splitting function
        Random_Noise += 1
    end

    Final_Run_Time::Int32 = 20000 #Long evolving of the cells after splitting to get the final epigenetic state
    Parameters = (ParameterMatrix, CArray,KArray,Int32(Splits)) 
    tspan = (Int32(0.0),Final_Run_Time)

    if Mean_Field
        prob = ODEProblem(Epigenetic_Evolution2,PositionArray,tspan,Parameters) #ODEdefining
    elseif Diffusing_System
        prob = ODEProblem(Epigenetic_Evolution4,PositionArray,tspan,Parameters) #ODEdefining
    elseif Stochastic_Noise
        prob = SDEProblem(Epigenetic_Evolution4, σ_Genes,PositionArray,tspan,Parameters)

    else
        prob = ODEProblem(Epigenetic_Evolution3,PositionArray,tspan,Parameters) #ODEdefining
    end    #Solving
    if !Stochastic_Noise
        sol = solve(prob,BS3())
    else
        #,dtmin=0.1,force_dtmin=true
        sol = solve(prob,SRA3(),dtmin=0.1,force_dtmin=true)
    end
    #sol = solve(prob,alg_hints = [:nonstiff]) #Solving
    #sol = solve(prob)
    Times = sol.t
    StepAmount = length(Times)
    GenesEpi = sol
    PositionArray = GenesEpi[StepAmount]

    OrbitalCollection = Times' .+ Splits*SplitTime 
    EpiCollected = Times' .+ Splits*SplitTime 
    for j=1:2^(Splits) #Together multiply, always 2^Splits orbits, these j represent different paths
        OrbitalCollection = vcat(OrbitalCollection,sol[1,j,:,:])  
        EpiCollected = vcat(EpiCollected,sol[2,j,:,:])   

    end
    Orbit_Collection = hcat(Orbit_Collection,OrbitalCollection)
    Epi_Collection = hcat(Epi_Collection,EpiCollected)
    #Orbit1 = [cat(Orbit1[1],Times.+SplitTime,dims=1),cat(Orbit1[2],GenesEpi[1,1,1:M,1:StepAmount],dims=2)]
    #Orbit2 = [cat(Orbit2[1],Times.+SplitTime,dims=1),cat(Orbit2[2],GenesEpi[1,2,1:M,1:StepAmount],dims=2)]
    #EpiOrbit1 = [cat(EpiOrbit1[1],Times.+SplitTime,dims=1),cat(EpiOrbit1[2],GenesEpi[2,1,1:M,1:StepAmount],dims=2)]
    #EpiOrbit2 = [cat(EpiOrbit2[1],Times.+SplitTime,dims=1),cat(EpiOrbit2[2],GenesEpi[2,2,1:M,1:StepAmount],dims=2)]

    return PositionArray[2,:,:],Random_Noise, Orbit_Collection, Epi_Collection
end


function Scoring_Indexes(N,Final_Epigenetic_Factors,Cell_Bit)
    "Function used to calculate the score of all the networks, distribution robustness and the cell type robustness. Note
    that we are still selecting on amount of different cells, not entropy.
    Input:
    N                       = How many copies of the same networks we have 
    Final_Epigentic_Factors = Array of epigenetic factors after running the simulations
    Cell_Bit                = Cell type tracker
    Output:
    Indices                   = Indices of networks sorted by score, we only select these
    AllIndices                = Longer copy of Indices, contains also those we will no longer use
    Evolution_Indices         = Gives us the score corresponding with the Indices we selected on
    [Cell_Types,Cell_Amounts] = Vector of the cell types and how many of these cell types there are
    Cell_Bit                  = Updated cell type tracker
    KLD_Distance              = Kullback-Lieber divergence score for the distribution robustness
    Bit_Distances             = Cell Type robustness measure
    "
    Scoring = zeros(2,N)
    Cell_Types = []
    Cell_Amounts = []
    Cell_Distribution = Array{Vector{Float64}}(undef, N) 
    Cell_Bit_Array = Array{Vector{Any}}(undef, N)

    for x=1:N
        Differentiation, Bit_Genes = Differentiaton_Detection_Parameter(M,Final_Epigenetic_Factors[x,:,:]) 
     
        #Here we save some cell types and bits to use for our robustness calculation after the for loop
        #Cell types
        alpha = countmap(Differentiation)
        Temp_Array = Vector{Float64}()
        for i in keys(alpha)
            Temp_Array = push!(Temp_Array,alpha[i])
        end
        #print(Temp_Array)
        Cell_Distribution[x] = sort(Temp_Array,rev=true)/2^Splits #We sort the distribution of our cells and normalise
        #Bits
        Cell_Bit_Array[x] = Bit_Genes
        
        
        
        
        #This part here is where we track the cell types we had before as well
        for tau in Bit_Genes
            if !(tau in Cell_Bit)

                push!(Cell_Bit,tau)


            end
        end
        for tau in Differentiation 
            if !(tau in Cell_Types)
                push!(Cell_Types,tau)
                push!(Cell_Amounts,1)
            else
                Cell_Amounts[findfirst(==(tau), Cell_Types)] += 1
            end
        end

        Scoring[1:2,x] = [length(countmap(Differentiation)),x]

    end

    #Robustness calculation
    KLD_Distance = 0
    
    KLD_Adder = 0
    for i=1:N
        for j=1:N #Loop over all and calculate KL for all distributions
            if i!=j #If the same we get no difference anyways
                Length_Differ = length(Cell_Distribution[i])-length(Cell_Distribution[j])
                if Length_Differ > 0 #i is longer so we need to add 0 to j
                    KLD_Adder = kldivergence(Cell_Distribution[i],vcat(Cell_Distribution[j],zeros(Length_Differ)))
                    if KLD_Adder != Inf #There are zeros in the second distribution (divide by 0 error)
                        KLD_Distance += kldivergence(Cell_Distribution[i],vcat(Cell_Distribution[j],zeros(Length_Differ)))
                    end
                else
                    KLD_Adder = kldivergence(vcat(Cell_Distribution[i],zeros(-Length_Differ)),Cell_Distribution[j])

                    KLD_Distance += kldivergence(vcat(Cell_Distribution[i],zeros(-Length_Differ)),Cell_Distribution[j])
                end
            end
        end
    end
    KLD_Distance = KLD_Distance/(N*(N-1))
    
    #Now we see the closest type in the other groups
    Bit_Distances = 0
    for x =1:N
        for Bit_Inside in Cell_Bit_Array[x] #These are all the cell types in bit form
            for y =1:N #We loop over the others
                if x!=y #Must look at different otherwise distance 0 again
                    Bit_Distan_Tem = M
                    for Bit_Outside in Cell_Bit_Array[y] #We loop over all the cell types in this other one
                        Bit_Distan_Temp = sum(abs.(Bit_Inside .- Bit_Outside)) 
                        if Bit_Distan_Temp < Bit_Distan_Tem #Distance must be smaller if it is the best fit
                            Bit_Distan_Tem = Bit_Distan_Temp
                        end
                    end
                    Bit_Distances += sqrt(Bit_Distan_Tem)/(N*(N-1)) #Sqrt to make euclidean distance, N*(N-1) for normalisation
                end
            end
        end
    end
    
    
    #Here we mess a bit with the indices to just have the ones with the highest score later
    
    Indices = sortslices(Scoring,dims=2,rev=true)
    Evolution_Indices = Indices[1,1:K]

    AllIndices = deepcopy(Indices[2,:])
    Indices = Indices[2,1:K]
    Indices = Int.(Indices)
    #print(Indices[2,1:K])
    return Indices, AllIndices, Evolution_Indices, [Cell_Types,Cell_Amounts],Cell_Bit, KLD_Distance, Bit_Distances
end

function Epi_Plotter(Indices,K,Final_Epigenetic_Factors)
    "Function used to plot the epigenetic factors of different cells
    Input:
    Indices                 = Indices of highest scoring runs  
    k                       = 4 or 8 plots
    Final_Epigentic_Factors = Epigenetic factors that we will plot
    Output:
    k = Plot element, simply use display on the output to see the requested plot
    "
    pa = plot(Final_Epigenetic_Factors[Indices[1],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pb = plot(Final_Epigenetic_Factors[Indices[2],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pc = plot(Final_Epigenetic_Factors[Indices[3],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pd = plot(Final_Epigenetic_Factors[Indices[4],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    if K==8
        pe = plot(Final_Epigenetic_Factors[Indices[5],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        pf = plot(Final_Epigenetic_Factors[Indices[6],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        pg = plot(Final_Epigenetic_Factors[Indices[7],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        ph = plot(Final_Epigenetic_Factors[Indices[8],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        k = plot(pa,pb,pc,pd,pe,pf,pg,ph,layout=(4,2),legend=false)
    else
        k = plot(pa,pb,pc,pd,layout=(2,2),legend=false)
    end
    return k
end

function Gene_Variance(Final_Epigenetic_Factors,M,N)
    "Function used to calcualte the variance in expressed genes
    Input:
    Final_epigenetic_Factors = Final epigenetic values which represent gene expression
    M                        = Amount of Genes
    N                        = How many copies of same network do we have
    Output:
    Deviation = Variance in these genes (normalised by 2^Splits*N)
    "
    BitterMatrix = signbit.(-Final_Epigenetic_Factors[:,:,:])
    Average_Bits = zeros(M)
    for i=1:N
        for j=1:2^Splits
            Average_Bits += BitterMatrix[i,j,:]
        end
    end
    Average_Bits /= N*2^Splits
    Deviation = 0

    for i=1:N
        for j=1:2^Splits
            Deviation += sum((Average_Bits .- BitterMatrix[i,j,:]).^2)/(2^Splits*N-1)
        end
    end
    
    return Deviation
end

function PCA_Distribution_Creator(PCA_Dimensions,Transformed,Dimensions_Distribution,Cell_Distribution)
    
    
    PCA_Dimensional_Values = zeros(PCA_Dimensions,findmax(Dimensions_Distribution)[1]) 
    #Here we will save all the values and use these for indexes later
    B = Array{Vector{Tuple{Float64, Int64}}}(undef, PCA_Dimensions)
    for j=1:PCA_Dimensions
        B[j] = [(i, count(==(i), Transformed)) for i in unique(Transformed[j,:])] #Here we find what the arguments are for the
        #different dimensions
        for k = 1:length(B[j])
            PCA_Dimensional_Values[j,k] = B[j][k][1] #Here we save all the individual arguments in the corresponding dimension
        end
    end
    #print(PCA_Dimensional_Values)   #If we wanna see them

    Components_Length = length(Transformed[1,:]) 
    for k in 1:Components_Length
        Index_PCA = zeros(PCA_Dimensions)

        for l=1:PCA_Dimensions
            Index_PCA[l] = (findfirst(==(Transformed[l,k]), PCA_Dimensional_Values[l,:]))
        end 
        Index_PCA = Int.(Index_PCA)
        
        
        if PCA_Dimensions==2 #I really tried but I couldn't fix it so just throwing in this hack
            Cell_Distribution[Index_PCA[1],Index_PCA[2]] += 1/Components_Length
        elseif PCA_Dimensions==1
            Cell_Distribution[Index_PCA[1]] += 1/Components_Length   
        elseif PCA_Dimensions==3
            Cell_Distribution[Index_PCA[1],Index_PCA[2],Index_PCA[3]] += 1/Components_Length
        elseif PCA_Dimensions==4
            Cell_Distribution[Index_PCA[1],Index_PCA[2],Index_PCA[3],Index_PCA[4]] += 1/Components_Length
        elseif PCA_Dimensions==5
            Cell_Distribution[Index_PCA[1],Index_PCA[2],Index_PCA[3],Index_PCA[4],Index_PCA[5]] += 1/Components_Length
        else
        print("ADD MORE HERE THERE IS A MISTAKE, DO NOT GO OVER 5 PCA DIMENSIONS YET")
        end
    end
    

    
    return Cell_Distribution
end



function Earth_Movers_Distance_1D(Vector1,Vector2)

    #Takes 2 distribution vectors containing bins with a position and their %
    Length_V = length(Vector1[1,:]) #Both should be same length anyways
    EMD = zeros(Length_V+1) #We add an extra one because of the following loop
    
    for i =1:Length_V
        EMD[i+1] = Vector1[2,i] - Vector2[2,i] + EMD[i]
    end
    EMD_Distance = 0
    
    for i =1:Length_V-1
        EMD_Distance += abs(EMD[i+1])*(Vector1[1,i+1] - Vector1[1,i]) #We take moved distribution times moved distance

    end

    
    return EMD_Distance
end


function Robustness_Distribution_Scoring()
    
    
    #Here we create a very general robustness measure
    #This is done by taking the PCA components over all the different generations
    #Then we categorise all cell types and measure how similar their distribution
    #is in a given network

    #!!!!!!! This robustness does not consider cell type distances
    
    Robustness_Score = zeros(Generations,P)
    for Genere=1:Generations  #We want to calculate this for all generations
        for i = 1:P #We have to repeat this for all different networks in a generation
            #This is used to find the maximum of different cell types along every dimension in this network
            Dimensions_Distribution = zeros(Int32,PCA_Dimensions)
            for j=1:N
                Transformed = transform(model,reshape(signbit.(-History_Final_Epigenetic_Factors[Genere,i,j,:,:]),2^Splits,M)')

                for k=1:PCA_Dimensions
                    if Int.(length(countmap(Transformed[k,:]))) >  Dimensions_Distribution[k]
                        Dimensions_Distribution[k] = Int.(length(countmap(Transformed[k,:])))
                    end
                end   
            end

            #Now we have the max dimensions, now we want to make all distributions
            #Of this specific network

            Dimensions_Distribution = (Dimensions_Distribution...,) #Magically makes a tuple!
            #print(Dimensions_Distribution) #if we wanna visualise

            Cell_Distribution_Collection = Array{Array{Float64, PCA_Dimensions}}(undef, N)
            Vectorised_Cell_Distribution_Collection = Array{Vector{Float64}}(undef, N)

            for j=1:N #Now we collect all the distributions
                Transformed = transform(model,reshape(signbit.(-History_Final_Epigenetic_Factors[Genere,i,j,:,:]),2^Splits,M)')

                Cell_Distribution = zeros(Dimensions_Distribution) #This is where we need it to be a tuple!
                Cell_Distribution_Collection[j] = PCA_Distribution_Creator(PCA_Dimensions,Transformed,Dimensions_Distribution,Cell_Distribution)

                #Now we want to convert all of these into vectors to apply Jensen-Shannon
                Vectorised_Cell_Distribution_Collection[j] = sort(vec(Cell_Distribution_Collection[j]),rev=true)
            end

            #JS-divergence
            for tau = 1:N-1
                for pau = tau:N
                    Robustness_Score[Genere,i] += js_divergence(Vectorised_Cell_Distribution_Collection[tau],Vectorised_Cell_Distribution_Collection[pau])
                end
            end


        end
    end
    return Robustness_Score
end



function EMD_Generational_Calculator()
    
    #Here we will create a Robustness measure that takes into account how far
    #away different cell types are for the same network.
    #we will do this by creating a PCA analysis for EVERY single network
    #for all the different generations
    #(Alternatively we see if we can use the same PCA analysis within one generation)
    #We will create a one dimensional distribution and use our simplified
    #EMD algorithm
    
    Vectorised_Cell_Network_Collection = Array{Matrix{Float64}}(undef, N) #Here we save these distributions
    #This loops over copies of same network and makes all our distributions
    EMD_Distances = zeros(Generations)
    for generation_p = 1:Generations
        EMD_Dist = 0
        for P_Valu = 1:P
            FullNetworkBits = reshape(signbit.(-History_Final_Epigenetic_Factors[generation_p,P_Valu,:,:,:]),2^Splits*N,M)'
            Networkmodel= fit(PCA,FullNetworkBits;maxoutdim=1)
            Networkmodel_EigenV = eigvals(Networkmodel::PCA)
            for i=1:N
                #This creates the Distribution of a single individual
                #We start to apply this PCA to one of these individuals
                NetworkBits = reshape(signbit.(-History_Final_Epigenetic_Factors[generation_p,P_Valu,i,:,:]),2^Splits,M)'
                Transformed_Network = transform(Networkmodel,NetworkBits)
                Distribution_Netw = zeros(0)
                X_Values = zeros(0)
                Transformed_Network = round.(Transformed_Network,digits=4) #Otherwise ugly floating point numbers
                Length_Netw_Normal = 1/length(Transformed_Network)
                for i in Transformed_Network

                    if i in X_Values
                        Distribution_Netw[findfirst(==(i), X_Values)] += Length_Netw_Normal
                    else
                        X_Values = vcat(X_Values,i)
                        Distribution_Netw = vcat(Distribution_Netw,Length_Netw_Normal)
                    end
                end



                Combined_Distribution = zeros(2,length(X_Values))
                Combined_Distribution[1,:] = round.(X_Values/Networkmodel_EigenV,digits=4)
                Combined_Distribution[2,:] = Distribution_Netw
                Combined_Distribution = sortslices(Combined_Distribution,dims=2)

                Vectorised_Cell_Network_Collection[i] = Combined_Distribution
            end


            #Now we need to apply EMD to these distributions and save the result

            for tau = 1:N-1   #Preparing the vectors to have the same X values and length
                for pau = tau+1:N
                    Vector1 = Vectorised_Cell_Network_Collection[tau]
                    Vector2 = Vectorised_Cell_Network_Collection[pau]
                    for i in Vector1[1,:]
                        if !(i in Vector2[1,:])
                            Vector2 = hcat(Vector2, [i,0]) #Adding empty position
                        end
                    end
                    for i in Vector2[1,:]
                        if !(i in Vector1[1,:])
                            Vector1 = hcat(Vector1, [i,0]) #Adding empty position
                        end
                    end

                    Vector1 = (sortslices(Vector1,dims=2)) #Sorting them with the added X values
                    Vector2 = (sortslices(Vector2,dims=2))
                    EMD_Dist += Earth_Movers_Distance_1D(Vector1,Vector2) #Calculating EDM for 1D

                end
            end
        end
        EMD_Dist = 2*EMD_Dist/(P*N*(N-1))  #Normalisation
        EMD_Distances[generation_p] = EMD_Dist
    end
    return EMD_Distances
end


function Two_Generational_Plotter(Generation,Cut_Off,Duration,Length_Orbits,Opacity)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1+2*(Generation-1):2+2*(Generation-1)
        for j=1:Amount_Copies
            if Shortest>size(Orbit_Array[i,j])[2]
                Shortest = size(Orbit_Array[i,j])[2] #Used later to check which data set is the shortest when plotting
            end

            Total_Length += size(Orbit_Array[i,j])[2]

        end
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2*8*2^Splits)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1+2*(Generation-1):2+2*(Generation-1)
        for j=1:Amount_Copies
            if Cut_Off
                
                for p=0:2^Splits-1
                    PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i,j][2+p*M:1+(p+1)*M,1:Length_Orbits]
                end
            else
                Length_Orbits =  size(Orbit_Array[i,j])[2]
                for p=0:2^Splits-1
                    PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i,j][2+p*M:1+(p+1)*M,:]
                end
            end

            Collection_Index += Length_Orbits*2^Splits
        end
    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,2,Amount_Copies,2^Splits,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:2
        for j=1:Amount_Copies

            for tau=1:2^Splits
                tester = Orbit_Array[i+2*(Generation-1),j][2+(tau-1)*M:1+tau*M,1:Shortest]
                Transformed = transform(Orbital_Model,tester)
                PCA_Orbits_2Gen[i,j,tau,:,:] = Transformed
            end
        end
    end
    
    
    Colours = [:red,:orange,:maroon,:chocolate4]
    @gif for k=1:Duration
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen[1,1,1,1,1:k],PCA_Orbits_2Gen[1,1,1,2,1:k],PCA_Orbits_2Gen[1,1,1,3,1:k],legend=false,c=:blue,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)


        for j=1:8
            for tau=1:2^Splits
                PCAPlots = plot!(PCA_Orbits_2Gen[i,j,tau,1,1:k],PCA_Orbits_2Gen[i,j,tau,2,1:k],PCA_Orbits_2Gen[i,j,tau,3,1:k],legend=false,c=:blue,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)
            end
        end

        i=2
        for j=1:4
            colour = Colours[j]
            for tau=1:2^Splits
                PCAPlots = plot!(PCA_Orbits_2Gen[i,2*j-1,tau,1,1:k],PCA_Orbits_2Gen[i,2*j-1,tau,2,1:k],PCA_Orbits_2Gen[i,2*j-1,tau,3,1:k],legend=false,c=colour,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                PCAPlots = plot!(PCA_Orbits_2Gen[i,2*j,tau,1,1:k],PCA_Orbits_2Gen[i,2*j,tau,2,1:k],PCA_Orbits_2Gen[i,2*j,tau,3,1:k],legend=false,c=colour,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)

            end
        end


    end
end




function Generational_Tracker(Family_Distance,Original_Tracking,Types)
    Cell_Generational_Tracking = deepcopy(Original_Tracking)
    Original_Tracking = deepcopy(Cell_Generational_Tracking)
    size(Types[1,:])


    TypeFam = [Types[1,:],1]
    TypeFam = hcat(TypeFam,[Types[1,:],2])
    TypeFam = hcat(TypeFam,[Types[1,:],3])
    TypeFam = hcat(TypeFam,[Types[1,:],4])

    print(size(TypeFam[1,:]))

    TypeFam = [Types[1,:],1]
    Distances_Gen = zeros(length(Cell_Generational_Tracking[1,:]))
    Cell_Generational_Tracking[1,1] = 1
    for i=2:length(Cell_Generational_Tracking[1,:])
        NewFam = true
        for j=1:length(TypeFam[1,:]) 
            if sum(abs.(Types[i,:]-TypeFam[1,j])) < Family_Distance + 0.5
                if NewFam
                    NewFam = false
                    Cell_Generational_Tracking[1,i] = j
                    Distances_Gen[j] = sum(abs.(Types[i,:]-TypeFam[1,j]))
                elseif Distances_Gen[j] > sum(abs.(Types[i,:]-TypeFam[1,j]))
                    Cell_Generational_Tracking[1,i] = j
                    Distances_Gen[j] = sum(abs.(Types[i,:]-TypeFam[1,j]))
                end
            end
        end
        if NewFam
            Distances_Gen[i] = sum(abs.(Types[i,:]-TypeFam[1,1]))
            Cell_Generational_Tracking[1,i] = Int(length(TypeFam[1,:])+1)
            TypeFam = hcat(TypeFam,[Types[i,:],Int(length(TypeFam[1,:])+1)])

        end
    end

    Cell_Generational_Tracking = Cell_Generational_Tracking[:,sortperm(Cell_Generational_Tracking[1,:])]



    Amount_Types = Cell_Generational_Tracking[1,end]
    Top_Bound_Index = zeros(Amount_Types)
    for j=1:Amount_Types
        Top_Bound_Index[j] = findfirst(isequal(j),Cell_Generational_Tracking[1,:])
    end
    print(Top_Bound_Index[:])


    #print(Cell_Generational_Tracking)
    Copied_Tracker = deepcopy(Cell_Generational_Tracking[2:end,:])
    Copied_Tracker = Copied_Tracker/sum(Copied_Tracker[1,:])

    Boundaries = zeros(Generations,length(Copied_Tracker[1,:]))
    for beta in 2:length(Copied_Tracker[1,:])
        Copied_Tracker[:,beta] += Copied_Tracker[:,beta-1]
        Boundaries[:,beta] = Copied_Tracker[:,beta] - Copied_Tracker[:,beta-1]
    end

    Cell_Type_Plotter = plot(Copied_Tracker[:,1]; ribbon = (Copied_Tracker[:,1],LinRange(0, 0, Generations)))
    for beta in 2:length(Copied_Tracker[1,:])
        Cell_Type_Plotter = plot!(Copied_Tracker[:,beta]; ribbon = (Boundaries[:,beta],LinRange(0, 0, Generations)),legend=false)
    end
    display(Cell_Type_Plotter)




    

    #Now we do the same but we group them in families


    Family_Boundaries = ones(Generations,Amount_Types)
    Family_Boundaries[:,1] = Copied_Tracker[:,Int(Top_Bound_Index[2]-1)].-0
    for beta in 2:Amount_Types-1

        Family_Boundaries[:,beta] = Copied_Tracker[:,Int(Top_Bound_Index[beta+1]-1)] .- Copied_Tracker[:,Int(Top_Bound_Index[beta]-1)]
    end
    Family_Boundaries[:,end] = 1 .- Copied_Tracker[:,Int(Top_Bound_Index[Amount_Types]-1)]


    Cell_Type_Family_Plotter = plot(Copied_Tracker[:,Int(Top_Bound_Index[2]-1)]; ribbon = (Family_Boundaries[:,1],LinRange(0, 0, Generations)))
    for beta in 2:Amount_Types-1
        Cell_Type_Family_Plotter = plot!(Copied_Tracker[:,Int(Top_Bound_Index[beta+1]-1)]; ribbon = (Family_Boundaries[:,beta],LinRange(0, 0, Generations)),legend=false)
    end
    Cell_Type_Family_Plotter = plot!(ones(Generations); ribbon = (Family_Boundaries[:,Amount_Types],LinRange(0, 0, Generations)),legend=false)

    if false
        Cell_Type_Family_Plotter = plot!(Copied_Tracker[:,1],linestyle=:dash)
        for beta in 2:length(Copied_Tracker[1,:])
            Cell_Type_Family_Plotter = plot!(Copied_Tracker[:,beta],linestyle=:dash)
        end
    end

    display(Cell_Type_Family_Plotter)
end

function Generational_Tracker_Initialisation()
    TypesString = deepcopy(Cell_Generational_Tracking[1,:])
    TypesString = string.(TypesString,base=2,pad=100)
    Types = zeros(Int16,length(Cell_Generational_Tracking[1,:]),M)
    for i in 1:length(Cell_Generational_Tracking[1,:])
        for j=1:M
            Types[i,j] = (parse(Int16,TypesString[i][j]))
        end
    end
    Original_Tracking = deepcopy(Cell_Generational_Tracking)
    return Original_Tracking, Types
end

function Interpolator(Times,Play_Array)
    Vectors = zeros(length(Times),1+M*2^Splits)
    for i=1:length(Times)
        Index_Interp = findfirst(>=(Times[i]), Play_Array[1,:])
        Vectors[i,:] = Play_Array[:,Index_Interp-1] + (Times[i]-Play_Array[1,Index_Interp-1])*(Play_Array[:,Index_Interp] - Play_Array[:,Index_Interp-1])/(Play_Array[1,Index_Interp] - Play_Array[1,Index_Interp-1])
    end
    return Vectors
end


function One_Generational_Plotter(Cut_Off,Duration,Length_Orbits,Opacity,Name)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1:N_Perturbed

            if Shortest>size(Orbit_Array[i])[2]
                Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
            end
            Total_Length += size(Orbit_Array[i])[2]
        
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1:N_Perturbed
        
        if Cut_Off

            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
            end
        else
            Length_Orbits =  size(Orbit_Array[i,j])[2]
            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
            end
        end

        Collection_Index += Length_Orbits*2^Splits

    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,N_Perturbed,2^Splits,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:N_Perturbed
        
        for tau=1:2^Splits
            tester = Orbit_Array[i][2+(tau-1)*M:1+tau*M,1:Shortest]
            Transformed = transform(Orbital_Model,tester)
            PCA_Orbits_2Gen[i,tau,:,:] = Transformed
        end

    end
    
    
    anim = @animate for k=1:Duration
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen[1,1,1,1:k],PCA_Orbits_2Gen[1,1,2,1:k],PCA_Orbits_2Gen[1,1,3,1:k],legend=false,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)


        for i=1:N_Perturbed
            for tau=1:2^Splits
                PCAPlots = plot!(PCA_Orbits_2Gen[i,tau,1,1:k],PCA_Orbits_2Gen[i,tau,2,1:k],PCA_Orbits_2Gen[i,tau,3,1:k],legend=false,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)
            end
        end

        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 5
    gif(anim, Name, fps = 15)
    
end

function adjoint(a)
    return a'
end


function Coarsed_Variance(Variance_Length,Time_Orbital_Collection,Coarsing,Orbit_Length)
    
    
    #Variance[i] = sum((Time_Orbital_Collection[i,:,:] .- mean(Time_Orbital_Collection[i,:,:],dims=1)).^2)/(M-1)

    
    
    Variance = zeros(Variance_Length)
    Average = zeros(Variance_Length,M)
    
    for i = 1:Variance_Length #For all different times
       Average[i,:] =  mean(Time_Orbital_Collection[i,:,:],dims=1)[1,:]
        for k=1:Orbit_Length #Have to do it for all the orbits
            Temp_Var = 1000
            Add_Var = 0
            
            for j=max(1,i-Coarsing):i+Coarsing 
                Add_Var = sum((Time_Orbital_Collection[j,k,:] .- Average[i,:]).^2)/(M-1)

                if Add_Var < Temp_Var
                    Temp_Var = Add_Var               
                end
            end
            Variance[i] += Temp_Var  

        end
    end
    
    return Variance
end


function Multi_One_Generational_Plotter(Cut_Off,Duration,Length_Orbits,Opacity,Name,Amount_Perturbations,Partial,N_Perturbed)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1:N_Perturbed*Amount_Perturbations

            if Shortest>size(Orbit_Array[i])[2]
                Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
            end
            Total_Length += size(Orbit_Array[i])[2]
        
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed*Amount_Perturbations)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1:N_Perturbed*Amount_Perturbations
        
        if Cut_Off

            for p=0:2^Splits-1

                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
            end
        else
            Length_Orbits =  size(Orbit_Array[i,j])[2]
            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
            end
        end

        Collection_Index += Length_Orbits*2^Splits

    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,N_Perturbed*2^Splits,Amount_Perturbations,3,Shortest)
    PCA_Orbits_Partial = zeros(Float32,Int(N_Perturbed/Partial)*2^Splits,Amount_Perturbations,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:N_Perturbed
        for j=0:Amount_Perturbations-1
        for tau=1:2^Splits
            tester = Orbit_Array[i+N_Perturbed*j][2+(tau-1)*M:1+tau*M,1:Shortest]
            Transformed = transform(Orbital_Model,tester)
            PCA_Orbits_2Gen[(i-1)*2^Splits+tau,j+1,:,:] = Transformed
            if mod(i,Partial) == 0
                PCA_Orbits_Partial[(Int(i/Partial)-1)*2^Splits+tau,j+1,:,:] = Transformed
            end
        end
        end

    end
    Colours = [:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
    
    #PCA_Orbits_Partial = reshape(PCA_Orbits_Partial,(Int(2^Splits*N_Perturbed*Amount_Perturbations/Partial),3,Shortest))
    PCA_Orbits_2Gen_Plot = permutedims(PCA_Orbits_Partial,(4,3,2,1))
    Orbit_Range = Int(N_Perturbed/Partial)*2^Splits
    tick()
    anim = @animate for k=1:Int(Duration/5)
        
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,:],PCA_Orbits_2Gen_Plot[1:k,2,1,:],PCA_Orbits_2Gen_Plot[1:k,3,1,:],c=Colours[1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for p=1:Amount_Perturbations
            PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,:],PCA_Orbits_2Gen_Plot[1:k,2,p,:],PCA_Orbits_2Gen_Plot[1:k,3,p,:],c=Colours[p],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)

        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 5
    tock()
    tick()
    animlong = @animate for k=1:Duration
        
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,:],PCA_Orbits_2Gen_Plot[1:k,2,1,:],PCA_Orbits_2Gen_Plot[1:k,3,1,:],c=Colours[1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for p=1:Amount_Perturbations
            PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,:],PCA_Orbits_2Gen_Plot[1:k,2,p,:],PCA_Orbits_2Gen_Plot[1:k,3,p,:],c=Colours[p],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)

        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 50
    tock()
    abra = gif(anim, Name, fps = 10)
    display(abra)

    kadabra = gif(animlong, Name[1:end-4] * "_Long" *Name[end-3:end], fps = 3)
    display(kadabra)



end


function Multi_One_Generational_Plotter2(Cut_Off,Duration,Length_Orbits,Opacity,Name,Amount_Perturbations,Partial,N_Perturbed,Orbit_Array)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1:N_Perturbed*Amount_Perturbations

            if Shortest>size(Orbit_Array[i])[2]
                Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
            end
            Total_Length += size(Orbit_Array[i])[2]
        
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed*Amount_Perturbations)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1:N_Perturbed*Amount_Perturbations
        
        if Cut_Off

            for p=0:2^Splits-1

                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
            end
        else
            Length_Orbits =  size(Orbit_Array[i,j])[2]
            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
            end
        end

        Collection_Index += Length_Orbits*2^Splits

    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,N_Perturbed*2^Splits,Amount_Perturbations,3,Shortest)
    PCA_Orbits_Partial = zeros(Float32,Int(N_Perturbed/Partial)*2^Splits,Amount_Perturbations,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:N_Perturbed
        for j=0:Amount_Perturbations-1
        for tau=1:2^Splits
            tester = Orbit_Array[i+N_Perturbed*j][2+(tau-1)*M:1+tau*M,1:Shortest]
            Transformed = transform(Orbital_Model,tester)
            PCA_Orbits_2Gen[(i-1)*2^Splits+tau,j+1,:,:] = Transformed
            if mod(i,Partial) == 0
                PCA_Orbits_Partial[(Int(i/Partial)-1)*2^Splits+tau,j+1,:,:] = Transformed
            end
        end
        end

    end
    Colours = [:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
    
    #PCA_Orbits_Partial = reshape(PCA_Orbits_Partial,(Int(2^Splits*N_Perturbed*Amount_Perturbations/Partial),3,Shortest))
    PCA_Orbits_2Gen_Plot = permutedims(PCA_Orbits_Partial,(4,3,2,1))
    Orbit_Range = Int(N_Perturbed/Partial)*2^Splits
    tick()
    anim = @animate for k=1:Int(Duration/5)
        
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,:],PCA_Orbits_2Gen_Plot[1:k,2,1,:],PCA_Orbits_2Gen_Plot[1:k,3,1,:],c=Colours[1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for p=1:Amount_Perturbations
            PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,:],PCA_Orbits_2Gen_Plot[1:k,2,p,:],PCA_Orbits_2Gen_Plot[1:k,3,p,:],c=Colours[p],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)

        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 5
    tock()
    tick()
    animlong = @animate for k=1:Duration
        
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,:],PCA_Orbits_2Gen_Plot[1:k,2,1,:],PCA_Orbits_2Gen_Plot[1:k,3,1,:],c=Colours[1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for p=1:Amount_Perturbations
            PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,:],PCA_Orbits_2Gen_Plot[1:k,2,p,:],PCA_Orbits_2Gen_Plot[1:k,3,p,:],c=Colours[p],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)

        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 50
    tock()
    abra = gif(anim, Name, fps = 10)
    display(abra)

    kadabra = gif(animlong, Name[1:end-4] * "_Long" *Name[end-3:end], fps = 3)
    display(kadabra)

    
    PCAPlots_Final = scatter(PCA_Orbits_2Gen_Plot[end,1,1,:],PCA_Orbits_2Gen_Plot[end,2,1,:],PCA_Orbits_2Gen_Plot[end,3,1,:],title="Final Cell States",c=Colours[1],legend=false)
    for p=1:Amount_Perturbations
        PCAPlots_Final = scatter!(PCA_Orbits_2Gen_Plot[end,1,p,:],PCA_Orbits_2Gen_Plot[end,2,p,:],PCA_Orbits_2Gen_Plot[end,3,p,:],c=Colours[p],legend=false)

    end

    display(PCAPlots_Final)
    savefig(Name[1:end-4] * "_Final.png")
end

function Multi_One_Generational_Plotter3(Cut_Off,Duration,Length_Orbits,Opacity,Name,Amount_Perturbations,Partial)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1:N_Perturbed*Amount_Perturbations

            if Shortest>size(Orbit_Array[i])[2]
                Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
            end
            Total_Length += size(Orbit_Array[i])[2]
        
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed*Amount_Perturbations)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1:N_Perturbed*Amount_Perturbations
        
        if Cut_Off

            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
            end
        else
            Length_Orbits =  size(Orbit_Array[i,j])[2]
            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
            end
        end

        Collection_Index += Length_Orbits*2^Splits

    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,N_Perturbed*2^Splits,Amount_Perturbations,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:N_Perturbed
        for j=1:2^Splits
            for tau=1:Amount_Perturbations
                tester = Orbit_Array[i+ (tau-1)*N_Perturbed][2+(j-1)*M:1+j*M,1:Shortest]
                Transformed = transform(Orbital_Model,tester)
                PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,:,:] = Transformed

            end
        end
    end
    Colours = [:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
    
    PCA_Orbits_2Gen_Plot = permutedims(PCA_Orbits_2Gen,(4,3,2,1))
    anim = @animate for k=1:Duration
        
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1],c=Colours[1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for l=1:Amount_Perturbations
            
            PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,l,:],PCA_Orbits_2Gen_Plot[1:k,2,l,:],PCA_Orbits_2Gen_Plot[1:k,3,l,:],c=Colours[l],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
            
        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 15

    abra = gif(anim, Name, fps = 15)
end

function N_Dimensional_Euclid(Position1,Position2,Dimensions)
    Summed = 0
    for i =1:Dimensions
        Summed += (Position2[i]-Position1[i])^2
    end
    Distance = sqrt(Summed)
    return Distance
end



function Distance_Movement_Matrix(Movement_Matrix)
    
    for i=1:N_Sources #we must loop over all the sources to find a good sink for it

        #First we try to find the closest sinks
        Distance_Indices = (hcat(DistanceMatrix[i,:], 1:N_Sinks))
        Distance_Indices = Distance_Indices[sortperm(Distance_Indices[:, 1]), :]
        Distance_Indices = Int.(Distance_Indices[:,2])
        print(Distance_Indices)

        #Then we check if we can still send anything to this sink
        Displacing = true
        D_Index = 1
        while Displacing
            I_Sink = Distance_Indices[D_Index] #This is the index of the sink we are checking now
            Available_Sink = Sinks_Input[I_Sink] - sum(Movement_Matrix[:,I_Sink])
            print(Sinks_Input[I_Sink])
            if Available_Sink > 0 #It can have more added to it
                #We check if we can send everything to the sink or only partial
                Remaining_Flow = Sources_Output[i] - sum(Movement_Matrix[i,:])
                if Remaining_Flow > Available_Sink
                #If partial we fill a bit and go back to the start to find the next closest sink
                    Movement_Matrix[i,I_Sink] = Available_Sink
                    D_Index += 1
                    print("Partial.     ")
                    #BUT WE HAVE TO CHECK IF WE JUST FILLED IT ALL IN!!!! 
                    if Sources_Output[i] - sum(Movement_Matrix[i,:]) < 0.0001
                        Displacing = false
                        print("Finished After Partial")
                    end

                else  
                #If we can send it all we do and go to the next Source i+1
                    Displacing = false
                    Movement_Matrix[i,I_Sink] = Remaining_Flow
                    print("Finished.     ")

                end
            else #Sink is full
                print("Full.     ")

                #Cannot send more in the sink so we have to check the next one
                D_Index += 1
            end
        end
    end
    Movement_Matrix = round.(Movement_Matrix,digits=4)

    return Movement_Matrix
end

function Cost_Reducing_Rewiring(Attempts,Optimised_Indices)
    print("Initial Costs: ")
    print(sum(Cost_Matrix))
    for yota =1:Attempts
        println(" ")
        #We make a big while loop until a certain condition we decide later
        Cost_Matrix = DistanceMatrix .* Movement_Matrix
        print(sum(Cost_Matrix))
        for theta =1:1   #Old loop that can be removed

            if Optimised_Indices != []
                for kappa=1:length(Optimised_Indices[1,:])
                    Cost_Matrix[Optimised_Indices[1,kappa],Optimised_Indices[2,kappa]] = 0
                end
            end

            #We find the connection that has the highest cost, find the source and the corresponding sink
            Source_Index = argmax(Cost_Matrix)[1]
            Sink_Index = argmax(Cost_Matrix)[2]
            Improvement = 0 #Use this to keep track of best
            Displace = 0 #Keep track of best
            Best_Source = 0 #Keep track
            Best_Sink = 0 #Keep track

            #We let this source connect to other sinks instead and try to change another source that was connected to that sink
            for i=1:N_Sinks #We loop over all other sinks
                if i!=Sink_Index #Make sure it is a different sink
                    for j=1:N_Sources #We loop over all other Sources
                        if j!=Source_Index #Make sure it is a different source
                            #Check how much we may change 
                            Displace_Temp = min(Movement_Matrix[Source_Index,Sink_Index],Movement_Matrix[j,i])
                            #Calculate the improvement
                            #-DM[So,Si] + DM[So,i] We remove So -> Si and make So -> i
                            #-DM[j,i] + DM[j,Si] We remove j -> i and make j -> Si
                            Temp_Improvement = Displace_Temp*(-DistanceMatrix[Source_Index,Sink_Index] + DistanceMatrix[Source_Index,i] -DistanceMatrix[j,i] + DistanceMatrix[j,Sink_Index])

                            #Find the best improvement for this initial Source_Sink combo
                            if Temp_Improvement < Improvement #Found a better improvement
                                Improvement = Temp_Improvement
                                Displace = Displace_Temp
                                Best_Source = j
                                Best_Sink = i
                            end

                        end
                    end
                end
            end
            print("Better amount:  ")
            print(Improvement)
            if Improvement==0
                print("No improvement???")
                print(Displace)
                print("    ")
                print(Sink_Index)
                print(Source_Index)
                if Optimised_Indices == []
                    Optimised_Indices = vcat(Optimised_Indices,[Source_Index;Sink_Index])
                else
                    Optimised_Indices = hcat(Optimised_Indices,[Source_Index;Sink_Index])
                end

            else
                print("WE IMPROVED")
                Movement_Matrix[Source_Index,Sink_Index] -= Displace
                Movement_Matrix[Best_Source,Best_Sink] -= Displace
                Movement_Matrix[Best_Source,Sink_Index] += Displace
                Movement_Matrix[Source_Index,Best_Sink] += Displace


            end

            #Switch the best one and repeat
        end
        print("   ")
        Cost_Matrix = DistanceMatrix .* Movement_Matrix
        print(sum(Cost_Matrix))
    end

    println(" ")
    print("Final Costs: ")
    print(sum(Cost_Matrix)) 
    return sum(Cost_Matrix)
end

function Final_Cell_PCAScatterPlot(Orbit_Array,Single_Plot)
    Final_Cell_States_PCA = zeros(Amount_Large_Perturbations*Amount_Small_Perturbations*2^Splits,M)
    Transformed_Data = zeros(Amount_Small_Perturbations*2^Splits,Amount_Large_Perturbations,2)
    Before_Data = zeros(Amount_Small_Perturbations*2^Splits,Amount_Large_Perturbations,M)
    for j=1:Amount_Large_Perturbations*Amount_Small_Perturbations
        for i=1:2^Splits
            Final_Cell_States_PCA[j+Amount_Large_Perturbations*Amount_Small_Perturbations*(i-1),:] = Orbit_Array[j][2+(i-1)*M:1+i*M,end]
        end
    end
    #This is added in in case we only have the same cell types, then our code would crash
    Final_Cell_States_PCA[1,1] += 1
    
    
    for j=1:Amount_Large_Perturbations
        for i=1:Amount_Small_Perturbations
            for tau=1:2^Splits
                Before_Data[i+(tau-1)*Amount_Small_Perturbations,j,:] = Orbit_Array[i+(j-1)*Amount_Small_Perturbations][2+(tau-1)*M:1+tau*M,end]
            end
        end
    end

    Final_Cell_States_PCA[1:5,1:10] .+= 1
    Final_Cell_PCA = fit(PCA,Final_Cell_States_PCA';maxoutdim =2,pratio =1)
    print(size(Final_Cell_PCA))
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Final_Cell_PCA))
    Final_Cell_States_PCA[1:5,1:10] .-= 1
 

    for j=1:Amount_Large_Perturbations

        Transformed_Data[:,j,:] = transform(Final_Cell_PCA,Before_Data[:,j,:]')'

    end
    Plot_Legend = true
    Colours = [:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
    if Single_Plot == false
        j=1
        abrada = scatter((Transformed_Data[1,j,1],Transformed_Data[1,j,2]),xlabel="PCA 1",ylabel="PCA 2",title="Final Cell States in PCA landscape",c=Colours[j],alpha=0.4,legend=Plot_Legend,label="Initial Noise = " * string(Large_Pert_Radius[j]))
        for j=2:Amount_Large_Perturbations
            abrada = scatter!((Transformed_Data[1,j,1],Transformed_Data[1,j,2]),c=Colours[j],alpha=0.4,legend=Plot_Legend,label="Initial Noise = " * string(Large_Pert_Radius[j]))
        end
        for j=1:Amount_Large_Perturbations
            for l=2:length(Transformed_Data[:,j,1])
                for j=1:Amount_Large_Perturbations
                    abrada = scatter!((Transformed_Data[l,j,1],Transformed_Data[l,j,2]),c=Colours[j],alpha=0.4,label=false)
                end
            end
        end
        savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/PCA_Maps/Scatter_PCA"*Copy_String*".png")

        KernelData = reshape(Transformed_Data, (Amount_Small_Perturbations*2^Splits*Amount_Large_Perturbations, 2))
        
        k = kde((KernelData[:,1], KernelData[:,2]))


        display(contourf(k,
             c = :vik,title="Final Cell State PCA Heatmap",
             linewidth = 0))
        savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/PCA_Maps/Heatmap_"*Copy_String*".png")

        
    else
        j = Single_Plot
        abrada = scatter((Transformed_Data[:,j,1],Transformed_Data[:,j,2]),xlabel="PCA 1",ylabel="PCA 2",title="Final Cell States in PCA landscape",c=Colours[j],alpha=0.4,legend=Plot_Legend,label="Initial Noise = " * string(Large_Pert_Radius[j]))
        
    end
    return abrada
end



function Multi_One_Generational_Plotter4(Cut_Off,Duration,Length_Orbits,Opacity,Name,Amount_Perturbations,Partial,N_Perturbed,Orbit_Array,Final_Epigenetic_Factors,Gene_Colour_Collection)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1:N_Perturbed*Amount_Perturbations

            if Shortest>size(Orbit_Array[i])[2]
                Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
            end
            Total_Length += size(Orbit_Array[i])[2]
        
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed*Amount_Perturbations)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1:N_Perturbed*Amount_Perturbations
        
        if Cut_Off

            for p=0:2^Splits-1

                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
            end
        else
            Length_Orbits =  size(Orbit_Array[i,j])[2]
            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
            end
        end

        Collection_Index += Length_Orbits*2^Splits

    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,Amount_Perturbations,N_Perturbed,2^Splits,3,Shortest)
    PCA_Orbits_Partial = zeros(Float32,Amount_Perturbations,Int(N_Perturbed/Partial),2^Splits,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:N_Perturbed
        for j=0:Amount_Perturbations-1
        for tau=1:2^Splits
            tester = Orbit_Array[i+N_Perturbed*j][2+(tau-1)*M:1+tau*M,1:Shortest]
            Transformed = transform(Orbital_Model,tester)
            PCA_Orbits_2Gen[j+1,i,tau,:,:] = Transformed
            if mod(i-1,Partial) == 0
                PCA_Orbits_Partial[j+1,(Int(i/Partial)),tau,:,:] = Transformed
            end
        end
        end

    end
    Colours = [:red,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
     
    #PCA_Orbits_Partial = reshape(PCA_Orbits_Partial,(Int(2^Splits*N_Perturbed*Amount_Perturbations/Partial),3,Shortest))
    PCA_Orbits_2Gen_Plot = permutedims(PCA_Orbits_Partial,(5,4,3,2,1))
    Orbit_Range = Int(N_Perturbed/Partial)*2^Splits
    
    print(size(PCA_Orbits_2Gen_Plot))
    if false
        for Gene_Colour in Gene_Colour_Collection
            tick()
            
            anim = @animate for k=1:Int(Duration/5)
                i=1
                PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1,1],c=Colours[Final_Epigenetic_Factors[1,1,1,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                for q=1:Int(N_Perturbed/Partial)
                    for p=1:2^Splits
                        for t =1:Amount_Perturbations
                        PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,q,t],PCA_Orbits_2Gen_Plot[1:k,2,p,q,t],PCA_Orbits_2Gen_Plot[1:k,3,p,q,t],c=Colours[Final_Epigenetic_Factors[t,q,p,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                        end
                    end
                end
        
                #for i=1:Int(N_Perturbed/5)
                #    for j=1:Amount_Perturbations
                #        for tau=1:2^Splits
                #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                #        end
                #    end
                #end
                
                #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])
        
                title!("Gene = $Gene_Colour Time = $k")
            end every 5
            tock()
            tick()
            animlong = @animate for k=1:Duration
                
                i=1
                PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1,1],c=Colours[Final_Epigenetic_Factors[1,1,1,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                for q=1:Int(N_Perturbed/Partial)
                    for p=1:2^Splits
                        for t =1:Amount_Perturbations
                        PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,q,t],PCA_Orbits_2Gen_Plot[1:k,2,p,q,t],PCA_Orbits_2Gen_Plot[1:k,3,p,q,t],c=Colours[Final_Epigenetic_Factors[t,q,p,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                        end
                    end
                end
                #for i=1:Int(N_Perturbed/5)
                #    for j=1:Amount_Perturbations
                #        for tau=1:2^Splits
                #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                #        end
                #    end
                #end
                
                #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])
        
                title!("Gene = $Gene_Colour Time = $k")
            end every 50
            tock()
    
            abra = gif(anim, Name[1:end-4]*"_Gene:"*string(Gene_Colour)*Name[end-3:end], fps = 10)
            display(abra)
        
            kadabra = gif(animlong, Name[1:end-4] * "_Long"*"_Gene:"*string(Gene_Colour) *Name[end-3:end], fps = 3)
            display(kadabra)
        
            
            Total_Ratio = principalratio(Orbital_Model)
            Eigen_Values = eigvals(Orbital_Model)
            Eigen_Values = Eigen_Values/sum(Eigen_Values)
            Eigen1 = Eigen_Values[1]
            Eigen2 = Eigen_Values[2]
            Eigen3 = Eigen_Values[3]
            PCAPlots_Final = scatter(PCA_Orbits_2Gen_Plot[end,1,1,1,:],PCA_Orbits_2Gen_Plot[end,2,1,1,:],PCA_Orbits_2Gen_Plot[end,3,1,1,:],c=Colours[Final_Epigenetic_Factors[1,1,1,Gene_Colour]+1],legend=true,label="
            Principal 1: $Eigen1
            Principal 2: $Eigen2]
            Principal 3: $Eigen3")
        
            for q=1:Int(N_Perturbed/Partial)
                for p=1:2^Splits
                    for t =1:Amount_Perturbations
                        PCAPlots_Final = scatter!((PCA_Orbits_2Gen_Plot[end,1,p,q,t],PCA_Orbits_2Gen_Plot[end,2,p,q,t],PCA_Orbits_2Gen_Plot[end,3,p,q,t]),c=Colours[Final_Epigenetic_Factors[t,q,p,Gene_Colour]+1],legend=true,label="")
                    end
                end
            end
            
            
            title!("Gene = $Gene_Colour , Ratio: $Total_Ratio")
            display(PCAPlots_Final)
            savefig(Name[1:end-4] * "_Final_Static_$Gene_Colour.png")
            
        end
    end
    Multi_Colours = [:cyan,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:darkmagenta,:lightcyan4,:olive,:gold4,:lightsalmon,:navajowhite,:pink]
    Only3Types = false
    tick()
    anim = @animate for k=1:Int(Duration/5)
        
        i=1
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
        if Only3Types
            Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
        end
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1,1],c=Multi_Colours[Colour_Index],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for q=1:Int(N_Perturbed/Partial)
            for p=1:2^Splits
                for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,q,t],PCA_Orbits_2Gen_Plot[1:k,2,p,q,t],PCA_Orbits_2Gen_Plot[1:k,3,p,q,t],c=Multi_Colours[Colour_Index],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                end
            end
        end
    
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])
    
        title!("Gene = Multiple Time = $k")
    end every 5
    tock()
    tick()
    animlong = @animate for k=1:2*Duration
        
        i=1
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
        if Only3Types
            Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
        end
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1,1],c=Multi_Colours[Colour_Index],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for q=1:Int(N_Perturbed/Partial)
            for p=1:2^Splits
                for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,q,t],PCA_Orbits_2Gen_Plot[1:k,2,p,q,t],PCA_Orbits_2Gen_Plot[1:k,3,p,q,t],c=Multi_Colours[Colour_Index],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                end
            end
        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])
    
        title!("Gene = Multiple Time = $k")
    end every 50
    tock()
    abra = gif(anim, Name[1:end-4]*Name[end-3:end], fps = 10)
    display(abra)
    
    kadabra = gif(animlong, Name[1:end-4]*Name[end-3:end], fps = 3)
    display(kadabra)
    
    
    
    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    PCAPlots_Final = scatter(PCA_Orbits_2Gen_Plot[end,1,1,1,:],PCA_Orbits_2Gen_Plot[end,2,1,1,:],PCA_Orbits_2Gen_Plot[end,3,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Final = scatter!((PCA_Orbits_2Gen_Plot[end,1,p,q,t],PCA_Orbits_2Gen_Plot[end,2,p,q,t],PCA_Orbits_2Gen_Plot[end,3,p,q,t]),c=Multi_Colours[Colour_Index],legend=false)
            end
        end
    end
    Total_Ratio = principalratio(Orbital_Model)
    Eigen_Values = eigvals(Orbital_Model)
    Eigen_Values = Eigen_Values/sum(Eigen_Values)
    title!("Static Multi Colour, Ratio: $Total_Ratio")
    display(PCAPlots_Final)
    savefig(Name[1:end-4] * "Final_Multi_Colour.png")



    
    PcaPlots_Time_Length = 500
    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    
    PCAPlots_Time1 = plot(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,1,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Time1 = plot!(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,1,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA 1")
            end
        end
    end
    display(PCAPlots_Time1)

    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    
    PCAPlots_Time2 = plot(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,2,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Time2 = plot!(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,2,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA 1")
            end
        end
    end
    display(PCAPlots_Time2)

    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    
    PCAPlots_Time3 = plot(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,3,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Time3 = plot!(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,3,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA 1")
            end
        end
    end
    display(PCAPlots_Time3)
    return Orbital_Model
end

function Evolution_NoEpi(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    
    
    
    #⊗_(Wi, bi) = Wi .* bi
       

    du[1,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,:].+u[2,:].+CArray[:])) .- u[1,:]
    
    #Epigenetic Evolution, no need to worry about boundaries here so simple loop      

    
end




##############################################################################################################################
##############################################################################################################################
##############################################################################################################################








#Load_Generations = [70]
#Load_Generations = vcat(1,Load_Generations) #These are all the generations that we will analyse
Copies = [1] #Can reduce this to just 1,2 or just 1 if we don't want as many copies



#Main variables to change:
Stochastic_Noise = true
N::Int32 = 8 #How many copies of same network we have
#K::Int32 = N  Depricated parameter, do not change, if ever K not defined replace it by N
P = 4 #How many different networks will we use in the evolution run
Generations = 6 #How many generations in the evolution run

#DO NOT CHANGE THESE
Splits::Int32 = 6 #DO NOT CHANGE
SplitTime::Float32 = 0.1 #Time before a splitting event happens, DOES NOT MATTER HERE
M::Int32 = 40 #This is the amount of genes we want in our network
Splitting_Strength = 0.0f0

#CAREFUL WITH M>127 DO NOT INCREASE WITHOUT REWRITING DIFFERENTIATION FUNCTION
#BIT OVERFLOW!!!!!


#Constant and plotting tools


InvSqr::Float32 = 1/sqrt(M)
PCA_Plotting = false 
Orbit_Plotting = true  #Currently not used yet
Epi_Plotting = false #Currently not used yet
Counter = 0 #Used to randomise the mutations, leave to keep same seed
Saved_Generations = [2,4] #These are the generations where we want to save some orbits for


#Diffusion or mean field parameters
Mean_Field = false
Diffusing_System = false
KArray = zeros(Float32,M,1) #Diffusion/Mean Field array is set to zero everywhere
#KArray[1:10] .= 0.1 #Here we can manually adjust which genes will have diffusion















Amount_Large_Perturbations = 1 #How many large perturbations we will have each time
Amount_Small_Perturbations = 1 #How many smaller perturbations we will have around our large perturbation
Large_Pert_Radius = [0,1,5,20]
Small_Pert_Radius = 0
#Perturbations = zeros(32)









Generation_String = string(Load_Generations[1])
Copy_String = string(Copies[1])
Load_Data_String_Load = Data_To_Analyse
#Load_Data_String = Data_To_Analyse
#Copies = [1]
#Load_Generations = [10]
Orbit_Array = 0
Time_Orbital_Collection2 = 0
Orbit_Collection = 0
EpiThroughTime = 0

#To have access after the loop
Interpolated_Orbits = 0
Final_Epigenetic_Factors = 0
Gene_Cells_Colour = 0
Orbital_Model = 0
@showprogress 1 for zeta in Load_Generations
    GC.gc()
    for iota in Copies
        Generation_String = string(zeta)

        Copy_String = string(iota)
        Current_Network_String = "/Gen_"*Generation_String*"_"*Copy_String*".jld"
        Loaded_Data = load(Load_Data_String_Load*Current_Network_String)
        mkpath(Load_Data_String*"/Processed/Gen:"*Generation_String*"/PCA_Maps")
        mkpath(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Orbits")
        mkpath(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance")


        #Here we load and perturb the networks


        Duration = 1000 #For Animation should be 3000
        Length_Orbits = 3000 #For PCA should be 4000
        Opacity = 75
        Partial = 1

        Variance_Length = 2500
        Times = [i for i=1:Variance_Length+50]
        Times_Orbit = [i for i=1:Length_Orbits]


        Perturb_Seed = 12345
        rng=MersenneTwister(Perturb_Seed)
        #Amount_Large_Perturbations = 1 #How many large perturbations we will have each time
        #Amount_Small_Perturbations = 16 #How many smaller perturbations we will have around our large perturbation
        #Large_Pert_Radius = 10
        #Small_Pert_Radius = 3
        CArray_Collection = zeros(Float32,Amount_Large_Perturbations,M)
        PositionArray_Collection = zeros(Float32,Amount_Large_Perturbations,Amount_Small_Perturbations,2,1,M)
        ParameterMatrix_Collection = zeros(Int32,Amount_Large_Perturbations,M,M)
        Final_Epigenetic_Factors = zeros(Int8,Amount_Large_Perturbations,Amount_Small_Perturbations,2^(Splits),M)
        Epi_Factors = zeros(Float16,2^(Splits),M)
        for j=1:Amount_Large_Perturbations
            CArray_Collection[j,:] = Loaded_Data["CArray"]
            ParameterMatrix_Collection[j,:,:] = Loaded_Data["Network"]

            #Write large pert vector here

            Noise_Vector = randn(rng,M).-0.5
            Norm = sqrt(sum(Noise_Vector.^2))
            Noise_Vector = Large_Pert_Radius[j]*Noise_Vector/Norm
            for i=1:Amount_Small_Perturbations


                #Small pert vector here
                Noise_Vector2 = randn(rng,M).-0.5
                Norm = sqrt(sum(Noise_Vector2.^2))
                Noise_Vector2 = rand()^(1/M)*Small_Pert_Radius*Noise_Vector2/Norm

                PositionArray_Collection[j,i,1,1,:] = Loaded_Data["PositionArray"] + Noise_Vector' + Noise_Vector2'

            end
        end

        Evolutionary_Complexity = []


        Random_Noise = 574 #seed for splitting noise

        #Need to define outside for loop, otherwise it will be destroyed afterwards
        Genes = 0





        KLDistance_temp = 0
        BitDistance_temp =  0
        Evolutionary_Complexity = []
        Indices = 0
        Information = zeros(1)
        Ratio = zeros(1)
        Cell_Type_Tracker = []
        Cell_Collection = []
        Evolutionary_Complexity = zeros(N)
        Temporary = []
        Orbit_Array = Array{Matrix{Float16}}(undef,Amount_Large_Perturbations*Amount_Small_Perturbations)
        Interpolated_Orbits = Array{Matrix{Float16}}(undef,Amount_Large_Perturbations*Amount_Small_Perturbations)

        Time_Orbital_Collection = 0

        for yeta=1:Amount_Large_Perturbations
            for x=1:Amount_Small_Perturbations
                #CArray = CArray_Collection[x,:]
                #ParameterMatrix = ParameterMatrix_Collection[x,:,:]
                #PositionArray = PositionArray_Collection[x,:]
                Epi_Factors,Random_Noise,Temporary, EpiThroughTime = Running_Network(CArray_Collection[yeta,:],ParameterMatrix_Collection[yeta,:,:],PositionArray_Collection[yeta,x,:,:,:],Random_Noise)
                Final_Epigenetic_Factors[yeta,x,:,:] = signbit.(Epi_Factors)
                Orbit_Array[x+(yeta-1)*Amount_Small_Perturbations] = Temporary

            end

        end
        #Indices,AllIndices,Evolutionary_Complexity[:],Cell_Collection, Cell_Type_Tracker,KLDistance_temp,BitDistance_temp = Scoring_Indexes(N,Final_Epigenetic_Factors,Cell_Type_Tracker)


        #HERE WE DO THE VARIANCE DATA ANALYSIS

        Orbit_Collection = Orbit_Array

        #Collect all orbits in function of time,


        #Maybe apply PCA to these orbits or keep them like this
        Time_Orbital_Collection = zeros(Float16,length(Times),Amount_Small_Perturbations*2^Splits*Amount_Large_Perturbations,M)
        Time_Orbital_Collection2 = zeros(Float16,length(Times),Amount_Large_Perturbations,Amount_Small_Perturbations*2^Splits,M)



        Orbit_Counter=0
        Larger_Orbit_Counter = 1
        for Orbits in Orbit_Collection
            #Here we do it for the variance
            Interpolated_Vector = Interpolator(Times,Orbits) #We do linear interpolation to have all the same timesteps
            for i=1:2^Splits #We put the different splitted cells all together with the same networks.
                Time_Orbital_Collection[:,Orbit_Counter*2^Splits+i,:] = Interpolated_Vector[:,2+(i-1)*M:1+i*M]
            end
            for k=1:2^Splits
                Time_Orbital_Collection2[:,Larger_Orbit_Counter,mod(Orbit_Counter,Amount_Small_Perturbations)+k,:] = Interpolated_Vector[:,2+M*(k-1):1+M*k]
            end 
            Orbit_Counter += 1
            if mod(Orbit_Counter,Amount_Small_Perturbations) == 0
                Larger_Orbit_Counter += 1
            end


            #Here we do it for the orbits
            Interpolated_Vector2 = Interpolator(Times_Orbit,Orbits)
            Interpolated_Orbits[Orbit_Counter] = permutedims(Interpolated_Vector2,[2,1])

        end
        #Calculate variance in gene expression in function of time



        Variance = zeros(Variance_Length)
        for i=1:Variance_Length
            Variance[i] = sum((Time_Orbital_Collection[i,:,:] .- mean(Time_Orbital_Collection[i,:,:],dims=1)).^2)/(M-1)
        end



        Spatial_Var = Coarsed_Variance(Variance_Length,Time_Orbital_Collection,10,Amount_Large_Perturbations*Amount_Small_Perturbations*2^Splits)
        aba = plot(Variance,label="Temporal",xlabel="Time",ylabel="Variance",title="Spatial and Temporal Variance of all Orbits")
        aba = plot!(Spatial_Var,label="Spatial")
        aba =vline!(SplitTime*[i for i=1:Splits],colour=:"red",linealpha=:0.3,label=false)
        display(aba)
        
        savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance/Spatial_Temporal_"*Copy_String*".png")
        #savefig("Data_All/Data_Folder/TestFile/TestImage.png")



        aca = plot(Variance[1:250],label="Temporal",xlabel="Time",ylabel="Variance",title="Spatial and Temporal Variance of all Orbits")
        aca = plot!(Spatial_Var[1:250],label="Spatial")
        aca =vline!(SplitTime*[i for i=1:Splits],colour=:"red",linealpha=:0.3,label=false)
        ylims!(0,max(maximum(Variance[3:250]),maximum(Spatial_Var[3:250]))*1.3)

        display(aca)
        
        savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance/Spatial_Temporal_Zoomed_"*Copy_String*".png")


        if false

            Seperate_Variance = zeros(Amount_Large_Perturbations,Variance_Length)
            for j=1:Amount_Large_Perturbations
                for i=1:Variance_Length
                    Seperate_Variance[j,i] = sum((Time_Orbital_Collection2[i,j,:,:] .- mean(Time_Orbital_Collection2[i,j,:,:],dims=1)).^2)/(M-1)
                end

            end

            ada = plot(Seperate_Variance[1,1:250],label=false,xlabel="Time",ylabel="Variance",title="Temporal Variance of Specific I.C.")
            for j=2:Amount_Large_Perturbations
                ada = plot!(Seperate_Variance[j,1:250],label=false)
            end
            display(ada)

            savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance/Temporal_Zoomed_IC_"*Copy_String*".png")


            aea = plot(Seperate_Variance[1,1:2500],label=false,xlabel="Time",ylabel="Variance",title="Temporal Variance of Specific I.C.")
            for j=2:Amount_Large_Perturbations
                aea = plot!(Seperate_Variance[j,1:2500],label=false)
            end
            display(aea)

            savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance/Temporal_IC_"*Copy_String*".png")

            Spatial_Sep = Coarsed_Variance(Variance_Length,Time_Orbital_Collection[:,1:Amount_Small_Perturbations*2^Splits,:],25,Amount_Small_Perturbations*2^Splits)
            afa = plot(Spatial_Sep[1:250],label=false,xlabel="Time",ylabel="Variance",title="Spatial Variance of Specific I.C.")
            for j=1:Amount_Large_Perturbations-1
                Spatial_Sep = Coarsed_Variance(Variance_Length,Time_Orbital_Collection[:,1+j*Amount_Small_Perturbations*2^Splits:(j+1)*Amount_Small_Perturbations*2^Splits,:],25,Amount_Small_Perturbations*2^Splits)

                afa = plot!(Spatial_Sep[1:250],label=false)
            end

            display(afa)

            savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance/Spatial_Zoomed_IC_"*Copy_String*".png")
        end




        #Perturbations = [0 for i=1:32]
        for kappa=1:Amount_Large_Perturbations
            println(kappa)
            display(Final_Cell_PCAScatterPlot(Orbit_Array,kappa)) #Don't really need this one
        end
        
        display(Final_Cell_PCAScatterPlot(Orbit_Array,false));
        
        Gene_Cells_Colour = [1,2,3,4]
        print("Before the Multi Big One")
        GC.gc()
        Orbital_Model = (Multi_One_Generational_Plotter4(true,Duration,Length_Orbits,Opacity,Load_Data_String*"/Processed/Gen:"*Generation_String*"/Orbits/Genes_"*Copy_String*".gif",Amount_Large_Perturbations,Partial,Amount_Small_Perturbations,Interpolated_Orbits,Final_Epigenetic_Factors,Gene_Cells_Colour));
    






        Copy_String = string(1)
        Current_Network_String = "/Gen_"*Generation_String*"_"*Copy_String*".jld"
        Loaded_Data = load(Load_Data_String_Load*Current_Network_String)
        
        CArray = Loaded_Data["CArray"]
        ParameterMatrix = Loaded_Data["Network"]
        

        
        TimesWeCheck = [1,5,10,25,40,60,80,100,150,200,300,500,1000,4000,20000]
        TimeChecks = length(TimesWeCheck)
        Cell_Gatherings = zeros(TimeChecks,2,2^Splits,M)
        CycleAmount = zeros(TimeChecks)
        UniqueCycleAmount = zeros(TimeChecks)
        FixPointAmount = zeros(TimeChecks)
        
        Interpolated_Cells = Interpolator(TimesWeCheck,Orbit_Array[1])
        Interpolated_EPIs = Interpolator(TimesWeCheck,EpiThroughTime)
        for i=1:2^Splits
            Cell_Gatherings[:,1,i,:] = Interpolated_Cells[:,2+(i-1)*M:1+(i)*M]
            Cell_Gatherings[:,2,i,:] = Interpolated_EPIs[:,2+(i-1)*M:1+(i)*M]
        end
        
        
        Copy_String = string(1)
        Current_Network_String = "/Gen_"*Generation_String*"_"*Copy_String*".jld"
        Loaded_Data = load(Load_Data_String_Load*Current_Network_String)
        
        CArray = Loaded_Data["CArray"]
        ParameterMatrix = Loaded_Data["Network"]
        FixPoint_Positions = Array{Float64}(undef,0)
        Counter_Cycle = 0
        XPositionTol = 0
        
        
        
        
        for t = 1:TimeChecks
            Counter_Cycle = 0
            FixPoint_Positions = Array{Float64}(undef,0)
            VectorTest = []
            for k = 1:2^Splits
                PositionArray = Cell_Gatherings[t,:,k,:]
                
                Parameters = (ParameterMatrix, CArray) #We have to update the splits every rerun, rest stays same
                tspan = (Float32(0.0),100) #How long ODE will run
                
                prob = ODEProblem(Evolution_NoEpi,PositionArray,tspan,Parameters) #ODEdefining
                
                
                
                
                sol = solve(prob,BS3())
                aba = plot(sol.t,sol[1,1,:])
                for j=2:40
                    aba = plot!(sol.t,sol[1,j,:],legend=false,title=k)
                end
                #if t> 10
                #    display(aba)
                #end
                
                
                
                
                
                
                
                XVector = sol[1,:,end]
                EpiVector = sol[2,:,end]
                SumOfDerivates = sum(abs.(tanh.(40*(InvSqr*ParameterMatrix*XVector.+EpiVector.+CArray[:])) .- XVector))
                DerivativeTol = 0.1
                XPositionTol = 0.1
                
                #if t> 10   
                    #print(SumOfDerivates)
                #end
                
                if SumOfDerivates < DerivativeTol
                    if FixPoint_Positions == [] #Still empty
                        FixPoint_Positions = vcat(FixPoint_Positions,XVector)
                    else #Already has a point in it
                        NewFixPoint = true
                        #print("The distance to the fixpoints are:")
                        for i=1:length(FixPoint_Positions[1,:]) #Here we check over all the existing ones to see if it is not close to one already existing
                            #println(sqrt(sum((FixPoint_Positions[:,i] .- XVector).^2)))
                            if sqrt(sum((FixPoint_Positions[:,i] .- XVector).^2)) < XPositionTol
                                NewFixPoint = false
                            end    
                        end
                        if NewFixPoint #If it is a new fixpoint we add it in now
                            FixPoint_Positions = hcat(FixPoint_Positions,XVector)
                        end
                    end
                else
                    #Exception, we have a cycle? try to plot it
                    
                    CycleToleranceActive = 0.01
                    Active_Gene = zeros(M)
                    
                    prob2 = ODEProblem(Evolution_NoEpi,sol[:,:,end],(Float32(0.0),20),Parameters) #ODEdefining
                    sol2 = solve(prob2,BS3())
                    
                    for j=1:M
                        if abs(maximum(sol2[1,j,:].-mean(sol2[1,j,:]))) > CycleToleranceActive
                            Active_Gene[j] = 1
                        end
                    end
                    if VectorTest == []
                        VectorTest = vcat(VectorTest,Active_Gene)
                    else
                        New_Cycle = true
                        for tau in length(VectorTest[1,:])
                            if Active_Gene == VectorTest[:,tau]
                                New_Cycle = false
                            end
                        end
                        if New_Cycle
                            VectorTest = hcat(VectorTest,Active_Gene)
                        end
                    end
        
        
        
                    
                    Counter_Cycle += 1
                end
            end
            try FixPointAmount[t] = length(FixPoint_Positions[1,:])
        
            catch 
                FixPointAmount[t] = 0
            else
                FixPointAmount[t] = length(FixPoint_Positions[1,:])
            end
            CycleAmount[t] = Counter_Cycle
            try UniqueCycleAmount[t] = length(VectorTest[1,:])
        
            catch
                UniqueCycleAmount[t] = 0
            else
                UniqueCycleAmount[t] = length(VectorTest[1,:])
            end
        end
        
        aba = plot(TimesWeCheck,FixPointAmount,label="Fix Points")
        aba = plot!(TimesWeCheck,CycleAmount,label="Cycles (DoubleCounted)",xaxis=:log,title="PositionalTolerance="*string(XPositionTol))
        plot!(TimesWeCheck,UniqueCycleAmount,label="Cycles (Unique)")
        display(aba)

    end
end
    








In [ ]:
Gene_To_Plot = [1,2,3,4,6,30,32]
Gene_To_Plot = [i for i=1:40]
Cells = [i for i=1:8]

Orbit_Array = Interpolated_Orbits
#Multi_One_Generational_Plotter4(true,Duration,Length_Orbits,Opacity,Load_Data_String*"/Processed/Gen:"*Generation_String*"/Orbits/Genes_"*Copy_String*".gif",Amount_Large_Perturbations,Partial,Amount_Small_Perturbations,Interpolated_Orbits,Final_Epigenetic_Factors,Gene_Cells_Colour));
#Multi_One_Generational_Plotter4(Cut_Off,Duration,Length_Orbits,Opacity,Name,Amount_Perturbations,Partial,N_Perturbed,Orbit_Array,Final_Epigenetic_Factors,Gene_Colour_Collection)
Amount_Perturbations = Amount_Large_Perturbations #1
N_Perturbed = Amount_Small_Perturbations  #1
Cut_Off = true #Remove later times from the PCA dimension
Length_Orbits = 3000
Partial = 1
Total_Length = 0
Shortest = size(Orbit_Array[1,1])[2]




#In this option we take all the lengths and concatenate them, however this
#may create a bias in the PCA analysis towards only final cell states
for i=1:N_Perturbed*Amount_Perturbations

        if Shortest>size(Orbit_Array[i])[2]
            Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
        end
        Total_Length += size(Orbit_Array[i])[2]
    
end
if Cut_Off
    PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed*Amount_Perturbations)
else
    PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
end
Collection_Index=0


for i=1:N_Perturbed*Amount_Perturbations
    
    if Cut_Off

        for p=0:2^Splits-1

            PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
        end
    else
        Length_Orbits =  size(Orbit_Array[i,j])[2]
        for p=0:2^Splits-1
            PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
        end
    end

    Collection_Index += Length_Orbits*2^Splits

end


Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)

PCA_Initial_Orbit_Array = [] #Emptied to save space
PCA_Orbits_2Gen = zeros(Float32,Amount_Perturbations,N_Perturbed,2^Splits,3,Shortest)
PCA_Orbits_Partial = zeros(Float32,Amount_Perturbations,Int(N_Perturbed/Partial),2^Splits,3,Shortest)
print(" ")
print("The Principalratio is: ")
print(principalratio(Orbital_Model))
for i=1:N_Perturbed
    for j=0:Amount_Perturbations-1
    for tau=1:2^Splits
        tester = Orbit_Array[i+N_Perturbed*j][2+(tau-1)*M:1+tau*M,1:Shortest]
        Transformed = transform(Orbital_Model,tester)
        PCA_Orbits_2Gen[j+1,i,tau,:,:] = Transformed
        if mod(i-1,Partial) == 0
            PCA_Orbits_Partial[j+1,(Int(i/Partial)),tau,:,:] = Transformed
        end
    end
    end

end
Colours = [:red,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
 
#PCA_Orbits_Partial = reshape(PCA_Orbits_Partial,(Int(2^Splits*N_Perturbed*Amount_Perturbations/Partial),3,Shortest))
PCA_Orbits_2Gen_Plot = permutedims(PCA_Orbits_Partial,(5,4,3,2,1))
Orbit_Range = Int(N_Perturbed/Partial)*2^Splits

print(size(PCA_Orbits_2Gen_Plot))


function Interpolator(Times,Play_Array)
    Vectors = zeros(length(Times),1+M*2^Splits)
    for i=1:length(Times)
        Index_Interp = findfirst(>=(Times[i]), Play_Array[1,:])
        Vectors[i,:] = Play_Array[:,Index_Interp-1] + (Times[i]-Play_Array[1,Index_Interp-1])*(Play_Array[:,Index_Interp] - Play_Array[:,Index_Interp-1])/(Play_Array[1,Index_Interp] - Play_Array[1,Index_Interp-1])
    end
    return Vectors
end

Interpolated_Epi = Interpolator([i for i=1:Length_Orbits],EpiThroughTime)

Interpolated_Epi_Reshaped = reshape(Interpolated_Epi[:,2:end], (Length_Orbits,M,2^Splits))

Interpolated_Orbits_Reshaped = reshape(Interpolated_Orbits[1][2:end,:]', (Length_Orbits,M,2^Splits));


function Orbits_Final_Cell_Type(Time_Lower,Time_Upper,Dimension_To_Plot,Samples_To_Plot,Orbits_To_Plot,Updating)
    #Orbits[Times,Dimension,Different_Cells]
    Colours = [:red,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:orange,:magenta,:mistyrose3,:darkgreen,:gold4,:salmon1]
    if Updating
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1

        aca =  plot!([Time_Lower:Time_Upper],Orbits_To_Plot[Time_Lower:Time_Upper,Dimension_To_Plot,Samples_To_Plot[1]],legend=false,xscale=:log10,linewidth=2,linestyle=:dot,linecolor=Colours[Colour_Index])
        for j=2:size(Samples_To_Plot)[1]
            Colour_Index = Final_Epigenetic_Factors[1,1,j,1]+Final_Epigenetic_Factors[1,1,j,2]*2+Final_Epigenetic_Factors[1,1,j,3]*4+Final_Epigenetic_Factors[1,1,j,4]*8+1
            aca =  plot!([Time_Lower:Time_Upper],Orbits_To_Plot[Time_Lower:Time_Upper,Dimension_To_Plot,Samples_To_Plot[j]],legend=false,xscale=:log10,linewidth=2,linestyle=:dot,linecolor=Colours[Colour_Index])
        end
    else
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1

        aca =  plot([Time_Lower:Time_Upper],Orbits_To_Plot[Time_Lower:Time_Upper,Dimension_To_Plot,Samples_To_Plot[1]],legend=false,xscale=:log10,linecolor=Colours[Colour_Index],title="Dimension = $Dimension_To_Plot")
        for j=2:size(Samples_To_Plot)[1]
            Colour_Index = Final_Epigenetic_Factors[1,1,j,1]+Final_Epigenetic_Factors[1,1,j,2]*2+Final_Epigenetic_Factors[1,1,j,3]*4+Final_Epigenetic_Factors[1,1,j,4]*8+1
            aca =  plot!([Time_Lower:Time_Upper],Orbits_To_Plot[Time_Lower:Time_Upper,Dimension_To_Plot,Samples_To_Plot[j]],legend=false,xscale=:log10,linecolor=Colours[Colour_Index])
        end
    end
    return aca
end
Time_Lower = 10
#Orbits[Times,Dimension,Different_Cells]
aaa = Orbits_Final_Cell_Type(Time_Lower,3000,1,[1,3,4,6,11,12],PCA_Orbits_2Gen_Plot[:,:,:,1,1],false)
display(aaa)
aba = Orbits_Final_Cell_Type(Time_Lower,3000,2,[1,3,4,6,11,12],PCA_Orbits_2Gen_Plot[:,:,:,1,1],false)
display(aba)
aca = Orbits_Final_Cell_Type(Time_Lower,3000,3,[1,3,4,6,11,12],PCA_Orbits_2Gen_Plot[:,:,:,1,1],false)
display(aca)
for tau in Gene_To_Plot
    ada = Orbits_Final_Cell_Type(Time_Lower,3000,tau,Cells,Interpolated_Orbits_Reshaped,false)
    ada = Orbits_Final_Cell_Type(Time_Lower,3000,tau,Cells,Interpolated_Epi_Reshaped,true)
    display(ada)
end

Individual_Cells = reshape(Interpolated_Orbits[1][2:end,:],(2^Splits,M,Length_Orbits))
1+1

function Active_Counter(Tolerance, TimeFrame,Cell_Orbit)
    Activity = ones(Length_Orbits)*M
    for t=TimeFrame+1:Length_Orbits
        Mean_Single = mean(Cell_Orbit[:,t-TimeFrame:t],dims=2)
        Difference = abs.(Cell_Orbit[:,t-TimeFrame] .- Mean_Single)  
        BitMatrixValue = Difference .> ones(M)*Tolerance
        
        for l=1:TimeFrame
            Difference = abs.(Cell_Orbit[:,t-TimeFrame+l] .- Mean_Single)  
            BitMatrixValue2 = Difference .> ones(M)*Tolerance
            BitMatrixValue = BitMatrixValue .| BitMatrixValue2
        end
        Activity[t] = sum(BitMatrixValue)
    end
    return Activity
end

Test = Active_Counter(0.3, 10,Individual_Cells[1,:,:])
aba = plot(Test[1:end],xscale=:log10)
for j=2:2^Splits
    Test = Active_Counter(0.3, 10,Individual_Cells[j,:,:])
    aba = plot!(Test[1:end],xscale=:log10)
end
display(aba)

In [ ]:
using ChaosTools
using StaticArrays

function Gene_ODE(u, Parameters, time)
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    Epi_Factors = Parameters[3]
    test = InvSqr*ParameterMatrix[1,:]'*u[:]
    #vector = Vector{Float16}([])
    #append!(vector,tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1])
    vector = [tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1]]
    for c=2:M
        append!(vector,tanh(40*(InvSqr*ParameterMatrix[c,:]'*u[:]+Epi_Factors[c]+CArray[c])) - u[c])
    end
    dx = @SVector [vector[k] for k in 1:M]

    return dx
end  

Times = [10,30,50,70,90,110,130,150,200,300,500,1000,2000]
Lyapunov_In_Time = zeros(64,length(Times),M)

for Cell=1:64
    
    for l = 1:length(Times)
        Time = Times[l]
        lor = CoupledODEs(Gene_ODE, Float64.(Interpolated_Orbits_Reshaped[Time,:,Cell]), [ParameterMatrix,CArray,Interpolated_Epi_Reshaped[Time,:,Cell]])
        λλ = lyapunovspectrum(lor, 1000; Δt = 0.1)
        Lyapunov_In_Time[Cell,l,:] = λλ
    end
    aba = plot(Times,Lyapunov_In_Time[Cell,:,:],xscale=:log10,legend=false)
    display(aba)
end


In [ ]:
#### SMALL PERT IS SET TO 1 WE ONLY HAVE 1 SAMPLE NOW
### WE MESSED WITH SOME DATA_TO_ANALYSE and the loading ones

#Data_To_Analyse = "21_12_NoSplit_HighODE_SplitTime=0.1_2000Gen_Noise01"
#Data_To_Analyse = ARGS[1]
#Noise = parse(Float64, ARGS[2])
#Data_To_Analyse = "21_12_NoSplit_HighODE_SplitTime=0.1_2000Gen_Noise005_Mut=2"
#Noise = 0.005

Noise = 0.1
Data_To_Analyse = "07_01_NoSplit_HighODE_SplitTime=0.1_2000Gen_Noise0.1_Sample5"

Load_Generations = [325,525]
#Load_Generations = [24]
Load_Generations = [375]

Epi_Limiter = 1
a::Float32 = 1.0*Epi_Limiter   #Ratio of gene to epigenetic factor
v::Float32 = 6*10^-4/Epi_Limiter  #Speed at which epigenetic factor changes
#v::Float32 = 0  #Speed at which epigenetic factor changes

Load_Data_String = Data_To_Analyse*"Epi0.1"



println("OUTPUTTING MY INPUT")
println(Data_To_Analyse)
println(Noise)

using Random
using Plots
using Distributions
using DifferentialEquations
using Statistics
using ProgressMeter
using TickTock
using LinearAlgebra
using StatsBase
using LaTeXStrings
using MultivariateStats
using Dates
using Distances
using JLD
using KernelDensity
using Plots
using StatsPlots




function Setting_Up_Randomly(M,Sparse,rng=MersenneTwister(123))
    "Function used to randomly with seed define our c_i's, J_ij's and the initial positions
    Input:
    M      = Amount of Genes in our system
    Sparse = Bool to determine if we have a sparse graph
    rng    = Random Julia seed to make sure we can recreate the same initial conditions
    Output:
    CArray          = C_i's used, can only be between -0.5 and 0.5 now
    PositionArray   = Random Initial position, between -1 and 1 for all genes, epifactors = 0
    ParameterMatrix = J_ij's Currently a dense network with values between -0.5 and 0.5
    "
    ParameterMatrix = zeros(M,M)
    CArray = zeros(1,M)
    PositionArray = zeros(2,1,M)
    
    CArray = rand(rng,Float32,M).-0.5; #Random c_j between -0,5 and 0.5
    PositionArray[1,1,1:M] = 2*(rand(rng,Float32,M).-0.5); #Random Initial position 
    ParameterMatrix = 2 * bitrand(rng,M,M).-1; #J_jk's
    if Sparse
       for i=1:M
            for j=1:M
                if rand(rng) > 5/M #Average of 5 connections per gene

                    ParameterMatrix[i,j] = 0 
                end
            end
        end
    end
    return CArray, PositionArray, ParameterMatrix
end







function Noisy_Splitting(solution,Total_Splits,M,Seed)
    " Once our cell decides to split, we decide to introduce some random noise in how these genes get distributed.
    This function takes the cells, doubles them and then gives the daughters cell the same gene parameters +- noise
    one daughter gets + the other gets -. For the epigeneitc factors they are simply inherited and do not see any noise
    Input:
    solution     = These are the initial conditions of the parent cells that we need to split 
    (First genes of all cells, then all epigenetic factors)
    Total_Splits = Amount of times the cell has split (including this current split)
    M            = Amount of Genes
    Seed         = Random Noise seed for reproducing results
    Output:
    SplittedArray = New array of the genes and epigenetic factors of the daughter cells
    How is this structured? Suppose we take M=3 for convenience, we would have [g1,g2,g3,e1,e2,e3] for the first parent cell
    The daughters become: [g11,g12,g13,g21,g22,g23,e11,e12,e13,e21,e22,e23]
    Where we have genes of daughter 1, then genes of daughter 2, then epi of daughter 1 and epi of daughter 2.
    When the Daughters split we get the genes of Children of cell 1, then genes of Children of cell 2 and same for epi.
    "
    Random.seed!(Seed)
    s = Total_Splits #For shortening
    SplittedArray = zeros(Float32,2,2^(s),M) #2^(s+1), 2M is the normal length, 4M after 1 split, 8M after 2 splits etc.
    for g=1:2^(s-1) #looping over the length of the parent
        for t=1:M
            Gaussian = Normal{Float32}(0.0f0,0.0f0)
            Displace = rand(Gaussian) #Noise for split
            #index = Int32.(floor((g-1)/M)*M +g) #example M=10: Index 1-> 1 and 11, index 2->2 and 12 ... 
            #index 10->10 and 20, index 11->21 and 30 and so on. This is old method

            #Genes
            SplittedArray[1,2*g-1,t] = solution[1,g,t] + Displace
            SplittedArray[1,2*g,t] = solution[1,g,t] - Displace
            
        end
        #Epigenetic
        SplittedArray[2,2*g-1,:] = solution[2, g,:]
        SplittedArray[2,2*g,:] = solution[2, g,:]   
    end

    return SplittedArray  
end

function Differentiaton_Detection_Parameter(M,Epigenetic_Plot)
    "Function used to convert epigenetic factors into cell types and bits
    Input:
    M               = Amount of Genes 
    Epigenetic_Plot = Array of epigenetic factors which have to be converted to 0 and 1 representing on or off
    Output:
    b        = Array of scalar values representing cell types
    BitGenes = Array of vectors representing the genes of the cell types
    "
    b = Int128[]
    Counter = Int[]
    Epi_Bitwise = signbit.(-Epigenetic_Plot)

    
    BitGenes = [] #We will put the bit notation of the gene in here
    #Later rewrite to let this replace Bit_Represent
    
    for i=1:length(Epi_Bitwise[:,1])
        Bit_Represent::Int128 = 0 #CAREFUL WITH M>127
        for j=1:M
            Bit_Represent += Int128(Epi_Bitwise[i,j]*2)^(j-1)
        end
        #print(Bit_Represent)
        #print("     ")
        if !(Bit_Represent in b)

            push!(BitGenes,collect(Epi_Bitwise[i,:]))
            push!(Counter,1)
        end
        push!(b,Bit_Represent)

        
    end
    
    #print(b)
    Counter = sort(Counter,rev=true)
    #print(Counter)
    Score = 0
    #Make here the formula to turn b and the counter into a score


    return b,BitGenes
end

function Entropy_Score(M,Epigenetic_Plot)
    "Function used to convert epigenetic factors into cell types and bits to then calculate the entropic score.
    See previous function for alternative, this is currently not in use.
    Input:
    M               = Amount of Genes 
    Epigenetic_Plot = Array of epigenetic factors which have to be converted to 0 and 1 representing on or off
    Output:
    Score = Entropy score of the cell types
    "
    b = Int[]
    Counter = Int[]
    Epi_Bitwise = signbit.(-Epigenetic_Plot)
    
    for i=1:length(Epi_Bitwise[:,1])
        Bit_Represent::Int128 = 0 #CAREFUL WITH M>127
        for j=1:M
            Bit_Represent += Int128(Epi_Bitwise[i,j]*2)^(j-1)
        end
        #print(Bit_Represent)
        #print("     ")
        if !(Bit_Represent in b)
            push!(b,Bit_Represent)
            push!(Counter,1)
        else 
            Counter[findfirst(==(Bit_Represent), b)] += 1
        end
        
    end
    
    #print(b)
    Counter = sort(Counter,rev=true)
    #print(Counter)
    Score = 0
    #Make here the formula to turn b and the counter into a score
    for i=1:length(b)
       prob = Counter[i]/M
       Score -= prob*log(prob)
    end
    
    return Score
end



function Mutate_Parameters(ParameterMatrix_Mut,M,Mutations,Seed)
    "Function used to mutate the network (we turn a 1 or -1 to 0 and we turn a 0 to 1 or -1)
    Input:
    ParameterMatrix_Mut = Original Network that we want to mutate
    M                   = Amount of Genes 
    Mutations           = How many connections will we mutate
    Seed                = Randomness seed
    Output:
    ParameterMatrix_Mut = Mutated Network
    "
    Zero_Indices = findall(iszero,ParameterMatrix_Mut)
    One_Indices = findall(!iszero,ParameterMatrix_Mut)
    Random.seed!(Seed)
   
    for j=1:Mutations 
        t_zero = rand(1:length(Zero_Indices))
        t_one = rand(1:length(One_Indices))
        i_zero = Zero_Indices[t_zero][1]
        j_zero = Zero_Indices[t_zero][2]
        i_Value = One_Indices[t_one][1]
        j_Value = One_Indices[t_one][2]



        ParameterMatrix_Mut[i_zero,j_zero] = rand(0:1)*2-1 #turn a zero into 1 or -1
        ParameterMatrix_Mut[i_Value,j_Value] = 0 #turn 1 or -1 into 0




        
    end
    return ParameterMatrix_Mut
end


#Some Julia functions I played around with, may reconsider using more later
MultiplyBroad_(a,b) = a .* b

SummingDim2_(a) = sum(a,dims=2) 

HyperTanh_(a) = tanh.(40*a)



function Epigenetic_Evolution3(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    
    #⊗_(Wi, bi) = Wi .* bi
       
    for t=1:2^Splits

        du[2,t,:] = v*(a*u[1,t,:].-u[2,t,:])    
        du[1,t,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,t,:].+u[2,t,:].+CArray[:])) .- u[1,t,:] 
            #Epigenetic Evolution, no need to worry about boundaries here so simple loop      
    end  
    
end



function Gradient(u,Splits,t)
    "Function used to calculate the gradient of a cells neighbours, takes into accounts if there even are neighbours.
    Input:
    u      = Gene concentrations over which we take the gradient
    Splits = How many times have the cells split already (determines where neighbours are)
    t      = What location is our cell (boundaries only have 1 neighbour)
    Output:
    Multiple different version depending on the splits, this will be the gradient we want
    "
    if Splits == 0
        return 0
    elseif Splits ==1
        if t==1
            return u[t+1,:] - u[t,:]
        else
            return u[t-1,:] - u[t,:]
        end
    else
        if t==1
            return u[t+1,:] - u[t,:]
        elseif t==2^Splits
            return u[t-1,:] - u[t,:]
        else
            return u[t-1,:] - 2*u[t,:] + u[t+1,:]
        end
    end
    
end

function Epigenetic_Evolution4(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    
    #⊗_(Wi, bi) = Wi .* bi
       
    for t=1:2^Splits

        du[2,t,:] = v*(a*u[1,t,:].-u[2,t,:])    
        du[1,t,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,t,:].+u[2,t,:].+CArray[:])) .- u[1,t,:] +KArray.*Gradient(u[1,:,:],Splits,t)
            #Epigenetic Evolution, no need to worry about boundaries here so simple loop      
    end  
    
end

function σ_Genes(du, u,Parameters,Time)
    du[1,:,:] .= Noise
    
end



function Epigenetic_Evolution2(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    Cells::Int32 = 2^Splits
    #⊗_(Wi, bi) = Wi .* bi
       
    for t=1:Cells  

       du[2,t,:] = v*(a*u[1,t,:].-u[2,t,:])    

       du[1,t,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,t,:].+u[2,t,:].+CArray[:])) .- u[1,t,:] .+ KArray.*(vec(sum(u[1,:,:],dims=1))/Cells - u[1,t,:])

    end  
    
end

#Defining it all in function form


function Mutating_Network(N,M,K,ParameterMatrix_Collection,PositionArray_Collection, Indices, Counter, Genetics)
    "Function used to make mutated copies of the best performing networks. Currently not in use
    Input:
    N                          = How many copies of the network do we make
    M                          = Amount of Genes
    K                          = Supposed to work as a fraction of N, telling us how many mutant versions
    ParameterMatrix_Collection = The original networks
    PositionArray_Collection   = Original Positions
    Indices                    = Sorted indices giving the best networks
    Counter                    = Randomiser seed
    Genetics                   = Used to track heritance of networks
    Output:
    ParameterMatrix_Collection = Updated network
    PositionArray_Collection   = Updated initial positions
    Counter                    = Updated seed
    Updated_Genes              = Updated inheritance
    "
    Updated_Genes = zeros(N)
    Mutations = 1
    Seed_Mutations = 1234
    Indexation_Mutate = 1
    Temporary_Matrix = zeros(Float32,N,M,M)
    Temporary_Vector = zeros(Float32,N,2,1,M)
    for p = 1:K
        for q=1:N/(2*K)
            Temporary_Matrix[Indexation_Mutate,:,:] = Mutate_Parameters(ParameterMatrix_Collection[Indices[p],:,:],M,Mutations,Seed_Mutations+Counter)
            Temporary_Vector[Indexation_Mutate,:,:,:] = PositionArray_Collection[Indices[p],:,:,:]
            Updated_Genes[Indexation_Mutate] = Genetics[p]
            Temporary_Matrix[Indexation_Mutate+1,:,:] = Temporary_Matrix[Indexation_Mutate,:,:]
            Temporary_Vector[Indexation_Mutate+1,:,:,:] = Temporary_Vector[Indexation_Mutate,:,:,:]
            Updated_Genes[Indexation_Mutate+1] = Genetics[p]
            Counter += 1
            Indexation_Mutate +=2
        end
        Temporary_Matrix[Indexation_Mutate-1,:,:] = ParameterMatrix_Collection[Indices[p],:,:] #We overwrite it to add the original back
        Temporary_Matrix[Indexation_Mutate-2,:,:] = ParameterMatrix_Collection[Indices[p],:,:]
    end
    ParameterMatrix_Collection = deepcopy(Temporary_Matrix);
    PositionArray_Collection = deepcopy(Temporary_Vector);
    return ParameterMatrix_Collection, PositionArray_Collection, Counter, Updated_Genes
end



function Running_Network(CArray,ParameterMatrix,PositionArray,Random_Noise)
    "Function used to run our simulation
    Input:
    CArray          = Constants used in our ODE solver  
    ParameterMatrix = Collection of Networks
    PositionArray   = Original Positions
    Random_Noise    = Randomiser seed
    Output:
    PositionArray[2,:,:] = Epigenetic factors from our simulation
    Random_Noise         = Updated seed
    "
    #CArray = CArray_Collection[x,:]
    #ParameterMatrix = ParameterMatrix_Collection[x,:,:]
    #PositionArray = PositionArray_Collection[x,:]
    Orbit_Collection = zeros(Float32,2^Splits*M+1,0)
    Epi_Collection = zeros(Float32,2^Splits*M+1,0)
    
    
    
    
    for s=1:Splits #Looping over the splits
        Parameters = (ParameterMatrix, CArray,KArray,Int32(s-1)) #We have to update the splits every rerun, rest stays same
        tspan = (Float32(0.0),SplitTime) #How long ODE will run
        if Mean_Field
            prob = ODEProblem(Epigenetic_Evolution2,PositionArray,tspan,Parameters) #ODEdefining
        elseif Diffusing_System
            prob = ODEProblem(Epigenetic_Evolution4,PositionArray,tspan,Parameters) #ODEdefining
        elseif Stochastic_Noise
            prob = SDEProblem(Epigenetic_Evolution4, σ_Genes,PositionArray,tspan,Parameters)

        else
            prob = ODEProblem(Epigenetic_Evolution3,PositionArray,tspan,Parameters) #ODEdefining
        end    #Solving
        if !Stochastic_Noise
            sol = solve(prob,BS3())
        else
            sol = solve(prob,SRA3())
            #sol = solve(prob,LambaEulerHeun())
        end
        Times = sol.t #Save the time, doesn't get used yet though
        StepAmount = length(Times)  #Amount of data points per gene
        Genes = sol 
        PositionArray = Genes[StepAmount] #Saving the genes and epi factors
        
        OrbitalCollection = Times' .+ (s-1)*SplitTime
        EpiCollected = Times' .+ (s-1)*SplitTime
        for i=1:2^(Splits-s+1)  #s=1 this gives all 2^Splits, this is to say how much we repeat same path
            for j=1:2^(s-1) #Together multiply, always 2^Splits orbits, these j represent different paths
                OrbitalCollection = vcat(OrbitalCollection,sol[1,j,:,:])   
                EpiCollected = vcat(EpiCollected,sol[2,j,:,:])   
            end
        end
        Orbit_Collection = hcat(Orbit_Collection,OrbitalCollection)
        Epi_Collection = hcat(Epi_Collection,EpiCollected)
        #Orbit1 = [Times,Genes[1,1,1:M,1:StepAmount]]
        #Orbit2 = [Times,Genes[1,1,1:M,1:StepAmount]]
        #EpiOrbit1 = [Times,Genes[2,1,1:M,1:StepAmount]]
        #EpiOrbit2 = [Times,Genes[2,1,1:M,1:StepAmount]]
        #Now we must take all the values from the updated PositionArray and give appropriate terms to the daughter cells
        PositionArray = Noisy_Splitting(PositionArray,s,M,Random_Noise)  #Splitting function
        Random_Noise += 1
    end

    Final_Run_Time::Int32 = 20000 #Long evolving of the cells after splitting to get the final epigenetic state
    Parameters = (ParameterMatrix, CArray,KArray,Int32(Splits)) 
    tspan = (Int32(0.0),Final_Run_Time)

    if Mean_Field
        prob = ODEProblem(Epigenetic_Evolution2,PositionArray,tspan,Parameters) #ODEdefining
    elseif Diffusing_System
        prob = ODEProblem(Epigenetic_Evolution4,PositionArray,tspan,Parameters) #ODEdefining
    elseif Stochastic_Noise
        prob = SDEProblem(Epigenetic_Evolution4, σ_Genes,PositionArray,tspan,Parameters)

    else
        prob = ODEProblem(Epigenetic_Evolution3,PositionArray,tspan,Parameters) #ODEdefining
    end    #Solving
    if !Stochastic_Noise
        sol = solve(prob,BS3())
    else
        #,dtmin=0.1,force_dtmin=true
        sol = solve(prob,SRA3(),dtmin=0.1,force_dtmin=true)
    end
    #sol = solve(prob,alg_hints = [:nonstiff]) #Solving
    #sol = solve(prob)
    Times = sol.t
    StepAmount = length(Times)
    GenesEpi = sol
    PositionArray = GenesEpi[StepAmount]

    OrbitalCollection = Times' .+ Splits*SplitTime 
    EpiCollected = Times' .+ Splits*SplitTime 
    for j=1:2^(Splits) #Together multiply, always 2^Splits orbits, these j represent different paths
        OrbitalCollection = vcat(OrbitalCollection,sol[1,j,:,:])  
        EpiCollected = vcat(EpiCollected,sol[2,j,:,:])   

    end
    Orbit_Collection = hcat(Orbit_Collection,OrbitalCollection)
    Epi_Collection = hcat(Epi_Collection,EpiCollected)
    #Orbit1 = [cat(Orbit1[1],Times.+SplitTime,dims=1),cat(Orbit1[2],GenesEpi[1,1,1:M,1:StepAmount],dims=2)]
    #Orbit2 = [cat(Orbit2[1],Times.+SplitTime,dims=1),cat(Orbit2[2],GenesEpi[1,2,1:M,1:StepAmount],dims=2)]
    #EpiOrbit1 = [cat(EpiOrbit1[1],Times.+SplitTime,dims=1),cat(EpiOrbit1[2],GenesEpi[2,1,1:M,1:StepAmount],dims=2)]
    #EpiOrbit2 = [cat(EpiOrbit2[1],Times.+SplitTime,dims=1),cat(EpiOrbit2[2],GenesEpi[2,2,1:M,1:StepAmount],dims=2)]

    return PositionArray[2,:,:],Random_Noise, Orbit_Collection, Epi_Collection
end


function Scoring_Indexes(N,Final_Epigenetic_Factors,Cell_Bit)
    "Function used to calculate the score of all the networks, distribution robustness and the cell type robustness. Note
    that we are still selecting on amount of different cells, not entropy.
    Input:
    N                       = How many copies of the same networks we have 
    Final_Epigentic_Factors = Array of epigenetic factors after running the simulations
    Cell_Bit                = Cell type tracker
    Output:
    Indices                   = Indices of networks sorted by score, we only select these
    AllIndices                = Longer copy of Indices, contains also those we will no longer use
    Evolution_Indices         = Gives us the score corresponding with the Indices we selected on
    [Cell_Types,Cell_Amounts] = Vector of the cell types and how many of these cell types there are
    Cell_Bit                  = Updated cell type tracker
    KLD_Distance              = Kullback-Lieber divergence score for the distribution robustness
    Bit_Distances             = Cell Type robustness measure
    "
    Scoring = zeros(2,N)
    Cell_Types = []
    Cell_Amounts = []
    Cell_Distribution = Array{Vector{Float64}}(undef, N) 
    Cell_Bit_Array = Array{Vector{Any}}(undef, N)

    for x=1:N
        Differentiation, Bit_Genes = Differentiaton_Detection_Parameter(M,Final_Epigenetic_Factors[x,:,:]) 
     
        #Here we save some cell types and bits to use for our robustness calculation after the for loop
        #Cell types
        alpha = countmap(Differentiation)
        Temp_Array = Vector{Float64}()
        for i in keys(alpha)
            Temp_Array = push!(Temp_Array,alpha[i])
        end
        #print(Temp_Array)
        Cell_Distribution[x] = sort(Temp_Array,rev=true)/2^Splits #We sort the distribution of our cells and normalise
        #Bits
        Cell_Bit_Array[x] = Bit_Genes
        
        
        
        
        #This part here is where we track the cell types we had before as well
        for tau in Bit_Genes
            if !(tau in Cell_Bit)

                push!(Cell_Bit,tau)


            end
        end
        for tau in Differentiation 
            if !(tau in Cell_Types)
                push!(Cell_Types,tau)
                push!(Cell_Amounts,1)
            else
                Cell_Amounts[findfirst(==(tau), Cell_Types)] += 1
            end
        end

        Scoring[1:2,x] = [length(countmap(Differentiation)),x]

    end

    #Robustness calculation
    KLD_Distance = 0
    
    KLD_Adder = 0
    for i=1:N
        for j=1:N #Loop over all and calculate KL for all distributions
            if i!=j #If the same we get no difference anyways
                Length_Differ = length(Cell_Distribution[i])-length(Cell_Distribution[j])
                if Length_Differ > 0 #i is longer so we need to add 0 to j
                    KLD_Adder = kldivergence(Cell_Distribution[i],vcat(Cell_Distribution[j],zeros(Length_Differ)))
                    if KLD_Adder != Inf #There are zeros in the second distribution (divide by 0 error)
                        KLD_Distance += kldivergence(Cell_Distribution[i],vcat(Cell_Distribution[j],zeros(Length_Differ)))
                    end
                else
                    KLD_Adder = kldivergence(vcat(Cell_Distribution[i],zeros(-Length_Differ)),Cell_Distribution[j])

                    KLD_Distance += kldivergence(vcat(Cell_Distribution[i],zeros(-Length_Differ)),Cell_Distribution[j])
                end
            end
        end
    end
    KLD_Distance = KLD_Distance/(N*(N-1))
    
    #Now we see the closest type in the other groups
    Bit_Distances = 0
    for x =1:N
        for Bit_Inside in Cell_Bit_Array[x] #These are all the cell types in bit form
            for y =1:N #We loop over the others
                if x!=y #Must look at different otherwise distance 0 again
                    Bit_Distan_Tem = M
                    for Bit_Outside in Cell_Bit_Array[y] #We loop over all the cell types in this other one
                        Bit_Distan_Temp = sum(abs.(Bit_Inside .- Bit_Outside)) 
                        if Bit_Distan_Temp < Bit_Distan_Tem #Distance must be smaller if it is the best fit
                            Bit_Distan_Tem = Bit_Distan_Temp
                        end
                    end
                    Bit_Distances += sqrt(Bit_Distan_Tem)/(N*(N-1)) #Sqrt to make euclidean distance, N*(N-1) for normalisation
                end
            end
        end
    end
    
    
    #Here we mess a bit with the indices to just have the ones with the highest score later
    
    Indices = sortslices(Scoring,dims=2,rev=true)
    Evolution_Indices = Indices[1,1:K]

    AllIndices = deepcopy(Indices[2,:])
    Indices = Indices[2,1:K]
    Indices = Int.(Indices)
    #print(Indices[2,1:K])
    return Indices, AllIndices, Evolution_Indices, [Cell_Types,Cell_Amounts],Cell_Bit, KLD_Distance, Bit_Distances
end

function Epi_Plotter(Indices,K,Final_Epigenetic_Factors)
    "Function used to plot the epigenetic factors of different cells
    Input:
    Indices                 = Indices of highest scoring runs  
    k                       = 4 or 8 plots
    Final_Epigentic_Factors = Epigenetic factors that we will plot
    Output:
    k = Plot element, simply use display on the output to see the requested plot
    "
    pa = plot(Final_Epigenetic_Factors[Indices[1],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pb = plot(Final_Epigenetic_Factors[Indices[2],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pc = plot(Final_Epigenetic_Factors[Indices[3],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pd = plot(Final_Epigenetic_Factors[Indices[4],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    if K==8
        pe = plot(Final_Epigenetic_Factors[Indices[5],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        pf = plot(Final_Epigenetic_Factors[Indices[6],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        pg = plot(Final_Epigenetic_Factors[Indices[7],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        ph = plot(Final_Epigenetic_Factors[Indices[8],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        k = plot(pa,pb,pc,pd,pe,pf,pg,ph,layout=(4,2),legend=false)
    else
        k = plot(pa,pb,pc,pd,layout=(2,2),legend=false)
    end
    return k
end

function Gene_Variance(Final_Epigenetic_Factors,M,N)
    "Function used to calcualte the variance in expressed genes
    Input:
    Final_epigenetic_Factors = Final epigenetic values which represent gene expression
    M                        = Amount of Genes
    N                        = How many copies of same network do we have
    Output:
    Deviation = Variance in these genes (normalised by 2^Splits*N)
    "
    BitterMatrix = signbit.(-Final_Epigenetic_Factors[:,:,:])
    Average_Bits = zeros(M)
    for i=1:N
        for j=1:2^Splits
            Average_Bits += BitterMatrix[i,j,:]
        end
    end
    Average_Bits /= N*2^Splits
    Deviation = 0

    for i=1:N
        for j=1:2^Splits
            Deviation += sum((Average_Bits .- BitterMatrix[i,j,:]).^2)/(2^Splits*N-1)
        end
    end
    
    return Deviation
end

function PCA_Distribution_Creator(PCA_Dimensions,Transformed,Dimensions_Distribution,Cell_Distribution)
    
    
    PCA_Dimensional_Values = zeros(PCA_Dimensions,findmax(Dimensions_Distribution)[1]) 
    #Here we will save all the values and use these for indexes later
    B = Array{Vector{Tuple{Float64, Int64}}}(undef, PCA_Dimensions)
    for j=1:PCA_Dimensions
        B[j] = [(i, count(==(i), Transformed)) for i in unique(Transformed[j,:])] #Here we find what the arguments are for the
        #different dimensions
        for k = 1:length(B[j])
            PCA_Dimensional_Values[j,k] = B[j][k][1] #Here we save all the individual arguments in the corresponding dimension
        end
    end
    #print(PCA_Dimensional_Values)   #If we wanna see them

    Components_Length = length(Transformed[1,:]) 
    for k in 1:Components_Length
        Index_PCA = zeros(PCA_Dimensions)

        for l=1:PCA_Dimensions
            Index_PCA[l] = (findfirst(==(Transformed[l,k]), PCA_Dimensional_Values[l,:]))
        end 
        Index_PCA = Int.(Index_PCA)
        
        
        if PCA_Dimensions==2 #I really tried but I couldn't fix it so just throwing in this hack
            Cell_Distribution[Index_PCA[1],Index_PCA[2]] += 1/Components_Length
        elseif PCA_Dimensions==1
            Cell_Distribution[Index_PCA[1]] += 1/Components_Length   
        elseif PCA_Dimensions==3
            Cell_Distribution[Index_PCA[1],Index_PCA[2],Index_PCA[3]] += 1/Components_Length
        elseif PCA_Dimensions==4
            Cell_Distribution[Index_PCA[1],Index_PCA[2],Index_PCA[3],Index_PCA[4]] += 1/Components_Length
        elseif PCA_Dimensions==5
            Cell_Distribution[Index_PCA[1],Index_PCA[2],Index_PCA[3],Index_PCA[4],Index_PCA[5]] += 1/Components_Length
        else
        print("ADD MORE HERE THERE IS A MISTAKE, DO NOT GO OVER 5 PCA DIMENSIONS YET")
        end
    end
    

    
    return Cell_Distribution
end



function Earth_Movers_Distance_1D(Vector1,Vector2)

    #Takes 2 distribution vectors containing bins with a position and their %
    Length_V = length(Vector1[1,:]) #Both should be same length anyways
    EMD = zeros(Length_V+1) #We add an extra one because of the following loop
    
    for i =1:Length_V
        EMD[i+1] = Vector1[2,i] - Vector2[2,i] + EMD[i]
    end
    EMD_Distance = 0
    
    for i =1:Length_V-1
        EMD_Distance += abs(EMD[i+1])*(Vector1[1,i+1] - Vector1[1,i]) #We take moved distribution times moved distance

    end

    
    return EMD_Distance
end


function Robustness_Distribution_Scoring()
    
    
    #Here we create a very general robustness measure
    #This is done by taking the PCA components over all the different generations
    #Then we categorise all cell types and measure how similar their distribution
    #is in a given network

    #!!!!!!! This robustness does not consider cell type distances
    
    Robustness_Score = zeros(Generations,P)
    for Genere=1:Generations  #We want to calculate this for all generations
        for i = 1:P #We have to repeat this for all different networks in a generation
            #This is used to find the maximum of different cell types along every dimension in this network
            Dimensions_Distribution = zeros(Int32,PCA_Dimensions)
            for j=1:N
                Transformed = transform(model,reshape(signbit.(-History_Final_Epigenetic_Factors[Genere,i,j,:,:]),2^Splits,M)')

                for k=1:PCA_Dimensions
                    if Int.(length(countmap(Transformed[k,:]))) >  Dimensions_Distribution[k]
                        Dimensions_Distribution[k] = Int.(length(countmap(Transformed[k,:])))
                    end
                end   
            end

            #Now we have the max dimensions, now we want to make all distributions
            #Of this specific network

            Dimensions_Distribution = (Dimensions_Distribution...,) #Magically makes a tuple!
            #print(Dimensions_Distribution) #if we wanna visualise

            Cell_Distribution_Collection = Array{Array{Float64, PCA_Dimensions}}(undef, N)
            Vectorised_Cell_Distribution_Collection = Array{Vector{Float64}}(undef, N)

            for j=1:N #Now we collect all the distributions
                Transformed = transform(model,reshape(signbit.(-History_Final_Epigenetic_Factors[Genere,i,j,:,:]),2^Splits,M)')

                Cell_Distribution = zeros(Dimensions_Distribution) #This is where we need it to be a tuple!
                Cell_Distribution_Collection[j] = PCA_Distribution_Creator(PCA_Dimensions,Transformed,Dimensions_Distribution,Cell_Distribution)

                #Now we want to convert all of these into vectors to apply Jensen-Shannon
                Vectorised_Cell_Distribution_Collection[j] = sort(vec(Cell_Distribution_Collection[j]),rev=true)
            end

            #JS-divergence
            for tau = 1:N-1
                for pau = tau:N
                    Robustness_Score[Genere,i] += js_divergence(Vectorised_Cell_Distribution_Collection[tau],Vectorised_Cell_Distribution_Collection[pau])
                end
            end


        end
    end
    return Robustness_Score
end



function EMD_Generational_Calculator()
    
    #Here we will create a Robustness measure that takes into account how far
    #away different cell types are for the same network.
    #we will do this by creating a PCA analysis for EVERY single network
    #for all the different generations
    #(Alternatively we see if we can use the same PCA analysis within one generation)
    #We will create a one dimensional distribution and use our simplified
    #EMD algorithm
    
    Vectorised_Cell_Network_Collection = Array{Matrix{Float64}}(undef, N) #Here we save these distributions
    #This loops over copies of same network and makes all our distributions
    EMD_Distances = zeros(Generations)
    for generation_p = 1:Generations
        EMD_Dist = 0
        for P_Valu = 1:P
            FullNetworkBits = reshape(signbit.(-History_Final_Epigenetic_Factors[generation_p,P_Valu,:,:,:]),2^Splits*N,M)'
            Networkmodel= fit(PCA,FullNetworkBits;maxoutdim=1)
            Networkmodel_EigenV = eigvals(Networkmodel::PCA)
            for i=1:N
                #This creates the Distribution of a single individual
                #We start to apply this PCA to one of these individuals
                NetworkBits = reshape(signbit.(-History_Final_Epigenetic_Factors[generation_p,P_Valu,i,:,:]),2^Splits,M)'
                Transformed_Network = transform(Networkmodel,NetworkBits)
                Distribution_Netw = zeros(0)
                X_Values = zeros(0)
                Transformed_Network = round.(Transformed_Network,digits=4) #Otherwise ugly floating point numbers
                Length_Netw_Normal = 1/length(Transformed_Network)
                for i in Transformed_Network

                    if i in X_Values
                        Distribution_Netw[findfirst(==(i), X_Values)] += Length_Netw_Normal
                    else
                        X_Values = vcat(X_Values,i)
                        Distribution_Netw = vcat(Distribution_Netw,Length_Netw_Normal)
                    end
                end



                Combined_Distribution = zeros(2,length(X_Values))
                Combined_Distribution[1,:] = round.(X_Values/Networkmodel_EigenV,digits=4)
                Combined_Distribution[2,:] = Distribution_Netw
                Combined_Distribution = sortslices(Combined_Distribution,dims=2)

                Vectorised_Cell_Network_Collection[i] = Combined_Distribution
            end


            #Now we need to apply EMD to these distributions and save the result

            for tau = 1:N-1   #Preparing the vectors to have the same X values and length
                for pau = tau+1:N
                    Vector1 = Vectorised_Cell_Network_Collection[tau]
                    Vector2 = Vectorised_Cell_Network_Collection[pau]
                    for i in Vector1[1,:]
                        if !(i in Vector2[1,:])
                            Vector2 = hcat(Vector2, [i,0]) #Adding empty position
                        end
                    end
                    for i in Vector2[1,:]
                        if !(i in Vector1[1,:])
                            Vector1 = hcat(Vector1, [i,0]) #Adding empty position
                        end
                    end

                    Vector1 = (sortslices(Vector1,dims=2)) #Sorting them with the added X values
                    Vector2 = (sortslices(Vector2,dims=2))
                    EMD_Dist += Earth_Movers_Distance_1D(Vector1,Vector2) #Calculating EDM for 1D

                end
            end
        end
        EMD_Dist = 2*EMD_Dist/(P*N*(N-1))  #Normalisation
        EMD_Distances[generation_p] = EMD_Dist
    end
    return EMD_Distances
end


function Two_Generational_Plotter(Generation,Cut_Off,Duration,Length_Orbits,Opacity)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1+2*(Generation-1):2+2*(Generation-1)
        for j=1:Amount_Copies
            if Shortest>size(Orbit_Array[i,j])[2]
                Shortest = size(Orbit_Array[i,j])[2] #Used later to check which data set is the shortest when plotting
            end

            Total_Length += size(Orbit_Array[i,j])[2]

        end
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2*8*2^Splits)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1+2*(Generation-1):2+2*(Generation-1)
        for j=1:Amount_Copies
            if Cut_Off
                
                for p=0:2^Splits-1
                    PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i,j][2+p*M:1+(p+1)*M,1:Length_Orbits]
                end
            else
                Length_Orbits =  size(Orbit_Array[i,j])[2]
                for p=0:2^Splits-1
                    PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i,j][2+p*M:1+(p+1)*M,:]
                end
            end

            Collection_Index += Length_Orbits*2^Splits
        end
    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,2,Amount_Copies,2^Splits,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:2
        for j=1:Amount_Copies

            for tau=1:2^Splits
                tester = Orbit_Array[i+2*(Generation-1),j][2+(tau-1)*M:1+tau*M,1:Shortest]
                Transformed = transform(Orbital_Model,tester)
                PCA_Orbits_2Gen[i,j,tau,:,:] = Transformed
            end
        end
    end
    
    
    Colours = [:red,:orange,:maroon,:chocolate4]
    @gif for k=1:Duration
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen[1,1,1,1,1:k],PCA_Orbits_2Gen[1,1,1,2,1:k],PCA_Orbits_2Gen[1,1,1,3,1:k],legend=false,c=:blue,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)


        for j=1:8
            for tau=1:2^Splits
                PCAPlots = plot!(PCA_Orbits_2Gen[i,j,tau,1,1:k],PCA_Orbits_2Gen[i,j,tau,2,1:k],PCA_Orbits_2Gen[i,j,tau,3,1:k],legend=false,c=:blue,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)
            end
        end

        i=2
        for j=1:4
            colour = Colours[j]
            for tau=1:2^Splits
                PCAPlots = plot!(PCA_Orbits_2Gen[i,2*j-1,tau,1,1:k],PCA_Orbits_2Gen[i,2*j-1,tau,2,1:k],PCA_Orbits_2Gen[i,2*j-1,tau,3,1:k],legend=false,c=colour,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                PCAPlots = plot!(PCA_Orbits_2Gen[i,2*j,tau,1,1:k],PCA_Orbits_2Gen[i,2*j,tau,2,1:k],PCA_Orbits_2Gen[i,2*j,tau,3,1:k],legend=false,c=colour,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)

            end
        end


    end
end




function Generational_Tracker(Family_Distance,Original_Tracking,Types)
    Cell_Generational_Tracking = deepcopy(Original_Tracking)
    Original_Tracking = deepcopy(Cell_Generational_Tracking)
    size(Types[1,:])


    TypeFam = [Types[1,:],1]
    TypeFam = hcat(TypeFam,[Types[1,:],2])
    TypeFam = hcat(TypeFam,[Types[1,:],3])
    TypeFam = hcat(TypeFam,[Types[1,:],4])

    print(size(TypeFam[1,:]))

    TypeFam = [Types[1,:],1]
    Distances_Gen = zeros(length(Cell_Generational_Tracking[1,:]))
    Cell_Generational_Tracking[1,1] = 1
    for i=2:length(Cell_Generational_Tracking[1,:])
        NewFam = true
        for j=1:length(TypeFam[1,:]) 
            if sum(abs.(Types[i,:]-TypeFam[1,j])) < Family_Distance + 0.5
                if NewFam
                    NewFam = false
                    Cell_Generational_Tracking[1,i] = j
                    Distances_Gen[j] = sum(abs.(Types[i,:]-TypeFam[1,j]))
                elseif Distances_Gen[j] > sum(abs.(Types[i,:]-TypeFam[1,j]))
                    Cell_Generational_Tracking[1,i] = j
                    Distances_Gen[j] = sum(abs.(Types[i,:]-TypeFam[1,j]))
                end
            end
        end
        if NewFam
            Distances_Gen[i] = sum(abs.(Types[i,:]-TypeFam[1,1]))
            Cell_Generational_Tracking[1,i] = Int(length(TypeFam[1,:])+1)
            TypeFam = hcat(TypeFam,[Types[i,:],Int(length(TypeFam[1,:])+1)])

        end
    end

    Cell_Generational_Tracking = Cell_Generational_Tracking[:,sortperm(Cell_Generational_Tracking[1,:])]



    Amount_Types = Cell_Generational_Tracking[1,end]
    Top_Bound_Index = zeros(Amount_Types)
    for j=1:Amount_Types
        Top_Bound_Index[j] = findfirst(isequal(j),Cell_Generational_Tracking[1,:])
    end
    print(Top_Bound_Index[:])


    #print(Cell_Generational_Tracking)
    Copied_Tracker = deepcopy(Cell_Generational_Tracking[2:end,:])
    Copied_Tracker = Copied_Tracker/sum(Copied_Tracker[1,:])

    Boundaries = zeros(Generations,length(Copied_Tracker[1,:]))
    for beta in 2:length(Copied_Tracker[1,:])
        Copied_Tracker[:,beta] += Copied_Tracker[:,beta-1]
        Boundaries[:,beta] = Copied_Tracker[:,beta] - Copied_Tracker[:,beta-1]
    end

    Cell_Type_Plotter = plot(Copied_Tracker[:,1]; ribbon = (Copied_Tracker[:,1],LinRange(0, 0, Generations)))
    for beta in 2:length(Copied_Tracker[1,:])
        Cell_Type_Plotter = plot!(Copied_Tracker[:,beta]; ribbon = (Boundaries[:,beta],LinRange(0, 0, Generations)),legend=false)
    end
    display(Cell_Type_Plotter)




    

    #Now we do the same but we group them in families


    Family_Boundaries = ones(Generations,Amount_Types)
    Family_Boundaries[:,1] = Copied_Tracker[:,Int(Top_Bound_Index[2]-1)].-0
    for beta in 2:Amount_Types-1

        Family_Boundaries[:,beta] = Copied_Tracker[:,Int(Top_Bound_Index[beta+1]-1)] .- Copied_Tracker[:,Int(Top_Bound_Index[beta]-1)]
    end
    Family_Boundaries[:,end] = 1 .- Copied_Tracker[:,Int(Top_Bound_Index[Amount_Types]-1)]


    Cell_Type_Family_Plotter = plot(Copied_Tracker[:,Int(Top_Bound_Index[2]-1)]; ribbon = (Family_Boundaries[:,1],LinRange(0, 0, Generations)))
    for beta in 2:Amount_Types-1
        Cell_Type_Family_Plotter = plot!(Copied_Tracker[:,Int(Top_Bound_Index[beta+1]-1)]; ribbon = (Family_Boundaries[:,beta],LinRange(0, 0, Generations)),legend=false)
    end
    Cell_Type_Family_Plotter = plot!(ones(Generations); ribbon = (Family_Boundaries[:,Amount_Types],LinRange(0, 0, Generations)),legend=false)

    if false
        Cell_Type_Family_Plotter = plot!(Copied_Tracker[:,1],linestyle=:dash)
        for beta in 2:length(Copied_Tracker[1,:])
            Cell_Type_Family_Plotter = plot!(Copied_Tracker[:,beta],linestyle=:dash)
        end
    end

    display(Cell_Type_Family_Plotter)
end

function Generational_Tracker_Initialisation()
    TypesString = deepcopy(Cell_Generational_Tracking[1,:])
    TypesString = string.(TypesString,base=2,pad=100)
    Types = zeros(Int16,length(Cell_Generational_Tracking[1,:]),M)
    for i in 1:length(Cell_Generational_Tracking[1,:])
        for j=1:M
            Types[i,j] = (parse(Int16,TypesString[i][j]))
        end
    end
    Original_Tracking = deepcopy(Cell_Generational_Tracking)
    return Original_Tracking, Types
end

function Interpolator(Times,Play_Array)
    Vectors = zeros(length(Times),1+M*2^Splits)
    for i=1:length(Times)
        Index_Interp = findfirst(>=(Times[i]), Play_Array[1,:])
        Vectors[i,:] = Play_Array[:,Index_Interp-1] + (Times[i]-Play_Array[1,Index_Interp-1])*(Play_Array[:,Index_Interp] - Play_Array[:,Index_Interp-1])/(Play_Array[1,Index_Interp] - Play_Array[1,Index_Interp-1])
    end
    return Vectors
end


function One_Generational_Plotter(Cut_Off,Duration,Length_Orbits,Opacity,Name)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1:N_Perturbed

            if Shortest>size(Orbit_Array[i])[2]
                Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
            end
            Total_Length += size(Orbit_Array[i])[2]
        
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1:N_Perturbed
        
        if Cut_Off

            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
            end
        else
            Length_Orbits =  size(Orbit_Array[i,j])[2]
            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
            end
        end

        Collection_Index += Length_Orbits*2^Splits

    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,N_Perturbed,2^Splits,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:N_Perturbed
        
        for tau=1:2^Splits
            tester = Orbit_Array[i][2+(tau-1)*M:1+tau*M,1:Shortest]
            Transformed = transform(Orbital_Model,tester)
            PCA_Orbits_2Gen[i,tau,:,:] = Transformed
        end

    end
    
    
    anim = @animate for k=1:Duration
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen[1,1,1,1:k],PCA_Orbits_2Gen[1,1,2,1:k],PCA_Orbits_2Gen[1,1,3,1:k],legend=false,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)


        for i=1:N_Perturbed
            for tau=1:2^Splits
                PCAPlots = plot!(PCA_Orbits_2Gen[i,tau,1,1:k],PCA_Orbits_2Gen[i,tau,2,1:k],PCA_Orbits_2Gen[i,tau,3,1:k],legend=false,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)
            end
        end

        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 5
    gif(anim, Name, fps = 15)
    
end

function adjoint(a)
    return a'
end


function Coarsed_Variance(Variance_Length,Time_Orbital_Collection,Coarsing,Orbit_Length)
    
    
    #Variance[i] = sum((Time_Orbital_Collection[i,:,:] .- mean(Time_Orbital_Collection[i,:,:],dims=1)).^2)/(M-1)

    
    
    Variance = zeros(Variance_Length)
    Average = zeros(Variance_Length,M)
    
    for i = 1:Variance_Length #For all different times
       Average[i,:] =  mean(Time_Orbital_Collection[i,:,:],dims=1)[1,:]
        for k=1:Orbit_Length #Have to do it for all the orbits
            Temp_Var = 1000
            Add_Var = 0
            
            for j=max(1,i-Coarsing):i+Coarsing 
                Add_Var = sum((Time_Orbital_Collection[j,k,:] .- Average[i,:]).^2)/(M-1)

                if Add_Var < Temp_Var
                    Temp_Var = Add_Var               
                end
            end
            Variance[i] += Temp_Var  

        end
    end
    
    return Variance
end


function Multi_One_Generational_Plotter(Cut_Off,Duration,Length_Orbits,Opacity,Name,Amount_Perturbations,Partial,N_Perturbed)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1:N_Perturbed*Amount_Perturbations

            if Shortest>size(Orbit_Array[i])[2]
                Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
            end
            Total_Length += size(Orbit_Array[i])[2]
        
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed*Amount_Perturbations)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1:N_Perturbed*Amount_Perturbations
        
        if Cut_Off

            for p=0:2^Splits-1

                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
            end
        else
            Length_Orbits =  size(Orbit_Array[i,j])[2]
            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
            end
        end

        Collection_Index += Length_Orbits*2^Splits

    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,N_Perturbed*2^Splits,Amount_Perturbations,3,Shortest)
    PCA_Orbits_Partial = zeros(Float32,Int(N_Perturbed/Partial)*2^Splits,Amount_Perturbations,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:N_Perturbed
        for j=0:Amount_Perturbations-1
        for tau=1:2^Splits
            tester = Orbit_Array[i+N_Perturbed*j][2+(tau-1)*M:1+tau*M,1:Shortest]
            Transformed = transform(Orbital_Model,tester)
            PCA_Orbits_2Gen[(i-1)*2^Splits+tau,j+1,:,:] = Transformed
            if mod(i,Partial) == 0
                PCA_Orbits_Partial[(Int(i/Partial)-1)*2^Splits+tau,j+1,:,:] = Transformed
            end
        end
        end

    end
    Colours = [:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
    
    #PCA_Orbits_Partial = reshape(PCA_Orbits_Partial,(Int(2^Splits*N_Perturbed*Amount_Perturbations/Partial),3,Shortest))
    PCA_Orbits_2Gen_Plot = permutedims(PCA_Orbits_Partial,(4,3,2,1))
    Orbit_Range = Int(N_Perturbed/Partial)*2^Splits
    tick()
    anim = @animate for k=1:Int(Duration/5)
        
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,:],PCA_Orbits_2Gen_Plot[1:k,2,1,:],PCA_Orbits_2Gen_Plot[1:k,3,1,:],c=Colours[1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for p=1:Amount_Perturbations
            PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,:],PCA_Orbits_2Gen_Plot[1:k,2,p,:],PCA_Orbits_2Gen_Plot[1:k,3,p,:],c=Colours[p],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)

        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 5
    tock()
    tick()
    animlong = @animate for k=1:Duration
        
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,:],PCA_Orbits_2Gen_Plot[1:k,2,1,:],PCA_Orbits_2Gen_Plot[1:k,3,1,:],c=Colours[1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for p=1:Amount_Perturbations
            PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,:],PCA_Orbits_2Gen_Plot[1:k,2,p,:],PCA_Orbits_2Gen_Plot[1:k,3,p,:],c=Colours[p],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)

        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 50
    tock()
    abra = gif(anim, Name, fps = 10)
    display(abra)

    kadabra = gif(animlong, Name[1:end-4] * "_Long" *Name[end-3:end], fps = 3)
    display(kadabra)



end


function Multi_One_Generational_Plotter2(Cut_Off,Duration,Length_Orbits,Opacity,Name,Amount_Perturbations,Partial,N_Perturbed,Orbit_Array)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1:N_Perturbed*Amount_Perturbations

            if Shortest>size(Orbit_Array[i])[2]
                Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
            end
            Total_Length += size(Orbit_Array[i])[2]
        
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed*Amount_Perturbations)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1:N_Perturbed*Amount_Perturbations
        
        if Cut_Off

            for p=0:2^Splits-1

                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
            end
        else
            Length_Orbits =  size(Orbit_Array[i,j])[2]
            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
            end
        end

        Collection_Index += Length_Orbits*2^Splits

    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,N_Perturbed*2^Splits,Amount_Perturbations,3,Shortest)
    PCA_Orbits_Partial = zeros(Float32,Int(N_Perturbed/Partial)*2^Splits,Amount_Perturbations,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:N_Perturbed
        for j=0:Amount_Perturbations-1
        for tau=1:2^Splits
            tester = Orbit_Array[i+N_Perturbed*j][2+(tau-1)*M:1+tau*M,1:Shortest]
            Transformed = transform(Orbital_Model,tester)
            PCA_Orbits_2Gen[(i-1)*2^Splits+tau,j+1,:,:] = Transformed
            if mod(i,Partial) == 0
                PCA_Orbits_Partial[(Int(i/Partial)-1)*2^Splits+tau,j+1,:,:] = Transformed
            end
        end
        end

    end
    Colours = [:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
    
    #PCA_Orbits_Partial = reshape(PCA_Orbits_Partial,(Int(2^Splits*N_Perturbed*Amount_Perturbations/Partial),3,Shortest))
    PCA_Orbits_2Gen_Plot = permutedims(PCA_Orbits_Partial,(4,3,2,1))
    Orbit_Range = Int(N_Perturbed/Partial)*2^Splits
    tick()
    anim = @animate for k=1:Int(Duration/5)
        
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,:],PCA_Orbits_2Gen_Plot[1:k,2,1,:],PCA_Orbits_2Gen_Plot[1:k,3,1,:],c=Colours[1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for p=1:Amount_Perturbations
            PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,:],PCA_Orbits_2Gen_Plot[1:k,2,p,:],PCA_Orbits_2Gen_Plot[1:k,3,p,:],c=Colours[p],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)

        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 5
    tock()
    tick()
    animlong = @animate for k=1:Duration
        
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,:],PCA_Orbits_2Gen_Plot[1:k,2,1,:],PCA_Orbits_2Gen_Plot[1:k,3,1,:],c=Colours[1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for p=1:Amount_Perturbations
            PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,:],PCA_Orbits_2Gen_Plot[1:k,2,p,:],PCA_Orbits_2Gen_Plot[1:k,3,p,:],c=Colours[p],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)

        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 50
    tock()
    abra = gif(anim, Name, fps = 10)
    display(abra)

    kadabra = gif(animlong, Name[1:end-4] * "_Long" *Name[end-3:end], fps = 3)
    display(kadabra)

    
    PCAPlots_Final = scatter(PCA_Orbits_2Gen_Plot[end,1,1,:],PCA_Orbits_2Gen_Plot[end,2,1,:],PCA_Orbits_2Gen_Plot[end,3,1,:],title="Final Cell States",c=Colours[1],legend=false)
    for p=1:Amount_Perturbations
        PCAPlots_Final = scatter!(PCA_Orbits_2Gen_Plot[end,1,p,:],PCA_Orbits_2Gen_Plot[end,2,p,:],PCA_Orbits_2Gen_Plot[end,3,p,:],c=Colours[p],legend=false)

    end

    display(PCAPlots_Final)
    savefig(Name[1:end-4] * "_Final.png")
end

function Multi_One_Generational_Plotter3(Cut_Off,Duration,Length_Orbits,Opacity,Name,Amount_Perturbations,Partial)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1:N_Perturbed*Amount_Perturbations

            if Shortest>size(Orbit_Array[i])[2]
                Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
            end
            Total_Length += size(Orbit_Array[i])[2]
        
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed*Amount_Perturbations)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1:N_Perturbed*Amount_Perturbations
        
        if Cut_Off

            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
            end
        else
            Length_Orbits =  size(Orbit_Array[i,j])[2]
            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
            end
        end

        Collection_Index += Length_Orbits*2^Splits

    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,N_Perturbed*2^Splits,Amount_Perturbations,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:N_Perturbed
        for j=1:2^Splits
            for tau=1:Amount_Perturbations
                tester = Orbit_Array[i+ (tau-1)*N_Perturbed][2+(j-1)*M:1+j*M,1:Shortest]
                Transformed = transform(Orbital_Model,tester)
                PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,:,:] = Transformed

            end
        end
    end
    Colours = [:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
    
    PCA_Orbits_2Gen_Plot = permutedims(PCA_Orbits_2Gen,(4,3,2,1))
    anim = @animate for k=1:Duration
        
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1],c=Colours[1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for l=1:Amount_Perturbations
            
            PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,l,:],PCA_Orbits_2Gen_Plot[1:k,2,l,:],PCA_Orbits_2Gen_Plot[1:k,3,l,:],c=Colours[l],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
            
        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 15

    abra = gif(anim, Name, fps = 15)
end

function N_Dimensional_Euclid(Position1,Position2,Dimensions)
    Summed = 0
    for i =1:Dimensions
        Summed += (Position2[i]-Position1[i])^2
    end
    Distance = sqrt(Summed)
    return Distance
end



function Distance_Movement_Matrix(Movement_Matrix)
    
    for i=1:N_Sources #we must loop over all the sources to find a good sink for it

        #First we try to find the closest sinks
        Distance_Indices = (hcat(DistanceMatrix[i,:], 1:N_Sinks))
        Distance_Indices = Distance_Indices[sortperm(Distance_Indices[:, 1]), :]
        Distance_Indices = Int.(Distance_Indices[:,2])
        print(Distance_Indices)

        #Then we check if we can still send anything to this sink
        Displacing = true
        D_Index = 1
        while Displacing
            I_Sink = Distance_Indices[D_Index] #This is the index of the sink we are checking now
            Available_Sink = Sinks_Input[I_Sink] - sum(Movement_Matrix[:,I_Sink])
            print(Sinks_Input[I_Sink])
            if Available_Sink > 0 #It can have more added to it
                #We check if we can send everything to the sink or only partial
                Remaining_Flow = Sources_Output[i] - sum(Movement_Matrix[i,:])
                if Remaining_Flow > Available_Sink
                #If partial we fill a bit and go back to the start to find the next closest sink
                    Movement_Matrix[i,I_Sink] = Available_Sink
                    D_Index += 1
                    print("Partial.     ")
                    #BUT WE HAVE TO CHECK IF WE JUST FILLED IT ALL IN!!!! 
                    if Sources_Output[i] - sum(Movement_Matrix[i,:]) < 0.0001
                        Displacing = false
                        print("Finished After Partial")
                    end

                else  
                #If we can send it all we do and go to the next Source i+1
                    Displacing = false
                    Movement_Matrix[i,I_Sink] = Remaining_Flow
                    print("Finished.     ")

                end
            else #Sink is full
                print("Full.     ")

                #Cannot send more in the sink so we have to check the next one
                D_Index += 1
            end
        end
    end
    Movement_Matrix = round.(Movement_Matrix,digits=4)

    return Movement_Matrix
end

function Cost_Reducing_Rewiring(Attempts,Optimised_Indices)
    print("Initial Costs: ")
    print(sum(Cost_Matrix))
    for yota =1:Attempts
        println(" ")
        #We make a big while loop until a certain condition we decide later
        Cost_Matrix = DistanceMatrix .* Movement_Matrix
        print(sum(Cost_Matrix))
        for theta =1:1   #Old loop that can be removed

            if Optimised_Indices != []
                for kappa=1:length(Optimised_Indices[1,:])
                    Cost_Matrix[Optimised_Indices[1,kappa],Optimised_Indices[2,kappa]] = 0
                end
            end

            #We find the connection that has the highest cost, find the source and the corresponding sink
            Source_Index = argmax(Cost_Matrix)[1]
            Sink_Index = argmax(Cost_Matrix)[2]
            Improvement = 0 #Use this to keep track of best
            Displace = 0 #Keep track of best
            Best_Source = 0 #Keep track
            Best_Sink = 0 #Keep track

            #We let this source connect to other sinks instead and try to change another source that was connected to that sink
            for i=1:N_Sinks #We loop over all other sinks
                if i!=Sink_Index #Make sure it is a different sink
                    for j=1:N_Sources #We loop over all other Sources
                        if j!=Source_Index #Make sure it is a different source
                            #Check how much we may change 
                            Displace_Temp = min(Movement_Matrix[Source_Index,Sink_Index],Movement_Matrix[j,i])
                            #Calculate the improvement
                            #-DM[So,Si] + DM[So,i] We remove So -> Si and make So -> i
                            #-DM[j,i] + DM[j,Si] We remove j -> i and make j -> Si
                            Temp_Improvement = Displace_Temp*(-DistanceMatrix[Source_Index,Sink_Index] + DistanceMatrix[Source_Index,i] -DistanceMatrix[j,i] + DistanceMatrix[j,Sink_Index])

                            #Find the best improvement for this initial Source_Sink combo
                            if Temp_Improvement < Improvement #Found a better improvement
                                Improvement = Temp_Improvement
                                Displace = Displace_Temp
                                Best_Source = j
                                Best_Sink = i
                            end

                        end
                    end
                end
            end
            print("Better amount:  ")
            print(Improvement)
            if Improvement==0
                print("No improvement???")
                print(Displace)
                print("    ")
                print(Sink_Index)
                print(Source_Index)
                if Optimised_Indices == []
                    Optimised_Indices = vcat(Optimised_Indices,[Source_Index;Sink_Index])
                else
                    Optimised_Indices = hcat(Optimised_Indices,[Source_Index;Sink_Index])
                end

            else
                print("WE IMPROVED")
                Movement_Matrix[Source_Index,Sink_Index] -= Displace
                Movement_Matrix[Best_Source,Best_Sink] -= Displace
                Movement_Matrix[Best_Source,Sink_Index] += Displace
                Movement_Matrix[Source_Index,Best_Sink] += Displace


            end

            #Switch the best one and repeat
        end
        print("   ")
        Cost_Matrix = DistanceMatrix .* Movement_Matrix
        print(sum(Cost_Matrix))
    end

    println(" ")
    print("Final Costs: ")
    print(sum(Cost_Matrix)) 
    return sum(Cost_Matrix)
end

function Final_Cell_PCAScatterPlot(Orbit_Array,Single_Plot)
    Final_Cell_States_PCA = zeros(Amount_Large_Perturbations*Amount_Small_Perturbations*2^Splits,M)
    Transformed_Data = zeros(Amount_Small_Perturbations*2^Splits,Amount_Large_Perturbations,2)
    Before_Data = zeros(Amount_Small_Perturbations*2^Splits,Amount_Large_Perturbations,M)
    for j=1:Amount_Large_Perturbations*Amount_Small_Perturbations
        for i=1:2^Splits
            Final_Cell_States_PCA[j+Amount_Large_Perturbations*Amount_Small_Perturbations*(i-1),:] = Orbit_Array[j][2+(i-1)*M:1+i*M,end]
        end
    end
    #This is added in in case we only have the same cell types, then our code would crash
    Final_Cell_States_PCA[1,1] += 1
    
    
    for j=1:Amount_Large_Perturbations
        for i=1:Amount_Small_Perturbations
            for tau=1:2^Splits
                Before_Data[i+(tau-1)*Amount_Small_Perturbations,j,:] = Orbit_Array[i+(j-1)*Amount_Small_Perturbations][2+(tau-1)*M:1+tau*M,end]
            end
        end
    end

    Final_Cell_States_PCA[1:5,1:10] .+= 1
    Final_Cell_PCA = fit(PCA,Final_Cell_States_PCA';maxoutdim =2,pratio =1)
    print(size(Final_Cell_PCA))
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Final_Cell_PCA))
    Final_Cell_States_PCA[1:5,1:10] .-= 1
 

    for j=1:Amount_Large_Perturbations

        Transformed_Data[:,j,:] = transform(Final_Cell_PCA,Before_Data[:,j,:]')'

    end
    Plot_Legend = true
    Colours = [:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
    if Single_Plot == false
        j=1
        abrada = scatter((Transformed_Data[1,j,1],Transformed_Data[1,j,2]),xlabel="PCA 1",ylabel="PCA 2",title="Final Cell States in PCA landscape",c=Colours[j],alpha=0.4,legend=Plot_Legend,label="Initial Noise = " * string(Large_Pert_Radius[j]))
        for j=2:Amount_Large_Perturbations
            abrada = scatter!((Transformed_Data[1,j,1],Transformed_Data[1,j,2]),c=Colours[j],alpha=0.4,legend=Plot_Legend,label="Initial Noise = " * string(Large_Pert_Radius[j]))
        end
        for j=1:Amount_Large_Perturbations
            for l=2:length(Transformed_Data[:,j,1])
                for j=1:Amount_Large_Perturbations
                    abrada = scatter!((Transformed_Data[l,j,1],Transformed_Data[l,j,2]),c=Colours[j],alpha=0.4,label=false)
                end
            end
        end
        savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/PCA_Maps/Scatter_PCA"*Copy_String*".png")

        KernelData = reshape(Transformed_Data, (Amount_Small_Perturbations*2^Splits*Amount_Large_Perturbations, 2))
        
        k = kde((KernelData[:,1], KernelData[:,2]))


        display(contourf(k,
             c = :vik,title="Final Cell State PCA Heatmap",
             linewidth = 0))
        savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/PCA_Maps/Heatmap_"*Copy_String*".png")

        
    else
        j = Single_Plot
        abrada = scatter((Transformed_Data[:,j,1],Transformed_Data[:,j,2]),xlabel="PCA 1",ylabel="PCA 2",title="Final Cell States in PCA landscape",c=Colours[j],alpha=0.4,legend=Plot_Legend,label="Initial Noise = " * string(Large_Pert_Radius[j]))
        
    end
    return abrada
end



function Multi_One_Generational_Plotter4(Cut_Off,Duration,Length_Orbits,Opacity,Name,Amount_Perturbations,Partial,N_Perturbed,Orbit_Array,Final_Epigenetic_Factors,Gene_Colour_Collection)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1:N_Perturbed*Amount_Perturbations

            if Shortest>size(Orbit_Array[i])[2]
                Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
            end
            Total_Length += size(Orbit_Array[i])[2]
        
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed*Amount_Perturbations)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1:N_Perturbed*Amount_Perturbations
        
        if Cut_Off

            for p=0:2^Splits-1

                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
            end
        else
            Length_Orbits =  size(Orbit_Array[i,j])[2]
            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
            end
        end

        Collection_Index += Length_Orbits*2^Splits

    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,Amount_Perturbations,N_Perturbed,2^Splits,3,Shortest)
    PCA_Orbits_Partial = zeros(Float32,Amount_Perturbations,Int(N_Perturbed/Partial),2^Splits,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:N_Perturbed
        for j=0:Amount_Perturbations-1
        for tau=1:2^Splits
            tester = Orbit_Array[i+N_Perturbed*j][2+(tau-1)*M:1+tau*M,1:Shortest]
            Transformed = transform(Orbital_Model,tester)
            PCA_Orbits_2Gen[j+1,i,tau,:,:] = Transformed
            if mod(i-1,Partial) == 0
                PCA_Orbits_Partial[j+1,(Int(i/Partial)),tau,:,:] = Transformed
            end
        end
        end

    end
    Colours = [:red,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
     
    #PCA_Orbits_Partial = reshape(PCA_Orbits_Partial,(Int(2^Splits*N_Perturbed*Amount_Perturbations/Partial),3,Shortest))
    PCA_Orbits_2Gen_Plot = permutedims(PCA_Orbits_Partial,(5,4,3,2,1))
    Orbit_Range = Int(N_Perturbed/Partial)*2^Splits
    
    print(size(PCA_Orbits_2Gen_Plot))
    if false
        for Gene_Colour in Gene_Colour_Collection
            tick()
            
            anim = @animate for k=1:Int(Duration/5)
                i=1
                PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1,1],c=Colours[Final_Epigenetic_Factors[1,1,1,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                for q=1:Int(N_Perturbed/Partial)
                    for p=1:2^Splits
                        for t =1:Amount_Perturbations
                        PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,q,t],PCA_Orbits_2Gen_Plot[1:k,2,p,q,t],PCA_Orbits_2Gen_Plot[1:k,3,p,q,t],c=Colours[Final_Epigenetic_Factors[t,q,p,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                        end
                    end
                end
        
                #for i=1:Int(N_Perturbed/5)
                #    for j=1:Amount_Perturbations
                #        for tau=1:2^Splits
                #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                #        end
                #    end
                #end
                
                #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])
        
                title!("Gene = $Gene_Colour Time = $k")
            end every 5
            tock()
            tick()
            animlong = @animate for k=1:Duration
                
                i=1
                PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1,1],c=Colours[Final_Epigenetic_Factors[1,1,1,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                for q=1:Int(N_Perturbed/Partial)
                    for p=1:2^Splits
                        for t =1:Amount_Perturbations
                        PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,q,t],PCA_Orbits_2Gen_Plot[1:k,2,p,q,t],PCA_Orbits_2Gen_Plot[1:k,3,p,q,t],c=Colours[Final_Epigenetic_Factors[t,q,p,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                        end
                    end
                end
                #for i=1:Int(N_Perturbed/5)
                #    for j=1:Amount_Perturbations
                #        for tau=1:2^Splits
                #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                #        end
                #    end
                #end
                
                #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])
        
                title!("Gene = $Gene_Colour Time = $k")
            end every 50
            tock()
    
            abra = gif(anim, Name[1:end-4]*"_Gene:"*string(Gene_Colour)*Name[end-3:end], fps = 10)
            display(abra)
        
            kadabra = gif(animlong, Name[1:end-4] * "_Long"*"_Gene:"*string(Gene_Colour) *Name[end-3:end], fps = 3)
            display(kadabra)
        
            
            Total_Ratio = principalratio(Orbital_Model)
            Eigen_Values = eigvals(Orbital_Model)
            Eigen_Values = Eigen_Values/sum(Eigen_Values)
            Eigen1 = Eigen_Values[1]
            Eigen2 = Eigen_Values[2]
            Eigen3 = Eigen_Values[3]
            PCAPlots_Final = scatter(PCA_Orbits_2Gen_Plot[end,1,1,1,:],PCA_Orbits_2Gen_Plot[end,2,1,1,:],PCA_Orbits_2Gen_Plot[end,3,1,1,:],c=Colours[Final_Epigenetic_Factors[1,1,1,Gene_Colour]+1],legend=true,label="
            Principal 1: $Eigen1
            Principal 2: $Eigen2]
            Principal 3: $Eigen3")
        
            for q=1:Int(N_Perturbed/Partial)
                for p=1:2^Splits
                    for t =1:Amount_Perturbations
                        PCAPlots_Final = scatter!((PCA_Orbits_2Gen_Plot[end,1,p,q,t],PCA_Orbits_2Gen_Plot[end,2,p,q,t],PCA_Orbits_2Gen_Plot[end,3,p,q,t]),c=Colours[Final_Epigenetic_Factors[t,q,p,Gene_Colour]+1],legend=true,label="")
                    end
                end
            end
            
            
            title!("Gene = $Gene_Colour , Ratio: $Total_Ratio")
            display(PCAPlots_Final)
            savefig(Name[1:end-4] * "_Final_Static_$Gene_Colour.png")
            
        end
    end
    Multi_Colours = [:cyan,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:darkmagenta,:lightcyan4,:olive,:gold4,:lightsalmon,:navajowhite,:pink]
    Only3Types = false
    tick()
    anim = @animate for k=1:Int(Duration/5)
        
        i=1
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
        if Only3Types
            Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
        end
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1,1],c=Multi_Colours[Colour_Index],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for q=1:Int(N_Perturbed/Partial)
            for p=1:2^Splits
                for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,q,t],PCA_Orbits_2Gen_Plot[1:k,2,p,q,t],PCA_Orbits_2Gen_Plot[1:k,3,p,q,t],c=Multi_Colours[Colour_Index],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                end
            end
        end
    
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])
    
        title!("Gene = Multiple Time = $k")
    end every 5
    tock()
    tick()
    animlong = @animate for k=1:2*Duration
        
        i=1
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
        if Only3Types
            Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
        end
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1,1],c=Multi_Colours[Colour_Index],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for q=1:Int(N_Perturbed/Partial)
            for p=1:2^Splits
                for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,q,t],PCA_Orbits_2Gen_Plot[1:k,2,p,q,t],PCA_Orbits_2Gen_Plot[1:k,3,p,q,t],c=Multi_Colours[Colour_Index],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                end
            end
        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])
    
        title!("Gene = Multiple Time = $k")
    end every 50
    tock()
    abra = gif(anim, Name[1:end-4]*Name[end-3:end], fps = 10)
    display(abra)
    
    kadabra = gif(animlong, Name[1:end-4]*Name[end-3:end], fps = 3)
    display(kadabra)
    
    
    
    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    PCAPlots_Final = scatter(PCA_Orbits_2Gen_Plot[end,1,1,1,:],PCA_Orbits_2Gen_Plot[end,2,1,1,:],PCA_Orbits_2Gen_Plot[end,3,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Final = scatter!((PCA_Orbits_2Gen_Plot[end,1,p,q,t],PCA_Orbits_2Gen_Plot[end,2,p,q,t],PCA_Orbits_2Gen_Plot[end,3,p,q,t]),c=Multi_Colours[Colour_Index],legend=false)
            end
        end
    end
    Total_Ratio = principalratio(Orbital_Model)
    Eigen_Values = eigvals(Orbital_Model)
    Eigen_Values = Eigen_Values/sum(Eigen_Values)
    title!("Static Multi Colour, Ratio: $Total_Ratio")
    display(PCAPlots_Final)
    savefig(Name[1:end-4] * "Final_Multi_Colour.png")



    
    PcaPlots_Time_Length = 500
    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    
    PCAPlots_Time1 = plot(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,1,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Time1 = plot!(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,1,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA 1")
            end
        end
    end
    display(PCAPlots_Time1)

    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    
    PCAPlots_Time2 = plot(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,2,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Time2 = plot!(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,2,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA 1")
            end
        end
    end
    display(PCAPlots_Time2)

    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    
    PCAPlots_Time3 = plot(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,3,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Time3 = plot!(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,3,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA 1")
            end
        end
    end
    display(PCAPlots_Time3)
    return Orbital_Model
end

function Evolution_NoEpi(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    
    
    
    #⊗_(Wi, bi) = Wi .* bi
       

    du[1,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,:].+u[2,:].+CArray[:])) .- u[1,:]
    
    #Epigenetic Evolution, no need to worry about boundaries here so simple loop      

    
end




##############################################################################################################################
##############################################################################################################################
##############################################################################################################################








#Load_Generations = [70]
#Load_Generations = vcat(1,Load_Generations) #These are all the generations that we will analyse
Copies = [1] #Can reduce this to just 1,2 or just 1 if we don't want as many copies



#Main variables to change:
Stochastic_Noise = true
N::Int32 = 8 #How many copies of same network we have
#K::Int32 = N  Depricated parameter, do not change, if ever K not defined replace it by N
P = 4 #How many different networks will we use in the evolution run
Generations = 6 #How many generations in the evolution run

#DO NOT CHANGE THESE
Splits::Int32 = 6 #DO NOT CHANGE
SplitTime::Float32 = 0.1 #Time before a splitting event happens, DOES NOT MATTER HERE
M::Int32 = 40 #This is the amount of genes we want in our network
Splitting_Strength = 0.0f0

#CAREFUL WITH M>127 DO NOT INCREASE WITHOUT REWRITING DIFFERENTIATION FUNCTION
#BIT OVERFLOW!!!!!


#Constant and plotting tools


InvSqr::Float32 = 1/sqrt(M)
PCA_Plotting = false 
Orbit_Plotting = true  #Currently not used yet
Epi_Plotting = false #Currently not used yet
Counter = 0 #Used to randomise the mutations, leave to keep same seed
Saved_Generations = [2,4] #These are the generations where we want to save some orbits for


#Diffusion or mean field parameters
Mean_Field = false
Diffusing_System = false
KArray = zeros(Float32,M,1) #Diffusion/Mean Field array is set to zero everywhere
#KArray[1:10] .= 0.1 #Here we can manually adjust which genes will have diffusion















Amount_Large_Perturbations = 1 #How many large perturbations we will have each time
Amount_Small_Perturbations = 1 #How many smaller perturbations we will have around our large perturbation
Large_Pert_Radius = [0,1,5,20]
Small_Pert_Radius = 0
#Perturbations = zeros(32)









Generation_String = string(Load_Generations[1])
Copy_String = string(Copies[1])
Load_Data_String_Load = Data_To_Analyse
#Load_Data_String = Data_To_Analyse
#Copies = [1]
#Load_Generations = [10]
Orbit_Array = 0
Time_Orbital_Collection2 = 0
Orbit_Collection = 0
EpiThroughTime = 0

#To have access after the loop
Interpolated_Orbits = 0
Final_Epigenetic_Factors = 0
Gene_Cells_Colour = 0
Orbital_Model = 0
@showprogress 1 for zeta in Load_Generations
    GC.gc()
    for iota in Copies
        Generation_String = string(zeta)

        Copy_String = string(iota)
        Current_Network_String = "/Gen_"*Generation_String*"_"*Copy_String*".jld"
        Loaded_Data = load(Load_Data_String_Load*Current_Network_String)
        mkpath(Load_Data_String*"/Processed/Gen:"*Generation_String*"/PCA_Maps")
        mkpath(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Orbits")
        mkpath(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance")


        #Here we load and perturb the networks


        Duration = 1000 #For Animation should be 3000
        Length_Orbits = 10000 #For PCA should be 4000
        Opacity = 75
        Partial = 1

        Variance_Length = 2500
        Times = [i for i=1:Variance_Length+50]
        Times_Orbit = [i for i=1:Length_Orbits]


        Perturb_Seed = 12345
        rng=MersenneTwister(Perturb_Seed)
        #Amount_Large_Perturbations = 1 #How many large perturbations we will have each time
        #Amount_Small_Perturbations = 16 #How many smaller perturbations we will have around our large perturbation
        #Large_Pert_Radius = 10
        #Small_Pert_Radius = 3
        CArray_Collection = zeros(Float32,Amount_Large_Perturbations,M)
        PositionArray_Collection = zeros(Float32,Amount_Large_Perturbations,Amount_Small_Perturbations,2,1,M)
        ParameterMatrix_Collection = zeros(Int32,Amount_Large_Perturbations,M,M)
        Final_Epigenetic_Factors = zeros(Int8,Amount_Large_Perturbations,Amount_Small_Perturbations,2^(Splits),M)
        Epi_Factors = zeros(Float16,2^(Splits),M)
        for j=1:Amount_Large_Perturbations
            CArray_Collection[j,:] = Loaded_Data["CArray"]
            ParameterMatrix_Collection[j,:,:] = Loaded_Data["Network"]

            #Write large pert vector here

            Noise_Vector = randn(rng,M).-0.5
            Norm = sqrt(sum(Noise_Vector.^2))
            Noise_Vector = Large_Pert_Radius[j]*Noise_Vector/Norm
            for i=1:Amount_Small_Perturbations


                #Small pert vector here
                Noise_Vector2 = randn(rng,M).-0.5
                Norm = sqrt(sum(Noise_Vector2.^2))
                Noise_Vector2 = rand()^(1/M)*Small_Pert_Radius*Noise_Vector2/Norm

                PositionArray_Collection[j,i,1,1,:] = Loaded_Data["PositionArray"] + Noise_Vector' + Noise_Vector2'

            end
        end

        Evolutionary_Complexity = []


        Random_Noise = 574 #seed for splitting noise

        #Need to define outside for loop, otherwise it will be destroyed afterwards
        Genes = 0





        KLDistance_temp = 0
        BitDistance_temp =  0
        Evolutionary_Complexity = []
        Indices = 0
        Information = zeros(1)
        Ratio = zeros(1)
        Cell_Type_Tracker = []
        Cell_Collection = []
        Evolutionary_Complexity = zeros(N)
        Temporary = []
        Orbit_Array = Array{Matrix{Float16}}(undef,Amount_Large_Perturbations*Amount_Small_Perturbations)
        Interpolated_Orbits = Array{Matrix{Float16}}(undef,Amount_Large_Perturbations*Amount_Small_Perturbations)

        Time_Orbital_Collection = 0

        for yeta=1:Amount_Large_Perturbations
            for x=1:Amount_Small_Perturbations
                #CArray = CArray_Collection[x,:]
                #ParameterMatrix = ParameterMatrix_Collection[x,:,:]
                #PositionArray = PositionArray_Collection[x,:]
                Epi_Factors,Random_Noise,Temporary, EpiThroughTime = Running_Network(CArray_Collection[yeta,:],ParameterMatrix_Collection[yeta,:,:],PositionArray_Collection[yeta,x,:,:,:],Random_Noise)
                Final_Epigenetic_Factors[yeta,x,:,:] = signbit.(Epi_Factors)
                Orbit_Array[x+(yeta-1)*Amount_Small_Perturbations] = Temporary

            end

        end
        #Indices,AllIndices,Evolutionary_Complexity[:],Cell_Collection, Cell_Type_Tracker,KLDistance_temp,BitDistance_temp = Scoring_Indexes(N,Final_Epigenetic_Factors,Cell_Type_Tracker)


        #HERE WE DO THE VARIANCE DATA ANALYSIS

        Orbit_Collection = Orbit_Array

        #Collect all orbits in function of time,


        #Maybe apply PCA to these orbits or keep them like this
        Time_Orbital_Collection = zeros(Float16,length(Times),Amount_Small_Perturbations*2^Splits*Amount_Large_Perturbations,M)
        Time_Orbital_Collection2 = zeros(Float16,length(Times),Amount_Large_Perturbations,Amount_Small_Perturbations*2^Splits,M)



        Orbit_Counter=0
        Larger_Orbit_Counter = 1
        for Orbits in Orbit_Collection
            #Here we do it for the variance
            Interpolated_Vector = Interpolator(Times,Orbits) #We do linear interpolation to have all the same timesteps
            for i=1:2^Splits #We put the different splitted cells all together with the same networks.
                Time_Orbital_Collection[:,Orbit_Counter*2^Splits+i,:] = Interpolated_Vector[:,2+(i-1)*M:1+i*M]
            end
            for k=1:2^Splits
                Time_Orbital_Collection2[:,Larger_Orbit_Counter,mod(Orbit_Counter,Amount_Small_Perturbations)+k,:] = Interpolated_Vector[:,2+M*(k-1):1+M*k]
            end 
            Orbit_Counter += 1
            if mod(Orbit_Counter,Amount_Small_Perturbations) == 0
                Larger_Orbit_Counter += 1
            end


            #Here we do it for the orbits
            Interpolated_Vector2 = Interpolator(Times_Orbit,Orbits)
            Interpolated_Orbits[Orbit_Counter] = permutedims(Interpolated_Vector2,[2,1])

        end
        #Calculate variance in gene expression in function of time



        Variance = zeros(Variance_Length)
        for i=1:Variance_Length
            Variance[i] = sum((Time_Orbital_Collection[i,:,:] .- mean(Time_Orbital_Collection[i,:,:],dims=1)).^2)/(M-1)
        end



        Spatial_Var = Coarsed_Variance(Variance_Length,Time_Orbital_Collection,10,Amount_Large_Perturbations*Amount_Small_Perturbations*2^Splits)
        aba = plot(Variance,label="Temporal",xlabel="Time",ylabel="Variance",title="Spatial and Temporal Variance of all Orbits")
        aba = plot!(Spatial_Var,label="Spatial")
        aba =vline!(SplitTime*[i for i=1:Splits],colour=:"red",linealpha=:0.3,label=false)
        display(aba)
        
        savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance/Spatial_Temporal_"*Copy_String*".png")
        #savefig("Data_All/Data_Folder/TestFile/TestImage.png")



        aca = plot(Variance[1:250],label="Temporal",xlabel="Time",ylabel="Variance",title="Spatial and Temporal Variance of all Orbits")
        aca = plot!(Spatial_Var[1:250],label="Spatial")
        aca =vline!(SplitTime*[i for i=1:Splits],colour=:"red",linealpha=:0.3,label=false)
        ylims!(0,max(maximum(Variance[3:250]),maximum(Spatial_Var[3:250]))*1.3)

        display(aca)
        
        savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance/Spatial_Temporal_Zoomed_"*Copy_String*".png")


        if false

            Seperate_Variance = zeros(Amount_Large_Perturbations,Variance_Length)
            for j=1:Amount_Large_Perturbations
                for i=1:Variance_Length
                    Seperate_Variance[j,i] = sum((Time_Orbital_Collection2[i,j,:,:] .- mean(Time_Orbital_Collection2[i,j,:,:],dims=1)).^2)/(M-1)
                end

            end

            ada = plot(Seperate_Variance[1,1:250],label=false,xlabel="Time",ylabel="Variance",title="Temporal Variance of Specific I.C.")
            for j=2:Amount_Large_Perturbations
                ada = plot!(Seperate_Variance[j,1:250],label=false)
            end
            display(ada)

            savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance/Temporal_Zoomed_IC_"*Copy_String*".png")


            aea = plot(Seperate_Variance[1,1:2500],label=false,xlabel="Time",ylabel="Variance",title="Temporal Variance of Specific I.C.")
            for j=2:Amount_Large_Perturbations
                aea = plot!(Seperate_Variance[j,1:2500],label=false)
            end
            display(aea)

            savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance/Temporal_IC_"*Copy_String*".png")

            Spatial_Sep = Coarsed_Variance(Variance_Length,Time_Orbital_Collection[:,1:Amount_Small_Perturbations*2^Splits,:],25,Amount_Small_Perturbations*2^Splits)
            afa = plot(Spatial_Sep[1:250],label=false,xlabel="Time",ylabel="Variance",title="Spatial Variance of Specific I.C.")
            for j=1:Amount_Large_Perturbations-1
                Spatial_Sep = Coarsed_Variance(Variance_Length,Time_Orbital_Collection[:,1+j*Amount_Small_Perturbations*2^Splits:(j+1)*Amount_Small_Perturbations*2^Splits,:],25,Amount_Small_Perturbations*2^Splits)

                afa = plot!(Spatial_Sep[1:250],label=false)
            end

            display(afa)

            savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance/Spatial_Zoomed_IC_"*Copy_String*".png")
        end




        #Perturbations = [0 for i=1:32]
        for kappa=1:Amount_Large_Perturbations
            println(kappa)
            display(Final_Cell_PCAScatterPlot(Orbit_Array,kappa)) #Don't really need this one
        end
        
        display(Final_Cell_PCAScatterPlot(Orbit_Array,false));
        
        Gene_Cells_Colour = [1,2,3,4]
        print("Before the Multi Big One")
        GC.gc()
        Orbital_Model = (Multi_One_Generational_Plotter4(true,Duration,Length_Orbits,Opacity,Load_Data_String*"/Processed/Gen:"*Generation_String*"/Orbits/Genes_"*Copy_String*".gif",Amount_Large_Perturbations,Partial,Amount_Small_Perturbations,Interpolated_Orbits,Final_Epigenetic_Factors,Gene_Cells_Colour));
    






        Copy_String = string(1)
        Current_Network_String = "/Gen_"*Generation_String*"_"*Copy_String*".jld"
        Loaded_Data = load(Load_Data_String_Load*Current_Network_String)
        
        CArray = Loaded_Data["CArray"]
        ParameterMatrix = Loaded_Data["Network"]
        

        
        TimesWeCheck = [1,5,10,25,40,60,80,100,150,200,300,500,1000,4000,20000]
        TimeChecks = length(TimesWeCheck)
        Cell_Gatherings = zeros(TimeChecks,2,2^Splits,M)
        CycleAmount = zeros(TimeChecks)
        UniqueCycleAmount = zeros(TimeChecks)
        FixPointAmount = zeros(TimeChecks)
        
        Interpolated_Cells = Interpolator(TimesWeCheck,Orbit_Array[1])
        Interpolated_EPIs = Interpolator(TimesWeCheck,EpiThroughTime)
        for i=1:2^Splits
            Cell_Gatherings[:,1,i,:] = Interpolated_Cells[:,2+(i-1)*M:1+(i)*M]
            Cell_Gatherings[:,2,i,:] = Interpolated_EPIs[:,2+(i-1)*M:1+(i)*M]
        end
        
        
        Copy_String = string(1)
        Current_Network_String = "/Gen_"*Generation_String*"_"*Copy_String*".jld"
        Loaded_Data = load(Load_Data_String_Load*Current_Network_String)
        
        CArray = Loaded_Data["CArray"]
        ParameterMatrix = Loaded_Data["Network"]
        FixPoint_Positions = Array{Float64}(undef,0)
        Counter_Cycle = 0
        XPositionTol = 0
        
        
        
        
        for t = 1:TimeChecks
            Counter_Cycle = 0
            FixPoint_Positions = Array{Float64}(undef,0)
            VectorTest = []
            for k = 1:2^Splits
                PositionArray = Cell_Gatherings[t,:,k,:]
                
                Parameters = (ParameterMatrix, CArray) #We have to update the splits every rerun, rest stays same
                tspan = (Float32(0.0),100) #How long ODE will run
                
                prob = ODEProblem(Evolution_NoEpi,PositionArray,tspan,Parameters) #ODEdefining
                
                
                
                
                sol = solve(prob,BS3())
                aba = plot(sol.t,sol[1,1,:])
                for j=2:40
                    aba = plot!(sol.t,sol[1,j,:],legend=false,title=k)
                end
                #if t> 10
                #    display(aba)
                #end
                
                
                
                
                
                
                
                XVector = sol[1,:,end]
                EpiVector = sol[2,:,end]
                SumOfDerivates = sum(abs.(tanh.(40*(InvSqr*ParameterMatrix*XVector.+EpiVector.+CArray[:])) .- XVector))
                DerivativeTol = 0.1
                XPositionTol = 0.1
                
                #if t> 10   
                    #print(SumOfDerivates)
                #end
                
                if SumOfDerivates < DerivativeTol
                    if FixPoint_Positions == [] #Still empty
                        FixPoint_Positions = vcat(FixPoint_Positions,XVector)
                    else #Already has a point in it
                        NewFixPoint = true
                        #print("The distance to the fixpoints are:")
                        for i=1:length(FixPoint_Positions[1,:]) #Here we check over all the existing ones to see if it is not close to one already existing
                            #println(sqrt(sum((FixPoint_Positions[:,i] .- XVector).^2)))
                            if sqrt(sum((FixPoint_Positions[:,i] .- XVector).^2)) < XPositionTol
                                NewFixPoint = false
                            end    
                        end
                        if NewFixPoint #If it is a new fixpoint we add it in now
                            FixPoint_Positions = hcat(FixPoint_Positions,XVector)
                        end
                    end
                else
                    #Exception, we have a cycle? try to plot it
                    
                    CycleToleranceActive = 0.01
                    Active_Gene = zeros(M)
                    
                    prob2 = ODEProblem(Evolution_NoEpi,sol[:,:,end],(Float32(0.0),20),Parameters) #ODEdefining
                    sol2 = solve(prob2,BS3())
                    
                    for j=1:M
                        if abs(maximum(sol2[1,j,:].-mean(sol2[1,j,:]))) > CycleToleranceActive
                            Active_Gene[j] = 1
                        end
                    end
                    if VectorTest == []
                        VectorTest = vcat(VectorTest,Active_Gene)
                    else
                        New_Cycle = true
                        for tau in length(VectorTest[1,:])
                            if Active_Gene == VectorTest[:,tau]
                                New_Cycle = false
                            end
                        end
                        if New_Cycle
                            VectorTest = hcat(VectorTest,Active_Gene)
                        end
                    end
        
        
        
                    
                    Counter_Cycle += 1
                end
            end
            try FixPointAmount[t] = length(FixPoint_Positions[1,:])
        
            catch 
                FixPointAmount[t] = 0
            else
                FixPointAmount[t] = length(FixPoint_Positions[1,:])
            end
            CycleAmount[t] = Counter_Cycle
            try UniqueCycleAmount[t] = length(VectorTest[1,:])
        
            catch
                UniqueCycleAmount[t] = 0
            else
                UniqueCycleAmount[t] = length(VectorTest[1,:])
            end
        end
        
        aba = plot(TimesWeCheck,FixPointAmount,label="Fix Points")
        aba = plot!(TimesWeCheck,CycleAmount,label="Cycles (DoubleCounted)",xaxis=:log,title="PositionalTolerance="*string(XPositionTol))
        plot!(TimesWeCheck,UniqueCycleAmount,label="Cycles (Unique)")
        display(aba)

    end
end
    








In [ ]:
Gene_To_Plot = [1,2,3,4,6,30,32]
Gene_To_Plot = [i for i=1:40]
Cells = [i for i=1:8]

Orbit_Array = Interpolated_Orbits
#Multi_One_Generational_Plotter4(true,Duration,Length_Orbits,Opacity,Load_Data_String*"/Processed/Gen:"*Generation_String*"/Orbits/Genes_"*Copy_String*".gif",Amount_Large_Perturbations,Partial,Amount_Small_Perturbations,Interpolated_Orbits,Final_Epigenetic_Factors,Gene_Cells_Colour));
#Multi_One_Generational_Plotter4(Cut_Off,Duration,Length_Orbits,Opacity,Name,Amount_Perturbations,Partial,N_Perturbed,Orbit_Array,Final_Epigenetic_Factors,Gene_Colour_Collection)
Amount_Perturbations = Amount_Large_Perturbations #1
N_Perturbed = Amount_Small_Perturbations  #1
Cut_Off = true #Remove later times from the PCA dimension
Length_Orbits = 10000
Partial = 1
Total_Length = 0
Shortest = size(Orbit_Array[1,1])[2]




#In this option we take all the lengths and concatenate them, however this
#may create a bias in the PCA analysis towards only final cell states
for i=1:N_Perturbed*Amount_Perturbations

        if Shortest>size(Orbit_Array[i])[2]
            Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
        end
        Total_Length += size(Orbit_Array[i])[2]
    
end
if Cut_Off
    PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed*Amount_Perturbations)
else
    PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
end
Collection_Index=0


for i=1:N_Perturbed*Amount_Perturbations
    
    if Cut_Off

        for p=0:2^Splits-1

            PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
        end
    else
        Length_Orbits =  size(Orbit_Array[i,j])[2]
        for p=0:2^Splits-1
            PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
        end
    end

    Collection_Index += Length_Orbits*2^Splits

end


Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)

PCA_Initial_Orbit_Array = [] #Emptied to save space
PCA_Orbits_2Gen = zeros(Float32,Amount_Perturbations,N_Perturbed,2^Splits,3,Shortest)
PCA_Orbits_Partial = zeros(Float32,Amount_Perturbations,Int(N_Perturbed/Partial),2^Splits,3,Shortest)
print(" ")
print("The Principalratio is: ")
print(principalratio(Orbital_Model))
for i=1:N_Perturbed
    for j=0:Amount_Perturbations-1
    for tau=1:2^Splits
        tester = Orbit_Array[i+N_Perturbed*j][2+(tau-1)*M:1+tau*M,1:Shortest]
        Transformed = transform(Orbital_Model,tester)
        PCA_Orbits_2Gen[j+1,i,tau,:,:] = Transformed
        if mod(i-1,Partial) == 0
            PCA_Orbits_Partial[j+1,(Int(i/Partial)),tau,:,:] = Transformed
        end
    end
    end

end
Colours = [:red,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
 
#PCA_Orbits_Partial = reshape(PCA_Orbits_Partial,(Int(2^Splits*N_Perturbed*Amount_Perturbations/Partial),3,Shortest))
PCA_Orbits_2Gen_Plot = permutedims(PCA_Orbits_Partial,(5,4,3,2,1))
Orbit_Range = Int(N_Perturbed/Partial)*2^Splits

print(size(PCA_Orbits_2Gen_Plot))


function Interpolator(Times,Play_Array)
    Vectors = zeros(length(Times),1+M*2^Splits)
    for i=1:length(Times)
        Index_Interp = findfirst(>=(Times[i]), Play_Array[1,:])
        Vectors[i,:] = Play_Array[:,Index_Interp-1] + (Times[i]-Play_Array[1,Index_Interp-1])*(Play_Array[:,Index_Interp] - Play_Array[:,Index_Interp-1])/(Play_Array[1,Index_Interp] - Play_Array[1,Index_Interp-1])
    end
    return Vectors
end

Interpolated_Epi = Interpolator([i for i=1:Length_Orbits],EpiThroughTime)

Interpolated_Epi_Reshaped = reshape(Interpolated_Epi[:,2:end], (Length_Orbits,M,2^Splits))

Interpolated_Orbits_Reshaped = reshape(Interpolated_Orbits[1][2:end,:]', (Length_Orbits,M,2^Splits));


function Orbits_Final_Cell_Type(Time_Lower,Time_Upper,Dimension_To_Plot,Samples_To_Plot,Orbits_To_Plot,Updating)
    #Orbits[Times,Dimension,Different_Cells]
    Colours = [:red,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:orange,:magenta,:mistyrose3,:darkgreen,:gold4,:salmon1]
    if Updating
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1

        aca =  plot!([Time_Lower:Time_Upper],Orbits_To_Plot[Time_Lower:Time_Upper,Dimension_To_Plot,Samples_To_Plot[1]],legend=false,xscale=:log10,linewidth=2,linestyle=:dot,linecolor=Colours[Colour_Index])
        for j=2:size(Samples_To_Plot)[1]
            Colour_Index = Final_Epigenetic_Factors[1,1,j,1]+Final_Epigenetic_Factors[1,1,j,2]*2+Final_Epigenetic_Factors[1,1,j,3]*4+Final_Epigenetic_Factors[1,1,j,4]*8+1
            aca =  plot!([Time_Lower:Time_Upper],Orbits_To_Plot[Time_Lower:Time_Upper,Dimension_To_Plot,Samples_To_Plot[j]],legend=false,xscale=:log10,linewidth=2,linestyle=:dot,linecolor=Colours[Colour_Index])
        end
    else
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1

        aca =  plot([Time_Lower:Time_Upper],Orbits_To_Plot[Time_Lower:Time_Upper,Dimension_To_Plot,Samples_To_Plot[1]],legend=false,xscale=:log10,linecolor=Colours[Colour_Index],title="Dimension = $Dimension_To_Plot")
        for j=2:size(Samples_To_Plot)[1]
            Colour_Index = Final_Epigenetic_Factors[1,1,j,1]+Final_Epigenetic_Factors[1,1,j,2]*2+Final_Epigenetic_Factors[1,1,j,3]*4+Final_Epigenetic_Factors[1,1,j,4]*8+1
            aca =  plot!([Time_Lower:Time_Upper],Orbits_To_Plot[Time_Lower:Time_Upper,Dimension_To_Plot,Samples_To_Plot[j]],legend=false,xscale=:log10,linecolor=Colours[Colour_Index])
        end
    end
    return aca
end
Time_Lower = 10
#Orbits[Times,Dimension,Different_Cells]
aaa = Orbits_Final_Cell_Type(Time_Lower,3000,1,[1,3,4,6,11,12],PCA_Orbits_2Gen_Plot[:,:,:,1,1],false)
display(aaa)
aba = Orbits_Final_Cell_Type(Time_Lower,3000,2,[1,3,4,6,11,12],PCA_Orbits_2Gen_Plot[:,:,:,1,1],false)
display(aba)
aca = Orbits_Final_Cell_Type(Time_Lower,3000,3,[1,3,4,6,11,12],PCA_Orbits_2Gen_Plot[:,:,:,1,1],false)
display(aca)
for tau in Gene_To_Plot
    ada = Orbits_Final_Cell_Type(Time_Lower,3000,tau,Cells,Interpolated_Orbits_Reshaped,false)
    ada = Orbits_Final_Cell_Type(Time_Lower,3000,tau,Cells,Interpolated_Epi_Reshaped,true)
    display(ada)
end

Individual_Cells = reshape(Interpolated_Orbits[1][2:end,:],(2^Splits,M,Length_Orbits))
1+1

function Active_Counter(Tolerance, TimeFrame,Cell_Orbit)
    Activity = ones(Length_Orbits)*M
    for t=TimeFrame+1:Length_Orbits
        Mean_Single = mean(Cell_Orbit[:,t-TimeFrame:t],dims=2)
        Difference = abs.(Cell_Orbit[:,t-TimeFrame] .- Mean_Single)  
        BitMatrixValue = Difference .> ones(M)*Tolerance
        
        for l=1:TimeFrame
            Difference = abs.(Cell_Orbit[:,t-TimeFrame+l] .- Mean_Single)  
            BitMatrixValue2 = Difference .> ones(M)*Tolerance
            BitMatrixValue = BitMatrixValue .| BitMatrixValue2
        end
        Activity[t] = sum(BitMatrixValue)
    end
    return Activity
end

Test = Active_Counter(0.3, 10,Individual_Cells[1,:,:])
aba = plot(Test[1:end],xscale=:log10)
for j=2:2^Splits
    Test = Active_Counter(0.3, 10,Individual_Cells[j,:,:])
    aba = plot!(Test[1:end],xscale=:log10)
end
display(aba)

In [ ]:
using ChaosTools
using StaticArrays

function Gene_ODE(u, Parameters, time)
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    Epi_Factors = Parameters[3]
    test = InvSqr*ParameterMatrix[1,:]'*u[:]
    #vector = Vector{Float16}([])
    #append!(vector,tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1])
    vector = [tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1]]
    for c=2:M
        append!(vector,tanh(40*(InvSqr*ParameterMatrix[c,:]'*u[:]+Epi_Factors[c]+CArray[c])) - u[c])
    end
    dx = @SVector [vector[k] for k in 1:M]

    return dx
end  

Times = [10,30,50,70,90,110,130,150,200,300,500,1000,2000,5000,10000]
Lyapunov_In_Time = zeros(64,length(Times),M)

for Cell=1:10
    
    for l = 1:length(Times)
        Time = Times[l]
        lor = CoupledODEs(Gene_ODE, Float64.(Interpolated_Orbits_Reshaped[Time,:,Cell]), [ParameterMatrix,CArray,Interpolated_Epi_Reshaped[Time,:,Cell]])
        λλ = lyapunovspectrum(lor, 100; Δt = 0.1)
        Lyapunov_In_Time[Cell,l,:] = λλ
    end
    aba = plot(Times,Lyapunov_In_Time[Cell,:,:],xscale=:log10,legend=false)
    display(aba)
end


In [ ]:
using ChaosTools
using StaticArrays

function Gene_ODE(u, Parameters, time)
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    Epi_Factors = Parameters[3]
    test = InvSqr*ParameterMatrix[1,:]'*u[:]
    #vector = Vector{Float16}([])
    #append!(vector,tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1])
    vector = [tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1]]
    for c=2:M
        append!(vector,tanh(40*(InvSqr*ParameterMatrix[c,:]'*u[:]+Epi_Factors[c]+CArray[c])) - u[c])
    end
    dx = @SVector [vector[k] for k in 1:M]

    return dx
end  

Times = [10,30,50,70,90,110,130,150,200,300,500,1000,2000]
Lyapunov_In_Time = zeros(64,length(Times),M)

for Cell=1:5
    
    for l = 1:length(Times)
        Time = Times[l]
        lor = CoupledODEs(Gene_ODE, Float64.(Interpolated_Orbits_Reshaped[Time,:,Cell]), [ParameterMatrix,CArray,Interpolated_Epi_Reshaped[Time,:,Cell]])
        λλ = lyapunovspectrum(lor, 1000; Δt = 0.1)
        Lyapunov_In_Time[Cell,l,:] = λλ
    end
    aba = plot(Times,Lyapunov_In_Time[Cell,:,:],xscale=:log10,legend=false)
    display(aba)
end


In [ ]:
using ChaosTools
using StaticArrays

function Gene_ODE(u, Parameters, time)
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    Epi_Factors = Parameters[3]
    test = InvSqr*ParameterMatrix[1,:]'*u[:]
    #vector = Vector{Float16}([])
    #append!(vector,tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1])
    vector = [tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1]]
    for c=2:M
        append!(vector,tanh(40*(InvSqr*ParameterMatrix[c,:]'*u[:]+Epi_Factors[c]+CArray[c])) - u[c])
    end
    dx = @SVector [vector[k] for k in 1:M]

    return dx
end  

Times = [10,30,50,70,90,110,130,150,200,300,500,1000,2000]
Lyapunov_In_Time = zeros(64,length(Times),M)

for Cell=1:5
    
    for l = 1:length(Times)
        Time = Times[l]
        lor = CoupledODEs(Gene_ODE, Float64.(Interpolated_Orbits_Reshaped[Time,:,Cell]), [ParameterMatrix,CArray,Interpolated_Epi_Reshaped[Time,:,Cell]])
        λλ = lyapunovspectrum(lor, 1000; Δt = 0.1)
        Lyapunov_In_Time[Cell,l,:] = λλ
    end
    aba = plot(Times,Lyapunov_In_Time[Cell,:,:],xscale=:log10,legend=false)
    aba = ylims!((-2,0.5))
    display(aba)
end


In [ ]:
using ChaosTools
using StaticArrays

function Gene_ODE(u, Parameters, time)
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    Epi_Factors = Parameters[3]
    test = InvSqr*ParameterMatrix[1,:]'*u[:]
    #vector = Vector{Float16}([])
    #append!(vector,tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1])
    vector = [tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1]]
    for c=2:M
        append!(vector,tanh(40*(InvSqr*ParameterMatrix[c,:]'*u[:]+Epi_Factors[c]+CArray[c])) - u[c])
    end
    dx = @SVector [vector[k] for k in 1:M]

    return dx
end  

Times = [10,30,50,70,90,110,130,150,200,300,500,1000,2000]
Lyapunov_In_Time = zeros(64,length(Times),M)

for Cell=1:5
    
    for l = 1:length(Times)
        Time = Times[l]
        lor = CoupledODEs(Gene_ODE, Float64.(Interpolated_Orbits_Reshaped[Time,:,Cell]), [ParameterMatrix,CArray,Interpolated_Epi_Reshaped[Time,:,Cell]])
        λλ = lyapunovspectrum(lor, 1000; Δt = 0.01)
        Lyapunov_In_Time[Cell,l,:] = λλ
    end
    aba = plot(Times,Lyapunov_In_Time[Cell,:,:],xscale=:log10,legend=false)
    aba = ylims!((-2,0.5))
    display(aba)
end


In [ ]:
using ChaosTools
using StaticArrays

function Gene_ODE(u, Parameters, time)
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    Epi_Factors = Parameters[3]
    test = InvSqr*ParameterMatrix[1,:]'*u[:]
    #vector = Vector{Float16}([])
    #append!(vector,tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1])
    vector = [tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1]]
    for c=2:M
        append!(vector,tanh(40*(InvSqr*ParameterMatrix[c,:]'*u[:]+Epi_Factors[c]+CArray[c])) - u[c])
    end
    dx = @SVector [vector[k] for k in 1:M]

    return dx
end  

Times = [10,30,50,70,90,110,130,150,200,300,500,1000,2000]
Lyapunov_In_Time = zeros(64,length(Times),M)

for Cell=1:5
    
    for l = 1:length(Times)
        Time = Times[l]
        lor = CoupledODEs(Gene_ODE, Float64.(Interpolated_Orbits_Reshaped[Time,:,Cell]), [ParameterMatrix,CArray,Interpolated_Epi_Reshaped[Time,:,Cell]])
        λλ = lyapunovspectrum(lor, 1000; Δt = 0.05)
        Lyapunov_In_Time[Cell,l,:] = λλ
    end
    aba = plot(Times,Lyapunov_In_Time[Cell,:,:],xscale=:log10,legend=false)
    aba = ylims!((-2,0.5))
    display(aba)
end


In [ ]:
using ChaosTools
using StaticArrays

function Gene_ODE(u, Parameters, time)
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    Epi_Factors = Parameters[3]
    test = InvSqr*ParameterMatrix[1,:]'*u[:]
    #vector = Vector{Float16}([])
    #append!(vector,tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1])
    vector = [tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1]]
    for c=2:M
        append!(vector,tanh(40*(InvSqr*ParameterMatrix[c,:]'*u[:]+Epi_Factors[c]+CArray[c])) - u[c])
    end
    dx = @SVector [vector[k] for k in 1:M]

    return dx
end  

Times = [10,30,40,50,60,70,80,90,100,110,120,130,140,150,175,200,300,500,1000,2000]
Lyapunov_In_Time = zeros(64,length(Times),M)

for Cell=1:5
    
    for l = 1:length(Times)
        Time = Times[l]
        lor = CoupledODEs(Gene_ODE, Float64.(Interpolated_Orbits_Reshaped[Time,:,Cell]), [ParameterMatrix,CArray,Interpolated_Epi_Reshaped[Time,:,Cell]])
        λλ = lyapunovspectrum(lor, 1000; Δt = 0.05)
        Lyapunov_In_Time[Cell,l,:] = λλ
    end
    aba = plot(Times,Lyapunov_In_Time[Cell,:,:],xscale=:log10,legend=false)
    aba = ylims!((-2,0.5))
    display(aba)
end


In [ ]:
using ChaosTools
using StaticArrays

function Gene_ODE(u, Parameters, time)
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    Epi_Factors = Parameters[3]
    test = InvSqr*ParameterMatrix[1,:]'*u[:]
    #vector = Vector{Float16}([])
    #append!(vector,tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1])
    vector = [tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1]]
    for c=2:M
        append!(vector,tanh(40*(InvSqr*ParameterMatrix[c,:]'*u[:]+Epi_Factors[c]+CArray[c])) - u[c])
    end
    dx = @SVector [vector[k] for k in 1:M]

    return dx
end  

Times = [10,30,40,50,60,70,80,90,100,110,120,130,140,150,175,200,300,500,1000,2000]
Lyapunov_In_Time = zeros(64,length(Times),M)

for Cell=1:5
    
    for l = 1:length(Times)
        Time = Times[l]
        lor = CoupledODEs(Gene_ODE, Float64.(Interpolated_Orbits_Reshaped[Time,:,Cell]), [ParameterMatrix,CArray,Interpolated_Epi_Reshaped[Time,:,Cell]])
        λλ = lyapunovspectrum(lor, 10000; Δt = 0.005)
        Lyapunov_In_Time[Cell,l,:] = λλ
    end
    aba = plot(Times,Lyapunov_In_Time[Cell,:,:],xscale=:log10,legend=false)
    aba = ylims!((-2,0.5))
    display(aba)
end


In [ ]:
using ChaosTools
using StaticArrays

function Gene_ODE(u, Parameters, time)
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    Epi_Factors = Parameters[3]
    test = InvSqr*ParameterMatrix[1,:]'*u[:]
    #vector = Vector{Float16}([])
    #append!(vector,tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1])
    vector = [tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1]]
    for c=2:M
        append!(vector,tanh(40*(InvSqr*ParameterMatrix[c,:]'*u[:]+Epi_Factors[c]+CArray[c])) - u[c])
    end
    dx = @SVector [vector[k] for k in 1:M]

    return dx
end  

Times = [10,30,40,50,60,70,80,90,100,110,120,130,140,150,175,200,300,500,1000,2000]
Lyapunov_In_Time = zeros(64,length(Times),M)

for Cell=1:5
    
    for l = 1:length(Times)
        Time = Times[l]
        lor = CoupledODEs(Gene_ODE, Float64.(Interpolated_Orbits_Reshaped[Time,:,Cell]), [ParameterMatrix,CArray,Interpolated_Epi_Reshaped[Time,:,Cell]])
        λλ = lyapunovspectrum(lor, 10000; Δt = 0.05)
        Lyapunov_In_Time[Cell,l,:] = λλ
    end
    aba = plot(Times,Lyapunov_In_Time[Cell,:,:],xscale=:log10,legend=false)
    aba = ylims!((-2,0.5))
    display(aba)
end


In [ ]:
for m=1:40
    ada = plot(Interpolated_Orbits_Reshaped[1:10000,m,1],title="Gene $m",xaxis=:log10)
    ada = plot!(Interpolated_Orbits_Reshaped[1:10000,m,2])
    ada = plot!(Interpolated_Orbits_Reshaped[1:10000,m,3])
    display(ada)
end

In [ ]:
using ChaosTools
using StaticArrays

function Gene_ODE(u, Parameters, time)
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    Epi_Factors = Parameters[3]
    test = InvSqr*ParameterMatrix[1,:]'*u[:]
    #vector = Vector{Float16}([])
    #append!(vector,tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1])
    vector = [tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1]]
    for c=2:M
        append!(vector,tanh(40*(InvSqr*ParameterMatrix[c,:]'*u[:]+Epi_Factors[c]+CArray[c])) - u[c])
    end
    dx = @SVector [vector[k] for k in 1:M]

    return dx
end  

Times = [10,40,60,80,100,120,140,175,300,500,1000,2000,9000]
Lyapunov_In_Time = zeros(64,length(Times),M)

for Cell=1:3
    
    for l = 1:length(Times)
        Time = Times[l]
        lor = CoupledODEs(Gene_ODE, Float64.(Interpolated_Orbits_Reshaped[Time,:,Cell]), [ParameterMatrix,CArray,Interpolated_Epi_Reshaped[Time,:,Cell]])
        λλ = lyapunovspectrum(lor, 10000; Δt = 0.005)
        Lyapunov_In_Time[Cell,l,:] = λλ
    end
    aba = plot(Times,Lyapunov_In_Time[Cell,:,:],xscale=:log10,legend=false)
    aba = ylims!((-2,0.5))
    display(aba)
end


In [ ]:
using ChaosTools
using StaticArrays

function Gene_ODE(u, Parameters, time)
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    Epi_Factors = Parameters[3]
    test = InvSqr*ParameterMatrix[1,:]'*u[:]
    #vector = Vector{Float16}([])
    #append!(vector,tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1])
    vector = [tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1]]
    for c=2:M
        append!(vector,tanh(40*(InvSqr*ParameterMatrix[c,:]'*u[:]+Epi_Factors[c]+CArray[c])) - u[c])
    end
    dx = @SVector [vector[k] for k in 1:M]

    return dx
end  

Times = [10,40,60,80,100,120,140,175,300,500,1000,2000,9000]
Lyapunov_In_Time = zeros(64,length(Times),M)

for Cell=1:3
    
    for l = 1:length(Times)
        Time = Times[l]
        lor = CoupledODEs(Gene_ODE, Float64.(Interpolated_Orbits_Reshaped[Time,:,Cell]), [ParameterMatrix,CArray,Interpolated_Epi_Reshaped[Time,:,Cell]])
        λλ = lyapunovspectrum(lor, 1000; Δt = 0.005)
        Lyapunov_In_Time[Cell,l,:] = λλ
    end
    aba = plot(Times,Lyapunov_In_Time[Cell,:,:],xscale=:log10,legend=false)
    aba = ylims!((-2,0.5))
    display(aba)
end


In [ ]:
#TimesWeCheck = [1,5,10,25,40,60,80,100,150,200,300,500,1000,4000,20000]
TimesWeCheck = [50,100,150,2000]
TimeChecks = length(TimesWeCheck)


Cell_Gatherings = zeros(TimeChecks,2,2^Splits,M)
CycleAmount = zeros(TimeChecks)
UniqueCycleAmount = zeros(TimeChecks)
FixPointAmount = zeros(TimeChecks)

Interpolated_Cells = Interpolator(TimesWeCheck,Orbit_Array[1])
Interpolated_EPIs = Interpolator(TimesWeCheck,EpiThroughTime)
for i=1:2^Splits
    Cell_Gatherings[:,1,i,:] = Interpolated_Cells[:,2+(i-1)*M:1+(i)*M]
    Cell_Gatherings[:,2,i,:] = Interpolated_EPIs[:,2+(i-1)*M:1+(i)*M]
end


Copy_String = string(1)
Current_Network_String = "/Gen_"*Generation_String*"_"*Copy_String*".jld"
Loaded_Data = load(Load_Data_String_Load*Current_Network_String)

CArray = Loaded_Data["CArray"]
ParameterMatrix = Loaded_Data["Network"]
FixPoint_Positions = Array{Float64}(undef,0)
Counter_Cycle = 0
XPositionTol = 0
Full_TimeSpan = 200
sol = 0
Sampling_Amounts = 8
Transformed_Data =  zeros(Float32,TimeChecks,2^Splits,3,Full_TimeSpan,Sampling_Amounts)

for q=1:Sampling_Amounts
    Saved_Orbits_All = zeros(Float16,TimeChecks,2^Splits,Full_TimeSpan,M)
    for t = 1:TimeChecks
        Counter_Cycle = 0
        FixPoint_Positions = Array{Float64}(undef,0)
        VectorTest = []
        for k = 1:2^Splits
            PositionArray = Cell_Gatherings[t,:,k,:]
            
            Parameters = (ParameterMatrix, CArray) #We have to update the splits every rerun, rest stays same
            tspan = (Float32(0.0),Full_TimeSpan) #How long ODE will run
            
            prob = SDEProblem(Evolution_NoEpi,σ_Genes,PositionArray,tspan,Parameters) #ODEdefining
            
            
            
            
            sol = solve(prob,SRA3(),dtmin=0.1,force_dtmin=true)
            if false 
                aba = plot(sol.t,sol[1,1,:])
                for j=2:40
                    aba = plot!(sol.t,sol[1,j,:],legend=false,title=TimesWeCheck[t])
                end
                if t> 1 && (k==1) 
                    
                    display(aba)
                end
            end
            for time=1:Full_TimeSpan
                Saved_Orbits_All[t,k,time,:] = sol(time)[1,:]
            end
            
            
            
            
            XVector = sol[1,:,end]
            EpiVector = sol[2,:,end]
            SumOfDerivates = sum(abs.(tanh.(40*(InvSqr*ParameterMatrix*XVector.+EpiVector.+CArray[:])) .- XVector))
            DerivativeTol = 0.1
            XPositionTol = 0.1
            
        end
            
    
    
    end

    for i=1:TimeChecks  
        for tau=1:2^Splits
            Tested_Transform = transform(Orbital_Model,Saved_Orbits_All[i,tau,:,:]')        
            Transformed_Data[i,tau,:,:,q] = Tested_Transform
        end
    end
    
end
## Save somehow the data/orbits of these frozen versions somewhere


# Plot this for the PCA as their axis

Multi_Colours = [:cyan,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:darkmagenta,:lightcyan4,:olive,:gold4,:lightsalmon,:navajowhite,:pink]




PCADim1 = 1
PCADim2 = 2





function PCA_Individual_Colour_Plotting(PcaDim1,PcaDim2,Time_Index_To_Plot,Colour_Index)
    Colour_Counter = 0
    for Colouring =Colour_Index:Colour_Index
        Plotting_Something = false
        plot([0])
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    
    
        if Colour_Index == Colouring
            Colour_Counter = mod(Colour_Counter,16) + 1
            Plotting_Something = true
            PCAPlots_TimeFilter = plot!(Transformed_Data[Time_Index_To_Plot,1,PcaDim1,:,1],Transformed_Data[Time_Index_To_Plot,1,PcaDim2,:,1],c=Multi_Colours[Colour_Counter],legend=false,opacity=0.2)
            PCAPlots_TimeFilter = scatter!((Transformed_Data[Time_Index_To_Plot,1,PcaDim1,end,1],Transformed_Data[Time_Index_To_Plot,1,PcaDim2,end,1]),c=Multi_Colours[Colour_Counter])

        end
        for q=1:Sampling_Amounts
            for p=1:2^Splits
                for t =1:1
                    Colour_Index = Final_Epigenetic_Factors[t,1,p,1]+Final_Epigenetic_Factors[t,1,p,2]*2+Final_Epigenetic_Factors[t,1,p,3]*4+Final_Epigenetic_Factors[t,1,p,4]*8+1
    
                    if Colour_Index == Colouring
                        Colour_Counter = mod(Colour_Counter,16) + 1
                        Plotting_Something = true
                        PCAPlots_TimeFilter = plot!(Transformed_Data[Time_Index_To_Plot,p,PcaDim1,:,q],Transformed_Data[Time_Index_To_Plot,p,PcaDim2,:,q],c=Multi_Colours[Colour_Counter],legend=false,opacity=0.2,title="Final Type Index $(Colouring) Time = $(TimesWeCheck[Time_Index_To_Plot])")
                        #PCAPlots_TimeFilter = scatter!((Transformed_Data[Time_Index_To_Plot,p,PcaDim1,end,q],Transformed_Data[Time_Index_To_Plot,p,PcaDim2,end,q]),c=Multi_Colours[Colour_Index])

                    end
                end
            end
        end
        if Plotting_Something
            display(PCAPlots_TimeFilter)
        end
        
        xlims!((-3,2))
        ylims!((-4,1))

    end
end






for t in [1,4,5,6,7]
    PCA_Individual_Colour_Plotting(PCADim1,PCADim2,t,3)
    PCA_Individual_Colour_Plotting(PCADim1,PCADim2,t,4)

end




Cell_Gatherings = zeros(TimeChecks,2,2^Splits,M)
CycleAmount = zeros(TimeChecks)
UniqueCycleAmount = zeros(TimeChecks)
FixPointAmount = zeros(TimeChecks)

Interpolated_Cells = Interpolator(TimesWeCheck,Orbit_Array[1])
Interpolated_EPIs = Interpolator(TimesWeCheck,EpiThroughTime)
for i=1:2^Splits
    Cell_Gatherings[:,1,i,:] = Interpolated_Cells[:,2+(i-1)*M:1+(i)*M]
    Cell_Gatherings[:,2,i,:] = Interpolated_EPIs[:,2+(i-1)*M:1+(i)*M]
end


Copy_String = string(1)
Current_Network_String = "/Gen_"*Generation_String*"_"*Copy_String*".jld"
Loaded_Data = load(Load_Data_String_Load*Current_Network_String)

CArray = Loaded_Data["CArray"]
ParameterMatrix = Loaded_Data["Network"]
FixPoint_Positions = Array{Float64}(undef,0)
Counter_Cycle = 0
XPositionTol = 0
Full_TimeSpan = 1000
sol = 0
Sampling_Amounts = 1
Transformed_Data_NoNoise =  zeros(Float32,TimeChecks,2^Splits,3,Full_TimeSpan,Sampling_Amounts)

for q=1:Sampling_Amounts
    Saved_Orbits_All = zeros(Float16,TimeChecks,2^Splits,Full_TimeSpan,M)
    for t = 1:TimeChecks
        Counter_Cycle = 0
        FixPoint_Positions = Array{Float64}(undef,0)
        VectorTest = []
        for k = 1:2^Splits
            PositionArray = Cell_Gatherings[t,:,k,:]
            
            Parameters = (ParameterMatrix, CArray) #We have to update the splits every rerun, rest stays same
            tspan = (Float32(0.0),Full_TimeSpan) #How long ODE will run
            
            prob = ODEProblem(Evolution_NoEpi,PositionArray,tspan,Parameters) #ODEdefining
            
            
            
            
            sol = solve(prob,BS3(),dtmin=0.1,force_dtmin=true)
            if false 
                aba = plot(sol.t,sol[1,1,:])
                for j=2:40
                    aba = plot!(sol.t,sol[1,j,:],legend=false,title=TimesWeCheck[t])
                end
                if t> 1 && (k==1) 
                    
                    display(aba)
                end
            end
            for time=1:Full_TimeSpan
                Saved_Orbits_All[t,k,time,:] = sol(time)[1,:]
            end
            
            
            
            
            XVector = sol[1,:,end]
            EpiVector = sol[2,:,end]
            SumOfDerivates = sum(abs.(tanh.(40*(InvSqr*ParameterMatrix*XVector.+EpiVector.+CArray[:])) .- XVector))
            DerivativeTol = 0.1
            XPositionTol = 0.1
            
        end
            
    
    
    end

    for i=1:TimeChecks  
        for tau=1:2^Splits
            Tested_Transform = transform(Orbital_Model,Saved_Orbits_All[i,tau,:,:]')        
            Transformed_Data_NoNoise[i,tau,:,:,q] = Tested_Transform
        end
    end
    
end
## Save somehow the data/orbits of these frozen versions somewhere


# Plot this for the PCA as their axis

Multi_Colours = [:cyan,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:darkmagenta,:lightcyan4,:olive,:gold4,:lightsalmon,:navajowhite,:pink]


function PCA_Individual_Colour_Plotting_NoNoise(PcaDim1,PcaDim2,Time_Index_To_Plot,Colour_Index)
    Colour_Counter = 0
    for Colouring = Colour_Index:Colour_Index
        
        Plotting_Something = false
        PCAPlots_TimeFilter = plot([0])
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    
    
        if Colour_Index == Colouring
            Colour_Counter = mod(Colour_Counter,16) + 1
            Plotting_Something = true
            PCAPlots_TimeFilter = plot!(Transformed_Data_NoNoise[Time_Index_To_Plot,1,PcaDim1,:,1],Transformed_Data_NoNoise[Time_Index_To_Plot,1,PcaDim2,:,1],c=Multi_Colours[Colour_Counter],legend=false,opacity=0.2)
            PCAPlots_TimeFilter = scatter!((Transformed_Data_NoNoise[Time_Index_To_Plot,1,PcaDim1,end,1],Transformed_Data_NoNoise[Time_Index_To_Plot,1,PcaDim2,end,1]),c=Multi_Colours[Colour_Counter],legend=false)
        end
        for q=1:Sampling_Amounts
            for p=1:2^Splits
                for t =1:1
                    Colour_Index = Final_Epigenetic_Factors[t,1,p,1]+Final_Epigenetic_Factors[t,1,p,2]*2+Final_Epigenetic_Factors[t,1,p,3]*4+Final_Epigenetic_Factors[t,1,p,4]*8+1
    
                    if Colour_Index == Colouring
                        Colour_Counter = mod(Colour_Counter,16) + 1

                        Plotting_Something = true
                        PCAPlots_TimeFilter = plot!(Transformed_Data_NoNoise[Time_Index_To_Plot,p,PcaDim1,:,q],Transformed_Data_NoNoise[Time_Index_To_Plot,p,PcaDim2,:,q],c=Multi_Colours[Colour_Counter],legend=false,opacity=0.2,title="Final Type Index $(Colouring) Time = $(TimesWeCheck[Time_Index_To_Plot]) No Noise")
                        PCAPlots_TimeFilter = scatter!((Transformed_Data_NoNoise[Time_Index_To_Plot,p,PcaDim1,end,q],Transformed_Data_NoNoise[Time_Index_To_Plot,p,PcaDim2,end,q]),c=Multi_Colours[Colour_Counter])

                    end
                end
            end
        end
        if Plotting_Something
            display(PCAPlots_TimeFilter)
        end
        
    end
end
PCADim1 = 1
PCADim2 = 2
for t = 1:4
    
        for p=1:1 #These are the different cell types we try to plot
            PCA_Individual_Colour_Plotting(PCADim1,PCADim2,t,p)
            PCA_Individual_Colour_Plotting_NoNoise(PCADim1,PCADim2,t,p)
    
        end
end




In [ ]:
Dimension1= 1
Dimension2 = 2
Cell = 1
plot(Transformed_Data_NoNoise[1,Cell,Dimension1,:,1],Transformed_Data_NoNoise[1,Cell,Dimension2,:,1],opacity=0.2,Colour=red)

In [ ]:
Sampling_Perturbs = 10
Perturbed_Orbits =  zeros(Float32,TimeChecks,2^Splits,3,Full_TimeSpan*10,Sampling_Perturbs)
Perturbation_Radius = 0.001

for q=1:Sampling_Perturbs
    Saved_Orbits_All = zeros(Float16,TimeChecks,2^Splits,Full_TimeSpan*10,M)
    for t = 1:TimeChecks
        Counter_Cycle = 0
        FixPoint_Positions = Array{Float64}(undef,0)
        VectorTest = []
        for k = 1:2^Splits
            PositionArray = Cell_Gatherings[t,:,k,:]
            Perturb = rand(40).-0.5
            Perturb /= sqrt(sum(Perturb.^2))
            Perturb *= Perturbation_Radius
            PositionArray[1,:] += Perturb
            Parameters = (ParameterMatrix, CArray) #We have to update the splits every rerun, rest stays same
            tspan = (Float32(0.0),Full_TimeSpan) #How long ODE will run
            
            prob = ODEProblem(Evolution_NoEpi,PositionArray,tspan,Parameters) #ODEdefining
            
            
            
            
            sol = solve(prob,BS3(),dtmin=0.1,force_dtmin=true)
            if false 
                aba = plot(sol.t,sol[1,1,:])
                for j=2:40
                    aba = plot!(sol.t,sol[1,j,:],legend=false,title=TimesWeCheck[t])
                end
                if t> 1 && (k==1) 
                    
                    display(aba)
                end
            end
            for time=1:Full_TimeSpan*10
                Saved_Orbits_All[t,k,time,:] = sol(time/10)[1,:]
            end
            
            
            
            
            XVector = sol[1,:,end]
            EpiVector = sol[2,:,end]
            SumOfDerivates = sum(abs.(tanh.(40*(InvSqr*ParameterMatrix*XVector.+EpiVector.+CArray[:])) .- XVector))
            DerivativeTol = 0.1
            XPositionTol = 0.1
            
        end
            
    
    
    end

    for i=1:TimeChecks  
        for tau=1:2^Splits
            Tested_Transform = transform(Orbital_Model,Saved_Orbits_All[i,tau,:,:]')        
            Perturbed_Orbits[i,tau,:,:,q] = Tested_Transform
        end
    end
    
end

In [ ]:
Dimension1= 1
Dimension2 = 2
Cell = 1

aba = plot(Transformed_Data_NoNoise[1,Cell,Dimension1,:,1],Transformed_Data_NoNoise[1,Cell,Dimension2,:,1])
for q=1:Sampling_Perturbs
    aba = plot!(Perturbed_Orbits[1,Cell,Dimension1,:,q],Perturbed_Orbits[1,Cell,Dimension2,:,q],opacity=0.2)
end
display(aba)

In [ ]:
Dimension1= 1
Dimension2 = 2
Cell = 1
anim = @animate for i ∈ 1:2000
    aba = plot(Transformed_Data_NoNoise[1,Cell,Dimension1,i:i+10,1],Transformed_Data_NoNoise[1,Cell,Dimension2,i:i+10,1])
    xlims!((-2,3))
    ylims!((-3,2))
    for q=1:Sampling_Perturbs
        aba = plot!(Perturbed_Orbits[1,Cell,Dimension1,i:i+10,q],Perturbed_Orbits[1,Cell,Dimension2,i:i+10,q],opacity=0.2)
    end
end
gif(anim, "anim_fps15.gif", fps = 30)

In [ ]:
Dimension1= 1
Dimension2 = 2
Cell = 2
anim = @animate for i ∈ 1:2000
    aba = plot(Transformed_Data_NoNoise[1,Cell,Dimension1,i:i+10,1],Transformed_Data_NoNoise[1,Cell,Dimension2,i:i+10,1])
    xlims!((-2,3))
    ylims!((-3,2))
    for q=1:Sampling_Perturbs
        aba = plot!(Perturbed_Orbits[1,Cell,Dimension1,i:i+10,q],Perturbed_Orbits[1,Cell,Dimension2,i:i+10,q],opacity=0.2)
    end
end
gif(anim, "anim_fps15.gif", fps = 30)

In [ ]:
Dimension1= 2
Dimension2 = 3
Cell = 1
anim = @animate for i ∈ 1:2000
    aba = plot(Transformed_Data_NoNoise[1,Cell,Dimension1,i:i+10,1],Transformed_Data_NoNoise[1,Cell,Dimension2,i:i+10,1])
    xlims!((-5,3))
    ylims!((-2,4))
    for q=1:Sampling_Perturbs
        aba = plot!(Perturbed_Orbits[1,Cell,Dimension1,i:i+10,q],Perturbed_Orbits[1,Cell,Dimension2,i:i+10,q],opacity=0.2)
    end
end
gif(anim, "anim_fps15.gif", fps = 30)

In [ ]:
Dimension1= 1
Dimension2 = 2
Cell = 3
for Cell = 1:20
    anim = @animate for i ∈ 1:2000
        aba = plot(Transformed_Data_NoNoise[1,Cell,Dimension1,i:i+10,1],Transformed_Data_NoNoise[1,Cell,Dimension2,i:i+10,1],title="Cell Number $Cell")
        xlims!((-2,3))
        ylims!((-3,2))
        for q=1:Sampling_Perturbs
            aba = plot!(Perturbed_Orbits[1,Cell,Dimension1,i:i+10,q],Perturbed_Orbits[1,Cell,Dimension2,i:i+10,q],opacity=0.2)
        end
    end
    display(gif(anim, "anim_fps15.gif", fps = 30))
end

In [ ]:
Dimension1= 1
Dimension2 = 2
Cell = 3
Time_Index = 2
for Cell = 1:6
    anim = @animate for i ∈ 1:2000
        aba = plot(Transformed_Data_NoNoise[Time_Index,Cell,Dimension1,i:i+10,1],Transformed_Data_NoNoise[Time_Index,Cell,Dimension2,i:i+10,1],title="Cell Number $Cell")
        xlims!((-2,3))
        ylims!((-3,2))
        for q=1:Sampling_Perturbs
            aba = plot!(Perturbed_Orbits[Time_Index,Cell,Dimension1,i:i+10,q],Perturbed_Orbits[Time_Index,Cell,Dimension2,i:i+10,q],opacity=0.2)
        end
    end
    display(gif(anim, "anim_fps15.gif", fps = 30))
end

In [ ]:
Dimension1= 1
Dimension2 = 2
Time_Index = 4
for Cell = 1:6
    anim = @animate for i ∈ 1:2000
        aba = plot(Transformed_Data_NoNoise[Time_Index,Cell,Dimension1,i:i+10,1],Transformed_Data_NoNoise[Time_Index,Cell,Dimension2,i:i+10,1],title="Cell Number $Cell")
        xlims!((-5,5))
        ylims!((-5,5))
        for q=1:Sampling_Perturbs
            aba = plot!(Perturbed_Orbits[Time_Index,Cell,Dimension1,i:i+10,q],Perturbed_Orbits[Time_Index,Cell,Dimension2,i:i+10,q],opacity=0.2)
        end
    end
    display(gif(anim, "anim_fps15.gif", fps = 30))
end

In [ ]:
Transformed_Data_NoNoise =  zeros(Float32,TimeChecks,2^Splits,3,Full_TimeSpan*10,Sampling_Amounts)

for q=1:1
    Saved_Orbits_All = zeros(Float16,TimeChecks,2^Splits,Full_TimeSpan*10,M)
    for t = 1:TimeChecks
        Counter_Cycle = 0
        FixPoint_Positions = Array{Float64}(undef,0)
        VectorTest = []
        for k = 1:2^Splits
            PositionArray = Cell_Gatherings[t,:,k,:]
            
            Parameters = (ParameterMatrix, CArray) #We have to update the splits every rerun, rest stays same
            tspan = (Float32(0.0),Full_TimeSpan) #How long ODE will run
            
            prob = ODEProblem(Evolution_NoEpi,PositionArray,tspan,Parameters) #ODEdefining
            
            
            
            
            sol = solve(prob,BS3(),dtmin=0.1,force_dtmin=true)
            if false 
                aba = plot(sol.t,sol[1,1,:])
                for j=2:40
                    aba = plot!(sol.t,sol[1,j,:],legend=false,title=TimesWeCheck[t])
                end
                if t> 1 && (k==1) 
                    
                    display(aba)
                end
            end
            for time=1:Full_TimeSpan*10
                Saved_Orbits_All[t,k,time,:] = sol(time/10)[1,:]
            end
            
            
            
            
            XVector = sol[1,:,end]
            EpiVector = sol[2,:,end]
            SumOfDerivates = sum(abs.(tanh.(40*(InvSqr*ParameterMatrix*XVector.+EpiVector.+CArray[:])) .- XVector))
            DerivativeTol = 0.1
            XPositionTol = 0.1
            
        end
            
    
    
    end

    for i=1:TimeChecks  
        for tau=1:2^Splits
            Tested_Transform = transform(Orbital_Model,Saved_Orbits_All[i,tau,:,:]')        
            Transformed_Data_NoNoise[i,tau,:,:,q] = Tested_Transform
        end
    end
    
end

In [ ]:
using ChaosTools
using StaticArrays

function Gene_ODE(u, Parameters, time)
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    Epi_Factors = Parameters[3]
    test = InvSqr*ParameterMatrix[1,:]'*u[:]
    #vector = Vector{Float16}([])
    #append!(vector,tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1])
    vector = [tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1]]
    for c=2:M
        append!(vector,tanh(40*(InvSqr*ParameterMatrix[c,:]'*u[:]+Epi_Factors[c]+CArray[c])) - u[c])
    end
    dx = @SVector [vector[k] for k in 1:M]

    return dx
end  

Times = [50]
Lyapunov_In_Time = zeros(64,length(Times),M)

for Cell=1:2
    
    for l = 1:length(Times)
        Time = Times[l]
        lor = CoupledODEs(Gene_ODE, Float64.(Interpolated_Orbits_Reshaped[Time,:,Cell]), [ParameterMatrix,CArray,Interpolated_Epi_Reshaped[Time,:,Cell]])
        λλ = lyapunovspectrum(lor, 10000; Δt = 0.1)
        print(λλ)
        Lyapunov_In_Time[Cell,l,:] = λλ
        
    end
    aba = plot(Times,Lyapunov_In_Time[Cell,:,:],xscale=:log10,legend=false)
    aba = ylims!((-2,0.5))
    display(aba)
end


In [ ]:
scatter(Lyapunov_In_Time[2,1,:])


In [ ]:
Cell=1

l=1
Time = Times[l]
lor = CoupledODEs(Gene_ODE, Float64.(Interpolated_Orbits_Reshaped[Time,:,Cell]), [ParameterMatrix,CArray,Interpolated_Epi_Reshaped[Time,:,Cell]])
λλ = lyapunov(lor, 1000, Δt = 0.1,d0=0.01)

In [ ]:
print(λλ)

In [ ]:
Cell=2

l=1
Time = Times[l]
lor = CoupledODEs(Gene_ODE, Float64.(Interpolated_Orbits_Reshaped[Time,:,Cell]), [ParameterMatrix,CArray,Interpolated_Epi_Reshaped[Time,:,Cell]])
λλ = lyapunov(lor, 1000, Δt = 0.1,d0=0.01)

In [ ]:
Dimension1= 1
Dimension2 = 2
Time_Index = 3
Cell = 1
Upper_Time = 300
aba = plot(Time_Plot_Difference[1:Upper_Time*10],sum(abs.(Transformed_Data_NoNoise[Time_Index,Cell,:,1:Upper_Time*10,1].-Perturbed_Orbits[Time_Index,Cell,:,1:Upper_Time*10,1]),dims=1)',yaxis=:log10,legend=false)
ylims!(10^-6,10)    
for q=2:Sampling_Perturbs
    aba = plot!(Time_Plot_Difference[1:Upper_Time*10],sum(abs.(Transformed_Data_NoNoise[Time_Index,Cell,:,1:Upper_Time*10,1].-Perturbed_Orbits[Time_Index,Cell,:,1:Upper_Time*10,q]),dims=1)')
end
display(aba)

Time_Index = 1
Cell = 1
Upper_Time = 30
for Cell =1:10
    aba  =plot(Time_Plot_Difference[1:Upper_Time*10],mean(sum(abs.(Transformed_Data_NoNoise[Time_Index,Cell,:,:,1].-Perturbed_Orbits[Time_Index,Cell,:,:,:]),dims=1),dims=3)[1,1:Upper_Time*10,1],yaxis=:log10,legend=false,title="Cell $Cell")
    ylims!(10^-4,10)    
    display(aba)
end

In [ ]:
Time_Plot_Difference = [0.1*i for i=1:10000]

In [ ]:
using ChaosTools
using StaticArrays
function lorenz_rule(u, p, t)
    σ = p[1]; ρ = p[2]; β = p[3]
    vector = [σ*(u[2]-u[1])]
    append!(vector,u[1]*(ρ-u[3]) - u[2])
    append!(vector,u[1]*u[2] - β*u[3])
    dx = @SVector [vector[k] for k in 1:3]
    return dx
end

lor = CoupledODEs(lorenz_rule, fill(10.0, 3), [10, 32, 8/3])
λλ = lyapunovspectrum(lor, 10000; Δt = 0.03)

In [ ]:

function lorenz_rule(u, p, t)
    σ = p[1]; ρ = p[2]; β = p[3]
    vector = [σ*(u[2]-u[1])]
    append!(vector,u[1]*(ρ-u[3]) - u[2])
    append!(vector,u[1]*u[2] - β*u[3])
    dx = [vector[k] for k in 1:3]
    return dx
end
Parameters = (ParameterMatrix, CArray) #We have to update the splits every rerun, rest stays same
tspan = (Float32(0.0),50) #How long ODE will run


prob = ODEProblem(lorenz_rule,fill(10.0, 3),tspan,[10, 32, 8/3]) #ODEdefining

sol = solve(prob,BS3())
Lorenz_Unperturbed = zeros(500,3)
for time=1:50*10
    Lorenz_Unperturbed[time,:] = sol(time/10)
end
Perturbs = 10
Lorenz_Perturbed = zeros(500,Perturbs,3)
Perturb_Radius = 0.0001
for i=1:Perturbs
    prob = ODEProblem(lorenz_rule,fill(10.0, 3) + Perturb_Radius*(rand(3).-0.5),tspan,[10, 32, 8/3]) #ODEdefining
    sol = solve(prob,BS3())
    for time=1:50*10
        Lorenz_Perturbed[time,i,:] = sol(time/10)
    end
end

In [ ]:
Mean_Difference = zeros(500)
plot()
for Perturb_Index=1:Perturbs
    Difference = sqrt.(sum((Lorenz_Perturbed[:,Perturb_Index,:] .- Lorenz_Unperturbed).^2,dims=2))
    Mean_Difference += Difference/Perturbs
    aba = plot!([0.1*i for i=1:500],Difference,yaxis=:log10)
end
display(aba)
plot([0.1*i for i=1:500],Mean_Difference,yaxis=:log10)
plot!([0.1*i for i=1:150],[2.71^(0.1*i*0.92)*Mean_Difference[1] for i=1:150])

In [ ]:
2.71^(0.1*1*0.92)

In [ ]:
Sampling_Perturbs = 100
Cells_Total = 4
Perturbed_Orbits =  zeros(Float32,TimeChecks,Cells_Total,3,Full_TimeSpan*10,Sampling_Perturbs)
Perturbation_Radius = 0.001
using ProgressMeter

@showprogress for q in 1:Sampling_Perturbs
    Saved_Orbits_All = zeros(Float16,TimeChecks,Cells_Total,Full_TimeSpan*10,M)
    for t = 1:TimeChecks
        Counter_Cycle = 0
        FixPoint_Positions = Array{Float64}(undef,0)
        VectorTest = []
        for k = 1:Cells_Total
            PositionArray = Cell_Gatherings[t,:,k,:]
            Perturb = rand(40).-0.5
            Perturb /= sqrt(sum(Perturb.^2))
            Perturb *= Perturbation_Radius
            PositionArray[1,:] += Perturb
            Parameters = (ParameterMatrix, CArray) #We have to update the splits every rerun, rest stays same
            tspan = (Float32(0.0),Full_TimeSpan) #How long ODE will run
            
            prob = ODEProblem(Evolution_NoEpi,PositionArray,tspan,Parameters) #ODEdefining
            
            
            
            
            sol = solve(prob,BS3(),dtmin=0.1,force_dtmin=true)
            if false 
                aba = plot(sol.t,sol[1,1,:])
                for j=2:40
                    aba = plot!(sol.t,sol[1,j,:],legend=false,title=TimesWeCheck[t])
                end
                if t> 1 && (k==1) 
                    
                    display(aba)
                end
            end
            for time=1:Full_TimeSpan*10
                Saved_Orbits_All[t,k,time,:] = sol(time/10)[1,:]
            end
            
            
            
            
            XVector = sol[1,:,end]
            EpiVector = sol[2,:,end]
            SumOfDerivates = sum(abs.(tanh.(40*(InvSqr*ParameterMatrix*XVector.+EpiVector.+CArray[:])) .- XVector))
            DerivativeTol = 0.1
            XPositionTol = 0.1
            
        end
            
    
    
    end

    for i=1:TimeChecks  
        for tau=1:Cells_Total
            Tested_Transform = transform(Orbital_Model,Saved_Orbits_All[i,tau,:,:]')        
            Perturbed_Orbits[i,tau,:,:,q] = Tested_Transform
        end
    end
    
end

In [ ]:
Dimension1= 1
Dimension2 = 2
Time_Index = 3
Cell = 1
Upper_Time = 300
aba = plot(Time_Plot_Difference[1:Upper_Time*10],sum(abs.(Transformed_Data_NoNoise[Time_Index,Cell,:,1:Upper_Time*10,1].-Perturbed_Orbits[Time_Index,Cell,:,1:Upper_Time*10,1]),dims=1)',yaxis=:log10,legend=false)
ylims!(10^-6,10)    
for q=2:Sampling_Perturbs
    aba = plot!(Time_Plot_Difference[1:Upper_Time*10],sum(abs.(Transformed_Data_NoNoise[Time_Index,Cell,:,1:Upper_Time*10,1].-Perturbed_Orbits[Time_Index,Cell,:,1:Upper_Time*10,q]),dims=1)')
end
display(aba)



In [ ]:
Time_Index = 1
Cell = 1
Upper_Time = 30
for Cell =1:4
    aba  =plot(Time_Plot_Difference[1:Upper_Time*10],mean(sum(abs.(Transformed_Data_NoNoise[Time_Index,Cell,:,:,1].-Perturbed_Orbits[Time_Index,Cell,:,:,:]),dims=1),dims=3)[1,1:Upper_Time*10,1],yaxis=:log10,legend=false,title="Cell $Cell")
    ylims!(10^-4,10)    
    display(aba)
end

In [ ]:
Time_Index = 2
Cell = 1
Upper_Time = 30
for Cell =1:4
    aba  =plot(Time_Plot_Difference[1:Upper_Time*10],mean(sum(abs.(Transformed_Data_NoNoise[Time_Index,Cell,:,:,1].-Perturbed_Orbits[Time_Index,Cell,:,:,:]),dims=1),dims=3)[1,1:Upper_Time*10,1],yaxis=:log10,legend=false,title="Cell $Cell")
    ylims!(10^-4,10)    
    display(aba)
end

In [ ]:
Time_Index = 4
Cell = 1
Upper_Time = 30
for Cell =1:4
    aba  =plot(Time_Plot_Difference[1:Upper_Time*10],mean(sum(abs.(Transformed_Data_NoNoise[Time_Index,Cell,:,:,1].-Perturbed_Orbits[Time_Index,Cell,:,:,:]),dims=1),dims=3)[1,1:Upper_Time*10,1],yaxis=:log10,legend=false,title="Cell $Cell")
    ylims!(10^-4,10)    
    display(aba)
end

In [ ]:
using ForwardDiff
import ForwardDiff.jacobian

function f(x)
    F = zero.(x)

    F[1] = x[1]^2 + x[3]
    F[2] = x[1] + x[2]
    F[3] = x[2]^2 + x[3]^2

    return F
end

x0 = [1,2,3]
J0 = jacobian(f, x0)

In [ ]:
function Gene_ODE(u)
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    Epi_Factors = Parameters[3]
    #vector = Vector{Float16}([])
    #append!(vector,tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1])
    vector = zero.(u)
    for c=1:M
        vector[c] = tanh(40*(InvSqr*ParameterMatrix[c,:]'*u[:]+Epi_Factors[c]+CArray[c])) - u[c]
    end

    return vector
end  


Parameters = [ParameterMatrix,CArray,Interpolated_Epi_Reshaped[Time,:,Cell]]
x0 = Interpolated_Orbits_Reshaped[Time,:,Cell]
J0 = jacobian(Gene_ODE, x0)
  

In [ ]:
Saved_Orbits_All = 0
Fine_Graining = 50
TimesWeCheck = [10,20,30,40,45,50,55,60,65,70,75,80,85,90,95,100,110,120,140,160,180,200,250,300,2000]
TimeChecks = length(TimesWeCheck)

Cell_Gatherings = zeros(TimeChecks,2,2^Splits,M)

Interpolated_Cells = Interpolator(TimesWeCheck,Orbit_Array[1])
Interpolated_EPIs = Interpolator(TimesWeCheck,EpiThroughTime)
for i=1:2^Splits
    Cell_Gatherings[:,1,i,:] = Interpolated_Cells[:,2+(i-1)*M:1+(i)*M]
    Cell_Gatherings[:,2,i,:] = Interpolated_EPIs[:,2+(i-1)*M:1+(i)*M]
end


Transformed_Data_NoNoise =  zeros(Float32,TimeChecks,2^Splits,3,Full_TimeSpan*Fine_Graining,Sampling_Amounts)
for q=1:1
    Saved_Orbits_All = zeros(Float16,TimeChecks,2^Splits,Full_TimeSpan*Fine_Graining,M)
    for t = 1:TimeChecks
        Counter_Cycle = 0
        FixPoint_Positions = Array{Float64}(undef,0)
        VectorTest = []
        for k = 1:2^Splits
            PositionArray = Cell_Gatherings[t,:,k,:]
            
            Parameters = (ParameterMatrix, CArray) #We have to update the splits every rerun, rest stays same
            tspan = (Float32(0.0),Full_TimeSpan) #How long ODE will run
            
            prob = ODEProblem(Evolution_NoEpi,PositionArray,tspan,Parameters) #ODEdefining
            
            
            
            
            sol = solve(prob,BS3(),dtmin=0.1,force_dtmin=true)
            if false 
                aba = plot(sol.t,sol[1,1,:])
                for j=2:40
                    aba = plot!(sol.t,sol[1,j,:],legend=false,title=TimesWeCheck[t])
                end
                if t> 1 && (k==1) 
                    
                    display(aba)
                end
            end
            for time=1:Full_TimeSpan*Fine_Graining
                Saved_Orbits_All[t,k,time,:] = sol(time/Fine_Graining)[1,:]
            end
            
            
            
            

        end
            
    
    
    end

    for i=1:TimeChecks  
        for tau=1:2^Splits
            Tested_Transform = transform(Orbital_Model,Saved_Orbits_All[i,tau,:,:]')        
            Transformed_Data_NoNoise[i,tau,:,:,q] = Tested_Transform
        end
    end
    
end

In [ ]:
function Gene_ODE(u)
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    Epi_Factors = Parameters[3]
    #vector = Vector{Float16}([])
    #append!(vector,tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1])
    vector = zero.(u)
    for c=1:M
        vector[c] = tanh(40*(InvSqr*ParameterMatrix[c,:]'*u[:]+Epi_Factors[c]+CArray[c])) - u[c]
    end

    return vector
end  
Cell = 1
Step_Size = 1
Delta_x = ones(40)/sqrt(40)
Amount_Lyapunov_Steps = 10000
Jacobian_Sampling_Points = zeros(Amount_Lyapunov_Steps,M,M)
Norms_Sampling_Points = zeros(Amount_Lyapunov_Steps)
for j=1:Amount_Lyapunov_Steps
    Parameters = [ParameterMatrix,CArray,Interpolated_Epi_Reshaped[TimesWeCheck[1],:,Cell]]
    x0 = Saved_Orbits_All[1,Cell,1+Step_Size*(j-1),:]
    J0 = jacobian(Gene_ODE, x0)
    Delta_x = J0*Delta_x
    Norm = sqrt(sum(Delta_x.^2))
    Norms_Sampling_Points[j] = Norm
    Jacobian_Sampling_Points[j,:,:] = J0
    Delta_x /= Norm
end

In [ ]:
ada = scatter((1/Fine_Graining,maximum(real.(eigvals(Jacobian_Sampling_Points[1,:,:])))),legend=false)
for i=2:1000
    ada = scatter!((i/Fine_Graining,maximum(real.(eigvals(Jacobian_Sampling_Points[i,:,:])))))

end
hline!((100,0))
xlabel!("Time")
ylabel!("Real part of largest Lyapunov exponent")
#plot!(([i/Fine_Graining for i=1:500],Norms_Sampling_Points[1:500]))
#hline!((100,1))

display(ada)

In [ ]:
function Gene_ODE(u)
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    Epi_Factors = Parameters[3]
    #vector = Vector{Float16}([])
    #append!(vector,tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1])
    vector = zero.(u)
    for c=1:M
        vector[c] = tanh(40*(InvSqr*ParameterMatrix[c,:]'*u[:]+Epi_Factors[c]+CArray[c])) - u[c]
    end

    return vector
end  
Cell = 2
Step_Size = 1
Delta_x = ones(40)/sqrt(40)
Amount_Lyapunov_Steps = 10000
Jacobian_Sampling_Points = zeros(Amount_Lyapunov_Steps,M,M)
Norms_Sampling_Points = zeros(Amount_Lyapunov_Steps)
for j=1:Amount_Lyapunov_Steps
    Parameters = [ParameterMatrix,CArray,Interpolated_Epi_Reshaped[TimesWeCheck[1],:,Cell]]
    x0 = Saved_Orbits_All[1,Cell,1+Step_Size*(j-1),:]
    J0 = jacobian(Gene_ODE, x0)
    Delta_x = J0*Delta_x
    Norm = sqrt(sum(Delta_x.^2))
    Norms_Sampling_Points[j] = Norm
    Jacobian_Sampling_Points[j,:,:] = J0
    Delta_x /= Norm
end

In [ ]:
ada = scatter((1/Fine_Graining,maximum(real.(eigvals(Jacobian_Sampling_Points[1,:,:])))),legend=false)
for i=2:1000
    ada = scatter!((i/Fine_Graining,maximum(real.(eigvals(Jacobian_Sampling_Points[i,:,:])))))

end
hline!((100,0))
xlabel!("Time")
ylabel!("Real part of largest Lyapunov exponent")
#plot!(([i/Fine_Graining for i=1:500],Norms_Sampling_Points[1:500]))
#hline!((100,1))

display(ada)

In [ ]:

function Gene_ODE(u)
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    Epi_Factors = Parameters[3]
    #vector = Vector{Float16}([])
    #append!(vector,tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1])
    vector = zero.(u)
    for c=1:M
        vector[c] = tanh(40*(InvSqr*ParameterMatrix[c,:]'*u[:]+Epi_Factors[c]+CArray[c])) - u[c]
    end

    return vector
end  
Amount_Lyapunov_Steps = 50000
Lyapunov_Mean = zeros(TimeChecks,Amount_Lyapunov_Steps)
for Time_Index=1:TimeChecks
    Cell = 1
    Step_Size = 1
    Delta_x = ones(40)/sqrt(40)
    Jacobian_Sampling_Points = zeros(Amount_Lyapunov_Steps,M,M)
    Norms_Sampling_Points = zeros(Amount_Lyapunov_Steps)
    for j=1:Amount_Lyapunov_Steps
        Parameters = [ParameterMatrix,CArray,Interpolated_Epi_Reshaped[TimesWeCheck[Time_Index],:,Cell]]
        x0 = Saved_Orbits_All[Time_Index,Cell,1+Step_Size*(j-1),:]
        J0 = jacobian(Gene_ODE, x0)
        Delta_x = J0*Delta_x
        Norm = sqrt(sum(Delta_x.^2))
        Norms_Sampling_Points[j] = Norm
        Jacobian_Sampling_Points[j,:,:] = J0
        Delta_x /= Norm
    end
    Lyapu = maximum(real.(eigvals(Jacobian_Sampling_Points[1,:,:])))
    Lyapunov_Mean[Time_Index,:] .+= Lyapu
    #ada = scatter((1/Fine_Graining,Lyapu),legend=false)
    for i=2:Amount_Lyapunov_Steps
        Lyapu = maximum(real.(eigvals(Jacobian_Sampling_Points[i,:,:])))
        Lyapunov_Mean[Time_Index,i:end] .+= Lyapu
        Lyapunov_Mean[Time_Index,i] /= i
        

        #ada = scatter!((i/Fine_Graining,Lyapunov_Mean[Time_Index,i]))
    end
    ada = plot([i/Fine_Graining for i=1:Amount_Lyapunov_Steps],Lyapunov_Mean[Time_Index,:])
    hline!((100,0))
    xlabel!("Time")
    ylabel!("Real part of largest Lyapunov exponent")
    
    #plot!(([i/Fine_Graining for i=1:500],Norms_Sampling_Points[1:500]))
    #hline!((100,1))
    
    Time_String = TimesWeCheck[Time_Index]
    title!("Cell = $Cell Time = $Time_String")
    display(ada)
end
plot(TimesWeCheck,Lyapunov_Mean[:,end])

In [ ]:

function Gene_ODE(u)
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    Epi_Factors = Parameters[3]
    #vector = Vector{Float16}([])
    #append!(vector,tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1])
    vector = zero.(u)
    for c=1:M
        vector[c] = tanh(40*(InvSqr*ParameterMatrix[c,:]'*u[:]+Epi_Factors[c]+CArray[c])) - u[c]
    end

    return vector
end  
Amount_Lyapunov_Steps = 50000
Lyapunov_Mean = zeros(TimeChecks,Amount_Lyapunov_Steps)
for Time_Index=1:TimeChecks
    Cell = 2
    Step_Size = 1
    Delta_x = ones(40)/sqrt(40)
    Jacobian_Sampling_Points = zeros(Amount_Lyapunov_Steps,M,M)
    Norms_Sampling_Points = zeros(Amount_Lyapunov_Steps)
    for j=1:Amount_Lyapunov_Steps
        Parameters = [ParameterMatrix,CArray,Interpolated_Epi_Reshaped[TimesWeCheck[Time_Index],:,Cell]]
        x0 = Saved_Orbits_All[Time_Index,Cell,1+Step_Size*(j-1),:]
        J0 = jacobian(Gene_ODE, x0)
        Delta_x = J0*Delta_x
        Norm = sqrt(sum(Delta_x.^2))
        Norms_Sampling_Points[j] = Norm
        Jacobian_Sampling_Points[j,:,:] = J0
        Delta_x /= Norm
    end
    Lyapu = maximum(real.(eigvals(Jacobian_Sampling_Points[1,:,:])))
    Lyapunov_Mean[Time_Index,:] .+= Lyapu
    #ada = scatter((1/Fine_Graining,Lyapu),legend=false)
    for i=2:Amount_Lyapunov_Steps
        Lyapu = maximum(real.(eigvals(Jacobian_Sampling_Points[i,:,:])))
        Lyapunov_Mean[Time_Index,i:end] .+= Lyapu
        Lyapunov_Mean[Time_Index,i] /= i
        

        #ada = scatter!((i/Fine_Graining,Lyapunov_Mean[Time_Index,i]))
    end
    ada = plot([i/Fine_Graining for i=1:Amount_Lyapunov_Steps],Lyapunov_Mean[Time_Index,:])
    hline!((100,0))
    xlabel!("Time")
    ylabel!("Real part of largest Lyapunov exponent")
    
    #plot!(([i/Fine_Graining for i=1:500],Norms_Sampling_Points[1:500]))
    #hline!((100,1))
    
    Time_String = TimesWeCheck[Time_Index]
    title!("Cell = $Cell Time = $Time_String")
    display(ada)
end
plot(TimesWeCheck,Lyapunov_Mean[:,end])

# Restart here!

In [ ]:
using ForwardDiff
import ForwardDiff.jacobian

Fine_Graining = 50
TimesWeCheck = [10,20,30,40,45,50,55,60,65,70,75,80,85,90,95,100,110,120,140,160,180,200,250,300,2000]
TimeChecks = length(TimesWeCheck)
Time_Index = 3
Cell = 1
Cell_Gatherings = zeros(TimeChecks,2,2^Splits,M)

Interpolated_Cells = Interpolator(TimesWeCheck,Orbit_Array[1])
Interpolated_EPIs = Interpolator(TimesWeCheck,EpiThroughTime)
for i=1:2^Splits
    Cell_Gatherings[:,1,i,:] = Interpolated_Cells[:,2+(i-1)*M:1+(i)*M]
    Cell_Gatherings[:,2,i,:] = Interpolated_EPIs[:,2+(i-1)*M:1+(i)*M]
end

Interpolated_Epi = Interpolator([i for i=1:Length_Orbits],EpiThroughTime)

Interpolated_Epi_Reshaped = reshape(Interpolated_Epi[:,2:end], (Length_Orbits,M,2^Splits))

Interpolated_Orbits_Reshaped = reshape(Interpolated_Orbits[1][2:end,:]', (Length_Orbits,M,2^Splits));

function Gene_ODE(u)
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    Epi_Factors = Parameters[3]
    #vector = Vector{Float16}([])
    #append!(vector,tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1])
    vector = zero.(u)
    for c=1:M
        vector[c] = tanh(40*(InvSqr*ParameterMatrix[c,:]'*u[:]+Epi_Factors[c]+CArray[c])) - u[c]
    end

    return vector
end  
Amount_Lyapunov_Steps = 200
Lyapunov_Mean = zeros(TimeChecks,Amount_Lyapunov_Steps)
Jacobian_Sampling_Points_Multiplied = 0
DeltaX_ThroughTime = zeros(Amount_Lyapunov_Steps,M)

Parameters = [ParameterMatrix,CArray,Interpolated_Epi_Reshaped[Time_Index,:,Cell]]




Lyapu_Maybe = zeros(500)
for j=1:500
    Lyapu_Maybe[j] = 1/(2*j)*log(DeltaX_ThroughTime[j,:]'*Jacobian_Sampling_Points_Multiplied[j,:,:]'*Jacobian_Sampling_Points_Multiplied[j,:,:]*DeltaX_ThroughTime[j,:])
end
plot(Lyapu_Maybe)


function lorenz_rule(u,p,t)
    σ = 10; ρ = 28; β = 8/3
    vector = zero.(u)
    vector[1] = σ*(u[2]-u[1])
    vector[2] = u[1]*(ρ-u[3]) - u[2]
    vector[3] = u[1]*u[2] - β*u[3]
    return vector
end
TimeSteps_Per_Second = 250
tspan = (Float32(0.0),1000) #How long ODE will run

prob = ODEProblem(lorenz_rule,[2,1,1],tspan) #ODEdefining

sol = solve(prob,BS3())
Lorenz_Unperturbed = zeros(1000*TimeSteps_Per_Second,3)
for time=1:1000*TimeSteps_Per_Second
    Lorenz_Unperturbed[time,:] = sol(time/TimeSteps_Per_Second)
end


function lorenz_rule(u)
    σ = 10; ρ = 28; β = 8/3
    vector = zero.(u)
    vector[1] = σ*(u[2]-u[1])
    vector[2] = u[1]*(ρ-u[3]) - u[2]
    vector[3] = u[1]*u[2] - β*u[3]
    return vector
end

Fine_Graining = 50
Cell = 1
Amount_Lyapunov_Steps = 1000000
Delta_T = 10^-4
Jacobian_Sampling_Points_Multiplied = 0
Step_Size = 1
d0 = 10^-7
Delta_x = d0*ones(3)/sqrt(3)
x0 = ones(3)
x0[1] += 1
X_Perturbed = x0 + Delta_x

Norm_Saved = zeros(Amount_Lyapunov_Steps)
for j=1:Amount_Lyapunov_Steps
    
    Step_1 = lorenz_rule(x0)
    Step_2 = lorenz_rule(X_Perturbed)
    x0 += Step_1*Delta_T
    X_Perturbed += Step_2*Delta_T

    Delta_x = X_Perturbed.-x0
    Norm_Saved[j] = sqrt(sum((X_Perturbed.-x0).^2))/d0

    
    Delta_x /= Norm_Saved[j]
    X_Perturbed = x0 + Delta_x
    if j>2:
        Moving_Average[j] = ((j-1)*Moving_Average[j-1] + logNorm_Saved[j])/j
    else
        Moving_Average[j] = 1/Delta_T.*log(Norm_Saved[j])
    end
end



# 0.01 Noise, type II

In [ ]:
TimesWeCheck = [10,30,50,70,90,120,150,200,300,500,1000,3000]
function Gene_ODE(u)
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    Epi_Factors = Parameters[3]
    #vector = Vector{Float16}([])
    #append!(vector,tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1])
    vector = zero.(u)
    for c=1:M
        vector[c] = tanh(40*(InvSqr*ParameterMatrix[c,:]'*u[:]+Epi_Factors[c]+CArray[c])) - u[c]
    end

    return vector
end  
Delta_T = 10^-3
D0 = 10^-6
Amount_Lyapunov_Steps = 500000
Norm_Saved = zeros(length(TimesWeCheck),Amount_Lyapunov_Steps)
Moving_Average = zeros(length(TimesWeCheck),Amount_Lyapunov_Steps)
Cell = 2

for t=1:length(TimesWeCheck)
    Time_Index=t
    Delta_X = D0*ones(M)/sqrt(M)
    Parameters = [ParameterMatrix,CArray,Interpolated_Epi_Reshaped[TimesWeCheck[Time_Index],:,Cell]]
    x0 = Interpolated_Orbits[1][2+(Cell-1)*M:1+Cell*M,TimesWeCheck[Time_Index]]
    
    for j=1:Amount_Lyapunov_Steps
        Stepx0 = Gene_ODE(x0)
        StepDeltax0 = Gene_ODE(x0+Delta_X)
    
    
        Delta_X = Delta_X + (StepDeltax0 - Stepx0)*Delta_T
        x0 += Stepx0*Delta_T
        Norm_Saved[t,j] = log(sqrt(sum(Delta_X.^2))/D0)
        if j==1
            Moving_Average[t,j] = Norm_Saved[t,j]
        else
            Moving_Average[t,j] = (Moving_Average[t,j-1]*(j-1) + Norm_Saved[t,j])/j
        end
        Delta_X /= sqrt(sum(Delta_X.^2))/D0
    end
    Start_Pont = 100
    TimeToString = TimesWeCheck[Time_Index]
    ada = plot([j*Delta_T for j=Start_Pont:Amount_Lyapunov_Steps],Moving_Average[t,Start_Pont:end],title="Time $TimeToString Cell = $Cell")
    ada = hline!((0,0))
    display(ada)
end


In [ ]:
plot(TimesWeCheck,Moving_Average[:,end],xaxis=:log10)
hline!((0,0))

# 0.1 Noise, type Ia

In [ ]:
#### SMALL PERT IS SET TO 1 WE ONLY HAVE 1 SAMPLE NOW
### WE MESSED WITH SOME DATA_TO_ANALYSE and the loading ones

#Data_To_Analyse = "21_12_NoSplit_HighODE_SplitTime=0.1_2000Gen_Noise01"
#Data_To_Analyse = ARGS[1]
#Noise = parse(Float64, ARGS[2])
#Data_To_Analyse = "21_12_NoSplit_HighODE_SplitTime=0.1_2000Gen_Noise005_Mut=2"
#Noise = 0.005

Noise = 0.1
Data_To_Analyse = "07_01_NoSplit_HighODE_SplitTime=0.1_2000Gen_Noise0.1_Sample5"

Load_Generations = [375]

Epi_Limiter = 1
a::Float32 = 1.0*Epi_Limiter   #Ratio of gene to epigenetic factor
v::Float32 = 6*10^-4/Epi_Limiter  #Speed at which epigenetic factor changes
#v::Float32 = 0  #Speed at which epigenetic factor changes

Load_Data_String = Data_To_Analyse*"Epi0.1"



println("OUTPUTTING MY INPUT")
println(Data_To_Analyse)
println(Noise)

using Random
using Plots
using Distributions
using DifferentialEquations
using Statistics
using ProgressMeter
using TickTock
using LinearAlgebra
using StatsBase
using LaTeXStrings
using MultivariateStats
using Dates
using Distances
using JLD
using KernelDensity
using Plots
using StatsPlots




function Setting_Up_Randomly(M,Sparse,rng=MersenneTwister(123))
    "Function used to randomly with seed define our c_i's, J_ij's and the initial positions
    Input:
    M      = Amount of Genes in our system
    Sparse = Bool to determine if we have a sparse graph
    rng    = Random Julia seed to make sure we can recreate the same initial conditions
    Output:
    CArray          = C_i's used, can only be between -0.5 and 0.5 now
    PositionArray   = Random Initial position, between -1 and 1 for all genes, epifactors = 0
    ParameterMatrix = J_ij's Currently a dense network with values between -0.5 and 0.5
    "
    ParameterMatrix = zeros(M,M)
    CArray = zeros(1,M)
    PositionArray = zeros(2,1,M)
    
    CArray = rand(rng,Float32,M).-0.5; #Random c_j between -0,5 and 0.5
    PositionArray[1,1,1:M] = 2*(rand(rng,Float32,M).-0.5); #Random Initial position 
    ParameterMatrix = 2 * bitrand(rng,M,M).-1; #J_jk's
    if Sparse
       for i=1:M
            for j=1:M
                if rand(rng) > 5/M #Average of 5 connections per gene

                    ParameterMatrix[i,j] = 0 
                end
            end
        end
    end
    return CArray, PositionArray, ParameterMatrix
end







function Noisy_Splitting(solution,Total_Splits,M,Seed)
    " Once our cell decides to split, we decide to introduce some random noise in how these genes get distributed.
    This function takes the cells, doubles them and then gives the daughters cell the same gene parameters +- noise
    one daughter gets + the other gets -. For the epigeneitc factors they are simply inherited and do not see any noise
    Input:
    solution     = These are the initial conditions of the parent cells that we need to split 
    (First genes of all cells, then all epigenetic factors)
    Total_Splits = Amount of times the cell has split (including this current split)
    M            = Amount of Genes
    Seed         = Random Noise seed for reproducing results
    Output:
    SplittedArray = New array of the genes and epigenetic factors of the daughter cells
    How is this structured? Suppose we take M=3 for convenience, we would have [g1,g2,g3,e1,e2,e3] for the first parent cell
    The daughters become: [g11,g12,g13,g21,g22,g23,e11,e12,e13,e21,e22,e23]
    Where we have genes of daughter 1, then genes of daughter 2, then epi of daughter 1 and epi of daughter 2.
    When the Daughters split we get the genes of Children of cell 1, then genes of Children of cell 2 and same for epi.
    "
    Random.seed!(Seed)
    s = Total_Splits #For shortening
    SplittedArray = zeros(Float32,2,2^(s),M) #2^(s+1), 2M is the normal length, 4M after 1 split, 8M after 2 splits etc.
    for g=1:2^(s-1) #looping over the length of the parent
        for t=1:M
            Gaussian = Normal{Float32}(0.0f0,0.0f0)
            Displace = rand(Gaussian) #Noise for split
            #index = Int32.(floor((g-1)/M)*M +g) #example M=10: Index 1-> 1 and 11, index 2->2 and 12 ... 
            #index 10->10 and 20, index 11->21 and 30 and so on. This is old method

            #Genes
            SplittedArray[1,2*g-1,t] = solution[1,g,t] + Displace
            SplittedArray[1,2*g,t] = solution[1,g,t] - Displace
            
        end
        #Epigenetic
        SplittedArray[2,2*g-1,:] = solution[2, g,:]
        SplittedArray[2,2*g,:] = solution[2, g,:]   
    end

    return SplittedArray  
end

function Differentiaton_Detection_Parameter(M,Epigenetic_Plot)
    "Function used to convert epigenetic factors into cell types and bits
    Input:
    M               = Amount of Genes 
    Epigenetic_Plot = Array of epigenetic factors which have to be converted to 0 and 1 representing on or off
    Output:
    b        = Array of scalar values representing cell types
    BitGenes = Array of vectors representing the genes of the cell types
    "
    b = Int128[]
    Counter = Int[]
    Epi_Bitwise = signbit.(-Epigenetic_Plot)

    
    BitGenes = [] #We will put the bit notation of the gene in here
    #Later rewrite to let this replace Bit_Represent
    
    for i=1:length(Epi_Bitwise[:,1])
        Bit_Represent::Int128 = 0 #CAREFUL WITH M>127
        for j=1:M
            Bit_Represent += Int128(Epi_Bitwise[i,j]*2)^(j-1)
        end
        #print(Bit_Represent)
        #print("     ")
        if !(Bit_Represent in b)

            push!(BitGenes,collect(Epi_Bitwise[i,:]))
            push!(Counter,1)
        end
        push!(b,Bit_Represent)

        
    end
    
    #print(b)
    Counter = sort(Counter,rev=true)
    #print(Counter)
    Score = 0
    #Make here the formula to turn b and the counter into a score


    return b,BitGenes
end

function Entropy_Score(M,Epigenetic_Plot)
    "Function used to convert epigenetic factors into cell types and bits to then calculate the entropic score.
    See previous function for alternative, this is currently not in use.
    Input:
    M               = Amount of Genes 
    Epigenetic_Plot = Array of epigenetic factors which have to be converted to 0 and 1 representing on or off
    Output:
    Score = Entropy score of the cell types
    "
    b = Int[]
    Counter = Int[]
    Epi_Bitwise = signbit.(-Epigenetic_Plot)
    
    for i=1:length(Epi_Bitwise[:,1])
        Bit_Represent::Int128 = 0 #CAREFUL WITH M>127
        for j=1:M
            Bit_Represent += Int128(Epi_Bitwise[i,j]*2)^(j-1)
        end
        #print(Bit_Represent)
        #print("     ")
        if !(Bit_Represent in b)
            push!(b,Bit_Represent)
            push!(Counter,1)
        else 
            Counter[findfirst(==(Bit_Represent), b)] += 1
        end
        
    end
    
    #print(b)
    Counter = sort(Counter,rev=true)
    #print(Counter)
    Score = 0
    #Make here the formula to turn b and the counter into a score
    for i=1:length(b)
       prob = Counter[i]/M
       Score -= prob*log(prob)
    end
    
    return Score
end



function Mutate_Parameters(ParameterMatrix_Mut,M,Mutations,Seed)
    "Function used to mutate the network (we turn a 1 or -1 to 0 and we turn a 0 to 1 or -1)
    Input:
    ParameterMatrix_Mut = Original Network that we want to mutate
    M                   = Amount of Genes 
    Mutations           = How many connections will we mutate
    Seed                = Randomness seed
    Output:
    ParameterMatrix_Mut = Mutated Network
    "
    Zero_Indices = findall(iszero,ParameterMatrix_Mut)
    One_Indices = findall(!iszero,ParameterMatrix_Mut)
    Random.seed!(Seed)
   
    for j=1:Mutations 
        t_zero = rand(1:length(Zero_Indices))
        t_one = rand(1:length(One_Indices))
        i_zero = Zero_Indices[t_zero][1]
        j_zero = Zero_Indices[t_zero][2]
        i_Value = One_Indices[t_one][1]
        j_Value = One_Indices[t_one][2]



        ParameterMatrix_Mut[i_zero,j_zero] = rand(0:1)*2-1 #turn a zero into 1 or -1
        ParameterMatrix_Mut[i_Value,j_Value] = 0 #turn 1 or -1 into 0




        
    end
    return ParameterMatrix_Mut
end


#Some Julia functions I played around with, may reconsider using more later
MultiplyBroad_(a,b) = a .* b

SummingDim2_(a) = sum(a,dims=2) 

HyperTanh_(a) = tanh.(40*a)



function Epigenetic_Evolution3(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    
    #⊗_(Wi, bi) = Wi .* bi
       
    for t=1:2^Splits

        du[2,t,:] = v*(a*u[1,t,:].-u[2,t,:])    
        du[1,t,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,t,:].+u[2,t,:].+CArray[:])) .- u[1,t,:] 
            #Epigenetic Evolution, no need to worry about boundaries here so simple loop      
    end  
    
end



function Gradient(u,Splits,t)
    "Function used to calculate the gradient of a cells neighbours, takes into accounts if there even are neighbours.
    Input:
    u      = Gene concentrations over which we take the gradient
    Splits = How many times have the cells split already (determines where neighbours are)
    t      = What location is our cell (boundaries only have 1 neighbour)
    Output:
    Multiple different version depending on the splits, this will be the gradient we want
    "
    if Splits == 0
        return 0
    elseif Splits ==1
        if t==1
            return u[t+1,:] - u[t,:]
        else
            return u[t-1,:] - u[t,:]
        end
    else
        if t==1
            return u[t+1,:] - u[t,:]
        elseif t==2^Splits
            return u[t-1,:] - u[t,:]
        else
            return u[t-1,:] - 2*u[t,:] + u[t+1,:]
        end
    end
    
end

function Epigenetic_Evolution4(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    
    #⊗_(Wi, bi) = Wi .* bi
       
    for t=1:2^Splits

        du[2,t,:] = v*(a*u[1,t,:].-u[2,t,:])    
        du[1,t,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,t,:].+u[2,t,:].+CArray[:])) .- u[1,t,:] +KArray.*Gradient(u[1,:,:],Splits,t)
            #Epigenetic Evolution, no need to worry about boundaries here so simple loop      
    end  
    
end

function σ_Genes(du, u,Parameters,Time)
    du[1,:,:] .= Noise
    
end



function Epigenetic_Evolution2(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    Cells::Int32 = 2^Splits
    #⊗_(Wi, bi) = Wi .* bi
       
    for t=1:Cells  

       du[2,t,:] = v*(a*u[1,t,:].-u[2,t,:])    

       du[1,t,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,t,:].+u[2,t,:].+CArray[:])) .- u[1,t,:] .+ KArray.*(vec(sum(u[1,:,:],dims=1))/Cells - u[1,t,:])

    end  
    
end

#Defining it all in function form


function Mutating_Network(N,M,K,ParameterMatrix_Collection,PositionArray_Collection, Indices, Counter, Genetics)
    "Function used to make mutated copies of the best performing networks. Currently not in use
    Input:
    N                          = How many copies of the network do we make
    M                          = Amount of Genes
    K                          = Supposed to work as a fraction of N, telling us how many mutant versions
    ParameterMatrix_Collection = The original networks
    PositionArray_Collection   = Original Positions
    Indices                    = Sorted indices giving the best networks
    Counter                    = Randomiser seed
    Genetics                   = Used to track heritance of networks
    Output:
    ParameterMatrix_Collection = Updated network
    PositionArray_Collection   = Updated initial positions
    Counter                    = Updated seed
    Updated_Genes              = Updated inheritance
    "
    Updated_Genes = zeros(N)
    Mutations = 1
    Seed_Mutations = 1234
    Indexation_Mutate = 1
    Temporary_Matrix = zeros(Float32,N,M,M)
    Temporary_Vector = zeros(Float32,N,2,1,M)
    for p = 1:K
        for q=1:N/(2*K)
            Temporary_Matrix[Indexation_Mutate,:,:] = Mutate_Parameters(ParameterMatrix_Collection[Indices[p],:,:],M,Mutations,Seed_Mutations+Counter)
            Temporary_Vector[Indexation_Mutate,:,:,:] = PositionArray_Collection[Indices[p],:,:,:]
            Updated_Genes[Indexation_Mutate] = Genetics[p]
            Temporary_Matrix[Indexation_Mutate+1,:,:] = Temporary_Matrix[Indexation_Mutate,:,:]
            Temporary_Vector[Indexation_Mutate+1,:,:,:] = Temporary_Vector[Indexation_Mutate,:,:,:]
            Updated_Genes[Indexation_Mutate+1] = Genetics[p]
            Counter += 1
            Indexation_Mutate +=2
        end
        Temporary_Matrix[Indexation_Mutate-1,:,:] = ParameterMatrix_Collection[Indices[p],:,:] #We overwrite it to add the original back
        Temporary_Matrix[Indexation_Mutate-2,:,:] = ParameterMatrix_Collection[Indices[p],:,:]
    end
    ParameterMatrix_Collection = deepcopy(Temporary_Matrix);
    PositionArray_Collection = deepcopy(Temporary_Vector);
    return ParameterMatrix_Collection, PositionArray_Collection, Counter, Updated_Genes
end



function Running_Network(CArray,ParameterMatrix,PositionArray,Random_Noise)
    "Function used to run our simulation
    Input:
    CArray          = Constants used in our ODE solver  
    ParameterMatrix = Collection of Networks
    PositionArray   = Original Positions
    Random_Noise    = Randomiser seed
    Output:
    PositionArray[2,:,:] = Epigenetic factors from our simulation
    Random_Noise         = Updated seed
    "
    #CArray = CArray_Collection[x,:]
    #ParameterMatrix = ParameterMatrix_Collection[x,:,:]
    #PositionArray = PositionArray_Collection[x,:]
    Orbit_Collection = zeros(Float32,2^Splits*M+1,0)
    Epi_Collection = zeros(Float32,2^Splits*M+1,0)
    
    
    
    
    for s=1:Splits #Looping over the splits
        Parameters = (ParameterMatrix, CArray,KArray,Int32(s-1)) #We have to update the splits every rerun, rest stays same
        tspan = (Float32(0.0),SplitTime) #How long ODE will run
        if Mean_Field
            prob = ODEProblem(Epigenetic_Evolution2,PositionArray,tspan,Parameters) #ODEdefining
        elseif Diffusing_System
            prob = ODEProblem(Epigenetic_Evolution4,PositionArray,tspan,Parameters) #ODEdefining
        elseif Stochastic_Noise
            prob = SDEProblem(Epigenetic_Evolution4, σ_Genes,PositionArray,tspan,Parameters)

        else
            prob = ODEProblem(Epigenetic_Evolution3,PositionArray,tspan,Parameters) #ODEdefining
        end    #Solving
        if !Stochastic_Noise
            sol = solve(prob,BS3())
        else
            sol = solve(prob,SRA3())
            #sol = solve(prob,LambaEulerHeun())
        end
        Times = sol.t #Save the time, doesn't get used yet though
        StepAmount = length(Times)  #Amount of data points per gene
        Genes = sol 
        PositionArray = Genes[StepAmount] #Saving the genes and epi factors
        
        OrbitalCollection = Times' .+ (s-1)*SplitTime
        EpiCollected = Times' .+ (s-1)*SplitTime
        for i=1:2^(Splits-s+1)  #s=1 this gives all 2^Splits, this is to say how much we repeat same path
            for j=1:2^(s-1) #Together multiply, always 2^Splits orbits, these j represent different paths
                OrbitalCollection = vcat(OrbitalCollection,sol[1,j,:,:])   
                EpiCollected = vcat(EpiCollected,sol[2,j,:,:])   
            end
        end
        Orbit_Collection = hcat(Orbit_Collection,OrbitalCollection)
        Epi_Collection = hcat(Epi_Collection,EpiCollected)
        #Orbit1 = [Times,Genes[1,1,1:M,1:StepAmount]]
        #Orbit2 = [Times,Genes[1,1,1:M,1:StepAmount]]
        #EpiOrbit1 = [Times,Genes[2,1,1:M,1:StepAmount]]
        #EpiOrbit2 = [Times,Genes[2,1,1:M,1:StepAmount]]
        #Now we must take all the values from the updated PositionArray and give appropriate terms to the daughter cells
        PositionArray = Noisy_Splitting(PositionArray,s,M,Random_Noise)  #Splitting function
        Random_Noise += 1
    end

    Final_Run_Time::Int32 = 20000 #Long evolving of the cells after splitting to get the final epigenetic state
    Parameters = (ParameterMatrix, CArray,KArray,Int32(Splits)) 
    tspan = (Int32(0.0),Final_Run_Time)

    if Mean_Field
        prob = ODEProblem(Epigenetic_Evolution2,PositionArray,tspan,Parameters) #ODEdefining
    elseif Diffusing_System
        prob = ODEProblem(Epigenetic_Evolution4,PositionArray,tspan,Parameters) #ODEdefining
    elseif Stochastic_Noise
        prob = SDEProblem(Epigenetic_Evolution4, σ_Genes,PositionArray,tspan,Parameters)

    else
        prob = ODEProblem(Epigenetic_Evolution3,PositionArray,tspan,Parameters) #ODEdefining
    end    #Solving
    if !Stochastic_Noise
        sol = solve(prob,BS3())
    else
        #,dtmin=0.1,force_dtmin=true
        sol = solve(prob,SRA3(),dtmin=0.1,force_dtmin=true)
    end
    #sol = solve(prob,alg_hints = [:nonstiff]) #Solving
    #sol = solve(prob)
    Times = sol.t
    StepAmount = length(Times)
    GenesEpi = sol
    PositionArray = GenesEpi[StepAmount]

    OrbitalCollection = Times' .+ Splits*SplitTime 
    EpiCollected = Times' .+ Splits*SplitTime 
    for j=1:2^(Splits) #Together multiply, always 2^Splits orbits, these j represent different paths
        OrbitalCollection = vcat(OrbitalCollection,sol[1,j,:,:])  
        EpiCollected = vcat(EpiCollected,sol[2,j,:,:])   

    end
    Orbit_Collection = hcat(Orbit_Collection,OrbitalCollection)
    Epi_Collection = hcat(Epi_Collection,EpiCollected)
    #Orbit1 = [cat(Orbit1[1],Times.+SplitTime,dims=1),cat(Orbit1[2],GenesEpi[1,1,1:M,1:StepAmount],dims=2)]
    #Orbit2 = [cat(Orbit2[1],Times.+SplitTime,dims=1),cat(Orbit2[2],GenesEpi[1,2,1:M,1:StepAmount],dims=2)]
    #EpiOrbit1 = [cat(EpiOrbit1[1],Times.+SplitTime,dims=1),cat(EpiOrbit1[2],GenesEpi[2,1,1:M,1:StepAmount],dims=2)]
    #EpiOrbit2 = [cat(EpiOrbit2[1],Times.+SplitTime,dims=1),cat(EpiOrbit2[2],GenesEpi[2,2,1:M,1:StepAmount],dims=2)]

    return PositionArray[2,:,:],Random_Noise, Orbit_Collection, Epi_Collection
end


function Scoring_Indexes(N,Final_Epigenetic_Factors,Cell_Bit)
    "Function used to calculate the score of all the networks, distribution robustness and the cell type robustness. Note
    that we are still selecting on amount of different cells, not entropy.
    Input:
    N                       = How many copies of the same networks we have 
    Final_Epigentic_Factors = Array of epigenetic factors after running the simulations
    Cell_Bit                = Cell type tracker
    Output:
    Indices                   = Indices of networks sorted by score, we only select these
    AllIndices                = Longer copy of Indices, contains also those we will no longer use
    Evolution_Indices         = Gives us the score corresponding with the Indices we selected on
    [Cell_Types,Cell_Amounts] = Vector of the cell types and how many of these cell types there are
    Cell_Bit                  = Updated cell type tracker
    KLD_Distance              = Kullback-Lieber divergence score for the distribution robustness
    Bit_Distances             = Cell Type robustness measure
    "
    Scoring = zeros(2,N)
    Cell_Types = []
    Cell_Amounts = []
    Cell_Distribution = Array{Vector{Float64}}(undef, N) 
    Cell_Bit_Array = Array{Vector{Any}}(undef, N)

    for x=1:N
        Differentiation, Bit_Genes = Differentiaton_Detection_Parameter(M,Final_Epigenetic_Factors[x,:,:]) 
     
        #Here we save some cell types and bits to use for our robustness calculation after the for loop
        #Cell types
        alpha = countmap(Differentiation)
        Temp_Array = Vector{Float64}()
        for i in keys(alpha)
            Temp_Array = push!(Temp_Array,alpha[i])
        end
        #print(Temp_Array)
        Cell_Distribution[x] = sort(Temp_Array,rev=true)/2^Splits #We sort the distribution of our cells and normalise
        #Bits
        Cell_Bit_Array[x] = Bit_Genes
        
        
        
        
        #This part here is where we track the cell types we had before as well
        for tau in Bit_Genes
            if !(tau in Cell_Bit)

                push!(Cell_Bit,tau)


            end
        end
        for tau in Differentiation 
            if !(tau in Cell_Types)
                push!(Cell_Types,tau)
                push!(Cell_Amounts,1)
            else
                Cell_Amounts[findfirst(==(tau), Cell_Types)] += 1
            end
        end

        Scoring[1:2,x] = [length(countmap(Differentiation)),x]

    end

    #Robustness calculation
    KLD_Distance = 0
    
    KLD_Adder = 0
    for i=1:N
        for j=1:N #Loop over all and calculate KL for all distributions
            if i!=j #If the same we get no difference anyways
                Length_Differ = length(Cell_Distribution[i])-length(Cell_Distribution[j])
                if Length_Differ > 0 #i is longer so we need to add 0 to j
                    KLD_Adder = kldivergence(Cell_Distribution[i],vcat(Cell_Distribution[j],zeros(Length_Differ)))
                    if KLD_Adder != Inf #There are zeros in the second distribution (divide by 0 error)
                        KLD_Distance += kldivergence(Cell_Distribution[i],vcat(Cell_Distribution[j],zeros(Length_Differ)))
                    end
                else
                    KLD_Adder = kldivergence(vcat(Cell_Distribution[i],zeros(-Length_Differ)),Cell_Distribution[j])

                    KLD_Distance += kldivergence(vcat(Cell_Distribution[i],zeros(-Length_Differ)),Cell_Distribution[j])
                end
            end
        end
    end
    KLD_Distance = KLD_Distance/(N*(N-1))
    
    #Now we see the closest type in the other groups
    Bit_Distances = 0
    for x =1:N
        for Bit_Inside in Cell_Bit_Array[x] #These are all the cell types in bit form
            for y =1:N #We loop over the others
                if x!=y #Must look at different otherwise distance 0 again
                    Bit_Distan_Tem = M
                    for Bit_Outside in Cell_Bit_Array[y] #We loop over all the cell types in this other one
                        Bit_Distan_Temp = sum(abs.(Bit_Inside .- Bit_Outside)) 
                        if Bit_Distan_Temp < Bit_Distan_Tem #Distance must be smaller if it is the best fit
                            Bit_Distan_Tem = Bit_Distan_Temp
                        end
                    end
                    Bit_Distances += sqrt(Bit_Distan_Tem)/(N*(N-1)) #Sqrt to make euclidean distance, N*(N-1) for normalisation
                end
            end
        end
    end
    
    
    #Here we mess a bit with the indices to just have the ones with the highest score later
    
    Indices = sortslices(Scoring,dims=2,rev=true)
    Evolution_Indices = Indices[1,1:K]

    AllIndices = deepcopy(Indices[2,:])
    Indices = Indices[2,1:K]
    Indices = Int.(Indices)
    #print(Indices[2,1:K])
    return Indices, AllIndices, Evolution_Indices, [Cell_Types,Cell_Amounts],Cell_Bit, KLD_Distance, Bit_Distances
end

function Epi_Plotter(Indices,K,Final_Epigenetic_Factors)
    "Function used to plot the epigenetic factors of different cells
    Input:
    Indices                 = Indices of highest scoring runs  
    k                       = 4 or 8 plots
    Final_Epigentic_Factors = Epigenetic factors that we will plot
    Output:
    k = Plot element, simply use display on the output to see the requested plot
    "
    pa = plot(Final_Epigenetic_Factors[Indices[1],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pb = plot(Final_Epigenetic_Factors[Indices[2],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pc = plot(Final_Epigenetic_Factors[Indices[3],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pd = plot(Final_Epigenetic_Factors[Indices[4],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    if K==8
        pe = plot(Final_Epigenetic_Factors[Indices[5],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        pf = plot(Final_Epigenetic_Factors[Indices[6],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        pg = plot(Final_Epigenetic_Factors[Indices[7],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        ph = plot(Final_Epigenetic_Factors[Indices[8],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        k = plot(pa,pb,pc,pd,pe,pf,pg,ph,layout=(4,2),legend=false)
    else
        k = plot(pa,pb,pc,pd,layout=(2,2),legend=false)
    end
    return k
end

function Gene_Variance(Final_Epigenetic_Factors,M,N)
    "Function used to calcualte the variance in expressed genes
    Input:
    Final_epigenetic_Factors = Final epigenetic values which represent gene expression
    M                        = Amount of Genes
    N                        = How many copies of same network do we have
    Output:
    Deviation = Variance in these genes (normalised by 2^Splits*N)
    "
    BitterMatrix = signbit.(-Final_Epigenetic_Factors[:,:,:])
    Average_Bits = zeros(M)
    for i=1:N
        for j=1:2^Splits
            Average_Bits += BitterMatrix[i,j,:]
        end
    end
    Average_Bits /= N*2^Splits
    Deviation = 0

    for i=1:N
        for j=1:2^Splits
            Deviation += sum((Average_Bits .- BitterMatrix[i,j,:]).^2)/(2^Splits*N-1)
        end
    end
    
    return Deviation
end

function PCA_Distribution_Creator(PCA_Dimensions,Transformed,Dimensions_Distribution,Cell_Distribution)
    
    
    PCA_Dimensional_Values = zeros(PCA_Dimensions,findmax(Dimensions_Distribution)[1]) 
    #Here we will save all the values and use these for indexes later
    B = Array{Vector{Tuple{Float64, Int64}}}(undef, PCA_Dimensions)
    for j=1:PCA_Dimensions
        B[j] = [(i, count(==(i), Transformed)) for i in unique(Transformed[j,:])] #Here we find what the arguments are for the
        #different dimensions
        for k = 1:length(B[j])
            PCA_Dimensional_Values[j,k] = B[j][k][1] #Here we save all the individual arguments in the corresponding dimension
        end
    end
    #print(PCA_Dimensional_Values)   #If we wanna see them

    Components_Length = length(Transformed[1,:]) 
    for k in 1:Components_Length
        Index_PCA = zeros(PCA_Dimensions)

        for l=1:PCA_Dimensions
            Index_PCA[l] = (findfirst(==(Transformed[l,k]), PCA_Dimensional_Values[l,:]))
        end 
        Index_PCA = Int.(Index_PCA)
        
        
        if PCA_Dimensions==2 #I really tried but I couldn't fix it so just throwing in this hack
            Cell_Distribution[Index_PCA[1],Index_PCA[2]] += 1/Components_Length
        elseif PCA_Dimensions==1
            Cell_Distribution[Index_PCA[1]] += 1/Components_Length   
        elseif PCA_Dimensions==3
            Cell_Distribution[Index_PCA[1],Index_PCA[2],Index_PCA[3]] += 1/Components_Length
        elseif PCA_Dimensions==4
            Cell_Distribution[Index_PCA[1],Index_PCA[2],Index_PCA[3],Index_PCA[4]] += 1/Components_Length
        elseif PCA_Dimensions==5
            Cell_Distribution[Index_PCA[1],Index_PCA[2],Index_PCA[3],Index_PCA[4],Index_PCA[5]] += 1/Components_Length
        else
        print("ADD MORE HERE THERE IS A MISTAKE, DO NOT GO OVER 5 PCA DIMENSIONS YET")
        end
    end
    

    
    return Cell_Distribution
end



function Earth_Movers_Distance_1D(Vector1,Vector2)

    #Takes 2 distribution vectors containing bins with a position and their %
    Length_V = length(Vector1[1,:]) #Both should be same length anyways
    EMD = zeros(Length_V+1) #We add an extra one because of the following loop
    
    for i =1:Length_V
        EMD[i+1] = Vector1[2,i] - Vector2[2,i] + EMD[i]
    end
    EMD_Distance = 0
    
    for i =1:Length_V-1
        EMD_Distance += abs(EMD[i+1])*(Vector1[1,i+1] - Vector1[1,i]) #We take moved distribution times moved distance

    end

    
    return EMD_Distance
end


function Robustness_Distribution_Scoring()
    
    
    #Here we create a very general robustness measure
    #This is done by taking the PCA components over all the different generations
    #Then we categorise all cell types and measure how similar their distribution
    #is in a given network

    #!!!!!!! This robustness does not consider cell type distances
    
    Robustness_Score = zeros(Generations,P)
    for Genere=1:Generations  #We want to calculate this for all generations
        for i = 1:P #We have to repeat this for all different networks in a generation
            #This is used to find the maximum of different cell types along every dimension in this network
            Dimensions_Distribution = zeros(Int32,PCA_Dimensions)
            for j=1:N
                Transformed = transform(model,reshape(signbit.(-History_Final_Epigenetic_Factors[Genere,i,j,:,:]),2^Splits,M)')

                for k=1:PCA_Dimensions
                    if Int.(length(countmap(Transformed[k,:]))) >  Dimensions_Distribution[k]
                        Dimensions_Distribution[k] = Int.(length(countmap(Transformed[k,:])))
                    end
                end   
            end

            #Now we have the max dimensions, now we want to make all distributions
            #Of this specific network

            Dimensions_Distribution = (Dimensions_Distribution...,) #Magically makes a tuple!
            #print(Dimensions_Distribution) #if we wanna visualise

            Cell_Distribution_Collection = Array{Array{Float64, PCA_Dimensions}}(undef, N)
            Vectorised_Cell_Distribution_Collection = Array{Vector{Float64}}(undef, N)

            for j=1:N #Now we collect all the distributions
                Transformed = transform(model,reshape(signbit.(-History_Final_Epigenetic_Factors[Genere,i,j,:,:]),2^Splits,M)')

                Cell_Distribution = zeros(Dimensions_Distribution) #This is where we need it to be a tuple!
                Cell_Distribution_Collection[j] = PCA_Distribution_Creator(PCA_Dimensions,Transformed,Dimensions_Distribution,Cell_Distribution)

                #Now we want to convert all of these into vectors to apply Jensen-Shannon
                Vectorised_Cell_Distribution_Collection[j] = sort(vec(Cell_Distribution_Collection[j]),rev=true)
            end

            #JS-divergence
            for tau = 1:N-1
                for pau = tau:N
                    Robustness_Score[Genere,i] += js_divergence(Vectorised_Cell_Distribution_Collection[tau],Vectorised_Cell_Distribution_Collection[pau])
                end
            end


        end
    end
    return Robustness_Score
end



function EMD_Generational_Calculator()
    
    #Here we will create a Robustness measure that takes into account how far
    #away different cell types are for the same network.
    #we will do this by creating a PCA analysis for EVERY single network
    #for all the different generations
    #(Alternatively we see if we can use the same PCA analysis within one generation)
    #We will create a one dimensional distribution and use our simplified
    #EMD algorithm
    
    Vectorised_Cell_Network_Collection = Array{Matrix{Float64}}(undef, N) #Here we save these distributions
    #This loops over copies of same network and makes all our distributions
    EMD_Distances = zeros(Generations)
    for generation_p = 1:Generations
        EMD_Dist = 0
        for P_Valu = 1:P
            FullNetworkBits = reshape(signbit.(-History_Final_Epigenetic_Factors[generation_p,P_Valu,:,:,:]),2^Splits*N,M)'
            Networkmodel= fit(PCA,FullNetworkBits;maxoutdim=1)
            Networkmodel_EigenV = eigvals(Networkmodel::PCA)
            for i=1:N
                #This creates the Distribution of a single individual
                #We start to apply this PCA to one of these individuals
                NetworkBits = reshape(signbit.(-History_Final_Epigenetic_Factors[generation_p,P_Valu,i,:,:]),2^Splits,M)'
                Transformed_Network = transform(Networkmodel,NetworkBits)
                Distribution_Netw = zeros(0)
                X_Values = zeros(0)
                Transformed_Network = round.(Transformed_Network,digits=4) #Otherwise ugly floating point numbers
                Length_Netw_Normal = 1/length(Transformed_Network)
                for i in Transformed_Network

                    if i in X_Values
                        Distribution_Netw[findfirst(==(i), X_Values)] += Length_Netw_Normal
                    else
                        X_Values = vcat(X_Values,i)
                        Distribution_Netw = vcat(Distribution_Netw,Length_Netw_Normal)
                    end
                end



                Combined_Distribution = zeros(2,length(X_Values))
                Combined_Distribution[1,:] = round.(X_Values/Networkmodel_EigenV,digits=4)
                Combined_Distribution[2,:] = Distribution_Netw
                Combined_Distribution = sortslices(Combined_Distribution,dims=2)

                Vectorised_Cell_Network_Collection[i] = Combined_Distribution
            end


            #Now we need to apply EMD to these distributions and save the result

            for tau = 1:N-1   #Preparing the vectors to have the same X values and length
                for pau = tau+1:N
                    Vector1 = Vectorised_Cell_Network_Collection[tau]
                    Vector2 = Vectorised_Cell_Network_Collection[pau]
                    for i in Vector1[1,:]
                        if !(i in Vector2[1,:])
                            Vector2 = hcat(Vector2, [i,0]) #Adding empty position
                        end
                    end
                    for i in Vector2[1,:]
                        if !(i in Vector1[1,:])
                            Vector1 = hcat(Vector1, [i,0]) #Adding empty position
                        end
                    end

                    Vector1 = (sortslices(Vector1,dims=2)) #Sorting them with the added X values
                    Vector2 = (sortslices(Vector2,dims=2))
                    EMD_Dist += Earth_Movers_Distance_1D(Vector1,Vector2) #Calculating EDM for 1D

                end
            end
        end
        EMD_Dist = 2*EMD_Dist/(P*N*(N-1))  #Normalisation
        EMD_Distances[generation_p] = EMD_Dist
    end
    return EMD_Distances
end


function Two_Generational_Plotter(Generation,Cut_Off,Duration,Length_Orbits,Opacity)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1+2*(Generation-1):2+2*(Generation-1)
        for j=1:Amount_Copies
            if Shortest>size(Orbit_Array[i,j])[2]
                Shortest = size(Orbit_Array[i,j])[2] #Used later to check which data set is the shortest when plotting
            end

            Total_Length += size(Orbit_Array[i,j])[2]

        end
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2*8*2^Splits)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1+2*(Generation-1):2+2*(Generation-1)
        for j=1:Amount_Copies
            if Cut_Off
                
                for p=0:2^Splits-1
                    PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i,j][2+p*M:1+(p+1)*M,1:Length_Orbits]
                end
            else
                Length_Orbits =  size(Orbit_Array[i,j])[2]
                for p=0:2^Splits-1
                    PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i,j][2+p*M:1+(p+1)*M,:]
                end
            end

            Collection_Index += Length_Orbits*2^Splits
        end
    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,2,Amount_Copies,2^Splits,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:2
        for j=1:Amount_Copies

            for tau=1:2^Splits
                tester = Orbit_Array[i+2*(Generation-1),j][2+(tau-1)*M:1+tau*M,1:Shortest]
                Transformed = transform(Orbital_Model,tester)
                PCA_Orbits_2Gen[i,j,tau,:,:] = Transformed
            end
        end
    end
    
    
    Colours = [:red,:orange,:maroon,:chocolate4]
    @gif for k=1:Duration
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen[1,1,1,1,1:k],PCA_Orbits_2Gen[1,1,1,2,1:k],PCA_Orbits_2Gen[1,1,1,3,1:k],legend=false,c=:blue,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)


        for j=1:8
            for tau=1:2^Splits
                PCAPlots = plot!(PCA_Orbits_2Gen[i,j,tau,1,1:k],PCA_Orbits_2Gen[i,j,tau,2,1:k],PCA_Orbits_2Gen[i,j,tau,3,1:k],legend=false,c=:blue,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)
            end
        end

        i=2
        for j=1:4
            colour = Colours[j]
            for tau=1:2^Splits
                PCAPlots = plot!(PCA_Orbits_2Gen[i,2*j-1,tau,1,1:k],PCA_Orbits_2Gen[i,2*j-1,tau,2,1:k],PCA_Orbits_2Gen[i,2*j-1,tau,3,1:k],legend=false,c=colour,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                PCAPlots = plot!(PCA_Orbits_2Gen[i,2*j,tau,1,1:k],PCA_Orbits_2Gen[i,2*j,tau,2,1:k],PCA_Orbits_2Gen[i,2*j,tau,3,1:k],legend=false,c=colour,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)

            end
        end


    end
end




function Generational_Tracker(Family_Distance,Original_Tracking,Types)
    Cell_Generational_Tracking = deepcopy(Original_Tracking)
    Original_Tracking = deepcopy(Cell_Generational_Tracking)
    size(Types[1,:])


    TypeFam = [Types[1,:],1]
    TypeFam = hcat(TypeFam,[Types[1,:],2])
    TypeFam = hcat(TypeFam,[Types[1,:],3])
    TypeFam = hcat(TypeFam,[Types[1,:],4])

    print(size(TypeFam[1,:]))

    TypeFam = [Types[1,:],1]
    Distances_Gen = zeros(length(Cell_Generational_Tracking[1,:]))
    Cell_Generational_Tracking[1,1] = 1
    for i=2:length(Cell_Generational_Tracking[1,:])
        NewFam = true
        for j=1:length(TypeFam[1,:]) 
            if sum(abs.(Types[i,:]-TypeFam[1,j])) < Family_Distance + 0.5
                if NewFam
                    NewFam = false
                    Cell_Generational_Tracking[1,i] = j
                    Distances_Gen[j] = sum(abs.(Types[i,:]-TypeFam[1,j]))
                elseif Distances_Gen[j] > sum(abs.(Types[i,:]-TypeFam[1,j]))
                    Cell_Generational_Tracking[1,i] = j
                    Distances_Gen[j] = sum(abs.(Types[i,:]-TypeFam[1,j]))
                end
            end
        end
        if NewFam
            Distances_Gen[i] = sum(abs.(Types[i,:]-TypeFam[1,1]))
            Cell_Generational_Tracking[1,i] = Int(length(TypeFam[1,:])+1)
            TypeFam = hcat(TypeFam,[Types[i,:],Int(length(TypeFam[1,:])+1)])

        end
    end

    Cell_Generational_Tracking = Cell_Generational_Tracking[:,sortperm(Cell_Generational_Tracking[1,:])]



    Amount_Types = Cell_Generational_Tracking[1,end]
    Top_Bound_Index = zeros(Amount_Types)
    for j=1:Amount_Types
        Top_Bound_Index[j] = findfirst(isequal(j),Cell_Generational_Tracking[1,:])
    end
    print(Top_Bound_Index[:])


    #print(Cell_Generational_Tracking)
    Copied_Tracker = deepcopy(Cell_Generational_Tracking[2:end,:])
    Copied_Tracker = Copied_Tracker/sum(Copied_Tracker[1,:])

    Boundaries = zeros(Generations,length(Copied_Tracker[1,:]))
    for beta in 2:length(Copied_Tracker[1,:])
        Copied_Tracker[:,beta] += Copied_Tracker[:,beta-1]
        Boundaries[:,beta] = Copied_Tracker[:,beta] - Copied_Tracker[:,beta-1]
    end

    Cell_Type_Plotter = plot(Copied_Tracker[:,1]; ribbon = (Copied_Tracker[:,1],LinRange(0, 0, Generations)))
    for beta in 2:length(Copied_Tracker[1,:])
        Cell_Type_Plotter = plot!(Copied_Tracker[:,beta]; ribbon = (Boundaries[:,beta],LinRange(0, 0, Generations)),legend=false)
    end
    display(Cell_Type_Plotter)




    

    #Now we do the same but we group them in families


    Family_Boundaries = ones(Generations,Amount_Types)
    Family_Boundaries[:,1] = Copied_Tracker[:,Int(Top_Bound_Index[2]-1)].-0
    for beta in 2:Amount_Types-1

        Family_Boundaries[:,beta] = Copied_Tracker[:,Int(Top_Bound_Index[beta+1]-1)] .- Copied_Tracker[:,Int(Top_Bound_Index[beta]-1)]
    end
    Family_Boundaries[:,end] = 1 .- Copied_Tracker[:,Int(Top_Bound_Index[Amount_Types]-1)]


    Cell_Type_Family_Plotter = plot(Copied_Tracker[:,Int(Top_Bound_Index[2]-1)]; ribbon = (Family_Boundaries[:,1],LinRange(0, 0, Generations)))
    for beta in 2:Amount_Types-1
        Cell_Type_Family_Plotter = plot!(Copied_Tracker[:,Int(Top_Bound_Index[beta+1]-1)]; ribbon = (Family_Boundaries[:,beta],LinRange(0, 0, Generations)),legend=false)
    end
    Cell_Type_Family_Plotter = plot!(ones(Generations); ribbon = (Family_Boundaries[:,Amount_Types],LinRange(0, 0, Generations)),legend=false)

    if false
        Cell_Type_Family_Plotter = plot!(Copied_Tracker[:,1],linestyle=:dash)
        for beta in 2:length(Copied_Tracker[1,:])
            Cell_Type_Family_Plotter = plot!(Copied_Tracker[:,beta],linestyle=:dash)
        end
    end

    display(Cell_Type_Family_Plotter)
end

function Generational_Tracker_Initialisation()
    TypesString = deepcopy(Cell_Generational_Tracking[1,:])
    TypesString = string.(TypesString,base=2,pad=100)
    Types = zeros(Int16,length(Cell_Generational_Tracking[1,:]),M)
    for i in 1:length(Cell_Generational_Tracking[1,:])
        for j=1:M
            Types[i,j] = (parse(Int16,TypesString[i][j]))
        end
    end
    Original_Tracking = deepcopy(Cell_Generational_Tracking)
    return Original_Tracking, Types
end

function Interpolator(Times,Play_Array)
    Vectors = zeros(length(Times),1+M*2^Splits)
    for i=1:length(Times)
        Index_Interp = findfirst(>=(Times[i]), Play_Array[1,:])
        Vectors[i,:] = Play_Array[:,Index_Interp-1] + (Times[i]-Play_Array[1,Index_Interp-1])*(Play_Array[:,Index_Interp] - Play_Array[:,Index_Interp-1])/(Play_Array[1,Index_Interp] - Play_Array[1,Index_Interp-1])
    end
    return Vectors
end


function One_Generational_Plotter(Cut_Off,Duration,Length_Orbits,Opacity,Name)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1:N_Perturbed

            if Shortest>size(Orbit_Array[i])[2]
                Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
            end
            Total_Length += size(Orbit_Array[i])[2]
        
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1:N_Perturbed
        
        if Cut_Off

            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
            end
        else
            Length_Orbits =  size(Orbit_Array[i,j])[2]
            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
            end
        end

        Collection_Index += Length_Orbits*2^Splits

    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,N_Perturbed,2^Splits,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:N_Perturbed
        
        for tau=1:2^Splits
            tester = Orbit_Array[i][2+(tau-1)*M:1+tau*M,1:Shortest]
            Transformed = transform(Orbital_Model,tester)
            PCA_Orbits_2Gen[i,tau,:,:] = Transformed
        end

    end
    
    
    anim = @animate for k=1:Duration
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen[1,1,1,1:k],PCA_Orbits_2Gen[1,1,2,1:k],PCA_Orbits_2Gen[1,1,3,1:k],legend=false,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)


        for i=1:N_Perturbed
            for tau=1:2^Splits
                PCAPlots = plot!(PCA_Orbits_2Gen[i,tau,1,1:k],PCA_Orbits_2Gen[i,tau,2,1:k],PCA_Orbits_2Gen[i,tau,3,1:k],legend=false,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)
            end
        end

        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 5
    gif(anim, Name, fps = 15)
    
end

function adjoint(a)
    return a'
end


function Coarsed_Variance(Variance_Length,Time_Orbital_Collection,Coarsing,Orbit_Length)
    
    
    #Variance[i] = sum((Time_Orbital_Collection[i,:,:] .- mean(Time_Orbital_Collection[i,:,:],dims=1)).^2)/(M-1)

    
    
    Variance = zeros(Variance_Length)
    Average = zeros(Variance_Length,M)
    
    for i = 1:Variance_Length #For all different times
       Average[i,:] =  mean(Time_Orbital_Collection[i,:,:],dims=1)[1,:]
        for k=1:Orbit_Length #Have to do it for all the orbits
            Temp_Var = 1000
            Add_Var = 0
            
            for j=max(1,i-Coarsing):i+Coarsing 
                Add_Var = sum((Time_Orbital_Collection[j,k,:] .- Average[i,:]).^2)/(M-1)

                if Add_Var < Temp_Var
                    Temp_Var = Add_Var               
                end
            end
            Variance[i] += Temp_Var  

        end
    end
    
    return Variance
end


function Multi_One_Generational_Plotter(Cut_Off,Duration,Length_Orbits,Opacity,Name,Amount_Perturbations,Partial,N_Perturbed)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1:N_Perturbed*Amount_Perturbations

            if Shortest>size(Orbit_Array[i])[2]
                Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
            end
            Total_Length += size(Orbit_Array[i])[2]
        
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed*Amount_Perturbations)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1:N_Perturbed*Amount_Perturbations
        
        if Cut_Off

            for p=0:2^Splits-1

                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
            end
        else
            Length_Orbits =  size(Orbit_Array[i,j])[2]
            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
            end
        end

        Collection_Index += Length_Orbits*2^Splits

    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,N_Perturbed*2^Splits,Amount_Perturbations,3,Shortest)
    PCA_Orbits_Partial = zeros(Float32,Int(N_Perturbed/Partial)*2^Splits,Amount_Perturbations,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:N_Perturbed
        for j=0:Amount_Perturbations-1
        for tau=1:2^Splits
            tester = Orbit_Array[i+N_Perturbed*j][2+(tau-1)*M:1+tau*M,1:Shortest]
            Transformed = transform(Orbital_Model,tester)
            PCA_Orbits_2Gen[(i-1)*2^Splits+tau,j+1,:,:] = Transformed
            if mod(i,Partial) == 0
                PCA_Orbits_Partial[(Int(i/Partial)-1)*2^Splits+tau,j+1,:,:] = Transformed
            end
        end
        end

    end
    Colours = [:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
    
    #PCA_Orbits_Partial = reshape(PCA_Orbits_Partial,(Int(2^Splits*N_Perturbed*Amount_Perturbations/Partial),3,Shortest))
    PCA_Orbits_2Gen_Plot = permutedims(PCA_Orbits_Partial,(4,3,2,1))
    Orbit_Range = Int(N_Perturbed/Partial)*2^Splits
    tick()
    anim = @animate for k=1:Int(Duration/5)
        
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,:],PCA_Orbits_2Gen_Plot[1:k,2,1,:],PCA_Orbits_2Gen_Plot[1:k,3,1,:],c=Colours[1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for p=1:Amount_Perturbations
            PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,:],PCA_Orbits_2Gen_Plot[1:k,2,p,:],PCA_Orbits_2Gen_Plot[1:k,3,p,:],c=Colours[p],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)

        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 5
    tock()
    tick()
    animlong = @animate for k=1:Duration
        
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,:],PCA_Orbits_2Gen_Plot[1:k,2,1,:],PCA_Orbits_2Gen_Plot[1:k,3,1,:],c=Colours[1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for p=1:Amount_Perturbations
            PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,:],PCA_Orbits_2Gen_Plot[1:k,2,p,:],PCA_Orbits_2Gen_Plot[1:k,3,p,:],c=Colours[p],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)

        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 50
    tock()
    abra = gif(anim, Name, fps = 10)
    display(abra)

    kadabra = gif(animlong, Name[1:end-4] * "_Long" *Name[end-3:end], fps = 3)
    display(kadabra)



end


function Multi_One_Generational_Plotter2(Cut_Off,Duration,Length_Orbits,Opacity,Name,Amount_Perturbations,Partial,N_Perturbed,Orbit_Array)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1:N_Perturbed*Amount_Perturbations

            if Shortest>size(Orbit_Array[i])[2]
                Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
            end
            Total_Length += size(Orbit_Array[i])[2]
        
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed*Amount_Perturbations)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1:N_Perturbed*Amount_Perturbations
        
        if Cut_Off

            for p=0:2^Splits-1

                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
            end
        else
            Length_Orbits =  size(Orbit_Array[i,j])[2]
            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
            end
        end

        Collection_Index += Length_Orbits*2^Splits

    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,N_Perturbed*2^Splits,Amount_Perturbations,3,Shortest)
    PCA_Orbits_Partial = zeros(Float32,Int(N_Perturbed/Partial)*2^Splits,Amount_Perturbations,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:N_Perturbed
        for j=0:Amount_Perturbations-1
        for tau=1:2^Splits
            tester = Orbit_Array[i+N_Perturbed*j][2+(tau-1)*M:1+tau*M,1:Shortest]
            Transformed = transform(Orbital_Model,tester)
            PCA_Orbits_2Gen[(i-1)*2^Splits+tau,j+1,:,:] = Transformed
            if mod(i,Partial) == 0
                PCA_Orbits_Partial[(Int(i/Partial)-1)*2^Splits+tau,j+1,:,:] = Transformed
            end
        end
        end

    end
    Colours = [:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
    
    #PCA_Orbits_Partial = reshape(PCA_Orbits_Partial,(Int(2^Splits*N_Perturbed*Amount_Perturbations/Partial),3,Shortest))
    PCA_Orbits_2Gen_Plot = permutedims(PCA_Orbits_Partial,(4,3,2,1))
    Orbit_Range = Int(N_Perturbed/Partial)*2^Splits
    tick()
    anim = @animate for k=1:Int(Duration/5)
        
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,:],PCA_Orbits_2Gen_Plot[1:k,2,1,:],PCA_Orbits_2Gen_Plot[1:k,3,1,:],c=Colours[1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for p=1:Amount_Perturbations
            PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,:],PCA_Orbits_2Gen_Plot[1:k,2,p,:],PCA_Orbits_2Gen_Plot[1:k,3,p,:],c=Colours[p],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)

        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 5
    tock()
    tick()
    animlong = @animate for k=1:Duration
        
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,:],PCA_Orbits_2Gen_Plot[1:k,2,1,:],PCA_Orbits_2Gen_Plot[1:k,3,1,:],c=Colours[1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for p=1:Amount_Perturbations
            PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,:],PCA_Orbits_2Gen_Plot[1:k,2,p,:],PCA_Orbits_2Gen_Plot[1:k,3,p,:],c=Colours[p],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)

        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 50
    tock()
    abra = gif(anim, Name, fps = 10)
    display(abra)

    kadabra = gif(animlong, Name[1:end-4] * "_Long" *Name[end-3:end], fps = 3)
    display(kadabra)

    
    PCAPlots_Final = scatter(PCA_Orbits_2Gen_Plot[end,1,1,:],PCA_Orbits_2Gen_Plot[end,2,1,:],PCA_Orbits_2Gen_Plot[end,3,1,:],title="Final Cell States",c=Colours[1],legend=false)
    for p=1:Amount_Perturbations
        PCAPlots_Final = scatter!(PCA_Orbits_2Gen_Plot[end,1,p,:],PCA_Orbits_2Gen_Plot[end,2,p,:],PCA_Orbits_2Gen_Plot[end,3,p,:],c=Colours[p],legend=false)

    end

    display(PCAPlots_Final)
    savefig(Name[1:end-4] * "_Final.png")
end

function Multi_One_Generational_Plotter3(Cut_Off,Duration,Length_Orbits,Opacity,Name,Amount_Perturbations,Partial)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1:N_Perturbed*Amount_Perturbations

            if Shortest>size(Orbit_Array[i])[2]
                Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
            end
            Total_Length += size(Orbit_Array[i])[2]
        
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed*Amount_Perturbations)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1:N_Perturbed*Amount_Perturbations
        
        if Cut_Off

            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
            end
        else
            Length_Orbits =  size(Orbit_Array[i,j])[2]
            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
            end
        end

        Collection_Index += Length_Orbits*2^Splits

    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,N_Perturbed*2^Splits,Amount_Perturbations,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:N_Perturbed
        for j=1:2^Splits
            for tau=1:Amount_Perturbations
                tester = Orbit_Array[i+ (tau-1)*N_Perturbed][2+(j-1)*M:1+j*M,1:Shortest]
                Transformed = transform(Orbital_Model,tester)
                PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,:,:] = Transformed

            end
        end
    end
    Colours = [:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
    
    PCA_Orbits_2Gen_Plot = permutedims(PCA_Orbits_2Gen,(4,3,2,1))
    anim = @animate for k=1:Duration
        
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1],c=Colours[1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for l=1:Amount_Perturbations
            
            PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,l,:],PCA_Orbits_2Gen_Plot[1:k,2,l,:],PCA_Orbits_2Gen_Plot[1:k,3,l,:],c=Colours[l],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
            
        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 15

    abra = gif(anim, Name, fps = 15)
end

function N_Dimensional_Euclid(Position1,Position2,Dimensions)
    Summed = 0
    for i =1:Dimensions
        Summed += (Position2[i]-Position1[i])^2
    end
    Distance = sqrt(Summed)
    return Distance
end



function Distance_Movement_Matrix(Movement_Matrix)
    
    for i=1:N_Sources #we must loop over all the sources to find a good sink for it

        #First we try to find the closest sinks
        Distance_Indices = (hcat(DistanceMatrix[i,:], 1:N_Sinks))
        Distance_Indices = Distance_Indices[sortperm(Distance_Indices[:, 1]), :]
        Distance_Indices = Int.(Distance_Indices[:,2])
        print(Distance_Indices)

        #Then we check if we can still send anything to this sink
        Displacing = true
        D_Index = 1
        while Displacing
            I_Sink = Distance_Indices[D_Index] #This is the index of the sink we are checking now
            Available_Sink = Sinks_Input[I_Sink] - sum(Movement_Matrix[:,I_Sink])
            print(Sinks_Input[I_Sink])
            if Available_Sink > 0 #It can have more added to it
                #We check if we can send everything to the sink or only partial
                Remaining_Flow = Sources_Output[i] - sum(Movement_Matrix[i,:])
                if Remaining_Flow > Available_Sink
                #If partial we fill a bit and go back to the start to find the next closest sink
                    Movement_Matrix[i,I_Sink] = Available_Sink
                    D_Index += 1
                    print("Partial.     ")
                    #BUT WE HAVE TO CHECK IF WE JUST FILLED IT ALL IN!!!! 
                    if Sources_Output[i] - sum(Movement_Matrix[i,:]) < 0.0001
                        Displacing = false
                        print("Finished After Partial")
                    end

                else  
                #If we can send it all we do and go to the next Source i+1
                    Displacing = false
                    Movement_Matrix[i,I_Sink] = Remaining_Flow
                    print("Finished.     ")

                end
            else #Sink is full
                print("Full.     ")

                #Cannot send more in the sink so we have to check the next one
                D_Index += 1
            end
        end
    end
    Movement_Matrix = round.(Movement_Matrix,digits=4)

    return Movement_Matrix
end

function Cost_Reducing_Rewiring(Attempts,Optimised_Indices)
    print("Initial Costs: ")
    print(sum(Cost_Matrix))
    for yota =1:Attempts
        println(" ")
        #We make a big while loop until a certain condition we decide later
        Cost_Matrix = DistanceMatrix .* Movement_Matrix
        print(sum(Cost_Matrix))
        for theta =1:1   #Old loop that can be removed

            if Optimised_Indices != []
                for kappa=1:length(Optimised_Indices[1,:])
                    Cost_Matrix[Optimised_Indices[1,kappa],Optimised_Indices[2,kappa]] = 0
                end
            end

            #We find the connection that has the highest cost, find the source and the corresponding sink
            Source_Index = argmax(Cost_Matrix)[1]
            Sink_Index = argmax(Cost_Matrix)[2]
            Improvement = 0 #Use this to keep track of best
            Displace = 0 #Keep track of best
            Best_Source = 0 #Keep track
            Best_Sink = 0 #Keep track

            #We let this source connect to other sinks instead and try to change another source that was connected to that sink
            for i=1:N_Sinks #We loop over all other sinks
                if i!=Sink_Index #Make sure it is a different sink
                    for j=1:N_Sources #We loop over all other Sources
                        if j!=Source_Index #Make sure it is a different source
                            #Check how much we may change 
                            Displace_Temp = min(Movement_Matrix[Source_Index,Sink_Index],Movement_Matrix[j,i])
                            #Calculate the improvement
                            #-DM[So,Si] + DM[So,i] We remove So -> Si and make So -> i
                            #-DM[j,i] + DM[j,Si] We remove j -> i and make j -> Si
                            Temp_Improvement = Displace_Temp*(-DistanceMatrix[Source_Index,Sink_Index] + DistanceMatrix[Source_Index,i] -DistanceMatrix[j,i] + DistanceMatrix[j,Sink_Index])

                            #Find the best improvement for this initial Source_Sink combo
                            if Temp_Improvement < Improvement #Found a better improvement
                                Improvement = Temp_Improvement
                                Displace = Displace_Temp
                                Best_Source = j
                                Best_Sink = i
                            end

                        end
                    end
                end
            end
            print("Better amount:  ")
            print(Improvement)
            if Improvement==0
                print("No improvement???")
                print(Displace)
                print("    ")
                print(Sink_Index)
                print(Source_Index)
                if Optimised_Indices == []
                    Optimised_Indices = vcat(Optimised_Indices,[Source_Index;Sink_Index])
                else
                    Optimised_Indices = hcat(Optimised_Indices,[Source_Index;Sink_Index])
                end

            else
                print("WE IMPROVED")
                Movement_Matrix[Source_Index,Sink_Index] -= Displace
                Movement_Matrix[Best_Source,Best_Sink] -= Displace
                Movement_Matrix[Best_Source,Sink_Index] += Displace
                Movement_Matrix[Source_Index,Best_Sink] += Displace


            end

            #Switch the best one and repeat
        end
        print("   ")
        Cost_Matrix = DistanceMatrix .* Movement_Matrix
        print(sum(Cost_Matrix))
    end

    println(" ")
    print("Final Costs: ")
    print(sum(Cost_Matrix)) 
    return sum(Cost_Matrix)
end

function Final_Cell_PCAScatterPlot(Orbit_Array,Single_Plot)
    Final_Cell_States_PCA = zeros(Amount_Large_Perturbations*Amount_Small_Perturbations*2^Splits,M)
    Transformed_Data = zeros(Amount_Small_Perturbations*2^Splits,Amount_Large_Perturbations,2)
    Before_Data = zeros(Amount_Small_Perturbations*2^Splits,Amount_Large_Perturbations,M)
    for j=1:Amount_Large_Perturbations*Amount_Small_Perturbations
        for i=1:2^Splits
            Final_Cell_States_PCA[j+Amount_Large_Perturbations*Amount_Small_Perturbations*(i-1),:] = Orbit_Array[j][2+(i-1)*M:1+i*M,end]
        end
    end
    #This is added in in case we only have the same cell types, then our code would crash
    Final_Cell_States_PCA[1,1] += 1
    
    
    for j=1:Amount_Large_Perturbations
        for i=1:Amount_Small_Perturbations
            for tau=1:2^Splits
                Before_Data[i+(tau-1)*Amount_Small_Perturbations,j,:] = Orbit_Array[i+(j-1)*Amount_Small_Perturbations][2+(tau-1)*M:1+tau*M,end]
            end
        end
    end

    Final_Cell_States_PCA[1:5,1:10] .+= 1
    Final_Cell_PCA = fit(PCA,Final_Cell_States_PCA';maxoutdim =2,pratio =1)
    print(size(Final_Cell_PCA))
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Final_Cell_PCA))
    Final_Cell_States_PCA[1:5,1:10] .-= 1
 

    for j=1:Amount_Large_Perturbations

        Transformed_Data[:,j,:] = transform(Final_Cell_PCA,Before_Data[:,j,:]')'

    end
    Plot_Legend = true
    Colours = [:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
    if Single_Plot == false
        j=1
        abrada = scatter((Transformed_Data[1,j,1],Transformed_Data[1,j,2]),xlabel="PCA 1",ylabel="PCA 2",title="Final Cell States in PCA landscape",c=Colours[j],alpha=0.4,legend=Plot_Legend,label="Initial Noise = " * string(Large_Pert_Radius[j]))
        for j=2:Amount_Large_Perturbations
            abrada = scatter!((Transformed_Data[1,j,1],Transformed_Data[1,j,2]),c=Colours[j],alpha=0.4,legend=Plot_Legend,label="Initial Noise = " * string(Large_Pert_Radius[j]))
        end
        for j=1:Amount_Large_Perturbations
            for l=2:length(Transformed_Data[:,j,1])
                for j=1:Amount_Large_Perturbations
                    abrada = scatter!((Transformed_Data[l,j,1],Transformed_Data[l,j,2]),c=Colours[j],alpha=0.4,label=false)
                end
            end
        end
        savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/PCA_Maps/Scatter_PCA"*Copy_String*".png")

        KernelData = reshape(Transformed_Data, (Amount_Small_Perturbations*2^Splits*Amount_Large_Perturbations, 2))
        
        k = kde((KernelData[:,1], KernelData[:,2]))


        display(contourf(k,
             c = :vik,title="Final Cell State PCA Heatmap",
             linewidth = 0))
        savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/PCA_Maps/Heatmap_"*Copy_String*".png")

        
    else
        j = Single_Plot
        abrada = scatter((Transformed_Data[:,j,1],Transformed_Data[:,j,2]),xlabel="PCA 1",ylabel="PCA 2",title="Final Cell States in PCA landscape",c=Colours[j],alpha=0.4,legend=Plot_Legend,label="Initial Noise = " * string(Large_Pert_Radius[j]))
        
    end
    return abrada
end



function Multi_One_Generational_Plotter4(Cut_Off,Duration,Length_Orbits,Opacity,Name,Amount_Perturbations,Partial,N_Perturbed,Orbit_Array,Final_Epigenetic_Factors,Gene_Colour_Collection)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1:N_Perturbed*Amount_Perturbations

            if Shortest>size(Orbit_Array[i])[2]
                Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
            end
            Total_Length += size(Orbit_Array[i])[2]
        
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed*Amount_Perturbations)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1:N_Perturbed*Amount_Perturbations
        
        if Cut_Off

            for p=0:2^Splits-1

                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
            end
        else
            Length_Orbits =  size(Orbit_Array[i,j])[2]
            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
            end
        end

        Collection_Index += Length_Orbits*2^Splits

    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,Amount_Perturbations,N_Perturbed,2^Splits,3,Shortest)
    PCA_Orbits_Partial = zeros(Float32,Amount_Perturbations,Int(N_Perturbed/Partial),2^Splits,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:N_Perturbed
        for j=0:Amount_Perturbations-1
        for tau=1:2^Splits
            tester = Orbit_Array[i+N_Perturbed*j][2+(tau-1)*M:1+tau*M,1:Shortest]
            Transformed = transform(Orbital_Model,tester)
            PCA_Orbits_2Gen[j+1,i,tau,:,:] = Transformed
            if mod(i-1,Partial) == 0
                PCA_Orbits_Partial[j+1,(Int(i/Partial)),tau,:,:] = Transformed
            end
        end
        end

    end
    Colours = [:red,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
     
    #PCA_Orbits_Partial = reshape(PCA_Orbits_Partial,(Int(2^Splits*N_Perturbed*Amount_Perturbations/Partial),3,Shortest))
    PCA_Orbits_2Gen_Plot = permutedims(PCA_Orbits_Partial,(5,4,3,2,1))
    Orbit_Range = Int(N_Perturbed/Partial)*2^Splits
    
    print(size(PCA_Orbits_2Gen_Plot))
    if false
        for Gene_Colour in Gene_Colour_Collection
            tick()
            
            anim = @animate for k=1:Int(Duration/5)
                i=1
                PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1,1],c=Colours[Final_Epigenetic_Factors[1,1,1,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                for q=1:Int(N_Perturbed/Partial)
                    for p=1:2^Splits
                        for t =1:Amount_Perturbations
                        PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,q,t],PCA_Orbits_2Gen_Plot[1:k,2,p,q,t],PCA_Orbits_2Gen_Plot[1:k,3,p,q,t],c=Colours[Final_Epigenetic_Factors[t,q,p,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                        end
                    end
                end
        
                #for i=1:Int(N_Perturbed/5)
                #    for j=1:Amount_Perturbations
                #        for tau=1:2^Splits
                #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                #        end
                #    end
                #end
                
                #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])
        
                title!("Gene = $Gene_Colour Time = $k")
            end every 5
            tock()
            tick()
            animlong = @animate for k=1:Duration
                
                i=1
                PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1,1],c=Colours[Final_Epigenetic_Factors[1,1,1,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                for q=1:Int(N_Perturbed/Partial)
                    for p=1:2^Splits
                        for t =1:Amount_Perturbations
                        PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,q,t],PCA_Orbits_2Gen_Plot[1:k,2,p,q,t],PCA_Orbits_2Gen_Plot[1:k,3,p,q,t],c=Colours[Final_Epigenetic_Factors[t,q,p,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                        end
                    end
                end
                #for i=1:Int(N_Perturbed/5)
                #    for j=1:Amount_Perturbations
                #        for tau=1:2^Splits
                #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                #        end
                #    end
                #end
                
                #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])
        
                title!("Gene = $Gene_Colour Time = $k")
            end every 50
            tock()
    
            abra = gif(anim, Name[1:end-4]*"_Gene:"*string(Gene_Colour)*Name[end-3:end], fps = 10)
            display(abra)
        
            kadabra = gif(animlong, Name[1:end-4] * "_Long"*"_Gene:"*string(Gene_Colour) *Name[end-3:end], fps = 3)
            display(kadabra)
        
            
            Total_Ratio = principalratio(Orbital_Model)
            Eigen_Values = eigvals(Orbital_Model)
            Eigen_Values = Eigen_Values/sum(Eigen_Values)
            Eigen1 = Eigen_Values[1]
            Eigen2 = Eigen_Values[2]
            Eigen3 = Eigen_Values[3]
            PCAPlots_Final = scatter(PCA_Orbits_2Gen_Plot[end,1,1,1,:],PCA_Orbits_2Gen_Plot[end,2,1,1,:],PCA_Orbits_2Gen_Plot[end,3,1,1,:],c=Colours[Final_Epigenetic_Factors[1,1,1,Gene_Colour]+1],legend=true,label="
            Principal 1: $Eigen1
            Principal 2: $Eigen2]
            Principal 3: $Eigen3")
        
            for q=1:Int(N_Perturbed/Partial)
                for p=1:2^Splits
                    for t =1:Amount_Perturbations
                        PCAPlots_Final = scatter!((PCA_Orbits_2Gen_Plot[end,1,p,q,t],PCA_Orbits_2Gen_Plot[end,2,p,q,t],PCA_Orbits_2Gen_Plot[end,3,p,q,t]),c=Colours[Final_Epigenetic_Factors[t,q,p,Gene_Colour]+1],legend=true,label="")
                    end
                end
            end
            
            
            title!("Gene = $Gene_Colour , Ratio: $Total_Ratio")
            display(PCAPlots_Final)
            savefig(Name[1:end-4] * "_Final_Static_$Gene_Colour.png")
            
        end
    end
    Multi_Colours = [:cyan,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:darkmagenta,:lightcyan4,:olive,:gold4,:lightsalmon,:navajowhite,:pink]
    Only3Types = false
    tick()
    anim = @animate for k=1:Int(Duration/5)
        
        i=1
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
        if Only3Types
            Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
        end
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1,1],c=Multi_Colours[Colour_Index],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for q=1:Int(N_Perturbed/Partial)
            for p=1:2^Splits
                for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,q,t],PCA_Orbits_2Gen_Plot[1:k,2,p,q,t],PCA_Orbits_2Gen_Plot[1:k,3,p,q,t],c=Multi_Colours[Colour_Index],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                end
            end
        end
    
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])
    
        title!("Gene = Multiple Time = $k")
    end every 5
    tock()
    tick()
    animlong = @animate for k=1:2*Duration
        
        i=1
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
        if Only3Types
            Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
        end
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1,1],c=Multi_Colours[Colour_Index],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for q=1:Int(N_Perturbed/Partial)
            for p=1:2^Splits
                for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,q,t],PCA_Orbits_2Gen_Plot[1:k,2,p,q,t],PCA_Orbits_2Gen_Plot[1:k,3,p,q,t],c=Multi_Colours[Colour_Index],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                end
            end
        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])
    
        title!("Gene = Multiple Time = $k")
    end every 50
    tock()
    abra = gif(anim, Name[1:end-4]*Name[end-3:end], fps = 10)
    display(abra)
    
    kadabra = gif(animlong, Name[1:end-4]*Name[end-3:end], fps = 3)
    display(kadabra)
    
    
    
    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    PCAPlots_Final = scatter(PCA_Orbits_2Gen_Plot[end,1,1,1,:],PCA_Orbits_2Gen_Plot[end,2,1,1,:],PCA_Orbits_2Gen_Plot[end,3,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Final = scatter!((PCA_Orbits_2Gen_Plot[end,1,p,q,t],PCA_Orbits_2Gen_Plot[end,2,p,q,t],PCA_Orbits_2Gen_Plot[end,3,p,q,t]),c=Multi_Colours[Colour_Index],legend=false)
            end
        end
    end
    Total_Ratio = principalratio(Orbital_Model)
    Eigen_Values = eigvals(Orbital_Model)
    Eigen_Values = Eigen_Values/sum(Eigen_Values)
    title!("Static Multi Colour, Ratio: $Total_Ratio")
    display(PCAPlots_Final)
    savefig(Name[1:end-4] * "Final_Multi_Colour.png")



    
    PcaPlots_Time_Length = 500
    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    
    PCAPlots_Time1 = plot(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,1,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Time1 = plot!(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,1,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA 1")
            end
        end
    end
    display(PCAPlots_Time1)

    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    
    PCAPlots_Time2 = plot(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,2,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Time2 = plot!(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,2,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA 1")
            end
        end
    end
    display(PCAPlots_Time2)

    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    
    PCAPlots_Time3 = plot(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,3,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Time3 = plot!(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,3,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA 1")
            end
        end
    end
    display(PCAPlots_Time3)
    
end

function Evolution_NoEpi(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    
    
    
    #⊗_(Wi, bi) = Wi .* bi
       

    du[1,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,:].+u[2,:].+CArray[:])) .- u[1,:]
    
    #Epigenetic Evolution, no need to worry about boundaries here so simple loop      

    
end




##############################################################################################################################
##############################################################################################################################
##############################################################################################################################








#Load_Generations = [70]
#Load_Generations = vcat(1,Load_Generations) #These are all the generations that we will analyse
Copies = [1] #Can reduce this to just 1,2 or just 1 if we don't want as many copies



#Main variables to change:
Stochastic_Noise = true
N::Int32 = 8 #How many copies of same network we have
#K::Int32 = N  Depricated parameter, do not change, if ever K not defined replace it by N
P = 4 #How many different networks will we use in the evolution run
Generations = 6 #How many generations in the evolution run

#DO NOT CHANGE THESE
Splits::Int32 = 6 #DO NOT CHANGE
SplitTime::Float32 = 0.1 #Time before a splitting event happens, DOES NOT MATTER HERE
M::Int32 = 40 #This is the amount of genes we want in our network
Splitting_Strength = 0.0f0

#CAREFUL WITH M>127 DO NOT INCREASE WITHOUT REWRITING DIFFERENTIATION FUNCTION
#BIT OVERFLOW!!!!!


#Constant and plotting tools


InvSqr::Float32 = 1/sqrt(M)
PCA_Plotting = false 
Orbit_Plotting = true  #Currently not used yet
Epi_Plotting = false #Currently not used yet
Counter = 0 #Used to randomise the mutations, leave to keep same seed
Saved_Generations = [2,4] #These are the generations where we want to save some orbits for


#Diffusion or mean field parameters
Mean_Field = false
Diffusing_System = false
KArray = zeros(Float32,M,1) #Diffusion/Mean Field array is set to zero everywhere
#KArray[1:10] .= 0.1 #Here we can manually adjust which genes will have diffusion















Amount_Large_Perturbations = 1 #How many large perturbations we will have each time
Amount_Small_Perturbations = 1 #How many smaller perturbations we will have around our large perturbation
Large_Pert_Radius = [0,1,5,20]
Small_Pert_Radius = 0
#Perturbations = zeros(32)









Generation_String = string(Load_Generations[1])
Copy_String = string(Copies[1])
Load_Data_String_Load = Data_To_Analyse
#Load_Data_String = Data_To_Analyse
#Copies = [1]
#Load_Generations = [10]
Orbit_Array = 0
Time_Orbital_Collection2 = 0
Orbit_Collection = 0
EpiThroughTime = 0

#To have access after the loop
Interpolated_Orbits = 0
Final_Epigenetic_Factors = 0
Gene_Cells_Colour = 0

@showprogress 1 for zeta in Load_Generations
    GC.gc()
    for iota in Copies
        Generation_String = string(zeta)

        Copy_String = string(iota)
        Current_Network_String = "/Gen_"*Generation_String*"_"*Copy_String*".jld"
        Loaded_Data = load(Load_Data_String_Load*Current_Network_String)
        mkpath(Load_Data_String*"/Processed/Gen:"*Generation_String*"/PCA_Maps")
        mkpath(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Orbits")
        mkpath(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance")


        #Here we load and perturb the networks


        Duration = 1000 #For Animation should be 3000
        Length_Orbits = 3000 #For PCA should be 4000
        Opacity = 75
        Partial = 1

        Variance_Length = 2500
        Times = [i for i=1:Variance_Length+50]
        Times_Orbit = [i for i=1:Length_Orbits]


        Perturb_Seed = 12345
        rng=MersenneTwister(Perturb_Seed)
        #Amount_Large_Perturbations = 1 #How many large perturbations we will have each time
        #Amount_Small_Perturbations = 16 #How many smaller perturbations we will have around our large perturbation
        #Large_Pert_Radius = 10
        #Small_Pert_Radius = 3
        CArray_Collection = zeros(Float32,Amount_Large_Perturbations,M)
        PositionArray_Collection = zeros(Float32,Amount_Large_Perturbations,Amount_Small_Perturbations,2,1,M)
        ParameterMatrix_Collection = zeros(Int32,Amount_Large_Perturbations,M,M)
        Final_Epigenetic_Factors = zeros(Int8,Amount_Large_Perturbations,Amount_Small_Perturbations,2^(Splits),M)
        Epi_Factors = zeros(Float16,2^(Splits),M)
        for j=1:Amount_Large_Perturbations
            CArray_Collection[j,:] = Loaded_Data["CArray"]
            ParameterMatrix_Collection[j,:,:] = Loaded_Data["Network"]

            #Write large pert vector here

            Noise_Vector = randn(rng,M).-0.5
            Norm = sqrt(sum(Noise_Vector.^2))
            Noise_Vector = Large_Pert_Radius[j]*Noise_Vector/Norm
            for i=1:Amount_Small_Perturbations


                #Small pert vector here
                Noise_Vector2 = randn(rng,M).-0.5
                Norm = sqrt(sum(Noise_Vector2.^2))
                Noise_Vector2 = rand()^(1/M)*Small_Pert_Radius*Noise_Vector2/Norm

                PositionArray_Collection[j,i,1,1,:] = Loaded_Data["PositionArray"] + Noise_Vector' + Noise_Vector2'

            end
        end

        Evolutionary_Complexity = []


        Random_Noise = 574 #seed for splitting noise

        #Need to define outside for loop, otherwise it will be destroyed afterwards
        Genes = 0





        KLDistance_temp = 0
        BitDistance_temp =  0
        Evolutionary_Complexity = []
        Indices = 0
        Information = zeros(1)
        Ratio = zeros(1)
        Cell_Type_Tracker = []
        Cell_Collection = []
        Evolutionary_Complexity = zeros(N)
        Temporary = []
        Orbit_Array = Array{Matrix{Float16}}(undef,Amount_Large_Perturbations*Amount_Small_Perturbations)
        Interpolated_Orbits = Array{Matrix{Float16}}(undef,Amount_Large_Perturbations*Amount_Small_Perturbations)

        Time_Orbital_Collection = 0

        for yeta=1:Amount_Large_Perturbations
            for x=1:Amount_Small_Perturbations
                #CArray = CArray_Collection[x,:]
                #ParameterMatrix = ParameterMatrix_Collection[x,:,:]
                #PositionArray = PositionArray_Collection[x,:]
                Epi_Factors,Random_Noise,Temporary, EpiThroughTime = Running_Network(CArray_Collection[yeta,:],ParameterMatrix_Collection[yeta,:,:],PositionArray_Collection[yeta,x,:,:,:],Random_Noise)
                Final_Epigenetic_Factors[yeta,x,:,:] = signbit.(Epi_Factors)
                Orbit_Array[x+(yeta-1)*Amount_Small_Perturbations] = Temporary

            end

        end
        #Indices,AllIndices,Evolutionary_Complexity[:],Cell_Collection, Cell_Type_Tracker,KLDistance_temp,BitDistance_temp = Scoring_Indexes(N,Final_Epigenetic_Factors,Cell_Type_Tracker)


        #HERE WE DO THE VARIANCE DATA ANALYSIS

        Orbit_Collection = Orbit_Array

        #Collect all orbits in function of time,


        #Maybe apply PCA to these orbits or keep them like this
        Time_Orbital_Collection = zeros(Float16,length(Times),Amount_Small_Perturbations*2^Splits*Amount_Large_Perturbations,M)
        Time_Orbital_Collection2 = zeros(Float16,length(Times),Amount_Large_Perturbations,Amount_Small_Perturbations*2^Splits,M)



        Orbit_Counter=0
        Larger_Orbit_Counter = 1
        for Orbits in Orbit_Collection
            #Here we do it for the variance
            Interpolated_Vector = Interpolator(Times,Orbits) #We do linear interpolation to have all the same timesteps
            for i=1:2^Splits #We put the different splitted cells all together with the same networks.
                Time_Orbital_Collection[:,Orbit_Counter*2^Splits+i,:] = Interpolated_Vector[:,2+(i-1)*M:1+i*M]
            end
            for k=1:2^Splits
                Time_Orbital_Collection2[:,Larger_Orbit_Counter,mod(Orbit_Counter,Amount_Small_Perturbations)+k,:] = Interpolated_Vector[:,2+M*(k-1):1+M*k]
            end 
            Orbit_Counter += 1
            if mod(Orbit_Counter,Amount_Small_Perturbations) == 0
                Larger_Orbit_Counter += 1
            end


            #Here we do it for the orbits
            Interpolated_Vector2 = Interpolator(Times_Orbit,Orbits)
            Interpolated_Orbits[Orbit_Counter] = permutedims(Interpolated_Vector2,[2,1])

        end
        #Calculate variance in gene expression in function of time



        Variance = zeros(Variance_Length)
        for i=1:Variance_Length
            Variance[i] = sum((Time_Orbital_Collection[i,:,:] .- mean(Time_Orbital_Collection[i,:,:],dims=1)).^2)/(M-1)
        end



        Spatial_Var = Coarsed_Variance(Variance_Length,Time_Orbital_Collection,10,Amount_Large_Perturbations*Amount_Small_Perturbations*2^Splits)
        aba = plot(Variance,label="Temporal",xlabel="Time",ylabel="Variance",title="Spatial and Temporal Variance of all Orbits")
        aba = plot!(Spatial_Var,label="Spatial")
        aba =vline!(SplitTime*[i for i=1:Splits],colour=:"red",linealpha=:0.3,label=false)
        display(aba)
        
        savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance/Spatial_Temporal_"*Copy_String*".png")
        #savefig("Data_All/Data_Folder/TestFile/TestImage.png")



        aca = plot(Variance[1:250],label="Temporal",xlabel="Time",ylabel="Variance",title="Spatial and Temporal Variance of all Orbits")
        aca = plot!(Spatial_Var[1:250],label="Spatial")
        aca =vline!(SplitTime*[i for i=1:Splits],colour=:"red",linealpha=:0.3,label=false)
        ylims!(0,max(maximum(Variance[3:250]),maximum(Spatial_Var[3:250]))*1.3)

        display(aca)
        
        savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance/Spatial_Temporal_Zoomed_"*Copy_String*".png")


        if false

            Seperate_Variance = zeros(Amount_Large_Perturbations,Variance_Length)
            for j=1:Amount_Large_Perturbations
                for i=1:Variance_Length
                    Seperate_Variance[j,i] = sum((Time_Orbital_Collection2[i,j,:,:] .- mean(Time_Orbital_Collection2[i,j,:,:],dims=1)).^2)/(M-1)
                end

            end

            ada = plot(Seperate_Variance[1,1:250],label=false,xlabel="Time",ylabel="Variance",title="Temporal Variance of Specific I.C.")
            for j=2:Amount_Large_Perturbations
                ada = plot!(Seperate_Variance[j,1:250],label=false)
            end
            display(ada)

            savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance/Temporal_Zoomed_IC_"*Copy_String*".png")


            aea = plot(Seperate_Variance[1,1:2500],label=false,xlabel="Time",ylabel="Variance",title="Temporal Variance of Specific I.C.")
            for j=2:Amount_Large_Perturbations
                aea = plot!(Seperate_Variance[j,1:2500],label=false)
            end
            display(aea)

            savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance/Temporal_IC_"*Copy_String*".png")

            Spatial_Sep = Coarsed_Variance(Variance_Length,Time_Orbital_Collection[:,1:Amount_Small_Perturbations*2^Splits,:],25,Amount_Small_Perturbations*2^Splits)
            afa = plot(Spatial_Sep[1:250],label=false,xlabel="Time",ylabel="Variance",title="Spatial Variance of Specific I.C.")
            for j=1:Amount_Large_Perturbations-1
                Spatial_Sep = Coarsed_Variance(Variance_Length,Time_Orbital_Collection[:,1+j*Amount_Small_Perturbations*2^Splits:(j+1)*Amount_Small_Perturbations*2^Splits,:],25,Amount_Small_Perturbations*2^Splits)

                afa = plot!(Spatial_Sep[1:250],label=false)
            end

            display(afa)

            savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance/Spatial_Zoomed_IC_"*Copy_String*".png")
        end




        #Perturbations = [0 for i=1:32]
        for kappa=1:Amount_Large_Perturbations
            println(kappa)
            display(Final_Cell_PCAScatterPlot(Orbit_Array,kappa)) #Don't really need this one
        end
        
        display(Final_Cell_PCAScatterPlot(Orbit_Array,false));
        
        Gene_Cells_Colour = [1,2,3,4]
        print("Before the Multi Big One")
        GC.gc()
        display(Multi_One_Generational_Plotter4(true,Duration,Length_Orbits,Opacity,Load_Data_String*"/Processed/Gen:"*Generation_String*"/Orbits/Genes_"*Copy_String*".gif",Amount_Large_Perturbations,Partial,Amount_Small_Perturbations,Interpolated_Orbits,Final_Epigenetic_Factors,Gene_Cells_Colour));
    






        Copy_String = string(1)
        Current_Network_String = "/Gen_"*Generation_String*"_"*Copy_String*".jld"
        Loaded_Data = load(Load_Data_String_Load*Current_Network_String)
        
        CArray = Loaded_Data["CArray"]
        ParameterMatrix = Loaded_Data["Network"]
        

        
        TimesWeCheck = [1,5,10,25,40,60,80,100,150,200,300,500,1000,4000,20000]
        TimeChecks = length(TimesWeCheck)
        Cell_Gatherings = zeros(TimeChecks,2,2^Splits,M)
        CycleAmount = zeros(TimeChecks)
        UniqueCycleAmount = zeros(TimeChecks)
        FixPointAmount = zeros(TimeChecks)
        
        Interpolated_Cells = Interpolator(TimesWeCheck,Orbit_Array[1])
        Interpolated_EPIs = Interpolator(TimesWeCheck,EpiThroughTime)
        for i=1:2^Splits
            Cell_Gatherings[:,1,i,:] = Interpolated_Cells[:,2+(i-1)*M:1+(i)*M]
            Cell_Gatherings[:,2,i,:] = Interpolated_EPIs[:,2+(i-1)*M:1+(i)*M]
        end
        
        
        Copy_String = string(1)
        Current_Network_String = "/Gen_"*Generation_String*"_"*Copy_String*".jld"
        Loaded_Data = load(Load_Data_String_Load*Current_Network_String)
        
        CArray = Loaded_Data["CArray"]
        ParameterMatrix = Loaded_Data["Network"]
        FixPoint_Positions = Array{Float64}(undef,0)
        Counter_Cycle = 0
        XPositionTol = 0
        
        
        
        
        for t = 1:TimeChecks
            Counter_Cycle = 0
            FixPoint_Positions = Array{Float64}(undef,0)
            VectorTest = []
            for k = 1:2^Splits
                PositionArray = Cell_Gatherings[t,:,k,:]
                
                Parameters = (ParameterMatrix, CArray) #We have to update the splits every rerun, rest stays same
                tspan = (Float32(0.0),100) #How long ODE will run
                
                prob = ODEProblem(Evolution_NoEpi,PositionArray,tspan,Parameters) #ODEdefining
                
                
                
                
                sol = solve(prob,BS3())
                aba = plot(sol.t,sol[1,1,:])
                for j=2:40
                    aba = plot!(sol.t,sol[1,j,:],legend=false,title=k)
                end
                #if t> 10
                #    display(aba)
                #end
                
                
                
                
                
                
                
                XVector = sol[1,:,end]
                EpiVector = sol[2,:,end]
                SumOfDerivates = sum(abs.(tanh.(40*(InvSqr*ParameterMatrix*XVector.+EpiVector.+CArray[:])) .- XVector))
                DerivativeTol = 0.1
                XPositionTol = 0.1
                
                #if t> 10   
                    #print(SumOfDerivates)
                #end
                
                if SumOfDerivates < DerivativeTol
                    if FixPoint_Positions == [] #Still empty
                        FixPoint_Positions = vcat(FixPoint_Positions,XVector)
                    else #Already has a point in it
                        NewFixPoint = true
                        #print("The distance to the fixpoints are:")
                        for i=1:length(FixPoint_Positions[1,:]) #Here we check over all the existing ones to see if it is not close to one already existing
                            #println(sqrt(sum((FixPoint_Positions[:,i] .- XVector).^2)))
                            if sqrt(sum((FixPoint_Positions[:,i] .- XVector).^2)) < XPositionTol
                                NewFixPoint = false
                            end    
                        end
                        if NewFixPoint #If it is a new fixpoint we add it in now
                            FixPoint_Positions = hcat(FixPoint_Positions,XVector)
                        end
                    end
                else
                    #Exception, we have a cycle? try to plot it
                    
                    CycleToleranceActive = 0.01
                    Active_Gene = zeros(M)
                    
                    prob2 = ODEProblem(Evolution_NoEpi,sol[:,:,end],(Float32(0.0),20),Parameters) #ODEdefining
                    sol2 = solve(prob2,BS3())
                    
                    for j=1:M
                        if abs(maximum(sol2[1,j,:].-mean(sol2[1,j,:]))) > CycleToleranceActive
                            Active_Gene[j] = 1
                        end
                    end
                    if VectorTest == []
                        VectorTest = vcat(VectorTest,Active_Gene)
                    else
                        New_Cycle = true
                        for tau in length(VectorTest[1,:])
                            if Active_Gene == VectorTest[:,tau]
                                New_Cycle = false
                            end
                        end
                        if New_Cycle
                            VectorTest = hcat(VectorTest,Active_Gene)
                        end
                    end
        
        
        
                    
                    Counter_Cycle += 1
                end
            end
            try FixPointAmount[t] = length(FixPoint_Positions[1,:])
        
            catch 
                FixPointAmount[t] = 0
            else
                FixPointAmount[t] = length(FixPoint_Positions[1,:])
            end
            CycleAmount[t] = Counter_Cycle
            try UniqueCycleAmount[t] = length(VectorTest[1,:])
        
            catch
                UniqueCycleAmount[t] = 0
            else
                UniqueCycleAmount[t] = length(VectorTest[1,:])
            end
        end
        
        aba = plot(TimesWeCheck,FixPointAmount,label="Fix Points")
        aba = plot!(TimesWeCheck,CycleAmount,label="Cycles (DoubleCounted)",xaxis=:log,title="PositionalTolerance="*string(XPositionTol))
        plot!(TimesWeCheck,UniqueCycleAmount,label="Cycles (Unique)")
        display(aba)

    end
end
    





function Multi_One_Generational_Plotter4(Cut_Off,Duration,Length_Orbits,Opacity,Name,Amount_Perturbations,Partial,N_Perturbed,Orbit_Array,Final_Epigenetic_Factors,Gene_Colour_Collection)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1:N_Perturbed*Amount_Perturbations

            if Shortest>size(Orbit_Array[i])[2]
                Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
            end
            Total_Length += size(Orbit_Array[i])[2]
        
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed*Amount_Perturbations)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1:N_Perturbed*Amount_Perturbations
        
        if Cut_Off

            for p=0:2^Splits-1

                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
            end
        else
            Length_Orbits =  size(Orbit_Array[i,j])[2]
            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
            end
        end

        Collection_Index += Length_Orbits*2^Splits

    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,Amount_Perturbations,N_Perturbed,2^Splits,3,Shortest)
    PCA_Orbits_Partial = zeros(Float32,Amount_Perturbations,Int(N_Perturbed/Partial),2^Splits,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:N_Perturbed
        for j=0:Amount_Perturbations-1
        for tau=1:2^Splits
            tester = Orbit_Array[i+N_Perturbed*j][2+(tau-1)*M:1+tau*M,1:Shortest]
            Transformed = transform(Orbital_Model,tester)
            PCA_Orbits_2Gen[j+1,i,tau,:,:] = Transformed
            if mod(i-1,Partial) == 0
                PCA_Orbits_Partial[j+1,(Int(i/Partial)),tau,:,:] = Transformed
            end
        end
        end

    end
    Colours = [:red,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
     
    #PCA_Orbits_Partial = reshape(PCA_Orbits_Partial,(Int(2^Splits*N_Perturbed*Amount_Perturbations/Partial),3,Shortest))
    PCA_Orbits_2Gen_Plot = permutedims(PCA_Orbits_Partial,(5,4,3,2,1))
    Orbit_Range = Int(N_Perturbed/Partial)*2^Splits
    
    print(size(PCA_Orbits_2Gen_Plot))
    if false
        for Gene_Colour in Gene_Colour_Collection
            tick()
            
            anim = @animate for k=1:Int(Duration/5)
                i=1
                PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1,1],c=Colours[Final_Epigenetic_Factors[1,1,1,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                for q=1:Int(N_Perturbed/Partial)
                    for p=1:2^Splits
                        for t =1:Amount_Perturbations
                        PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,q,t],PCA_Orbits_2Gen_Plot[1:k,2,p,q,t],PCA_Orbits_2Gen_Plot[1:k,3,p,q,t],c=Colours[Final_Epigenetic_Factors[t,q,p,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                        end
                    end
                end
        
                #for i=1:Int(N_Perturbed/5)
                #    for j=1:Amount_Perturbations
                #        for tau=1:2^Splits
                #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                #        end
                #    end
                #end
                
                #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])
        
                title!("Gene = $Gene_Colour Time = $k")
            end every 5
            tock()
            tick()
            animlong = @animate for k=1:Duration
                
                i=1
                PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1,1],c=Colours[Final_Epigenetic_Factors[1,1,1,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                for q=1:Int(N_Perturbed/Partial)
                    for p=1:2^Splits
                        for t =1:Amount_Perturbations
                        PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,q,t],PCA_Orbits_2Gen_Plot[1:k,2,p,q,t],PCA_Orbits_2Gen_Plot[1:k,3,p,q,t],c=Colours[Final_Epigenetic_Factors[t,q,p,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                        end
                    end
                end
                #for i=1:Int(N_Perturbed/5)
                #    for j=1:Amount_Perturbations
                #        for tau=1:2^Splits
                #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                #        end
                #    end
                #end
                
                #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])
        
                title!("Gene = $Gene_Colour Time = $k")
            end every 50
            tock()
    
            abra = gif(anim, Name[1:end-4]*"_Gene:"*string(Gene_Colour)*Name[end-3:end], fps = 10)
            display(abra)
        
            kadabra = gif(animlong, Name[1:end-4] * "_Long"*"_Gene:"*string(Gene_Colour) *Name[end-3:end], fps = 3)
            display(kadabra)
        
            
            Total_Ratio = principalratio(Orbital_Model)
            Eigen_Values = eigvals(Orbital_Model)
            Eigen_Values = Eigen_Values/sum(Eigen_Values)
            Eigen1 = Eigen_Values[1]
            Eigen2 = Eigen_Values[2]
            Eigen3 = Eigen_Values[3]
            PCAPlots_Final = scatter(PCA_Orbits_2Gen_Plot[end,1,1,1,:],PCA_Orbits_2Gen_Plot[end,2,1,1,:],PCA_Orbits_2Gen_Plot[end,3,1,1,:],c=Colours[Final_Epigenetic_Factors[1,1,1,Gene_Colour]+1],legend=true,label="
            Principal 1: $Eigen1
            Principal 2: $Eigen2]
            Principal 3: $Eigen3")
        
            for q=1:Int(N_Perturbed/Partial)
                for p=1:2^Splits
                    for t =1:Amount_Perturbations
                        PCAPlots_Final = scatter!((PCA_Orbits_2Gen_Plot[end,1,p,q,t],PCA_Orbits_2Gen_Plot[end,2,p,q,t],PCA_Orbits_2Gen_Plot[end,3,p,q,t]),c=Colours[Final_Epigenetic_Factors[t,q,p,Gene_Colour]+1],legend=true,label="")
                    end
                end
            end
            
            
            title!("Gene = $Gene_Colour , Ratio: $Total_Ratio")
            display(PCAPlots_Final)
            savefig(Name[1:end-4] * "_Final_Static_$Gene_Colour.png")
            
        end
    end
    Multi_Colours = [:cyan,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:darkmagenta,:lightcyan4,:olive,:gold4,:lightsalmon,:navajowhite,:pink]
    Only3Types = false


    
    
    
    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    PCAPlots_Final = scatter(PCA_Orbits_2Gen_Plot[end,1,1,1,:],PCA_Orbits_2Gen_Plot[end,2,1,1,:],PCA_Orbits_2Gen_Plot[end,3,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Final = scatter!((PCA_Orbits_2Gen_Plot[end,1,p,q,t],PCA_Orbits_2Gen_Plot[end,2,p,q,t],PCA_Orbits_2Gen_Plot[end,3,p,q,t]),c=Multi_Colours[Colour_Index],legend=false)
            end
        end
    end
    Total_Ratio = principalratio(Orbital_Model)
    Eigen_Values = eigvals(Orbital_Model)
    Eigen_Values = Eigen_Values/sum(Eigen_Values)
    title!("Static Multi Colour, Ratio: $Total_Ratio")
    display(PCAPlots_Final)
    savefig(Name[1:end-4] * "Final_Multi_Colour.png")



    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    
    PCAPlots_Time1 = plot([PcaPlots_Time_Start:PcaPlots_Time_Length],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,1,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Time1 = plot!([PcaPlots_Time_Start:PcaPlots_Time_Length],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,1,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA 1")
            end
        end
    end
    display(PCAPlots_Time1)

    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    
    PCAPlots_Time2 = plot([PcaPlots_Time_Start:PcaPlots_Time_Length],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,2,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Time2 = plot!([PcaPlots_Time_Start:PcaPlots_Time_Length],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,2,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA 2")
            end
        end
    end
    display(PCAPlots_Time2)

    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    
    PCAPlots_Time3 = plot([PcaPlots_Time_Start:PcaPlots_Time_Length],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,3,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Time3 = plot!([PcaPlots_Time_Start:PcaPlots_Time_Length],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,3,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA 3")
            end
        end
    end
    display(PCAPlots_Time3)



    for Colouring =1:16
        Plotting_Something = false
        PCAPlots_TimeFilter = plot(zeros(PcaPlots_Time_Start:PcaPlots_Time_Length))
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
        if Only3Types
            Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
        end

        if Colour_Index == Colouring
            Plotting_Something = true
            PCAPlots_TimeFilter = plot!([PcaPlots_Time_Start:PcaPlots_Time_Length],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,SeperatePCAIndex,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
        end
        for q=1:Int(N_Perturbed/Partial)
            for p=1:2^Splits
                for t =1:Amount_Perturbations
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                    if Only3Types
                        Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                    end
                    if Colour_Index == Colouring
                        Plotting_Something = true
                        PCAPlots_TimeFilter = plot!([PcaPlots_Time_Start:PcaPlots_Time_Length],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,SeperatePCAIndex,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA Colour Index = $(Colouring)")
                    end
                end
            end
        end
        if Plotting_Something
            display(PCAPlots_TimeFilter)
        end
        
    end

    PcaDim1 = 1
    PcaDim2 = 2
    PCAPlots_TimeFilter = plot([0])
    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end

    
    PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,1,1,:],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,1,1,:],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end

                PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,p,q,t],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,p,q,t],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
                
            end
        end
    end
    display(PCAPlots_TimeFilter)

    PcaDim1 = 2
    PcaDim2 = 3
    PCAPlots_TimeFilter = plot([0])
    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end

    
    PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,1,1,:],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,1,1,:],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end

                PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,p,q,t],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,p,q,t],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
                
            end
        end
    end
    display(PCAPlots_TimeFilter)

    PcaDim1 = 1
    PcaDim2 = 3
    PCAPlots_TimeFilter = plot([0])
    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end

    
    PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,1,1,:],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,1,1,:],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end

                PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,p,q,t],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,p,q,t],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
                
            end
        end
    end
    display(PCAPlots_TimeFilter)

    
    PcaDim1 = 1
    PcaDim2 = 2
    
    for Colouring =1:16
        Plotting_Something = false
        PCAPlots_TimeFilter = plot([0])
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
        if Only3Types
            Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
        end

        if Colour_Index == Colouring
            Plotting_Something = true
            PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,1,1,:],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,1,1,:],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
        end
        for q=1:Int(N_Perturbed/Partial)
            for p=1:2^Splits
                for t =1:Amount_Perturbations
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                    if Only3Types
                        Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                    end
                    if Colour_Index == Colouring
                        Plotting_Something = true
                        PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,p,q,t],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA Colour Index = $(Colouring)",opacity=0.1)
                    end
                end
            end
        end
        if Plotting_Something
            display(PCAPlots_TimeFilter)
        end
        
    end

    PcaDim1 = 2
    PcaDim2 = 3
    
    for Colouring =1:16
        Plotting_Something = false
        PCAPlots_TimeFilter = plot([0])
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
        if Only3Types
            Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
        end

        if Colour_Index == Colouring
            Plotting_Something = true
            PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,1,1,:],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,1,1,:],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
        end
        for q=1:Int(N_Perturbed/Partial)
            for p=1:2^Splits
                for t =1:Amount_Perturbations
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                    if Only3Types
                        Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                    end
                    if Colour_Index == Colouring
                        Plotting_Something = true
                        PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,p,q,t],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA Colour Index = $(Colouring)",opacity=0.1)
                    end
                end
            end
        end
        if Plotting_Something
            display(PCAPlots_TimeFilter)
        end
        
    end

    PcaDim1 = 1
    PcaDim2 = 3
    
    for Colouring =1:16
        Plotting_Something = false
        PCAPlots_TimeFilter = plot([0])
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
        if Only3Types
            Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
        end

        if Colour_Index == Colouring
            Plotting_Something = true
            PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,1,1,:],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,1,1,:],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
        end
        for q=1:Int(N_Perturbed/Partial)
            for p=1:2^Splits
                for t =1:Amount_Perturbations
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                    if Only3Types
                        Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                    end
                    if Colour_Index == Colouring
                        Plotting_Something = true
                        PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,p,q,t],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA Colour Index = $(Colouring)",opacity=0.1)
                    end
                end
            end
        end
        if Plotting_Something
            display(PCAPlots_TimeFilter)
        end
        
    end

    return Orbital_Model
    
end


Duration = 1000 #For Animation should be 3000
Length_Orbits = 3000 #For PCA should be 4000
Opacity = 75
Partial = 1

Variance_Length = 2500
Times = [i for i=1:Variance_Length+50]
Times_Orbit = [i for i=1:Length_Orbits]


PcaPlots_Time_Start = 100
PcaPlots_Time_Length = 600
SeperatePCAIndex = 3

Orbital_Model = Multi_One_Generational_Plotter4(true,Duration,Length_Orbits,Opacity,Load_Data_String*"/Processed/Gen:"*Generation_String*"/Orbits/Genes_"*Copy_String*".gif",Amount_Large_Perturbations,Partial,Amount_Small_Perturbations,Interpolated_Orbits,Final_Epigenetic_Factors,Gene_Cells_Colour);


#TimesWeCheck = [1,5,10,25,40,60,80,100,150,200,300,500,1000,4000,20000]
TimesWeCheck = [50,100,150,200,300,400,500]
TimeChecks = length(TimesWeCheck)


Cell_Gatherings = zeros(TimeChecks,2,2^Splits,M)
CycleAmount = zeros(TimeChecks)
UniqueCycleAmount = zeros(TimeChecks)
FixPointAmount = zeros(TimeChecks)

Interpolated_Cells = Interpolator(TimesWeCheck,Orbit_Array[1])
Interpolated_EPIs = Interpolator(TimesWeCheck,EpiThroughTime)
for i=1:2^Splits
    Cell_Gatherings[:,1,i,:] = Interpolated_Cells[:,2+(i-1)*M:1+(i)*M]
    Cell_Gatherings[:,2,i,:] = Interpolated_EPIs[:,2+(i-1)*M:1+(i)*M]
end


Copy_String = string(1)
Current_Network_String = "/Gen_"*Generation_String*"_"*Copy_String*".jld"
Loaded_Data = load(Load_Data_String_Load*Current_Network_String)

CArray = Loaded_Data["CArray"]
ParameterMatrix = Loaded_Data["Network"]
FixPoint_Positions = Array{Float64}(undef,0)
Counter_Cycle = 0
XPositionTol = 0
Full_TimeSpan = 200
sol = 0
Sampling_Amounts = 4
Transformed_Data =  zeros(Float32,TimeChecks,2^Splits,3,Full_TimeSpan,Sampling_Amounts)

for q=1:Sampling_Amounts
    Saved_Orbits_All = zeros(Float16,TimeChecks,2^Splits,Full_TimeSpan,M)
    for t = 1:TimeChecks
        Counter_Cycle = 0
        FixPoint_Positions = Array{Float64}(undef,0)
        VectorTest = []
        for k = 1:2^Splits
            PositionArray = Cell_Gatherings[t,:,k,:]
            
            Parameters = (ParameterMatrix, CArray) #We have to update the splits every rerun, rest stays same
            tspan = (Float32(0.0),Full_TimeSpan) #How long ODE will run
            
            prob = SDEProblem(Evolution_NoEpi,σ_Genes,PositionArray,tspan,Parameters) #ODEdefining
            
            
            
            
            sol = solve(prob,SRA3(),dtmin=0.1,force_dtmin=true)
            if false 
                aba = plot(sol.t,sol[1,1,:])
                for j=2:40
                    aba = plot!(sol.t,sol[1,j,:],legend=false,title=TimesWeCheck[t])
                end
                if t> 1 && (k==1) 
                    
                    display(aba)
                end
            end
            for time=1:Full_TimeSpan
                Saved_Orbits_All[t,k,time,:] = sol(time)[1,:]
            end
            
            
            
            
            XVector = sol[1,:,end]
            EpiVector = sol[2,:,end]
            SumOfDerivates = sum(abs.(tanh.(40*(InvSqr*ParameterMatrix*XVector.+EpiVector.+CArray[:])) .- XVector))
            DerivativeTol = 0.1
            XPositionTol = 0.1
            
        end
            
    
    
    end

    for i=1:TimeChecks  
        for tau=1:2^Splits
            Tested_Transform = transform(Orbital_Model,Saved_Orbits_All[i,tau,:,:]')        
            Transformed_Data[i,tau,:,:,q] = Tested_Transform
        end
    end
    
end
## Save somehow the data/orbits of these frozen versions somewhere


# Plot this for the PCA as their axis


Multi_Colours = [:cyan,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:darkmagenta,:lightcyan4,:olive,:gold4,:lightsalmon,:navajowhite,:pink]


function PCA_Individual_Colour_Plotting(PcaDim1,PcaDim2,Time_Index_To_Plot)
    for Colouring =1:16
        Plotting_Something = false
        PCAPlots_TimeFilter = plot([0])
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    
    
        if Colour_Index == Colouring
            Plotting_Something = true
            PCAPlots_TimeFilter = plot!(Transformed_Data[Time_Index_To_Plot,1,PcaDim1,:,1],Transformed_Data[Time_Index_To_Plot,1,PcaDim2,:,1],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
        end
        for q=1:Sampling_Amounts
            for p=1:2^Splits
                for t =1:1
                    Colour_Index = Final_Epigenetic_Factors[t,1,p,1]+Final_Epigenetic_Factors[t,1,p,2]*2+Final_Epigenetic_Factors[t,1,p,3]*4+Final_Epigenetic_Factors[t,1,p,4]*8+1
    
                    if Colour_Index == Colouring
                        Plotting_Something = true
                        PCAPlots_TimeFilter = plot!(Transformed_Data[Time_Index_To_Plot,p,PcaDim1,:,q],Transformed_Data[Time_Index_To_Plot,p,PcaDim2,:,q],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1,title="Final Type Index $(Colouring) Time = $(TimesWeCheck[Time_Index_To_Plot])")
                    end
                end
            end
        end
        if Plotting_Something
            display(PCAPlots_TimeFilter)
        end
        
    end
end
PCADim1 = 1
PCADim2 = 2
for t in [1,4,5,6,7]
    PCA_Individual_Colour_Plotting(PCADim1,PCADim2,t)
end

#TimesWeCheck = [1,5,10,25,40,60,80,100,150,200,300,500,1000,4000,20000]
TimesWeCheck = [50,100,150,200,300,320,340,360,380,400,500,600,700,800,1000,1500,2000]
TimeChecks = length(TimesWeCheck)


Cell_Gatherings = zeros(TimeChecks,2,2^Splits,M)
CycleAmount = zeros(TimeChecks)
UniqueCycleAmount = zeros(TimeChecks)
FixPointAmount = zeros(TimeChecks)

Interpolated_Cells = Interpolator(TimesWeCheck,Orbit_Array[1])
Interpolated_EPIs = Interpolator(TimesWeCheck,EpiThroughTime)
for i=1:2^Splits
    Cell_Gatherings[:,1,i,:] = Interpolated_Cells[:,2+(i-1)*M:1+(i)*M]
    Cell_Gatherings[:,2,i,:] = Interpolated_EPIs[:,2+(i-1)*M:1+(i)*M]
end


Copy_String = string(1)
Current_Network_String = "/Gen_"*Generation_String*"_"*Copy_String*".jld"
Loaded_Data = load(Load_Data_String_Load*Current_Network_String)

CArray = Loaded_Data["CArray"]
ParameterMatrix = Loaded_Data["Network"]
FixPoint_Positions = Array{Float64}(undef,0)
Counter_Cycle = 0
XPositionTol = 0
Full_TimeSpan = 200
sol = 0
Sampling_Amounts = 8
Transformed_Data =  zeros(Float32,TimeChecks,2^Splits,3,Full_TimeSpan,Sampling_Amounts)

for q=1:Sampling_Amounts
    Saved_Orbits_All = zeros(Float16,TimeChecks,2^Splits,Full_TimeSpan,M)
    for t = 1:TimeChecks
        Counter_Cycle = 0
        FixPoint_Positions = Array{Float64}(undef,0)
        VectorTest = []
        for k = 1:2^Splits
            PositionArray = Cell_Gatherings[t,:,k,:]
            
            Parameters = (ParameterMatrix, CArray) #We have to update the splits every rerun, rest stays same
            tspan = (Float32(0.0),Full_TimeSpan) #How long ODE will run
            
            prob = SDEProblem(Evolution_NoEpi,σ_Genes,PositionArray,tspan,Parameters) #ODEdefining
            
            
            
            
            sol = solve(prob,SRA3(),dtmin=0.1,force_dtmin=true)
            if false 
                aba = plot(sol.t,sol[1,1,:])
                for j=2:40
                    aba = plot!(sol.t,sol[1,j,:],legend=false,title=TimesWeCheck[t])
                end
                if t> 1 && (k==1) 
                    
                    display(aba)
                end
            end
            for time=1:Full_TimeSpan
                Saved_Orbits_All[t,k,time,:] = sol(time)[1,:]
            end
            
            
            
            
            XVector = sol[1,:,end]
            EpiVector = sol[2,:,end]
            SumOfDerivates = sum(abs.(tanh.(40*(InvSqr*ParameterMatrix*XVector.+EpiVector.+CArray[:])) .- XVector))
            DerivativeTol = 0.1
            XPositionTol = 0.1
            
        end
            
    
    
    end

    for i=1:TimeChecks  
        for tau=1:2^Splits
            Tested_Transform = transform(Orbital_Model,Saved_Orbits_All[i,tau,:,:]')        
            Transformed_Data[i,tau,:,:,q] = Tested_Transform
        end
    end
    
end
## Save somehow the data/orbits of these frozen versions somewhere


# Plot this for the PCA as their axis

Multi_Colours = [:cyan,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:darkmagenta,:lightcyan4,:olive,:gold4,:lightsalmon,:navajowhite,:pink]


function PCA_Individual_Colour_Plotting(PcaDim1,PcaDim2,Time_Index_To_Plot)
    for Colouring =1:16
        Plotting_Something = false
        PCAPlots_TimeFilter = plot([0])
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    
    
        if Colour_Index == Colouring
            Plotting_Something = true
            PCAPlots_TimeFilter = plot!(Transformed_Data[Time_Index_To_Plot,1,PcaDim1,:,1],Transformed_Data[Time_Index_To_Plot,1,PcaDim2,:,1],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
            PCAPlots_TimeFilter = scatter!((Transformed_Data[Time_Index_To_Plot,1,PcaDim1,end,1],Transformed_Data[Time_Index_To_Plot,1,PcaDim2,end,1]),c=Multi_Colours[Colour_Index])

        end
        for q=1:Sampling_Amounts
            for p=1:2^Splits
                for t =1:1
                    Colour_Index = Final_Epigenetic_Factors[t,1,p,1]+Final_Epigenetic_Factors[t,1,p,2]*2+Final_Epigenetic_Factors[t,1,p,3]*4+Final_Epigenetic_Factors[t,1,p,4]*8+1
    
                    if Colour_Index == Colouring
                        Plotting_Something = true
                        PCAPlots_TimeFilter = plot!(Transformed_Data[Time_Index_To_Plot,p,PcaDim1,:,q],Transformed_Data[Time_Index_To_Plot,p,PcaDim2,:,q],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1,title="Final Type Index $(Colouring) Time = $(TimesWeCheck[Time_Index_To_Plot])")
                        PCAPlots_TimeFilter = scatter!((Transformed_Data[Time_Index_To_Plot,p,PcaDim1,end,q],Transformed_Data[Time_Index_To_Plot,p,PcaDim2,end,q]),c=Multi_Colours[Colour_Index])

                    end
                end
            end
        end
        if Plotting_Something
            display(PCAPlots_TimeFilter)
        end
        
    end
end
PCADim1 = 1
PCADim2 = 2
for t in [1,4,5,6,7]
    PCA_Individual_Colour_Plotting(PCADim1,PCADim2,t)
end



#TimesWeCheck = [1,5,10,25,40,60,80,100,150,200,300,500,1000,4000,20000]
TimesWeCheck = [50,100,150,200,300,320,340,360,380,400,450,500,550,600,650,700,750,1000,1500,2000]
TimeChecks = length(TimesWeCheck)


Cell_Gatherings = zeros(TimeChecks,2,2^Splits,M)
CycleAmount = zeros(TimeChecks)
UniqueCycleAmount = zeros(TimeChecks)
FixPointAmount = zeros(TimeChecks)

Interpolated_Cells = Interpolator(TimesWeCheck,Orbit_Array[1])
Interpolated_EPIs = Interpolator(TimesWeCheck,EpiThroughTime)
for i=1:2^Splits
    Cell_Gatherings[:,1,i,:] = Interpolated_Cells[:,2+(i-1)*M:1+(i)*M]
    Cell_Gatherings[:,2,i,:] = Interpolated_EPIs[:,2+(i-1)*M:1+(i)*M]
end


Copy_String = string(1)
Current_Network_String = "/Gen_"*Generation_String*"_"*Copy_String*".jld"
Loaded_Data = load(Load_Data_String_Load*Current_Network_String)

CArray = Loaded_Data["CArray"]
ParameterMatrix = Loaded_Data["Network"]
FixPoint_Positions = Array{Float64}(undef,0)
Counter_Cycle = 0
XPositionTol = 0
Full_TimeSpan = 200
sol = 0
Sampling_Amounts = 8
Transformed_Data =  zeros(Float32,TimeChecks,2^Splits,3,Full_TimeSpan,Sampling_Amounts)
Saved_Orbits_All = 0
for q=1:Sampling_Amounts
    Saved_Orbits_All = zeros(Float16,TimeChecks,2^Splits,Full_TimeSpan,M)
    for t = 1:TimeChecks
        Counter_Cycle = 0
        FixPoint_Positions = Array{Float64}(undef,0)
        VectorTest = []
        for k = 1:2^Splits
            PositionArray = Cell_Gatherings[t,:,k,:]
            
            Parameters = (ParameterMatrix, CArray) #We have to update the splits every rerun, rest stays same
            tspan = (Float32(0.0),Full_TimeSpan) #How long ODE will run
            
            prob = SDEProblem(Evolution_NoEpi,σ_Genes,PositionArray,tspan,Parameters) #ODEdefining
            
            
            
            
            sol = solve(prob,SRA3(),dtmin=0.1,force_dtmin=true)
            if false 
                aba = plot(sol.t,sol[1,1,:])
                for j=2:40
                    aba = plot!(sol.t,sol[1,j,:],legend=false,title=TimesWeCheck[t])
                end
                if t> 1 && (k==1) 
                    
                    display(aba)
                end
            end
            for time=1:Full_TimeSpan
                Saved_Orbits_All[t,k,time,:] = sol(time)[1,:]
            end
            
            
            
            
            XVector = sol[1,:,end]
            EpiVector = sol[2,:,end]
            SumOfDerivates = sum(abs.(tanh.(40*(InvSqr*ParameterMatrix*XVector.+EpiVector.+CArray[:])) .- XVector))
            DerivativeTol = 0.1
            XPositionTol = 0.1
            
        end
            
    
    
    end

    for i=1:TimeChecks  
        for tau=1:2^Splits
            Tested_Transform = transform(Orbital_Model,Saved_Orbits_All[i,tau,:,:]')        
            Transformed_Data[i,tau,:,:,q] = Tested_Transform
        end
    end
    
end
## Save somehow the data/orbits of these frozen versions somewhere


# Plot this for the PCA as their axis

Multi_Colours = [:cyan,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:darkmagenta,:lightcyan4,:olive,:gold4,:lightsalmon,:navajowhite,:pink]




PCADim1 = 1
PCADim2 = 2





function PCA_Individual_Colour_Plotting(PcaDim1,PcaDim2,Time_Index_To_Plot,Colour_Index)
    Colour_Counter = 0
    for Colouring =Colour_Index:Colour_Index
        Plotting_Something = false
        plot([0])
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    
    
        if Colour_Index == Colouring
            Colour_Counter = mod(Colour_Counter,16) + 1
            Plotting_Something = true
            PCAPlots_TimeFilter = plot!(Transformed_Data[Time_Index_To_Plot,1,PcaDim1,:,1],Transformed_Data[Time_Index_To_Plot,1,PcaDim2,:,1],c=Multi_Colours[Colour_Counter],legend=false,opacity=0.2)
            PCAPlots_TimeFilter = scatter!((Transformed_Data[Time_Index_To_Plot,1,PcaDim1,end,1],Transformed_Data[Time_Index_To_Plot,1,PcaDim2,end,1]),c=Multi_Colours[Colour_Counter])

        end
        for q=1:Sampling_Amounts
            for p=1:2^Splits
                for t =1:1
                    Colour_Index = Final_Epigenetic_Factors[t,1,p,1]+Final_Epigenetic_Factors[t,1,p,2]*2+Final_Epigenetic_Factors[t,1,p,3]*4+Final_Epigenetic_Factors[t,1,p,4]*8+1
    
                    if Colour_Index == Colouring
                        Colour_Counter = mod(Colour_Counter,16) + 1
                        Plotting_Something = true
                        PCAPlots_TimeFilter = plot!(Transformed_Data[Time_Index_To_Plot,p,PcaDim1,:,q],Transformed_Data[Time_Index_To_Plot,p,PcaDim2,:,q],c=Multi_Colours[Colour_Counter],legend=false,opacity=0.2,title="Final Type Index $(Colouring) Time = $(TimesWeCheck[Time_Index_To_Plot])")
                        #PCAPlots_TimeFilter = scatter!((Transformed_Data[Time_Index_To_Plot,p,PcaDim1,end,q],Transformed_Data[Time_Index_To_Plot,p,PcaDim2,end,q]),c=Multi_Colours[Colour_Index])

                    end
                end
            end
        end
        if Plotting_Something
            display(PCAPlots_TimeFilter)
        end
        
        xlims!((-3,2))
        ylims!((-4,1))

    end
end






for t in [1,4,5,6,7]
    PCA_Individual_Colour_Plotting(PCADim1,PCADim2,t,3)
    PCA_Individual_Colour_Plotting(PCADim1,PCADim2,t,4)

end



In [ ]:
using ForwardDiff
import ForwardDiff.jacobian

Fine_Graining = 50
TimesWeCheck = [10,20,30,40,45,50,55,60,65,70,75,80,85,90,95,100,110,120,140,160,180,200,250,300,2000]
TimeChecks = length(TimesWeCheck)
Time_Index = 3
Cell = 1
Cell_Gatherings = zeros(TimeChecks,2,2^Splits,M)

Interpolated_Cells = Interpolator(TimesWeCheck,Orbit_Array[1])
Interpolated_EPIs = Interpolator(TimesWeCheck,EpiThroughTime)
for i=1:2^Splits
    Cell_Gatherings[:,1,i,:] = Interpolated_Cells[:,2+(i-1)*M:1+(i)*M]
    Cell_Gatherings[:,2,i,:] = Interpolated_EPIs[:,2+(i-1)*M:1+(i)*M]
end

Interpolated_Epi = Interpolator([i for i=1:Length_Orbits],EpiThroughTime)

Interpolated_Epi_Reshaped = reshape(Interpolated_Epi[:,2:end], (Length_Orbits,M,2^Splits))

Interpolated_Orbits_Reshaped = reshape(Interpolated_Orbits[1][2:end,:]', (Length_Orbits,M,2^Splits));

function Gene_ODE(u)
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    Epi_Factors = Parameters[3]
    #vector = Vector{Float16}([])
    #append!(vector,tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1])
    vector = zero.(u)
    for c=1:M
        vector[c] = tanh(40*(InvSqr*ParameterMatrix[c,:]'*u[:]+Epi_Factors[c]+CArray[c])) - u[c]
    end

    return vector
end  
Amount_Lyapunov_Steps = 200
Lyapunov_Mean = zeros(TimeChecks,Amount_Lyapunov_Steps)
Jacobian_Sampling_Points_Multiplied = 0
DeltaX_ThroughTime = zeros(Amount_Lyapunov_Steps,M)

Parameters = [ParameterMatrix,CArray,Interpolated_Epi_Reshaped[Time_Index,:,Cell]]

TimesWeCheck = [10,30,50,70,90,120,150,200,300,500,1000,3000]
function Gene_ODE(u)
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    Epi_Factors = Parameters[3]
    #vector = Vector{Float16}([])
    #append!(vector,tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1])
    vector = zero.(u)
    for c=1:M
        vector[c] = tanh(40*(InvSqr*ParameterMatrix[c,:]'*u[:]+Epi_Factors[c]+CArray[c])) - u[c]
    end

    return vector
end  
Delta_T = 10^-3
D0 = 10^-6
Amount_Lyapunov_Steps = 500000
Norm_Saved = zeros(length(TimesWeCheck),Amount_Lyapunov_Steps)
Moving_Average = zeros(length(TimesWeCheck),Amount_Lyapunov_Steps)
Cell = 2

for t=1:length(TimesWeCheck)
    Time_Index=t
    Delta_X = D0*ones(M)/sqrt(M)
    Parameters = [ParameterMatrix,CArray,Interpolated_Epi_Reshaped[TimesWeCheck[Time_Index],:,Cell]]
    x0 = Interpolated_Orbits[1][2+(Cell-1)*M:1+Cell*M,TimesWeCheck[Time_Index]]
    
    for j=1:Amount_Lyapunov_Steps
        Stepx0 = Gene_ODE(x0)
        StepDeltax0 = Gene_ODE(x0+Delta_X)
    
    
        Delta_X = Delta_X + (StepDeltax0 - Stepx0)*Delta_T
        x0 += Stepx0*Delta_T
        Norm_Saved[t,j] = log(sqrt(sum(Delta_X.^2))/D0)
        if j==1
            Moving_Average[t,j] = Norm_Saved[t,j]
        else
            Moving_Average[t,j] = (Moving_Average[t,j-1]*(j-1) + Norm_Saved[t,j])/j
        end
        Delta_X /= sqrt(sum(Delta_X.^2))/D0
    end
    Start_Pont = 100
    TimeToString = TimesWeCheck[Time_Index]
    ada = plot([j*Delta_T for j=Start_Pont:Amount_Lyapunov_Steps],Moving_Average[t,Start_Pont:end],title="Time $TimeToString Cell = $Cell")
    ada = hline!((0,0))
    display(ada)
end



In [ ]:
plot(TimesWeCheck,Moving_Average[:,end],xaxis=:log10)
hline!((0,0))

# 0.1 Noise, type Ib

In [ ]:
#### SMALL PERT IS SET TO 1 WE ONLY HAVE 1 SAMPLE NOW
### WE MESSED WITH SOME DATA_TO_ANALYSE and the loading ones

#Data_To_Analyse = "21_12_NoSplit_HighODE_SplitTime=0.1_2000Gen_Noise01"
#Data_To_Analyse = ARGS[1]
#Noise = parse(Float64, ARGS[2])
#Data_To_Analyse = "21_12_NoSplit_HighODE_SplitTime=0.1_2000Gen_Noise005_Mut=2"
#Noise = 0.005

Noise = 0.1
Data_To_Analyse = "07_01_NoSplit_HighODE_SplitTime=0.1_2000Gen_Noise0.1"

Load_Generations = [325] #525 also an option

Epi_Limiter = 1
a::Float32 = 1.0*Epi_Limiter   #Ratio of gene to epigenetic factor
v::Float32 = 6*10^-4/Epi_Limiter  #Speed at which epigenetic factor changes
#v::Float32 = 0  #Speed at which epigenetic factor changes

Load_Data_String = Data_To_Analyse*"Epi0.1"



println("OUTPUTTING MY INPUT")
println(Data_To_Analyse)
println(Noise)

using Random
using Plots
using Distributions
using DifferentialEquations
using Statistics
using ProgressMeter
using TickTock
using LinearAlgebra
using StatsBase
using LaTeXStrings
using MultivariateStats
using Dates
using Distances
using JLD
using KernelDensity
using Plots
using StatsPlots




function Setting_Up_Randomly(M,Sparse,rng=MersenneTwister(123))
    "Function used to randomly with seed define our c_i's, J_ij's and the initial positions
    Input:
    M      = Amount of Genes in our system
    Sparse = Bool to determine if we have a sparse graph
    rng    = Random Julia seed to make sure we can recreate the same initial conditions
    Output:
    CArray          = C_i's used, can only be between -0.5 and 0.5 now
    PositionArray   = Random Initial position, between -1 and 1 for all genes, epifactors = 0
    ParameterMatrix = J_ij's Currently a dense network with values between -0.5 and 0.5
    "
    ParameterMatrix = zeros(M,M)
    CArray = zeros(1,M)
    PositionArray = zeros(2,1,M)
    
    CArray = rand(rng,Float32,M).-0.5; #Random c_j between -0,5 and 0.5
    PositionArray[1,1,1:M] = 2*(rand(rng,Float32,M).-0.5); #Random Initial position 
    ParameterMatrix = 2 * bitrand(rng,M,M).-1; #J_jk's
    if Sparse
       for i=1:M
            for j=1:M
                if rand(rng) > 5/M #Average of 5 connections per gene

                    ParameterMatrix[i,j] = 0 
                end
            end
        end
    end
    return CArray, PositionArray, ParameterMatrix
end







function Noisy_Splitting(solution,Total_Splits,M,Seed)
    " Once our cell decides to split, we decide to introduce some random noise in how these genes get distributed.
    This function takes the cells, doubles them and then gives the daughters cell the same gene parameters +- noise
    one daughter gets + the other gets -. For the epigeneitc factors they are simply inherited and do not see any noise
    Input:
    solution     = These are the initial conditions of the parent cells that we need to split 
    (First genes of all cells, then all epigenetic factors)
    Total_Splits = Amount of times the cell has split (including this current split)
    M            = Amount of Genes
    Seed         = Random Noise seed for reproducing results
    Output:
    SplittedArray = New array of the genes and epigenetic factors of the daughter cells
    How is this structured? Suppose we take M=3 for convenience, we would have [g1,g2,g3,e1,e2,e3] for the first parent cell
    The daughters become: [g11,g12,g13,g21,g22,g23,e11,e12,e13,e21,e22,e23]
    Where we have genes of daughter 1, then genes of daughter 2, then epi of daughter 1 and epi of daughter 2.
    When the Daughters split we get the genes of Children of cell 1, then genes of Children of cell 2 and same for epi.
    "
    Random.seed!(Seed)
    s = Total_Splits #For shortening
    SplittedArray = zeros(Float32,2,2^(s),M) #2^(s+1), 2M is the normal length, 4M after 1 split, 8M after 2 splits etc.
    for g=1:2^(s-1) #looping over the length of the parent
        for t=1:M
            Gaussian = Normal{Float32}(0.0f0,0.0f0)
            Displace = rand(Gaussian) #Noise for split
            #index = Int32.(floor((g-1)/M)*M +g) #example M=10: Index 1-> 1 and 11, index 2->2 and 12 ... 
            #index 10->10 and 20, index 11->21 and 30 and so on. This is old method

            #Genes
            SplittedArray[1,2*g-1,t] = solution[1,g,t] + Displace
            SplittedArray[1,2*g,t] = solution[1,g,t] - Displace
            
        end
        #Epigenetic
        SplittedArray[2,2*g-1,:] = solution[2, g,:]
        SplittedArray[2,2*g,:] = solution[2, g,:]   
    end

    return SplittedArray  
end

function Differentiaton_Detection_Parameter(M,Epigenetic_Plot)
    "Function used to convert epigenetic factors into cell types and bits
    Input:
    M               = Amount of Genes 
    Epigenetic_Plot = Array of epigenetic factors which have to be converted to 0 and 1 representing on or off
    Output:
    b        = Array of scalar values representing cell types
    BitGenes = Array of vectors representing the genes of the cell types
    "
    b = Int128[]
    Counter = Int[]
    Epi_Bitwise = signbit.(-Epigenetic_Plot)

    
    BitGenes = [] #We will put the bit notation of the gene in here
    #Later rewrite to let this replace Bit_Represent
    
    for i=1:length(Epi_Bitwise[:,1])
        Bit_Represent::Int128 = 0 #CAREFUL WITH M>127
        for j=1:M
            Bit_Represent += Int128(Epi_Bitwise[i,j]*2)^(j-1)
        end
        #print(Bit_Represent)
        #print("     ")
        if !(Bit_Represent in b)

            push!(BitGenes,collect(Epi_Bitwise[i,:]))
            push!(Counter,1)
        end
        push!(b,Bit_Represent)

        
    end
    
    #print(b)
    Counter = sort(Counter,rev=true)
    #print(Counter)
    Score = 0
    #Make here the formula to turn b and the counter into a score


    return b,BitGenes
end

function Entropy_Score(M,Epigenetic_Plot)
    "Function used to convert epigenetic factors into cell types and bits to then calculate the entropic score.
    See previous function for alternative, this is currently not in use.
    Input:
    M               = Amount of Genes 
    Epigenetic_Plot = Array of epigenetic factors which have to be converted to 0 and 1 representing on or off
    Output:
    Score = Entropy score of the cell types
    "
    b = Int[]
    Counter = Int[]
    Epi_Bitwise = signbit.(-Epigenetic_Plot)
    
    for i=1:length(Epi_Bitwise[:,1])
        Bit_Represent::Int128 = 0 #CAREFUL WITH M>127
        for j=1:M
            Bit_Represent += Int128(Epi_Bitwise[i,j]*2)^(j-1)
        end
        #print(Bit_Represent)
        #print("     ")
        if !(Bit_Represent in b)
            push!(b,Bit_Represent)
            push!(Counter,1)
        else 
            Counter[findfirst(==(Bit_Represent), b)] += 1
        end
        
    end
    
    #print(b)
    Counter = sort(Counter,rev=true)
    #print(Counter)
    Score = 0
    #Make here the formula to turn b and the counter into a score
    for i=1:length(b)
       prob = Counter[i]/M
       Score -= prob*log(prob)
    end
    
    return Score
end



function Mutate_Parameters(ParameterMatrix_Mut,M,Mutations,Seed)
    "Function used to mutate the network (we turn a 1 or -1 to 0 and we turn a 0 to 1 or -1)
    Input:
    ParameterMatrix_Mut = Original Network that we want to mutate
    M                   = Amount of Genes 
    Mutations           = How many connections will we mutate
    Seed                = Randomness seed
    Output:
    ParameterMatrix_Mut = Mutated Network
    "
    Zero_Indices = findall(iszero,ParameterMatrix_Mut)
    One_Indices = findall(!iszero,ParameterMatrix_Mut)
    Random.seed!(Seed)
   
    for j=1:Mutations 
        t_zero = rand(1:length(Zero_Indices))
        t_one = rand(1:length(One_Indices))
        i_zero = Zero_Indices[t_zero][1]
        j_zero = Zero_Indices[t_zero][2]
        i_Value = One_Indices[t_one][1]
        j_Value = One_Indices[t_one][2]



        ParameterMatrix_Mut[i_zero,j_zero] = rand(0:1)*2-1 #turn a zero into 1 or -1
        ParameterMatrix_Mut[i_Value,j_Value] = 0 #turn 1 or -1 into 0




        
    end
    return ParameterMatrix_Mut
end


#Some Julia functions I played around with, may reconsider using more later
MultiplyBroad_(a,b) = a .* b

SummingDim2_(a) = sum(a,dims=2) 

HyperTanh_(a) = tanh.(40*a)



function Epigenetic_Evolution3(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    
    #⊗_(Wi, bi) = Wi .* bi
       
    for t=1:2^Splits

        du[2,t,:] = v*(a*u[1,t,:].-u[2,t,:])    
        du[1,t,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,t,:].+u[2,t,:].+CArray[:])) .- u[1,t,:] 
            #Epigenetic Evolution, no need to worry about boundaries here so simple loop      
    end  
    
end



function Gradient(u,Splits,t)
    "Function used to calculate the gradient of a cells neighbours, takes into accounts if there even are neighbours.
    Input:
    u      = Gene concentrations over which we take the gradient
    Splits = How many times have the cells split already (determines where neighbours are)
    t      = What location is our cell (boundaries only have 1 neighbour)
    Output:
    Multiple different version depending on the splits, this will be the gradient we want
    "
    if Splits == 0
        return 0
    elseif Splits ==1
        if t==1
            return u[t+1,:] - u[t,:]
        else
            return u[t-1,:] - u[t,:]
        end
    else
        if t==1
            return u[t+1,:] - u[t,:]
        elseif t==2^Splits
            return u[t-1,:] - u[t,:]
        else
            return u[t-1,:] - 2*u[t,:] + u[t+1,:]
        end
    end
    
end

function Epigenetic_Evolution4(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    
    #⊗_(Wi, bi) = Wi .* bi
       
    for t=1:2^Splits

        du[2,t,:] = v*(a*u[1,t,:].-u[2,t,:])    
        du[1,t,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,t,:].+u[2,t,:].+CArray[:])) .- u[1,t,:] +KArray.*Gradient(u[1,:,:],Splits,t)
            #Epigenetic Evolution, no need to worry about boundaries here so simple loop      
    end  
    
end

function σ_Genes(du, u,Parameters,Time)
    du[1,:,:] .= Noise
    
end



function Epigenetic_Evolution2(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    KArray = Parameters[3]
    Splits = Parameters[4]
    Cells::Int32 = 2^Splits
    #⊗_(Wi, bi) = Wi .* bi
       
    for t=1:Cells  

       du[2,t,:] = v*(a*u[1,t,:].-u[2,t,:])    

       du[1,t,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,t,:].+u[2,t,:].+CArray[:])) .- u[1,t,:] .+ KArray.*(vec(sum(u[1,:,:],dims=1))/Cells - u[1,t,:])

    end  
    
end

#Defining it all in function form


function Mutating_Network(N,M,K,ParameterMatrix_Collection,PositionArray_Collection, Indices, Counter, Genetics)
    "Function used to make mutated copies of the best performing networks. Currently not in use
    Input:
    N                          = How many copies of the network do we make
    M                          = Amount of Genes
    K                          = Supposed to work as a fraction of N, telling us how many mutant versions
    ParameterMatrix_Collection = The original networks
    PositionArray_Collection   = Original Positions
    Indices                    = Sorted indices giving the best networks
    Counter                    = Randomiser seed
    Genetics                   = Used to track heritance of networks
    Output:
    ParameterMatrix_Collection = Updated network
    PositionArray_Collection   = Updated initial positions
    Counter                    = Updated seed
    Updated_Genes              = Updated inheritance
    "
    Updated_Genes = zeros(N)
    Mutations = 1
    Seed_Mutations = 1234
    Indexation_Mutate = 1
    Temporary_Matrix = zeros(Float32,N,M,M)
    Temporary_Vector = zeros(Float32,N,2,1,M)
    for p = 1:K
        for q=1:N/(2*K)
            Temporary_Matrix[Indexation_Mutate,:,:] = Mutate_Parameters(ParameterMatrix_Collection[Indices[p],:,:],M,Mutations,Seed_Mutations+Counter)
            Temporary_Vector[Indexation_Mutate,:,:,:] = PositionArray_Collection[Indices[p],:,:,:]
            Updated_Genes[Indexation_Mutate] = Genetics[p]
            Temporary_Matrix[Indexation_Mutate+1,:,:] = Temporary_Matrix[Indexation_Mutate,:,:]
            Temporary_Vector[Indexation_Mutate+1,:,:,:] = Temporary_Vector[Indexation_Mutate,:,:,:]
            Updated_Genes[Indexation_Mutate+1] = Genetics[p]
            Counter += 1
            Indexation_Mutate +=2
        end
        Temporary_Matrix[Indexation_Mutate-1,:,:] = ParameterMatrix_Collection[Indices[p],:,:] #We overwrite it to add the original back
        Temporary_Matrix[Indexation_Mutate-2,:,:] = ParameterMatrix_Collection[Indices[p],:,:]
    end
    ParameterMatrix_Collection = deepcopy(Temporary_Matrix);
    PositionArray_Collection = deepcopy(Temporary_Vector);
    return ParameterMatrix_Collection, PositionArray_Collection, Counter, Updated_Genes
end



function Running_Network(CArray,ParameterMatrix,PositionArray,Random_Noise)
    "Function used to run our simulation
    Input:
    CArray          = Constants used in our ODE solver  
    ParameterMatrix = Collection of Networks
    PositionArray   = Original Positions
    Random_Noise    = Randomiser seed
    Output:
    PositionArray[2,:,:] = Epigenetic factors from our simulation
    Random_Noise         = Updated seed
    "
    #CArray = CArray_Collection[x,:]
    #ParameterMatrix = ParameterMatrix_Collection[x,:,:]
    #PositionArray = PositionArray_Collection[x,:]
    Orbit_Collection = zeros(Float32,2^Splits*M+1,0)
    Epi_Collection = zeros(Float32,2^Splits*M+1,0)
    
    
    
    
    for s=1:Splits #Looping over the splits
        Parameters = (ParameterMatrix, CArray,KArray,Int32(s-1)) #We have to update the splits every rerun, rest stays same
        tspan = (Float32(0.0),SplitTime) #How long ODE will run
        if Mean_Field
            prob = ODEProblem(Epigenetic_Evolution2,PositionArray,tspan,Parameters) #ODEdefining
        elseif Diffusing_System
            prob = ODEProblem(Epigenetic_Evolution4,PositionArray,tspan,Parameters) #ODEdefining
        elseif Stochastic_Noise
            prob = SDEProblem(Epigenetic_Evolution4, σ_Genes,PositionArray,tspan,Parameters)

        else
            prob = ODEProblem(Epigenetic_Evolution3,PositionArray,tspan,Parameters) #ODEdefining
        end    #Solving
        if !Stochastic_Noise
            sol = solve(prob,BS3())
        else
            sol = solve(prob,SRA3())
            #sol = solve(prob,LambaEulerHeun())
        end
        Times = sol.t #Save the time, doesn't get used yet though
        StepAmount = length(Times)  #Amount of data points per gene
        Genes = sol 
        PositionArray = Genes[StepAmount] #Saving the genes and epi factors
        
        OrbitalCollection = Times' .+ (s-1)*SplitTime
        EpiCollected = Times' .+ (s-1)*SplitTime
        for i=1:2^(Splits-s+1)  #s=1 this gives all 2^Splits, this is to say how much we repeat same path
            for j=1:2^(s-1) #Together multiply, always 2^Splits orbits, these j represent different paths
                OrbitalCollection = vcat(OrbitalCollection,sol[1,j,:,:])   
                EpiCollected = vcat(EpiCollected,sol[2,j,:,:])   
            end
        end
        Orbit_Collection = hcat(Orbit_Collection,OrbitalCollection)
        Epi_Collection = hcat(Epi_Collection,EpiCollected)
        #Orbit1 = [Times,Genes[1,1,1:M,1:StepAmount]]
        #Orbit2 = [Times,Genes[1,1,1:M,1:StepAmount]]
        #EpiOrbit1 = [Times,Genes[2,1,1:M,1:StepAmount]]
        #EpiOrbit2 = [Times,Genes[2,1,1:M,1:StepAmount]]
        #Now we must take all the values from the updated PositionArray and give appropriate terms to the daughter cells
        PositionArray = Noisy_Splitting(PositionArray,s,M,Random_Noise)  #Splitting function
        Random_Noise += 1
    end

    Final_Run_Time::Int32 = 20000 #Long evolving of the cells after splitting to get the final epigenetic state
    Parameters = (ParameterMatrix, CArray,KArray,Int32(Splits)) 
    tspan = (Int32(0.0),Final_Run_Time)

    if Mean_Field
        prob = ODEProblem(Epigenetic_Evolution2,PositionArray,tspan,Parameters) #ODEdefining
    elseif Diffusing_System
        prob = ODEProblem(Epigenetic_Evolution4,PositionArray,tspan,Parameters) #ODEdefining
    elseif Stochastic_Noise
        prob = SDEProblem(Epigenetic_Evolution4, σ_Genes,PositionArray,tspan,Parameters)

    else
        prob = ODEProblem(Epigenetic_Evolution3,PositionArray,tspan,Parameters) #ODEdefining
    end    #Solving
    if !Stochastic_Noise
        sol = solve(prob,BS3())
    else
        #,dtmin=0.1,force_dtmin=true
        sol = solve(prob,SRA3(),dtmin=0.1,force_dtmin=true)
    end
    #sol = solve(prob,alg_hints = [:nonstiff]) #Solving
    #sol = solve(prob)
    Times = sol.t
    StepAmount = length(Times)
    GenesEpi = sol
    PositionArray = GenesEpi[StepAmount]

    OrbitalCollection = Times' .+ Splits*SplitTime 
    EpiCollected = Times' .+ Splits*SplitTime 
    for j=1:2^(Splits) #Together multiply, always 2^Splits orbits, these j represent different paths
        OrbitalCollection = vcat(OrbitalCollection,sol[1,j,:,:])  
        EpiCollected = vcat(EpiCollected,sol[2,j,:,:])   

    end
    Orbit_Collection = hcat(Orbit_Collection,OrbitalCollection)
    Epi_Collection = hcat(Epi_Collection,EpiCollected)
    #Orbit1 = [cat(Orbit1[1],Times.+SplitTime,dims=1),cat(Orbit1[2],GenesEpi[1,1,1:M,1:StepAmount],dims=2)]
    #Orbit2 = [cat(Orbit2[1],Times.+SplitTime,dims=1),cat(Orbit2[2],GenesEpi[1,2,1:M,1:StepAmount],dims=2)]
    #EpiOrbit1 = [cat(EpiOrbit1[1],Times.+SplitTime,dims=1),cat(EpiOrbit1[2],GenesEpi[2,1,1:M,1:StepAmount],dims=2)]
    #EpiOrbit2 = [cat(EpiOrbit2[1],Times.+SplitTime,dims=1),cat(EpiOrbit2[2],GenesEpi[2,2,1:M,1:StepAmount],dims=2)]

    return PositionArray[2,:,:],Random_Noise, Orbit_Collection, Epi_Collection
end


function Scoring_Indexes(N,Final_Epigenetic_Factors,Cell_Bit)
    "Function used to calculate the score of all the networks, distribution robustness and the cell type robustness. Note
    that we are still selecting on amount of different cells, not entropy.
    Input:
    N                       = How many copies of the same networks we have 
    Final_Epigentic_Factors = Array of epigenetic factors after running the simulations
    Cell_Bit                = Cell type tracker
    Output:
    Indices                   = Indices of networks sorted by score, we only select these
    AllIndices                = Longer copy of Indices, contains also those we will no longer use
    Evolution_Indices         = Gives us the score corresponding with the Indices we selected on
    [Cell_Types,Cell_Amounts] = Vector of the cell types and how many of these cell types there are
    Cell_Bit                  = Updated cell type tracker
    KLD_Distance              = Kullback-Lieber divergence score for the distribution robustness
    Bit_Distances             = Cell Type robustness measure
    "
    Scoring = zeros(2,N)
    Cell_Types = []
    Cell_Amounts = []
    Cell_Distribution = Array{Vector{Float64}}(undef, N) 
    Cell_Bit_Array = Array{Vector{Any}}(undef, N)

    for x=1:N
        Differentiation, Bit_Genes = Differentiaton_Detection_Parameter(M,Final_Epigenetic_Factors[x,:,:]) 
     
        #Here we save some cell types and bits to use for our robustness calculation after the for loop
        #Cell types
        alpha = countmap(Differentiation)
        Temp_Array = Vector{Float64}()
        for i in keys(alpha)
            Temp_Array = push!(Temp_Array,alpha[i])
        end
        #print(Temp_Array)
        Cell_Distribution[x] = sort(Temp_Array,rev=true)/2^Splits #We sort the distribution of our cells and normalise
        #Bits
        Cell_Bit_Array[x] = Bit_Genes
        
        
        
        
        #This part here is where we track the cell types we had before as well
        for tau in Bit_Genes
            if !(tau in Cell_Bit)

                push!(Cell_Bit,tau)


            end
        end
        for tau in Differentiation 
            if !(tau in Cell_Types)
                push!(Cell_Types,tau)
                push!(Cell_Amounts,1)
            else
                Cell_Amounts[findfirst(==(tau), Cell_Types)] += 1
            end
        end

        Scoring[1:2,x] = [length(countmap(Differentiation)),x]

    end

    #Robustness calculation
    KLD_Distance = 0
    
    KLD_Adder = 0
    for i=1:N
        for j=1:N #Loop over all and calculate KL for all distributions
            if i!=j #If the same we get no difference anyways
                Length_Differ = length(Cell_Distribution[i])-length(Cell_Distribution[j])
                if Length_Differ > 0 #i is longer so we need to add 0 to j
                    KLD_Adder = kldivergence(Cell_Distribution[i],vcat(Cell_Distribution[j],zeros(Length_Differ)))
                    if KLD_Adder != Inf #There are zeros in the second distribution (divide by 0 error)
                        KLD_Distance += kldivergence(Cell_Distribution[i],vcat(Cell_Distribution[j],zeros(Length_Differ)))
                    end
                else
                    KLD_Adder = kldivergence(vcat(Cell_Distribution[i],zeros(-Length_Differ)),Cell_Distribution[j])

                    KLD_Distance += kldivergence(vcat(Cell_Distribution[i],zeros(-Length_Differ)),Cell_Distribution[j])
                end
            end
        end
    end
    KLD_Distance = KLD_Distance/(N*(N-1))
    
    #Now we see the closest type in the other groups
    Bit_Distances = 0
    for x =1:N
        for Bit_Inside in Cell_Bit_Array[x] #These are all the cell types in bit form
            for y =1:N #We loop over the others
                if x!=y #Must look at different otherwise distance 0 again
                    Bit_Distan_Tem = M
                    for Bit_Outside in Cell_Bit_Array[y] #We loop over all the cell types in this other one
                        Bit_Distan_Temp = sum(abs.(Bit_Inside .- Bit_Outside)) 
                        if Bit_Distan_Temp < Bit_Distan_Tem #Distance must be smaller if it is the best fit
                            Bit_Distan_Tem = Bit_Distan_Temp
                        end
                    end
                    Bit_Distances += sqrt(Bit_Distan_Tem)/(N*(N-1)) #Sqrt to make euclidean distance, N*(N-1) for normalisation
                end
            end
        end
    end
    
    
    #Here we mess a bit with the indices to just have the ones with the highest score later
    
    Indices = sortslices(Scoring,dims=2,rev=true)
    Evolution_Indices = Indices[1,1:K]

    AllIndices = deepcopy(Indices[2,:])
    Indices = Indices[2,1:K]
    Indices = Int.(Indices)
    #print(Indices[2,1:K])
    return Indices, AllIndices, Evolution_Indices, [Cell_Types,Cell_Amounts],Cell_Bit, KLD_Distance, Bit_Distances
end

function Epi_Plotter(Indices,K,Final_Epigenetic_Factors)
    "Function used to plot the epigenetic factors of different cells
    Input:
    Indices                 = Indices of highest scoring runs  
    k                       = 4 or 8 plots
    Final_Epigentic_Factors = Epigenetic factors that we will plot
    Output:
    k = Plot element, simply use display on the output to see the requested plot
    "
    pa = plot(Final_Epigenetic_Factors[Indices[1],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pb = plot(Final_Epigenetic_Factors[Indices[2],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pc = plot(Final_Epigenetic_Factors[Indices[3],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    pd = plot(Final_Epigenetic_Factors[Indices[4],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
    xlabel!("Cell Position")
    ylabel!("Epigenetic")
    ylims!(-1, 1)

    if K==8
        pe = plot(Final_Epigenetic_Factors[Indices[5],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        pf = plot(Final_Epigenetic_Factors[Indices[6],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        pg = plot(Final_Epigenetic_Factors[Indices[7],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        ph = plot(Final_Epigenetic_Factors[Indices[8],:,:],lw=1.5, palette = :tab10) #Final plot (palette is written for M=10, adjust manually if needed)
        xlabel!("Cell Position")
        ylabel!("Epigenetic")
        ylims!(-1, 1)

        k = plot(pa,pb,pc,pd,pe,pf,pg,ph,layout=(4,2),legend=false)
    else
        k = plot(pa,pb,pc,pd,layout=(2,2),legend=false)
    end
    return k
end

function Gene_Variance(Final_Epigenetic_Factors,M,N)
    "Function used to calcualte the variance in expressed genes
    Input:
    Final_epigenetic_Factors = Final epigenetic values which represent gene expression
    M                        = Amount of Genes
    N                        = How many copies of same network do we have
    Output:
    Deviation = Variance in these genes (normalised by 2^Splits*N)
    "
    BitterMatrix = signbit.(-Final_Epigenetic_Factors[:,:,:])
    Average_Bits = zeros(M)
    for i=1:N
        for j=1:2^Splits
            Average_Bits += BitterMatrix[i,j,:]
        end
    end
    Average_Bits /= N*2^Splits
    Deviation = 0

    for i=1:N
        for j=1:2^Splits
            Deviation += sum((Average_Bits .- BitterMatrix[i,j,:]).^2)/(2^Splits*N-1)
        end
    end
    
    return Deviation
end

function PCA_Distribution_Creator(PCA_Dimensions,Transformed,Dimensions_Distribution,Cell_Distribution)
    
    
    PCA_Dimensional_Values = zeros(PCA_Dimensions,findmax(Dimensions_Distribution)[1]) 
    #Here we will save all the values and use these for indexes later
    B = Array{Vector{Tuple{Float64, Int64}}}(undef, PCA_Dimensions)
    for j=1:PCA_Dimensions
        B[j] = [(i, count(==(i), Transformed)) for i in unique(Transformed[j,:])] #Here we find what the arguments are for the
        #different dimensions
        for k = 1:length(B[j])
            PCA_Dimensional_Values[j,k] = B[j][k][1] #Here we save all the individual arguments in the corresponding dimension
        end
    end
    #print(PCA_Dimensional_Values)   #If we wanna see them

    Components_Length = length(Transformed[1,:]) 
    for k in 1:Components_Length
        Index_PCA = zeros(PCA_Dimensions)

        for l=1:PCA_Dimensions
            Index_PCA[l] = (findfirst(==(Transformed[l,k]), PCA_Dimensional_Values[l,:]))
        end 
        Index_PCA = Int.(Index_PCA)
        
        
        if PCA_Dimensions==2 #I really tried but I couldn't fix it so just throwing in this hack
            Cell_Distribution[Index_PCA[1],Index_PCA[2]] += 1/Components_Length
        elseif PCA_Dimensions==1
            Cell_Distribution[Index_PCA[1]] += 1/Components_Length   
        elseif PCA_Dimensions==3
            Cell_Distribution[Index_PCA[1],Index_PCA[2],Index_PCA[3]] += 1/Components_Length
        elseif PCA_Dimensions==4
            Cell_Distribution[Index_PCA[1],Index_PCA[2],Index_PCA[3],Index_PCA[4]] += 1/Components_Length
        elseif PCA_Dimensions==5
            Cell_Distribution[Index_PCA[1],Index_PCA[2],Index_PCA[3],Index_PCA[4],Index_PCA[5]] += 1/Components_Length
        else
        print("ADD MORE HERE THERE IS A MISTAKE, DO NOT GO OVER 5 PCA DIMENSIONS YET")
        end
    end
    

    
    return Cell_Distribution
end



function Earth_Movers_Distance_1D(Vector1,Vector2)

    #Takes 2 distribution vectors containing bins with a position and their %
    Length_V = length(Vector1[1,:]) #Both should be same length anyways
    EMD = zeros(Length_V+1) #We add an extra one because of the following loop
    
    for i =1:Length_V
        EMD[i+1] = Vector1[2,i] - Vector2[2,i] + EMD[i]
    end
    EMD_Distance = 0
    
    for i =1:Length_V-1
        EMD_Distance += abs(EMD[i+1])*(Vector1[1,i+1] - Vector1[1,i]) #We take moved distribution times moved distance

    end

    
    return EMD_Distance
end


function Robustness_Distribution_Scoring()
    
    
    #Here we create a very general robustness measure
    #This is done by taking the PCA components over all the different generations
    #Then we categorise all cell types and measure how similar their distribution
    #is in a given network

    #!!!!!!! This robustness does not consider cell type distances
    
    Robustness_Score = zeros(Generations,P)
    for Genere=1:Generations  #We want to calculate this for all generations
        for i = 1:P #We have to repeat this for all different networks in a generation
            #This is used to find the maximum of different cell types along every dimension in this network
            Dimensions_Distribution = zeros(Int32,PCA_Dimensions)
            for j=1:N
                Transformed = transform(model,reshape(signbit.(-History_Final_Epigenetic_Factors[Genere,i,j,:,:]),2^Splits,M)')

                for k=1:PCA_Dimensions
                    if Int.(length(countmap(Transformed[k,:]))) >  Dimensions_Distribution[k]
                        Dimensions_Distribution[k] = Int.(length(countmap(Transformed[k,:])))
                    end
                end   
            end

            #Now we have the max dimensions, now we want to make all distributions
            #Of this specific network

            Dimensions_Distribution = (Dimensions_Distribution...,) #Magically makes a tuple!
            #print(Dimensions_Distribution) #if we wanna visualise

            Cell_Distribution_Collection = Array{Array{Float64, PCA_Dimensions}}(undef, N)
            Vectorised_Cell_Distribution_Collection = Array{Vector{Float64}}(undef, N)

            for j=1:N #Now we collect all the distributions
                Transformed = transform(model,reshape(signbit.(-History_Final_Epigenetic_Factors[Genere,i,j,:,:]),2^Splits,M)')

                Cell_Distribution = zeros(Dimensions_Distribution) #This is where we need it to be a tuple!
                Cell_Distribution_Collection[j] = PCA_Distribution_Creator(PCA_Dimensions,Transformed,Dimensions_Distribution,Cell_Distribution)

                #Now we want to convert all of these into vectors to apply Jensen-Shannon
                Vectorised_Cell_Distribution_Collection[j] = sort(vec(Cell_Distribution_Collection[j]),rev=true)
            end

            #JS-divergence
            for tau = 1:N-1
                for pau = tau:N
                    Robustness_Score[Genere,i] += js_divergence(Vectorised_Cell_Distribution_Collection[tau],Vectorised_Cell_Distribution_Collection[pau])
                end
            end


        end
    end
    return Robustness_Score
end



function EMD_Generational_Calculator()
    
    #Here we will create a Robustness measure that takes into account how far
    #away different cell types are for the same network.
    #we will do this by creating a PCA analysis for EVERY single network
    #for all the different generations
    #(Alternatively we see if we can use the same PCA analysis within one generation)
    #We will create a one dimensional distribution and use our simplified
    #EMD algorithm
    
    Vectorised_Cell_Network_Collection = Array{Matrix{Float64}}(undef, N) #Here we save these distributions
    #This loops over copies of same network and makes all our distributions
    EMD_Distances = zeros(Generations)
    for generation_p = 1:Generations
        EMD_Dist = 0
        for P_Valu = 1:P
            FullNetworkBits = reshape(signbit.(-History_Final_Epigenetic_Factors[generation_p,P_Valu,:,:,:]),2^Splits*N,M)'
            Networkmodel= fit(PCA,FullNetworkBits;maxoutdim=1)
            Networkmodel_EigenV = eigvals(Networkmodel::PCA)
            for i=1:N
                #This creates the Distribution of a single individual
                #We start to apply this PCA to one of these individuals
                NetworkBits = reshape(signbit.(-History_Final_Epigenetic_Factors[generation_p,P_Valu,i,:,:]),2^Splits,M)'
                Transformed_Network = transform(Networkmodel,NetworkBits)
                Distribution_Netw = zeros(0)
                X_Values = zeros(0)
                Transformed_Network = round.(Transformed_Network,digits=4) #Otherwise ugly floating point numbers
                Length_Netw_Normal = 1/length(Transformed_Network)
                for i in Transformed_Network

                    if i in X_Values
                        Distribution_Netw[findfirst(==(i), X_Values)] += Length_Netw_Normal
                    else
                        X_Values = vcat(X_Values,i)
                        Distribution_Netw = vcat(Distribution_Netw,Length_Netw_Normal)
                    end
                end



                Combined_Distribution = zeros(2,length(X_Values))
                Combined_Distribution[1,:] = round.(X_Values/Networkmodel_EigenV,digits=4)
                Combined_Distribution[2,:] = Distribution_Netw
                Combined_Distribution = sortslices(Combined_Distribution,dims=2)

                Vectorised_Cell_Network_Collection[i] = Combined_Distribution
            end


            #Now we need to apply EMD to these distributions and save the result

            for tau = 1:N-1   #Preparing the vectors to have the same X values and length
                for pau = tau+1:N
                    Vector1 = Vectorised_Cell_Network_Collection[tau]
                    Vector2 = Vectorised_Cell_Network_Collection[pau]
                    for i in Vector1[1,:]
                        if !(i in Vector2[1,:])
                            Vector2 = hcat(Vector2, [i,0]) #Adding empty position
                        end
                    end
                    for i in Vector2[1,:]
                        if !(i in Vector1[1,:])
                            Vector1 = hcat(Vector1, [i,0]) #Adding empty position
                        end
                    end

                    Vector1 = (sortslices(Vector1,dims=2)) #Sorting them with the added X values
                    Vector2 = (sortslices(Vector2,dims=2))
                    EMD_Dist += Earth_Movers_Distance_1D(Vector1,Vector2) #Calculating EDM for 1D

                end
            end
        end
        EMD_Dist = 2*EMD_Dist/(P*N*(N-1))  #Normalisation
        EMD_Distances[generation_p] = EMD_Dist
    end
    return EMD_Distances
end


function Two_Generational_Plotter(Generation,Cut_Off,Duration,Length_Orbits,Opacity)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1+2*(Generation-1):2+2*(Generation-1)
        for j=1:Amount_Copies
            if Shortest>size(Orbit_Array[i,j])[2]
                Shortest = size(Orbit_Array[i,j])[2] #Used later to check which data set is the shortest when plotting
            end

            Total_Length += size(Orbit_Array[i,j])[2]

        end
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2*8*2^Splits)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1+2*(Generation-1):2+2*(Generation-1)
        for j=1:Amount_Copies
            if Cut_Off
                
                for p=0:2^Splits-1
                    PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i,j][2+p*M:1+(p+1)*M,1:Length_Orbits]
                end
            else
                Length_Orbits =  size(Orbit_Array[i,j])[2]
                for p=0:2^Splits-1
                    PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i,j][2+p*M:1+(p+1)*M,:]
                end
            end

            Collection_Index += Length_Orbits*2^Splits
        end
    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,2,Amount_Copies,2^Splits,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:2
        for j=1:Amount_Copies

            for tau=1:2^Splits
                tester = Orbit_Array[i+2*(Generation-1),j][2+(tau-1)*M:1+tau*M,1:Shortest]
                Transformed = transform(Orbital_Model,tester)
                PCA_Orbits_2Gen[i,j,tau,:,:] = Transformed
            end
        end
    end
    
    
    Colours = [:red,:orange,:maroon,:chocolate4]
    @gif for k=1:Duration
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen[1,1,1,1,1:k],PCA_Orbits_2Gen[1,1,1,2,1:k],PCA_Orbits_2Gen[1,1,1,3,1:k],legend=false,c=:blue,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)


        for j=1:8
            for tau=1:2^Splits
                PCAPlots = plot!(PCA_Orbits_2Gen[i,j,tau,1,1:k],PCA_Orbits_2Gen[i,j,tau,2,1:k],PCA_Orbits_2Gen[i,j,tau,3,1:k],legend=false,c=:blue,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)
            end
        end

        i=2
        for j=1:4
            colour = Colours[j]
            for tau=1:2^Splits
                PCAPlots = plot!(PCA_Orbits_2Gen[i,2*j-1,tau,1,1:k],PCA_Orbits_2Gen[i,2*j-1,tau,2,1:k],PCA_Orbits_2Gen[i,2*j-1,tau,3,1:k],legend=false,c=colour,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                PCAPlots = plot!(PCA_Orbits_2Gen[i,2*j,tau,1,1:k],PCA_Orbits_2Gen[i,2*j,tau,2,1:k],PCA_Orbits_2Gen[i,2*j,tau,3,1:k],legend=false,c=colour,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)

            end
        end


    end
end




function Generational_Tracker(Family_Distance,Original_Tracking,Types)
    Cell_Generational_Tracking = deepcopy(Original_Tracking)
    Original_Tracking = deepcopy(Cell_Generational_Tracking)
    size(Types[1,:])


    TypeFam = [Types[1,:],1]
    TypeFam = hcat(TypeFam,[Types[1,:],2])
    TypeFam = hcat(TypeFam,[Types[1,:],3])
    TypeFam = hcat(TypeFam,[Types[1,:],4])

    print(size(TypeFam[1,:]))

    TypeFam = [Types[1,:],1]
    Distances_Gen = zeros(length(Cell_Generational_Tracking[1,:]))
    Cell_Generational_Tracking[1,1] = 1
    for i=2:length(Cell_Generational_Tracking[1,:])
        NewFam = true
        for j=1:length(TypeFam[1,:]) 
            if sum(abs.(Types[i,:]-TypeFam[1,j])) < Family_Distance + 0.5
                if NewFam
                    NewFam = false
                    Cell_Generational_Tracking[1,i] = j
                    Distances_Gen[j] = sum(abs.(Types[i,:]-TypeFam[1,j]))
                elseif Distances_Gen[j] > sum(abs.(Types[i,:]-TypeFam[1,j]))
                    Cell_Generational_Tracking[1,i] = j
                    Distances_Gen[j] = sum(abs.(Types[i,:]-TypeFam[1,j]))
                end
            end
        end
        if NewFam
            Distances_Gen[i] = sum(abs.(Types[i,:]-TypeFam[1,1]))
            Cell_Generational_Tracking[1,i] = Int(length(TypeFam[1,:])+1)
            TypeFam = hcat(TypeFam,[Types[i,:],Int(length(TypeFam[1,:])+1)])

        end
    end

    Cell_Generational_Tracking = Cell_Generational_Tracking[:,sortperm(Cell_Generational_Tracking[1,:])]



    Amount_Types = Cell_Generational_Tracking[1,end]
    Top_Bound_Index = zeros(Amount_Types)
    for j=1:Amount_Types
        Top_Bound_Index[j] = findfirst(isequal(j),Cell_Generational_Tracking[1,:])
    end
    print(Top_Bound_Index[:])


    #print(Cell_Generational_Tracking)
    Copied_Tracker = deepcopy(Cell_Generational_Tracking[2:end,:])
    Copied_Tracker = Copied_Tracker/sum(Copied_Tracker[1,:])

    Boundaries = zeros(Generations,length(Copied_Tracker[1,:]))
    for beta in 2:length(Copied_Tracker[1,:])
        Copied_Tracker[:,beta] += Copied_Tracker[:,beta-1]
        Boundaries[:,beta] = Copied_Tracker[:,beta] - Copied_Tracker[:,beta-1]
    end

    Cell_Type_Plotter = plot(Copied_Tracker[:,1]; ribbon = (Copied_Tracker[:,1],LinRange(0, 0, Generations)))
    for beta in 2:length(Copied_Tracker[1,:])
        Cell_Type_Plotter = plot!(Copied_Tracker[:,beta]; ribbon = (Boundaries[:,beta],LinRange(0, 0, Generations)),legend=false)
    end
    display(Cell_Type_Plotter)




    

    #Now we do the same but we group them in families


    Family_Boundaries = ones(Generations,Amount_Types)
    Family_Boundaries[:,1] = Copied_Tracker[:,Int(Top_Bound_Index[2]-1)].-0
    for beta in 2:Amount_Types-1

        Family_Boundaries[:,beta] = Copied_Tracker[:,Int(Top_Bound_Index[beta+1]-1)] .- Copied_Tracker[:,Int(Top_Bound_Index[beta]-1)]
    end
    Family_Boundaries[:,end] = 1 .- Copied_Tracker[:,Int(Top_Bound_Index[Amount_Types]-1)]


    Cell_Type_Family_Plotter = plot(Copied_Tracker[:,Int(Top_Bound_Index[2]-1)]; ribbon = (Family_Boundaries[:,1],LinRange(0, 0, Generations)))
    for beta in 2:Amount_Types-1
        Cell_Type_Family_Plotter = plot!(Copied_Tracker[:,Int(Top_Bound_Index[beta+1]-1)]; ribbon = (Family_Boundaries[:,beta],LinRange(0, 0, Generations)),legend=false)
    end
    Cell_Type_Family_Plotter = plot!(ones(Generations); ribbon = (Family_Boundaries[:,Amount_Types],LinRange(0, 0, Generations)),legend=false)

    if false
        Cell_Type_Family_Plotter = plot!(Copied_Tracker[:,1],linestyle=:dash)
        for beta in 2:length(Copied_Tracker[1,:])
            Cell_Type_Family_Plotter = plot!(Copied_Tracker[:,beta],linestyle=:dash)
        end
    end

    display(Cell_Type_Family_Plotter)
end

function Generational_Tracker_Initialisation()
    TypesString = deepcopy(Cell_Generational_Tracking[1,:])
    TypesString = string.(TypesString,base=2,pad=100)
    Types = zeros(Int16,length(Cell_Generational_Tracking[1,:]),M)
    for i in 1:length(Cell_Generational_Tracking[1,:])
        for j=1:M
            Types[i,j] = (parse(Int16,TypesString[i][j]))
        end
    end
    Original_Tracking = deepcopy(Cell_Generational_Tracking)
    return Original_Tracking, Types
end

function Interpolator(Times,Play_Array)
    Vectors = zeros(length(Times),1+M*2^Splits)
    for i=1:length(Times)
        Index_Interp = findfirst(>=(Times[i]), Play_Array[1,:])
        Vectors[i,:] = Play_Array[:,Index_Interp-1] + (Times[i]-Play_Array[1,Index_Interp-1])*(Play_Array[:,Index_Interp] - Play_Array[:,Index_Interp-1])/(Play_Array[1,Index_Interp] - Play_Array[1,Index_Interp-1])
    end
    return Vectors
end


function One_Generational_Plotter(Cut_Off,Duration,Length_Orbits,Opacity,Name)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1:N_Perturbed

            if Shortest>size(Orbit_Array[i])[2]
                Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
            end
            Total_Length += size(Orbit_Array[i])[2]
        
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1:N_Perturbed
        
        if Cut_Off

            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
            end
        else
            Length_Orbits =  size(Orbit_Array[i,j])[2]
            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
            end
        end

        Collection_Index += Length_Orbits*2^Splits

    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,N_Perturbed,2^Splits,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:N_Perturbed
        
        for tau=1:2^Splits
            tester = Orbit_Array[i][2+(tau-1)*M:1+tau*M,1:Shortest]
            Transformed = transform(Orbital_Model,tester)
            PCA_Orbits_2Gen[i,tau,:,:] = Transformed
        end

    end
    
    
    anim = @animate for k=1:Duration
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen[1,1,1,1:k],PCA_Orbits_2Gen[1,1,2,1:k],PCA_Orbits_2Gen[1,1,3,1:k],legend=false,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)


        for i=1:N_Perturbed
            for tau=1:2^Splits
                PCAPlots = plot!(PCA_Orbits_2Gen[i,tau,1,1:k],PCA_Orbits_2Gen[i,tau,2,1:k],PCA_Orbits_2Gen[i,tau,3,1:k],legend=false,alpha = 0.3*max.((1:k) .+ Opacity .- k, 0) / Opacity)
            end
        end

        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 5
    gif(anim, Name, fps = 15)
    
end

function adjoint(a)
    return a'
end


function Coarsed_Variance(Variance_Length,Time_Orbital_Collection,Coarsing,Orbit_Length)
    
    
    #Variance[i] = sum((Time_Orbital_Collection[i,:,:] .- mean(Time_Orbital_Collection[i,:,:],dims=1)).^2)/(M-1)

    
    
    Variance = zeros(Variance_Length)
    Average = zeros(Variance_Length,M)
    
    for i = 1:Variance_Length #For all different times
       Average[i,:] =  mean(Time_Orbital_Collection[i,:,:],dims=1)[1,:]
        for k=1:Orbit_Length #Have to do it for all the orbits
            Temp_Var = 1000
            Add_Var = 0
            
            for j=max(1,i-Coarsing):i+Coarsing 
                Add_Var = sum((Time_Orbital_Collection[j,k,:] .- Average[i,:]).^2)/(M-1)

                if Add_Var < Temp_Var
                    Temp_Var = Add_Var               
                end
            end
            Variance[i] += Temp_Var  

        end
    end
    
    return Variance
end


function Multi_One_Generational_Plotter(Cut_Off,Duration,Length_Orbits,Opacity,Name,Amount_Perturbations,Partial,N_Perturbed)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1:N_Perturbed*Amount_Perturbations

            if Shortest>size(Orbit_Array[i])[2]
                Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
            end
            Total_Length += size(Orbit_Array[i])[2]
        
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed*Amount_Perturbations)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1:N_Perturbed*Amount_Perturbations
        
        if Cut_Off

            for p=0:2^Splits-1

                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
            end
        else
            Length_Orbits =  size(Orbit_Array[i,j])[2]
            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
            end
        end

        Collection_Index += Length_Orbits*2^Splits

    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,N_Perturbed*2^Splits,Amount_Perturbations,3,Shortest)
    PCA_Orbits_Partial = zeros(Float32,Int(N_Perturbed/Partial)*2^Splits,Amount_Perturbations,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:N_Perturbed
        for j=0:Amount_Perturbations-1
        for tau=1:2^Splits
            tester = Orbit_Array[i+N_Perturbed*j][2+(tau-1)*M:1+tau*M,1:Shortest]
            Transformed = transform(Orbital_Model,tester)
            PCA_Orbits_2Gen[(i-1)*2^Splits+tau,j+1,:,:] = Transformed
            if mod(i,Partial) == 0
                PCA_Orbits_Partial[(Int(i/Partial)-1)*2^Splits+tau,j+1,:,:] = Transformed
            end
        end
        end

    end
    Colours = [:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
    
    #PCA_Orbits_Partial = reshape(PCA_Orbits_Partial,(Int(2^Splits*N_Perturbed*Amount_Perturbations/Partial),3,Shortest))
    PCA_Orbits_2Gen_Plot = permutedims(PCA_Orbits_Partial,(4,3,2,1))
    Orbit_Range = Int(N_Perturbed/Partial)*2^Splits
    tick()
    anim = @animate for k=1:Int(Duration/5)
        
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,:],PCA_Orbits_2Gen_Plot[1:k,2,1,:],PCA_Orbits_2Gen_Plot[1:k,3,1,:],c=Colours[1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for p=1:Amount_Perturbations
            PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,:],PCA_Orbits_2Gen_Plot[1:k,2,p,:],PCA_Orbits_2Gen_Plot[1:k,3,p,:],c=Colours[p],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)

        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 5
    tock()
    tick()
    animlong = @animate for k=1:Duration
        
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,:],PCA_Orbits_2Gen_Plot[1:k,2,1,:],PCA_Orbits_2Gen_Plot[1:k,3,1,:],c=Colours[1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for p=1:Amount_Perturbations
            PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,:],PCA_Orbits_2Gen_Plot[1:k,2,p,:],PCA_Orbits_2Gen_Plot[1:k,3,p,:],c=Colours[p],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)

        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 50
    tock()
    abra = gif(anim, Name, fps = 10)
    display(abra)

    kadabra = gif(animlong, Name[1:end-4] * "_Long" *Name[end-3:end], fps = 3)
    display(kadabra)



end


function Multi_One_Generational_Plotter2(Cut_Off,Duration,Length_Orbits,Opacity,Name,Amount_Perturbations,Partial,N_Perturbed,Orbit_Array)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1:N_Perturbed*Amount_Perturbations

            if Shortest>size(Orbit_Array[i])[2]
                Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
            end
            Total_Length += size(Orbit_Array[i])[2]
        
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed*Amount_Perturbations)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1:N_Perturbed*Amount_Perturbations
        
        if Cut_Off

            for p=0:2^Splits-1

                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
            end
        else
            Length_Orbits =  size(Orbit_Array[i,j])[2]
            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
            end
        end

        Collection_Index += Length_Orbits*2^Splits

    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,N_Perturbed*2^Splits,Amount_Perturbations,3,Shortest)
    PCA_Orbits_Partial = zeros(Float32,Int(N_Perturbed/Partial)*2^Splits,Amount_Perturbations,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:N_Perturbed
        for j=0:Amount_Perturbations-1
        for tau=1:2^Splits
            tester = Orbit_Array[i+N_Perturbed*j][2+(tau-1)*M:1+tau*M,1:Shortest]
            Transformed = transform(Orbital_Model,tester)
            PCA_Orbits_2Gen[(i-1)*2^Splits+tau,j+1,:,:] = Transformed
            if mod(i,Partial) == 0
                PCA_Orbits_Partial[(Int(i/Partial)-1)*2^Splits+tau,j+1,:,:] = Transformed
            end
        end
        end

    end
    Colours = [:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
    
    #PCA_Orbits_Partial = reshape(PCA_Orbits_Partial,(Int(2^Splits*N_Perturbed*Amount_Perturbations/Partial),3,Shortest))
    PCA_Orbits_2Gen_Plot = permutedims(PCA_Orbits_Partial,(4,3,2,1))
    Orbit_Range = Int(N_Perturbed/Partial)*2^Splits
    tick()
    anim = @animate for k=1:Int(Duration/5)
        
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,:],PCA_Orbits_2Gen_Plot[1:k,2,1,:],PCA_Orbits_2Gen_Plot[1:k,3,1,:],c=Colours[1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for p=1:Amount_Perturbations
            PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,:],PCA_Orbits_2Gen_Plot[1:k,2,p,:],PCA_Orbits_2Gen_Plot[1:k,3,p,:],c=Colours[p],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)

        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 5
    tock()
    tick()
    animlong = @animate for k=1:Duration
        
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,:],PCA_Orbits_2Gen_Plot[1:k,2,1,:],PCA_Orbits_2Gen_Plot[1:k,3,1,:],c=Colours[1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for p=1:Amount_Perturbations
            PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,:],PCA_Orbits_2Gen_Plot[1:k,2,p,:],PCA_Orbits_2Gen_Plot[1:k,3,p,:],c=Colours[p],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)

        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 50
    tock()
    abra = gif(anim, Name, fps = 10)
    display(abra)

    kadabra = gif(animlong, Name[1:end-4] * "_Long" *Name[end-3:end], fps = 3)
    display(kadabra)

    
    PCAPlots_Final = scatter(PCA_Orbits_2Gen_Plot[end,1,1,:],PCA_Orbits_2Gen_Plot[end,2,1,:],PCA_Orbits_2Gen_Plot[end,3,1,:],title="Final Cell States",c=Colours[1],legend=false)
    for p=1:Amount_Perturbations
        PCAPlots_Final = scatter!(PCA_Orbits_2Gen_Plot[end,1,p,:],PCA_Orbits_2Gen_Plot[end,2,p,:],PCA_Orbits_2Gen_Plot[end,3,p,:],c=Colours[p],legend=false)

    end

    display(PCAPlots_Final)
    savefig(Name[1:end-4] * "_Final.png")
end

function Multi_One_Generational_Plotter3(Cut_Off,Duration,Length_Orbits,Opacity,Name,Amount_Perturbations,Partial)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1:N_Perturbed*Amount_Perturbations

            if Shortest>size(Orbit_Array[i])[2]
                Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
            end
            Total_Length += size(Orbit_Array[i])[2]
        
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed*Amount_Perturbations)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1:N_Perturbed*Amount_Perturbations
        
        if Cut_Off

            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
            end
        else
            Length_Orbits =  size(Orbit_Array[i,j])[2]
            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
            end
        end

        Collection_Index += Length_Orbits*2^Splits

    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,N_Perturbed*2^Splits,Amount_Perturbations,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:N_Perturbed
        for j=1:2^Splits
            for tau=1:Amount_Perturbations
                tester = Orbit_Array[i+ (tau-1)*N_Perturbed][2+(j-1)*M:1+j*M,1:Shortest]
                Transformed = transform(Orbital_Model,tester)
                PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,:,:] = Transformed

            end
        end
    end
    Colours = [:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
    
    PCA_Orbits_2Gen_Plot = permutedims(PCA_Orbits_2Gen,(4,3,2,1))
    anim = @animate for k=1:Duration
        
        i=1
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1],c=Colours[1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for l=1:Amount_Perturbations
            
            PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,l,:],PCA_Orbits_2Gen_Plot[1:k,2,l,:],PCA_Orbits_2Gen_Plot[1:k,3,l,:],c=Colours[l],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
            
        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])

        title!("Time = $k")
    end every 15

    abra = gif(anim, Name, fps = 15)
end

function N_Dimensional_Euclid(Position1,Position2,Dimensions)
    Summed = 0
    for i =1:Dimensions
        Summed += (Position2[i]-Position1[i])^2
    end
    Distance = sqrt(Summed)
    return Distance
end



function Distance_Movement_Matrix(Movement_Matrix)
    
    for i=1:N_Sources #we must loop over all the sources to find a good sink for it

        #First we try to find the closest sinks
        Distance_Indices = (hcat(DistanceMatrix[i,:], 1:N_Sinks))
        Distance_Indices = Distance_Indices[sortperm(Distance_Indices[:, 1]), :]
        Distance_Indices = Int.(Distance_Indices[:,2])
        print(Distance_Indices)

        #Then we check if we can still send anything to this sink
        Displacing = true
        D_Index = 1
        while Displacing
            I_Sink = Distance_Indices[D_Index] #This is the index of the sink we are checking now
            Available_Sink = Sinks_Input[I_Sink] - sum(Movement_Matrix[:,I_Sink])
            print(Sinks_Input[I_Sink])
            if Available_Sink > 0 #It can have more added to it
                #We check if we can send everything to the sink or only partial
                Remaining_Flow = Sources_Output[i] - sum(Movement_Matrix[i,:])
                if Remaining_Flow > Available_Sink
                #If partial we fill a bit and go back to the start to find the next closest sink
                    Movement_Matrix[i,I_Sink] = Available_Sink
                    D_Index += 1
                    print("Partial.     ")
                    #BUT WE HAVE TO CHECK IF WE JUST FILLED IT ALL IN!!!! 
                    if Sources_Output[i] - sum(Movement_Matrix[i,:]) < 0.0001
                        Displacing = false
                        print("Finished After Partial")
                    end

                else  
                #If we can send it all we do and go to the next Source i+1
                    Displacing = false
                    Movement_Matrix[i,I_Sink] = Remaining_Flow
                    print("Finished.     ")

                end
            else #Sink is full
                print("Full.     ")

                #Cannot send more in the sink so we have to check the next one
                D_Index += 1
            end
        end
    end
    Movement_Matrix = round.(Movement_Matrix,digits=4)

    return Movement_Matrix
end

function Cost_Reducing_Rewiring(Attempts,Optimised_Indices)
    print("Initial Costs: ")
    print(sum(Cost_Matrix))
    for yota =1:Attempts
        println(" ")
        #We make a big while loop until a certain condition we decide later
        Cost_Matrix = DistanceMatrix .* Movement_Matrix
        print(sum(Cost_Matrix))
        for theta =1:1   #Old loop that can be removed

            if Optimised_Indices != []
                for kappa=1:length(Optimised_Indices[1,:])
                    Cost_Matrix[Optimised_Indices[1,kappa],Optimised_Indices[2,kappa]] = 0
                end
            end

            #We find the connection that has the highest cost, find the source and the corresponding sink
            Source_Index = argmax(Cost_Matrix)[1]
            Sink_Index = argmax(Cost_Matrix)[2]
            Improvement = 0 #Use this to keep track of best
            Displace = 0 #Keep track of best
            Best_Source = 0 #Keep track
            Best_Sink = 0 #Keep track

            #We let this source connect to other sinks instead and try to change another source that was connected to that sink
            for i=1:N_Sinks #We loop over all other sinks
                if i!=Sink_Index #Make sure it is a different sink
                    for j=1:N_Sources #We loop over all other Sources
                        if j!=Source_Index #Make sure it is a different source
                            #Check how much we may change 
                            Displace_Temp = min(Movement_Matrix[Source_Index,Sink_Index],Movement_Matrix[j,i])
                            #Calculate the improvement
                            #-DM[So,Si] + DM[So,i] We remove So -> Si and make So -> i
                            #-DM[j,i] + DM[j,Si] We remove j -> i and make j -> Si
                            Temp_Improvement = Displace_Temp*(-DistanceMatrix[Source_Index,Sink_Index] + DistanceMatrix[Source_Index,i] -DistanceMatrix[j,i] + DistanceMatrix[j,Sink_Index])

                            #Find the best improvement for this initial Source_Sink combo
                            if Temp_Improvement < Improvement #Found a better improvement
                                Improvement = Temp_Improvement
                                Displace = Displace_Temp
                                Best_Source = j
                                Best_Sink = i
                            end

                        end
                    end
                end
            end
            print("Better amount:  ")
            print(Improvement)
            if Improvement==0
                print("No improvement???")
                print(Displace)
                print("    ")
                print(Sink_Index)
                print(Source_Index)
                if Optimised_Indices == []
                    Optimised_Indices = vcat(Optimised_Indices,[Source_Index;Sink_Index])
                else
                    Optimised_Indices = hcat(Optimised_Indices,[Source_Index;Sink_Index])
                end

            else
                print("WE IMPROVED")
                Movement_Matrix[Source_Index,Sink_Index] -= Displace
                Movement_Matrix[Best_Source,Best_Sink] -= Displace
                Movement_Matrix[Best_Source,Sink_Index] += Displace
                Movement_Matrix[Source_Index,Best_Sink] += Displace


            end

            #Switch the best one and repeat
        end
        print("   ")
        Cost_Matrix = DistanceMatrix .* Movement_Matrix
        print(sum(Cost_Matrix))
    end

    println(" ")
    print("Final Costs: ")
    print(sum(Cost_Matrix)) 
    return sum(Cost_Matrix)
end

function Final_Cell_PCAScatterPlot(Orbit_Array,Single_Plot)
    Final_Cell_States_PCA = zeros(Amount_Large_Perturbations*Amount_Small_Perturbations*2^Splits,M)
    Transformed_Data = zeros(Amount_Small_Perturbations*2^Splits,Amount_Large_Perturbations,2)
    Before_Data = zeros(Amount_Small_Perturbations*2^Splits,Amount_Large_Perturbations,M)
    for j=1:Amount_Large_Perturbations*Amount_Small_Perturbations
        for i=1:2^Splits
            Final_Cell_States_PCA[j+Amount_Large_Perturbations*Amount_Small_Perturbations*(i-1),:] = Orbit_Array[j][2+(i-1)*M:1+i*M,end]
        end
    end
    #This is added in in case we only have the same cell types, then our code would crash
    Final_Cell_States_PCA[1,1] += 1
    
    
    for j=1:Amount_Large_Perturbations
        for i=1:Amount_Small_Perturbations
            for tau=1:2^Splits
                Before_Data[i+(tau-1)*Amount_Small_Perturbations,j,:] = Orbit_Array[i+(j-1)*Amount_Small_Perturbations][2+(tau-1)*M:1+tau*M,end]
            end
        end
    end

    Final_Cell_States_PCA[1:5,1:10] .+= 1
    Final_Cell_PCA = fit(PCA,Final_Cell_States_PCA';maxoutdim =2,pratio =1)
    print(size(Final_Cell_PCA))
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Final_Cell_PCA))
    Final_Cell_States_PCA[1:5,1:10] .-= 1
 

    for j=1:Amount_Large_Perturbations

        Transformed_Data[:,j,:] = transform(Final_Cell_PCA,Before_Data[:,j,:]')'

    end
    Plot_Legend = true
    Colours = [:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
    if Single_Plot == false
        j=1
        abrada = scatter((Transformed_Data[1,j,1],Transformed_Data[1,j,2]),xlabel="PCA 1",ylabel="PCA 2",title="Final Cell States in PCA landscape",c=Colours[j],alpha=0.4,legend=Plot_Legend,label="Initial Noise = " * string(Large_Pert_Radius[j]))
        for j=2:Amount_Large_Perturbations
            abrada = scatter!((Transformed_Data[1,j,1],Transformed_Data[1,j,2]),c=Colours[j],alpha=0.4,legend=Plot_Legend,label="Initial Noise = " * string(Large_Pert_Radius[j]))
        end
        for j=1:Amount_Large_Perturbations
            for l=2:length(Transformed_Data[:,j,1])
                for j=1:Amount_Large_Perturbations
                    abrada = scatter!((Transformed_Data[l,j,1],Transformed_Data[l,j,2]),c=Colours[j],alpha=0.4,label=false)
                end
            end
        end
        savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/PCA_Maps/Scatter_PCA"*Copy_String*".png")

        KernelData = reshape(Transformed_Data, (Amount_Small_Perturbations*2^Splits*Amount_Large_Perturbations, 2))
        
        k = kde((KernelData[:,1], KernelData[:,2]))


        display(contourf(k,
             c = :vik,title="Final Cell State PCA Heatmap",
             linewidth = 0))
        savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/PCA_Maps/Heatmap_"*Copy_String*".png")

        
    else
        j = Single_Plot
        abrada = scatter((Transformed_Data[:,j,1],Transformed_Data[:,j,2]),xlabel="PCA 1",ylabel="PCA 2",title="Final Cell States in PCA landscape",c=Colours[j],alpha=0.4,legend=Plot_Legend,label="Initial Noise = " * string(Large_Pert_Radius[j]))
        
    end
    return abrada
end



function Multi_One_Generational_Plotter4(Cut_Off,Duration,Length_Orbits,Opacity,Name,Amount_Perturbations,Partial,N_Perturbed,Orbit_Array,Final_Epigenetic_Factors,Gene_Colour_Collection)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1:N_Perturbed*Amount_Perturbations

            if Shortest>size(Orbit_Array[i])[2]
                Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
            end
            Total_Length += size(Orbit_Array[i])[2]
        
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed*Amount_Perturbations)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1:N_Perturbed*Amount_Perturbations
        
        if Cut_Off

            for p=0:2^Splits-1

                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
            end
        else
            Length_Orbits =  size(Orbit_Array[i,j])[2]
            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
            end
        end

        Collection_Index += Length_Orbits*2^Splits

    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,Amount_Perturbations,N_Perturbed,2^Splits,3,Shortest)
    PCA_Orbits_Partial = zeros(Float32,Amount_Perturbations,Int(N_Perturbed/Partial),2^Splits,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:N_Perturbed
        for j=0:Amount_Perturbations-1
        for tau=1:2^Splits
            tester = Orbit_Array[i+N_Perturbed*j][2+(tau-1)*M:1+tau*M,1:Shortest]
            Transformed = transform(Orbital_Model,tester)
            PCA_Orbits_2Gen[j+1,i,tau,:,:] = Transformed
            if mod(i-1,Partial) == 0
                PCA_Orbits_Partial[j+1,(Int(i/Partial)),tau,:,:] = Transformed
            end
        end
        end

    end
    Colours = [:red,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
     
    #PCA_Orbits_Partial = reshape(PCA_Orbits_Partial,(Int(2^Splits*N_Perturbed*Amount_Perturbations/Partial),3,Shortest))
    PCA_Orbits_2Gen_Plot = permutedims(PCA_Orbits_Partial,(5,4,3,2,1))
    Orbit_Range = Int(N_Perturbed/Partial)*2^Splits
    
    print(size(PCA_Orbits_2Gen_Plot))
    if false
        for Gene_Colour in Gene_Colour_Collection
            tick()
            
            anim = @animate for k=1:Int(Duration/5)
                i=1
                PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1,1],c=Colours[Final_Epigenetic_Factors[1,1,1,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                for q=1:Int(N_Perturbed/Partial)
                    for p=1:2^Splits
                        for t =1:Amount_Perturbations
                        PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,q,t],PCA_Orbits_2Gen_Plot[1:k,2,p,q,t],PCA_Orbits_2Gen_Plot[1:k,3,p,q,t],c=Colours[Final_Epigenetic_Factors[t,q,p,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                        end
                    end
                end
        
                #for i=1:Int(N_Perturbed/5)
                #    for j=1:Amount_Perturbations
                #        for tau=1:2^Splits
                #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                #        end
                #    end
                #end
                
                #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])
        
                title!("Gene = $Gene_Colour Time = $k")
            end every 5
            tock()
            tick()
            animlong = @animate for k=1:Duration
                
                i=1
                PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1,1],c=Colours[Final_Epigenetic_Factors[1,1,1,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                for q=1:Int(N_Perturbed/Partial)
                    for p=1:2^Splits
                        for t =1:Amount_Perturbations
                        PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,q,t],PCA_Orbits_2Gen_Plot[1:k,2,p,q,t],PCA_Orbits_2Gen_Plot[1:k,3,p,q,t],c=Colours[Final_Epigenetic_Factors[t,q,p,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                        end
                    end
                end
                #for i=1:Int(N_Perturbed/5)
                #    for j=1:Amount_Perturbations
                #        for tau=1:2^Splits
                #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                #        end
                #    end
                #end
                
                #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])
        
                title!("Gene = $Gene_Colour Time = $k")
            end every 50
            tock()
    
            abra = gif(anim, Name[1:end-4]*"_Gene:"*string(Gene_Colour)*Name[end-3:end], fps = 10)
            display(abra)
        
            kadabra = gif(animlong, Name[1:end-4] * "_Long"*"_Gene:"*string(Gene_Colour) *Name[end-3:end], fps = 3)
            display(kadabra)
        
            
            Total_Ratio = principalratio(Orbital_Model)
            Eigen_Values = eigvals(Orbital_Model)
            Eigen_Values = Eigen_Values/sum(Eigen_Values)
            Eigen1 = Eigen_Values[1]
            Eigen2 = Eigen_Values[2]
            Eigen3 = Eigen_Values[3]
            PCAPlots_Final = scatter(PCA_Orbits_2Gen_Plot[end,1,1,1,:],PCA_Orbits_2Gen_Plot[end,2,1,1,:],PCA_Orbits_2Gen_Plot[end,3,1,1,:],c=Colours[Final_Epigenetic_Factors[1,1,1,Gene_Colour]+1],legend=true,label="
            Principal 1: $Eigen1
            Principal 2: $Eigen2]
            Principal 3: $Eigen3")
        
            for q=1:Int(N_Perturbed/Partial)
                for p=1:2^Splits
                    for t =1:Amount_Perturbations
                        PCAPlots_Final = scatter!((PCA_Orbits_2Gen_Plot[end,1,p,q,t],PCA_Orbits_2Gen_Plot[end,2,p,q,t],PCA_Orbits_2Gen_Plot[end,3,p,q,t]),c=Colours[Final_Epigenetic_Factors[t,q,p,Gene_Colour]+1],legend=true,label="")
                    end
                end
            end
            
            
            title!("Gene = $Gene_Colour , Ratio: $Total_Ratio")
            display(PCAPlots_Final)
            savefig(Name[1:end-4] * "_Final_Static_$Gene_Colour.png")
            
        end
    end
    Multi_Colours = [:cyan,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:darkmagenta,:lightcyan4,:olive,:gold4,:lightsalmon,:navajowhite,:pink]
    Only3Types = false
    tick()
    anim = @animate for k=1:Int(Duration/5)
        
        i=1
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
        if Only3Types
            Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
        end
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1,1],c=Multi_Colours[Colour_Index],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for q=1:Int(N_Perturbed/Partial)
            for p=1:2^Splits
                for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,q,t],PCA_Orbits_2Gen_Plot[1:k,2,p,q,t],PCA_Orbits_2Gen_Plot[1:k,3,p,q,t],c=Multi_Colours[Colour_Index],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                end
            end
        end
    
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])
    
        title!("Gene = Multiple Time = $k")
    end every 5
    tock()
    tick()
    animlong = @animate for k=1:2*Duration
        
        i=1
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
        if Only3Types
            Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
        end
        PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1,1],c=Multi_Colours[Colour_Index],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        for q=1:Int(N_Perturbed/Partial)
            for p=1:2^Splits
                for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,q,t],PCA_Orbits_2Gen_Plot[1:k,2,p,q,t],PCA_Orbits_2Gen_Plot[1:k,3,p,q,t],c=Multi_Colours[Colour_Index],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                end
            end
        end
        #for i=1:Int(N_Perturbed/5)
        #    for j=1:Amount_Perturbations
        #        for tau=1:2^Splits
        #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
        #        end
        #    end
        #end
        
        #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])
    
        title!("Gene = Multiple Time = $k")
    end every 50
    tock()
    abra = gif(anim, Name[1:end-4]*Name[end-3:end], fps = 10)
    display(abra)
    
    kadabra = gif(animlong, Name[1:end-4]*Name[end-3:end], fps = 3)
    display(kadabra)
    
    
    
    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    PCAPlots_Final = scatter(PCA_Orbits_2Gen_Plot[end,1,1,1,:],PCA_Orbits_2Gen_Plot[end,2,1,1,:],PCA_Orbits_2Gen_Plot[end,3,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Final = scatter!((PCA_Orbits_2Gen_Plot[end,1,p,q,t],PCA_Orbits_2Gen_Plot[end,2,p,q,t],PCA_Orbits_2Gen_Plot[end,3,p,q,t]),c=Multi_Colours[Colour_Index],legend=false)
            end
        end
    end
    Total_Ratio = principalratio(Orbital_Model)
    Eigen_Values = eigvals(Orbital_Model)
    Eigen_Values = Eigen_Values/sum(Eigen_Values)
    title!("Static Multi Colour, Ratio: $Total_Ratio")
    display(PCAPlots_Final)
    savefig(Name[1:end-4] * "Final_Multi_Colour.png")



    
    PcaPlots_Time_Length = 500
    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    
    PCAPlots_Time1 = plot(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,1,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Time1 = plot!(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,1,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA 1")
            end
        end
    end
    display(PCAPlots_Time1)

    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    
    PCAPlots_Time2 = plot(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,2,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Time2 = plot!(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,2,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA 1")
            end
        end
    end
    display(PCAPlots_Time2)

    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    
    PCAPlots_Time3 = plot(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,3,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Time3 = plot!(PCA_Orbits_2Gen_Plot[1:PcaPlots_Time_Length,3,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA 1")
            end
        end
    end
    display(PCAPlots_Time3)
    
end

function Evolution_NoEpi(du,u,Parameters,Time)
    "Function used to define the ODE, needs to differentiate the boundaries and splits for diffusion.
    Input:
    du         = Used by ODESolver to calculate step, don't need to give input
    u          = These are our x's for the dx = f(x) solving. Basically the parameters that change
    Parameters = These are constants we have to give for our solver, it's a tuple consisting of J_ij's, C_i's, Diffusion, Splits
    Time       = This determines for how long the ODE will run
    Output is done through ODESolver, not this function
    "
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    
    
    
    #⊗_(Wi, bi) = Wi .* bi
       

    du[1,:] = tanh.(40*(InvSqr*ParameterMatrix*u[1,:].+u[2,:].+CArray[:])) .- u[1,:]
    
    #Epigenetic Evolution, no need to worry about boundaries here so simple loop      

    
end




##############################################################################################################################
##############################################################################################################################
##############################################################################################################################








#Load_Generations = [70]
#Load_Generations = vcat(1,Load_Generations) #These are all the generations that we will analyse
Copies = [1] #Can reduce this to just 1,2 or just 1 if we don't want as many copies



#Main variables to change:
Stochastic_Noise = true
N::Int32 = 8 #How many copies of same network we have
#K::Int32 = N  Depricated parameter, do not change, if ever K not defined replace it by N
P = 4 #How many different networks will we use in the evolution run
Generations = 6 #How many generations in the evolution run

#DO NOT CHANGE THESE
Splits::Int32 = 6 #DO NOT CHANGE
SplitTime::Float32 = 0.1 #Time before a splitting event happens, DOES NOT MATTER HERE
M::Int32 = 40 #This is the amount of genes we want in our network
Splitting_Strength = 0.0f0

#CAREFUL WITH M>127 DO NOT INCREASE WITHOUT REWRITING DIFFERENTIATION FUNCTION
#BIT OVERFLOW!!!!!


#Constant and plotting tools


InvSqr::Float32 = 1/sqrt(M)
PCA_Plotting = false 
Orbit_Plotting = true  #Currently not used yet
Epi_Plotting = false #Currently not used yet
Counter = 0 #Used to randomise the mutations, leave to keep same seed
Saved_Generations = [2,4] #These are the generations where we want to save some orbits for


#Diffusion or mean field parameters
Mean_Field = false
Diffusing_System = false
KArray = zeros(Float32,M,1) #Diffusion/Mean Field array is set to zero everywhere
#KArray[1:10] .= 0.1 #Here we can manually adjust which genes will have diffusion















Amount_Large_Perturbations = 1 #How many large perturbations we will have each time
Amount_Small_Perturbations = 1 #How many smaller perturbations we will have around our large perturbation
Large_Pert_Radius = [0,1,5,20]
Small_Pert_Radius = 0
#Perturbations = zeros(32)









Generation_String = string(Load_Generations[1])
Copy_String = string(Copies[1])
Load_Data_String_Load = Data_To_Analyse
#Load_Data_String = Data_To_Analyse
#Copies = [1]
#Load_Generations = [10]
Orbit_Array = 0
Time_Orbital_Collection2 = 0
Orbit_Collection = 0
EpiThroughTime = 0

#To have access after the loop
Interpolated_Orbits = 0
Final_Epigenetic_Factors = 0
Gene_Cells_Colour = 0

@showprogress 1 for zeta in Load_Generations
    GC.gc()
    for iota in Copies
        Generation_String = string(zeta)

        Copy_String = string(iota)
        Current_Network_String = "/Gen_"*Generation_String*"_"*Copy_String*".jld"
        Loaded_Data = load(Load_Data_String_Load*Current_Network_String)
        mkpath(Load_Data_String*"/Processed/Gen:"*Generation_String*"/PCA_Maps")
        mkpath(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Orbits")
        mkpath(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance")


        #Here we load and perturb the networks


        Duration = 1000 #For Animation should be 3000
        Length_Orbits = 3000 #For PCA should be 4000
        Opacity = 75
        Partial = 1

        Variance_Length = 2500
        Times = [i for i=1:Variance_Length+50]
        Times_Orbit = [i for i=1:Length_Orbits]


        Perturb_Seed = 12345
        rng=MersenneTwister(Perturb_Seed)
        #Amount_Large_Perturbations = 1 #How many large perturbations we will have each time
        #Amount_Small_Perturbations = 16 #How many smaller perturbations we will have around our large perturbation
        #Large_Pert_Radius = 10
        #Small_Pert_Radius = 3
        CArray_Collection = zeros(Float32,Amount_Large_Perturbations,M)
        PositionArray_Collection = zeros(Float32,Amount_Large_Perturbations,Amount_Small_Perturbations,2,1,M)
        ParameterMatrix_Collection = zeros(Int32,Amount_Large_Perturbations,M,M)
        Final_Epigenetic_Factors = zeros(Int8,Amount_Large_Perturbations,Amount_Small_Perturbations,2^(Splits),M)
        Epi_Factors = zeros(Float16,2^(Splits),M)
        for j=1:Amount_Large_Perturbations
            CArray_Collection[j,:] = Loaded_Data["CArray"]
            ParameterMatrix_Collection[j,:,:] = Loaded_Data["Network"]

            #Write large pert vector here

            Noise_Vector = randn(rng,M).-0.5
            Norm = sqrt(sum(Noise_Vector.^2))
            Noise_Vector = Large_Pert_Radius[j]*Noise_Vector/Norm
            for i=1:Amount_Small_Perturbations


                #Small pert vector here
                Noise_Vector2 = randn(rng,M).-0.5
                Norm = sqrt(sum(Noise_Vector2.^2))
                Noise_Vector2 = rand()^(1/M)*Small_Pert_Radius*Noise_Vector2/Norm

                PositionArray_Collection[j,i,1,1,:] = Loaded_Data["PositionArray"] + Noise_Vector' + Noise_Vector2'

            end
        end

        Evolutionary_Complexity = []


        Random_Noise = 574 #seed for splitting noise

        #Need to define outside for loop, otherwise it will be destroyed afterwards
        Genes = 0





        KLDistance_temp = 0
        BitDistance_temp =  0
        Evolutionary_Complexity = []
        Indices = 0
        Information = zeros(1)
        Ratio = zeros(1)
        Cell_Type_Tracker = []
        Cell_Collection = []
        Evolutionary_Complexity = zeros(N)
        Temporary = []
        Orbit_Array = Array{Matrix{Float16}}(undef,Amount_Large_Perturbations*Amount_Small_Perturbations)
        Interpolated_Orbits = Array{Matrix{Float16}}(undef,Amount_Large_Perturbations*Amount_Small_Perturbations)

        Time_Orbital_Collection = 0

        for yeta=1:Amount_Large_Perturbations
            for x=1:Amount_Small_Perturbations
                #CArray = CArray_Collection[x,:]
                #ParameterMatrix = ParameterMatrix_Collection[x,:,:]
                #PositionArray = PositionArray_Collection[x,:]
                Epi_Factors,Random_Noise,Temporary, EpiThroughTime = Running_Network(CArray_Collection[yeta,:],ParameterMatrix_Collection[yeta,:,:],PositionArray_Collection[yeta,x,:,:,:],Random_Noise)
                Final_Epigenetic_Factors[yeta,x,:,:] = signbit.(Epi_Factors)
                Orbit_Array[x+(yeta-1)*Amount_Small_Perturbations] = Temporary

            end

        end
        #Indices,AllIndices,Evolutionary_Complexity[:],Cell_Collection, Cell_Type_Tracker,KLDistance_temp,BitDistance_temp = Scoring_Indexes(N,Final_Epigenetic_Factors,Cell_Type_Tracker)


        #HERE WE DO THE VARIANCE DATA ANALYSIS

        Orbit_Collection = Orbit_Array

        #Collect all orbits in function of time,


        #Maybe apply PCA to these orbits or keep them like this
        Time_Orbital_Collection = zeros(Float16,length(Times),Amount_Small_Perturbations*2^Splits*Amount_Large_Perturbations,M)
        Time_Orbital_Collection2 = zeros(Float16,length(Times),Amount_Large_Perturbations,Amount_Small_Perturbations*2^Splits,M)



        Orbit_Counter=0
        Larger_Orbit_Counter = 1
        for Orbits in Orbit_Collection
            #Here we do it for the variance
            Interpolated_Vector = Interpolator(Times,Orbits) #We do linear interpolation to have all the same timesteps
            for i=1:2^Splits #We put the different splitted cells all together with the same networks.
                Time_Orbital_Collection[:,Orbit_Counter*2^Splits+i,:] = Interpolated_Vector[:,2+(i-1)*M:1+i*M]
            end
            for k=1:2^Splits
                Time_Orbital_Collection2[:,Larger_Orbit_Counter,mod(Orbit_Counter,Amount_Small_Perturbations)+k,:] = Interpolated_Vector[:,2+M*(k-1):1+M*k]
            end 
            Orbit_Counter += 1
            if mod(Orbit_Counter,Amount_Small_Perturbations) == 0
                Larger_Orbit_Counter += 1
            end


            #Here we do it for the orbits
            Interpolated_Vector2 = Interpolator(Times_Orbit,Orbits)
            Interpolated_Orbits[Orbit_Counter] = permutedims(Interpolated_Vector2,[2,1])

        end
        #Calculate variance in gene expression in function of time



        Variance = zeros(Variance_Length)
        for i=1:Variance_Length
            Variance[i] = sum((Time_Orbital_Collection[i,:,:] .- mean(Time_Orbital_Collection[i,:,:],dims=1)).^2)/(M-1)
        end



        Spatial_Var = Coarsed_Variance(Variance_Length,Time_Orbital_Collection,10,Amount_Large_Perturbations*Amount_Small_Perturbations*2^Splits)
        aba = plot(Variance,label="Temporal",xlabel="Time",ylabel="Variance",title="Spatial and Temporal Variance of all Orbits")
        aba = plot!(Spatial_Var,label="Spatial")
        aba =vline!(SplitTime*[i for i=1:Splits],colour=:"red",linealpha=:0.3,label=false)
        display(aba)
        
        savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance/Spatial_Temporal_"*Copy_String*".png")
        #savefig("Data_All/Data_Folder/TestFile/TestImage.png")



        aca = plot(Variance[1:250],label="Temporal",xlabel="Time",ylabel="Variance",title="Spatial and Temporal Variance of all Orbits")
        aca = plot!(Spatial_Var[1:250],label="Spatial")
        aca =vline!(SplitTime*[i for i=1:Splits],colour=:"red",linealpha=:0.3,label=false)
        ylims!(0,max(maximum(Variance[3:250]),maximum(Spatial_Var[3:250]))*1.3)

        display(aca)
        
        savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance/Spatial_Temporal_Zoomed_"*Copy_String*".png")


        if false

            Seperate_Variance = zeros(Amount_Large_Perturbations,Variance_Length)
            for j=1:Amount_Large_Perturbations
                for i=1:Variance_Length
                    Seperate_Variance[j,i] = sum((Time_Orbital_Collection2[i,j,:,:] .- mean(Time_Orbital_Collection2[i,j,:,:],dims=1)).^2)/(M-1)
                end

            end

            ada = plot(Seperate_Variance[1,1:250],label=false,xlabel="Time",ylabel="Variance",title="Temporal Variance of Specific I.C.")
            for j=2:Amount_Large_Perturbations
                ada = plot!(Seperate_Variance[j,1:250],label=false)
            end
            display(ada)

            savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance/Temporal_Zoomed_IC_"*Copy_String*".png")


            aea = plot(Seperate_Variance[1,1:2500],label=false,xlabel="Time",ylabel="Variance",title="Temporal Variance of Specific I.C.")
            for j=2:Amount_Large_Perturbations
                aea = plot!(Seperate_Variance[j,1:2500],label=false)
            end
            display(aea)

            savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance/Temporal_IC_"*Copy_String*".png")

            Spatial_Sep = Coarsed_Variance(Variance_Length,Time_Orbital_Collection[:,1:Amount_Small_Perturbations*2^Splits,:],25,Amount_Small_Perturbations*2^Splits)
            afa = plot(Spatial_Sep[1:250],label=false,xlabel="Time",ylabel="Variance",title="Spatial Variance of Specific I.C.")
            for j=1:Amount_Large_Perturbations-1
                Spatial_Sep = Coarsed_Variance(Variance_Length,Time_Orbital_Collection[:,1+j*Amount_Small_Perturbations*2^Splits:(j+1)*Amount_Small_Perturbations*2^Splits,:],25,Amount_Small_Perturbations*2^Splits)

                afa = plot!(Spatial_Sep[1:250],label=false)
            end

            display(afa)

            savefig(Load_Data_String*"/Processed/Gen:"*Generation_String*"/Variance/Spatial_Zoomed_IC_"*Copy_String*".png")
        end




        #Perturbations = [0 for i=1:32]
        for kappa=1:Amount_Large_Perturbations
            println(kappa)
            display(Final_Cell_PCAScatterPlot(Orbit_Array,kappa)) #Don't really need this one
        end
        
        display(Final_Cell_PCAScatterPlot(Orbit_Array,false));
        
        Gene_Cells_Colour = [1,2,3,4]
        print("Before the Multi Big One")
        GC.gc()
        display(Multi_One_Generational_Plotter4(true,Duration,Length_Orbits,Opacity,Load_Data_String*"/Processed/Gen:"*Generation_String*"/Orbits/Genes_"*Copy_String*".gif",Amount_Large_Perturbations,Partial,Amount_Small_Perturbations,Interpolated_Orbits,Final_Epigenetic_Factors,Gene_Cells_Colour));
    






        Copy_String = string(1)
        Current_Network_String = "/Gen_"*Generation_String*"_"*Copy_String*".jld"
        Loaded_Data = load(Load_Data_String_Load*Current_Network_String)
        
        CArray = Loaded_Data["CArray"]
        ParameterMatrix = Loaded_Data["Network"]
        

        
        TimesWeCheck = [1,5,10,25,40,60,80,100,150,200,300,500,1000,4000,20000]
        TimeChecks = length(TimesWeCheck)
        Cell_Gatherings = zeros(TimeChecks,2,2^Splits,M)
        CycleAmount = zeros(TimeChecks)
        UniqueCycleAmount = zeros(TimeChecks)
        FixPointAmount = zeros(TimeChecks)
        
        Interpolated_Cells = Interpolator(TimesWeCheck,Orbit_Array[1])
        Interpolated_EPIs = Interpolator(TimesWeCheck,EpiThroughTime)
        for i=1:2^Splits
            Cell_Gatherings[:,1,i,:] = Interpolated_Cells[:,2+(i-1)*M:1+(i)*M]
            Cell_Gatherings[:,2,i,:] = Interpolated_EPIs[:,2+(i-1)*M:1+(i)*M]
        end
        
        
        Copy_String = string(1)
        Current_Network_String = "/Gen_"*Generation_String*"_"*Copy_String*".jld"
        Loaded_Data = load(Load_Data_String_Load*Current_Network_String)
        
        CArray = Loaded_Data["CArray"]
        ParameterMatrix = Loaded_Data["Network"]
        FixPoint_Positions = Array{Float64}(undef,0)
        Counter_Cycle = 0
        XPositionTol = 0
        
        
        
        
        for t = 1:TimeChecks
            Counter_Cycle = 0
            FixPoint_Positions = Array{Float64}(undef,0)
            VectorTest = []
            for k = 1:2^Splits
                PositionArray = Cell_Gatherings[t,:,k,:]
                
                Parameters = (ParameterMatrix, CArray) #We have to update the splits every rerun, rest stays same
                tspan = (Float32(0.0),100) #How long ODE will run
                
                prob = ODEProblem(Evolution_NoEpi,PositionArray,tspan,Parameters) #ODEdefining
                
                
                
                
                sol = solve(prob,BS3())
                aba = plot(sol.t,sol[1,1,:])
                for j=2:40
                    aba = plot!(sol.t,sol[1,j,:],legend=false,title=k)
                end
                #if t> 10
                #    display(aba)
                #end
                
                
                
                
                
                
                
                XVector = sol[1,:,end]
                EpiVector = sol[2,:,end]
                SumOfDerivates = sum(abs.(tanh.(40*(InvSqr*ParameterMatrix*XVector.+EpiVector.+CArray[:])) .- XVector))
                DerivativeTol = 0.1
                XPositionTol = 0.1
                
                #if t> 10   
                    #print(SumOfDerivates)
                #end
                
                if SumOfDerivates < DerivativeTol
                    if FixPoint_Positions == [] #Still empty
                        FixPoint_Positions = vcat(FixPoint_Positions,XVector)
                    else #Already has a point in it
                        NewFixPoint = true
                        #print("The distance to the fixpoints are:")
                        for i=1:length(FixPoint_Positions[1,:]) #Here we check over all the existing ones to see if it is not close to one already existing
                            #println(sqrt(sum((FixPoint_Positions[:,i] .- XVector).^2)))
                            if sqrt(sum((FixPoint_Positions[:,i] .- XVector).^2)) < XPositionTol
                                NewFixPoint = false
                            end    
                        end
                        if NewFixPoint #If it is a new fixpoint we add it in now
                            FixPoint_Positions = hcat(FixPoint_Positions,XVector)
                        end
                    end
                else
                    #Exception, we have a cycle? try to plot it
                    
                    CycleToleranceActive = 0.01
                    Active_Gene = zeros(M)
                    
                    prob2 = ODEProblem(Evolution_NoEpi,sol[:,:,end],(Float32(0.0),20),Parameters) #ODEdefining
                    sol2 = solve(prob2,BS3())
                    
                    for j=1:M
                        if abs(maximum(sol2[1,j,:].-mean(sol2[1,j,:]))) > CycleToleranceActive
                            Active_Gene[j] = 1
                        end
                    end
                    if VectorTest == []
                        VectorTest = vcat(VectorTest,Active_Gene)
                    else
                        New_Cycle = true
                        for tau in length(VectorTest[1,:])
                            if Active_Gene == VectorTest[:,tau]
                                New_Cycle = false
                            end
                        end
                        if New_Cycle
                            VectorTest = hcat(VectorTest,Active_Gene)
                        end
                    end
        
        
        
                    
                    Counter_Cycle += 1
                end
            end
            try FixPointAmount[t] = length(FixPoint_Positions[1,:])
        
            catch 
                FixPointAmount[t] = 0
            else
                FixPointAmount[t] = length(FixPoint_Positions[1,:])
            end
            CycleAmount[t] = Counter_Cycle
            try UniqueCycleAmount[t] = length(VectorTest[1,:])
        
            catch
                UniqueCycleAmount[t] = 0
            else
                UniqueCycleAmount[t] = length(VectorTest[1,:])
            end
        end
        
        aba = plot(TimesWeCheck,FixPointAmount,label="Fix Points")
        aba = plot!(TimesWeCheck,CycleAmount,label="Cycles (DoubleCounted)",xaxis=:log,title="PositionalTolerance="*string(XPositionTol))
        plot!(TimesWeCheck,UniqueCycleAmount,label="Cycles (Unique)")
        display(aba)

    end
end
    





function Multi_One_Generational_Plotter4(Cut_Off,Duration,Length_Orbits,Opacity,Name,Amount_Perturbations,Partial,N_Perturbed,Orbit_Array,Final_Epigenetic_Factors,Gene_Colour_Collection)
    Total_Length = 0
    Shortest = size(Orbit_Array[1,1])[2]
    
    
    #In this option we take all the lengths and concatenate them, however this
    #may create a bias in the PCA analysis towards only final cell states
    for i=1:N_Perturbed*Amount_Perturbations

            if Shortest>size(Orbit_Array[i])[2]
                Shortest = size(Orbit_Array[i])[2] #Used later to check which data set is the shortest when plotting
            end
            Total_Length += size(Orbit_Array[i])[2]
        
    end
    if Cut_Off
        PCA_Initial_Orbit_Array = zeros(Float32,M,Length_Orbits*2^Splits*N_Perturbed*Amount_Perturbations)
    else
        PCA_Initial_Orbit_Array = zeros(Float32,M,Total_Length*2^Splits)
    end
    Collection_Index=0
    
    
    for i=1:N_Perturbed*Amount_Perturbations
        
        if Cut_Off

            for p=0:2^Splits-1

                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,1:Length_Orbits]
            end
        else
            Length_Orbits =  size(Orbit_Array[i,j])[2]
            for p=0:2^Splits-1
                PCA_Initial_Orbit_Array[:,Collection_Index+1+p*Length_Orbits:Collection_Index+(p+1)*Length_Orbits] = Orbit_Array[i][2+p*M:1+(p+1)*M,:]
            end
        end

        Collection_Index += Length_Orbits*2^Splits

    end


    Orbital_Model= fit(PCA,PCA_Initial_Orbit_Array;maxoutdim=3)
    
    PCA_Initial_Orbit_Array = [] #Emptied to save space
    PCA_Orbits_2Gen = zeros(Float32,Amount_Perturbations,N_Perturbed,2^Splits,3,Shortest)
    PCA_Orbits_Partial = zeros(Float32,Amount_Perturbations,Int(N_Perturbed/Partial),2^Splits,3,Shortest)
    print(" ")
    print("The Principalratio is: ")
    print(principalratio(Orbital_Model))
    for i=1:N_Perturbed
        for j=0:Amount_Perturbations-1
        for tau=1:2^Splits
            tester = Orbit_Array[i+N_Perturbed*j][2+(tau-1)*M:1+tau*M,1:Shortest]
            Transformed = transform(Orbital_Model,tester)
            PCA_Orbits_2Gen[j+1,i,tau,:,:] = Transformed
            if mod(i-1,Partial) == 0
                PCA_Orbits_Partial[j+1,(Int(i/Partial)),tau,:,:] = Transformed
            end
        end
        end

    end
    Colours = [:red,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1,:purple,:cyan,:green,:orange,:red,:brown,:black,:maroon1]
     
    #PCA_Orbits_Partial = reshape(PCA_Orbits_Partial,(Int(2^Splits*N_Perturbed*Amount_Perturbations/Partial),3,Shortest))
    PCA_Orbits_2Gen_Plot = permutedims(PCA_Orbits_Partial,(5,4,3,2,1))
    Orbit_Range = Int(N_Perturbed/Partial)*2^Splits
    
    print(size(PCA_Orbits_2Gen_Plot))
    if false
        for Gene_Colour in Gene_Colour_Collection
            tick()
            
            anim = @animate for k=1:Int(Duration/5)
                i=1
                PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1,1],c=Colours[Final_Epigenetic_Factors[1,1,1,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                for q=1:Int(N_Perturbed/Partial)
                    for p=1:2^Splits
                        for t =1:Amount_Perturbations
                        PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,q,t],PCA_Orbits_2Gen_Plot[1:k,2,p,q,t],PCA_Orbits_2Gen_Plot[1:k,3,p,q,t],c=Colours[Final_Epigenetic_Factors[t,q,p,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                        end
                    end
                end
        
                #for i=1:Int(N_Perturbed/5)
                #    for j=1:Amount_Perturbations
                #        for tau=1:2^Splits
                #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                #        end
                #    end
                #end
                
                #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])
        
                title!("Gene = $Gene_Colour Time = $k")
            end every 5
            tock()
            tick()
            animlong = @animate for k=1:Duration
                
                i=1
                PCAPlots = plot(PCA_Orbits_2Gen_Plot[1:k,1,1,1,1],PCA_Orbits_2Gen_Plot[1:k,2,1,1,1],PCA_Orbits_2Gen_Plot[1:k,3,1,1,1],c=Colours[Final_Epigenetic_Factors[1,1,1,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                for q=1:Int(N_Perturbed/Partial)
                    for p=1:2^Splits
                        for t =1:Amount_Perturbations
                        PCAPlots = plot!(PCA_Orbits_2Gen_Plot[1:k,1,p,q,t],PCA_Orbits_2Gen_Plot[1:k,2,p,q,t],PCA_Orbits_2Gen_Plot[1:k,3,p,q,t],c=Colours[Final_Epigenetic_Factors[t,q,p,Gene_Colour]+1],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                        end
                    end
                end
                #for i=1:Int(N_Perturbed/5)
                #    for j=1:Amount_Perturbations
                #        for tau=1:2^Splits
                #            PCAPlots = plot!(PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,1,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,2,1:k],PCA_Orbits_2Gen[i+(j-1)*N_Perturbed,tau,3,1:k],c=Colours[j],legend=false,alpha = 0.05*max.((1:k) .+ Opacity .- k, 0) / Opacity)
                #        end
                #    end
                #end
                
                #annotate!([(PCA_Orbits_2Gen[1,1,1,1],PCA_Orbits_2Gen[1,1,2,1],PCA_Orbits_2Gen[1,1,3,1],"Time = $k" )])
        
                title!("Gene = $Gene_Colour Time = $k")
            end every 50
            tock()
    
            abra = gif(anim, Name[1:end-4]*"_Gene:"*string(Gene_Colour)*Name[end-3:end], fps = 10)
            display(abra)
        
            kadabra = gif(animlong, Name[1:end-4] * "_Long"*"_Gene:"*string(Gene_Colour) *Name[end-3:end], fps = 3)
            display(kadabra)
        
            
            Total_Ratio = principalratio(Orbital_Model)
            Eigen_Values = eigvals(Orbital_Model)
            Eigen_Values = Eigen_Values/sum(Eigen_Values)
            Eigen1 = Eigen_Values[1]
            Eigen2 = Eigen_Values[2]
            Eigen3 = Eigen_Values[3]
            PCAPlots_Final = scatter(PCA_Orbits_2Gen_Plot[end,1,1,1,:],PCA_Orbits_2Gen_Plot[end,2,1,1,:],PCA_Orbits_2Gen_Plot[end,3,1,1,:],c=Colours[Final_Epigenetic_Factors[1,1,1,Gene_Colour]+1],legend=true,label="
            Principal 1: $Eigen1
            Principal 2: $Eigen2]
            Principal 3: $Eigen3")
        
            for q=1:Int(N_Perturbed/Partial)
                for p=1:2^Splits
                    for t =1:Amount_Perturbations
                        PCAPlots_Final = scatter!((PCA_Orbits_2Gen_Plot[end,1,p,q,t],PCA_Orbits_2Gen_Plot[end,2,p,q,t],PCA_Orbits_2Gen_Plot[end,3,p,q,t]),c=Colours[Final_Epigenetic_Factors[t,q,p,Gene_Colour]+1],legend=true,label="")
                    end
                end
            end
            
            
            title!("Gene = $Gene_Colour , Ratio: $Total_Ratio")
            display(PCAPlots_Final)
            savefig(Name[1:end-4] * "_Final_Static_$Gene_Colour.png")
            
        end
    end
    Multi_Colours = [:cyan,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:darkmagenta,:lightcyan4,:olive,:gold4,:lightsalmon,:navajowhite,:pink]
    Only3Types = false


    
    
    
    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    PCAPlots_Final = scatter(PCA_Orbits_2Gen_Plot[end,1,1,1,:],PCA_Orbits_2Gen_Plot[end,2,1,1,:],PCA_Orbits_2Gen_Plot[end,3,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Final = scatter!((PCA_Orbits_2Gen_Plot[end,1,p,q,t],PCA_Orbits_2Gen_Plot[end,2,p,q,t],PCA_Orbits_2Gen_Plot[end,3,p,q,t]),c=Multi_Colours[Colour_Index],legend=false)
            end
        end
    end
    Total_Ratio = principalratio(Orbital_Model)
    Eigen_Values = eigvals(Orbital_Model)
    Eigen_Values = Eigen_Values/sum(Eigen_Values)
    title!("Static Multi Colour, Ratio: $Total_Ratio")
    display(PCAPlots_Final)
    savefig(Name[1:end-4] * "Final_Multi_Colour.png")



    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    
    PCAPlots_Time1 = plot([PcaPlots_Time_Start:PcaPlots_Time_Length],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,1,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Time1 = plot!([PcaPlots_Time_Start:PcaPlots_Time_Length],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,1,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA 1")
            end
        end
    end
    display(PCAPlots_Time1)

    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    
    PCAPlots_Time2 = plot([PcaPlots_Time_Start:PcaPlots_Time_Length],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,2,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Time2 = plot!([PcaPlots_Time_Start:PcaPlots_Time_Length],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,2,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA 2")
            end
        end
    end
    display(PCAPlots_Time2)

    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end
    
    PCAPlots_Time3 = plot([PcaPlots_Time_Start:PcaPlots_Time_Length],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,3,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end
                PCAPlots_Time3 = plot!([PcaPlots_Time_Start:PcaPlots_Time_Length],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,3,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA 3")
            end
        end
    end
    display(PCAPlots_Time3)



    for Colouring =1:16
        Plotting_Something = false
        PCAPlots_TimeFilter = plot(zeros(PcaPlots_Time_Start:PcaPlots_Time_Length))
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
        if Only3Types
            Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
        end

        if Colour_Index == Colouring
            Plotting_Something = true
            PCAPlots_TimeFilter = plot!([PcaPlots_Time_Start:PcaPlots_Time_Length],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,SeperatePCAIndex,1,1,:],c=Multi_Colours[Colour_Index],legend=false)
        end
        for q=1:Int(N_Perturbed/Partial)
            for p=1:2^Splits
                for t =1:Amount_Perturbations
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                    if Only3Types
                        Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                    end
                    if Colour_Index == Colouring
                        Plotting_Something = true
                        PCAPlots_TimeFilter = plot!([PcaPlots_Time_Start:PcaPlots_Time_Length],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,SeperatePCAIndex,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA Colour Index = $(Colouring)")
                    end
                end
            end
        end
        if Plotting_Something
            display(PCAPlots_TimeFilter)
        end
        
    end

    PcaDim1 = 1
    PcaDim2 = 2
    PCAPlots_TimeFilter = plot([0])
    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end

    
    PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,1,1,:],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,1,1,:],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end

                PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,p,q,t],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,p,q,t],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
                
            end
        end
    end
    display(PCAPlots_TimeFilter)

    PcaDim1 = 2
    PcaDim2 = 3
    PCAPlots_TimeFilter = plot([0])
    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end

    
    PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,1,1,:],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,1,1,:],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end

                PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,p,q,t],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,p,q,t],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
                
            end
        end
    end
    display(PCAPlots_TimeFilter)

    PcaDim1 = 1
    PcaDim2 = 3
    PCAPlots_TimeFilter = plot([0])
    Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    if Only3Types
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
    end

    
    PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,1,1,:],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,1,1,:],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
    
    for q=1:Int(N_Perturbed/Partial)
        for p=1:2^Splits
            for t =1:Amount_Perturbations
                Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                if Only3Types
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                end

                PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,p,q,t],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,p,q,t],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
                
            end
        end
    end
    display(PCAPlots_TimeFilter)

    
    PcaDim1 = 1
    PcaDim2 = 2
    
    for Colouring =1:16
        Plotting_Something = false
        PCAPlots_TimeFilter = plot([0])
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
        if Only3Types
            Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
        end

        if Colour_Index == Colouring
            Plotting_Something = true
            PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,1,1,:],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,1,1,:],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
        end
        for q=1:Int(N_Perturbed/Partial)
            for p=1:2^Splits
                for t =1:Amount_Perturbations
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                    if Only3Types
                        Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                    end
                    if Colour_Index == Colouring
                        Plotting_Something = true
                        PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,p,q,t],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA Colour Index = $(Colouring)",opacity=0.1)
                    end
                end
            end
        end
        if Plotting_Something
            display(PCAPlots_TimeFilter)
        end
        
    end

    PcaDim1 = 2
    PcaDim2 = 3
    
    for Colouring =1:16
        Plotting_Something = false
        PCAPlots_TimeFilter = plot([0])
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
        if Only3Types
            Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
        end

        if Colour_Index == Colouring
            Plotting_Something = true
            PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,1,1,:],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,1,1,:],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
        end
        for q=1:Int(N_Perturbed/Partial)
            for p=1:2^Splits
                for t =1:Amount_Perturbations
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                    if Only3Types
                        Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                    end
                    if Colour_Index == Colouring
                        Plotting_Something = true
                        PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,p,q,t],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA Colour Index = $(Colouring)",opacity=0.1)
                    end
                end
            end
        end
        if Plotting_Something
            display(PCAPlots_TimeFilter)
        end
        
    end

    PcaDim1 = 1
    PcaDim2 = 3
    
    for Colouring =1:16
        Plotting_Something = false
        PCAPlots_TimeFilter = plot([0])
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
        if Only3Types
            Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+1
        end

        if Colour_Index == Colouring
            Plotting_Something = true
            PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,1,1,:],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,1,1,:],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
        end
        for q=1:Int(N_Perturbed/Partial)
            for p=1:2^Splits
                for t =1:Amount_Perturbations
                    Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+Final_Epigenetic_Factors[t,q,p,4]*8+1
                    if Only3Types
                        Colour_Index = Final_Epigenetic_Factors[t,q,p,1]+Final_Epigenetic_Factors[t,q,p,2]*2+Final_Epigenetic_Factors[t,q,p,3]*4+1
                    end
                    if Colour_Index == Colouring
                        Plotting_Something = true
                        PCAPlots_TimeFilter = plot!(PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim1,p,q,t],PCA_Orbits_2Gen_Plot[PcaPlots_Time_Start:PcaPlots_Time_Length,PcaDim2,p,q,t],c=Multi_Colours[Colour_Index],legend=false,title="PCA Colour Index = $(Colouring)",opacity=0.1)
                    end
                end
            end
        end
        if Plotting_Something
            display(PCAPlots_TimeFilter)
        end
        
    end

    return Orbital_Model
    
end


Duration = 1000 #For Animation should be 3000
Length_Orbits = 3000 #For PCA should be 4000
Opacity = 75
Partial = 1

Variance_Length = 2500
Times = [i for i=1:Variance_Length+50]
Times_Orbit = [i for i=1:Length_Orbits]


PcaPlots_Time_Start = 100
PcaPlots_Time_Length = 600
SeperatePCAIndex = 3

Orbital_Model = Multi_One_Generational_Plotter4(true,Duration,Length_Orbits,Opacity,Load_Data_String*"/Processed/Gen:"*Generation_String*"/Orbits/Genes_"*Copy_String*".gif",Amount_Large_Perturbations,Partial,Amount_Small_Perturbations,Interpolated_Orbits,Final_Epigenetic_Factors,Gene_Cells_Colour);


#TimesWeCheck = [1,5,10,25,40,60,80,100,150,200,300,500,1000,4000,20000]
TimesWeCheck = [50,100,150,200,300,400,500]
TimeChecks = length(TimesWeCheck)


Cell_Gatherings = zeros(TimeChecks,2,2^Splits,M)
CycleAmount = zeros(TimeChecks)
UniqueCycleAmount = zeros(TimeChecks)
FixPointAmount = zeros(TimeChecks)

Interpolated_Cells = Interpolator(TimesWeCheck,Orbit_Array[1])
Interpolated_EPIs = Interpolator(TimesWeCheck,EpiThroughTime)
for i=1:2^Splits
    Cell_Gatherings[:,1,i,:] = Interpolated_Cells[:,2+(i-1)*M:1+(i)*M]
    Cell_Gatherings[:,2,i,:] = Interpolated_EPIs[:,2+(i-1)*M:1+(i)*M]
end


Copy_String = string(1)
Current_Network_String = "/Gen_"*Generation_String*"_"*Copy_String*".jld"
Loaded_Data = load(Load_Data_String_Load*Current_Network_String)

CArray = Loaded_Data["CArray"]
ParameterMatrix = Loaded_Data["Network"]
FixPoint_Positions = Array{Float64}(undef,0)
Counter_Cycle = 0
XPositionTol = 0
Full_TimeSpan = 200
sol = 0
Sampling_Amounts = 4
Transformed_Data =  zeros(Float32,TimeChecks,2^Splits,3,Full_TimeSpan,Sampling_Amounts)

for q=1:Sampling_Amounts
    Saved_Orbits_All = zeros(Float16,TimeChecks,2^Splits,Full_TimeSpan,M)
    for t = 1:TimeChecks
        Counter_Cycle = 0
        FixPoint_Positions = Array{Float64}(undef,0)
        VectorTest = []
        for k = 1:2^Splits
            PositionArray = Cell_Gatherings[t,:,k,:]
            
            Parameters = (ParameterMatrix, CArray) #We have to update the splits every rerun, rest stays same
            tspan = (Float32(0.0),Full_TimeSpan) #How long ODE will run
            
            prob = SDEProblem(Evolution_NoEpi,σ_Genes,PositionArray,tspan,Parameters) #ODEdefining
            
            
            
            
            sol = solve(prob,SRA3(),dtmin=0.1,force_dtmin=true)
            if false 
                aba = plot(sol.t,sol[1,1,:])
                for j=2:40
                    aba = plot!(sol.t,sol[1,j,:],legend=false,title=TimesWeCheck[t])
                end
                if t> 1 && (k==1) 
                    
                    display(aba)
                end
            end
            for time=1:Full_TimeSpan
                Saved_Orbits_All[t,k,time,:] = sol(time)[1,:]
            end
            
            
            
            
            XVector = sol[1,:,end]
            EpiVector = sol[2,:,end]
            SumOfDerivates = sum(abs.(tanh.(40*(InvSqr*ParameterMatrix*XVector.+EpiVector.+CArray[:])) .- XVector))
            DerivativeTol = 0.1
            XPositionTol = 0.1
            
        end
            
    
    
    end

    for i=1:TimeChecks  
        for tau=1:2^Splits
            Tested_Transform = transform(Orbital_Model,Saved_Orbits_All[i,tau,:,:]')        
            Transformed_Data[i,tau,:,:,q] = Tested_Transform
        end
    end
    
end
## Save somehow the data/orbits of these frozen versions somewhere


# Plot this for the PCA as their axis


Multi_Colours = [:cyan,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:darkmagenta,:lightcyan4,:olive,:gold4,:lightsalmon,:navajowhite,:pink]


function PCA_Individual_Colour_Plotting(PcaDim1,PcaDim2,Time_Index_To_Plot)
    for Colouring =1:16
        Plotting_Something = false
        PCAPlots_TimeFilter = plot([0])
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    
    
        if Colour_Index == Colouring
            Plotting_Something = true
            PCAPlots_TimeFilter = plot!(Transformed_Data[Time_Index_To_Plot,1,PcaDim1,:,1],Transformed_Data[Time_Index_To_Plot,1,PcaDim2,:,1],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
        end
        for q=1:Sampling_Amounts
            for p=1:2^Splits
                for t =1:1
                    Colour_Index = Final_Epigenetic_Factors[t,1,p,1]+Final_Epigenetic_Factors[t,1,p,2]*2+Final_Epigenetic_Factors[t,1,p,3]*4+Final_Epigenetic_Factors[t,1,p,4]*8+1
    
                    if Colour_Index == Colouring
                        Plotting_Something = true
                        PCAPlots_TimeFilter = plot!(Transformed_Data[Time_Index_To_Plot,p,PcaDim1,:,q],Transformed_Data[Time_Index_To_Plot,p,PcaDim2,:,q],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1,title="Final Type Index $(Colouring) Time = $(TimesWeCheck[Time_Index_To_Plot])")
                    end
                end
            end
        end
        if Plotting_Something
            display(PCAPlots_TimeFilter)
        end
        
    end
end
PCADim1 = 1
PCADim2 = 2
for t in [1,4,5,6,7]
    PCA_Individual_Colour_Plotting(PCADim1,PCADim2,t)
end

#TimesWeCheck = [1,5,10,25,40,60,80,100,150,200,300,500,1000,4000,20000]
TimesWeCheck = [50,100,150,200,300,320,340,360,380,400,500,600,700,800,1000,1500,2000]
TimeChecks = length(TimesWeCheck)


Cell_Gatherings = zeros(TimeChecks,2,2^Splits,M)
CycleAmount = zeros(TimeChecks)
UniqueCycleAmount = zeros(TimeChecks)
FixPointAmount = zeros(TimeChecks)

Interpolated_Cells = Interpolator(TimesWeCheck,Orbit_Array[1])
Interpolated_EPIs = Interpolator(TimesWeCheck,EpiThroughTime)
for i=1:2^Splits
    Cell_Gatherings[:,1,i,:] = Interpolated_Cells[:,2+(i-1)*M:1+(i)*M]
    Cell_Gatherings[:,2,i,:] = Interpolated_EPIs[:,2+(i-1)*M:1+(i)*M]
end


Copy_String = string(1)
Current_Network_String = "/Gen_"*Generation_String*"_"*Copy_String*".jld"
Loaded_Data = load(Load_Data_String_Load*Current_Network_String)

CArray = Loaded_Data["CArray"]
ParameterMatrix = Loaded_Data["Network"]
FixPoint_Positions = Array{Float64}(undef,0)
Counter_Cycle = 0
XPositionTol = 0
Full_TimeSpan = 200
sol = 0
Sampling_Amounts = 8
Transformed_Data =  zeros(Float32,TimeChecks,2^Splits,3,Full_TimeSpan,Sampling_Amounts)

for q=1:Sampling_Amounts
    Saved_Orbits_All = zeros(Float16,TimeChecks,2^Splits,Full_TimeSpan,M)
    for t = 1:TimeChecks
        Counter_Cycle = 0
        FixPoint_Positions = Array{Float64}(undef,0)
        VectorTest = []
        for k = 1:2^Splits
            PositionArray = Cell_Gatherings[t,:,k,:]
            
            Parameters = (ParameterMatrix, CArray) #We have to update the splits every rerun, rest stays same
            tspan = (Float32(0.0),Full_TimeSpan) #How long ODE will run
            
            prob = SDEProblem(Evolution_NoEpi,σ_Genes,PositionArray,tspan,Parameters) #ODEdefining
            
            
            
            
            sol = solve(prob,SRA3(),dtmin=0.1,force_dtmin=true)
            if false 
                aba = plot(sol.t,sol[1,1,:])
                for j=2:40
                    aba = plot!(sol.t,sol[1,j,:],legend=false,title=TimesWeCheck[t])
                end
                if t> 1 && (k==1) 
                    
                    display(aba)
                end
            end
            for time=1:Full_TimeSpan
                Saved_Orbits_All[t,k,time,:] = sol(time)[1,:]
            end
            
            
            
            
            XVector = sol[1,:,end]
            EpiVector = sol[2,:,end]
            SumOfDerivates = sum(abs.(tanh.(40*(InvSqr*ParameterMatrix*XVector.+EpiVector.+CArray[:])) .- XVector))
            DerivativeTol = 0.1
            XPositionTol = 0.1
            
        end
            
    
    
    end

    for i=1:TimeChecks  
        for tau=1:2^Splits
            Tested_Transform = transform(Orbital_Model,Saved_Orbits_All[i,tau,:,:]')        
            Transformed_Data[i,tau,:,:,q] = Tested_Transform
        end
    end
    
end
## Save somehow the data/orbits of these frozen versions somewhere


# Plot this for the PCA as their axis

Multi_Colours = [:cyan,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:darkmagenta,:lightcyan4,:olive,:gold4,:lightsalmon,:navajowhite,:pink]


function PCA_Individual_Colour_Plotting(PcaDim1,PcaDim2,Time_Index_To_Plot)
    for Colouring =1:16
        Plotting_Something = false
        PCAPlots_TimeFilter = plot([0])
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    
    
        if Colour_Index == Colouring
            Plotting_Something = true
            PCAPlots_TimeFilter = plot!(Transformed_Data[Time_Index_To_Plot,1,PcaDim1,:,1],Transformed_Data[Time_Index_To_Plot,1,PcaDim2,:,1],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1)
            PCAPlots_TimeFilter = scatter!((Transformed_Data[Time_Index_To_Plot,1,PcaDim1,end,1],Transformed_Data[Time_Index_To_Plot,1,PcaDim2,end,1]),c=Multi_Colours[Colour_Index])

        end
        for q=1:Sampling_Amounts
            for p=1:2^Splits
                for t =1:1
                    Colour_Index = Final_Epigenetic_Factors[t,1,p,1]+Final_Epigenetic_Factors[t,1,p,2]*2+Final_Epigenetic_Factors[t,1,p,3]*4+Final_Epigenetic_Factors[t,1,p,4]*8+1
    
                    if Colour_Index == Colouring
                        Plotting_Something = true
                        PCAPlots_TimeFilter = plot!(Transformed_Data[Time_Index_To_Plot,p,PcaDim1,:,q],Transformed_Data[Time_Index_To_Plot,p,PcaDim2,:,q],c=Multi_Colours[Colour_Index],legend=false,opacity=0.1,title="Final Type Index $(Colouring) Time = $(TimesWeCheck[Time_Index_To_Plot])")
                        PCAPlots_TimeFilter = scatter!((Transformed_Data[Time_Index_To_Plot,p,PcaDim1,end,q],Transformed_Data[Time_Index_To_Plot,p,PcaDim2,end,q]),c=Multi_Colours[Colour_Index])

                    end
                end
            end
        end
        if Plotting_Something
            display(PCAPlots_TimeFilter)
        end
        
    end
end
PCADim1 = 1
PCADim2 = 2
for t in [1,4,5,6,7]
    PCA_Individual_Colour_Plotting(PCADim1,PCADim2,t)
end



#TimesWeCheck = [1,5,10,25,40,60,80,100,150,200,300,500,1000,4000,20000]
TimesWeCheck = [50,100,150,200,300,320,340,360,380,400,450,500,550,600,650,700,750,1000,1500,2000]
TimeChecks = length(TimesWeCheck)


Cell_Gatherings = zeros(TimeChecks,2,2^Splits,M)
CycleAmount = zeros(TimeChecks)
UniqueCycleAmount = zeros(TimeChecks)
FixPointAmount = zeros(TimeChecks)

Interpolated_Cells = Interpolator(TimesWeCheck,Orbit_Array[1])
Interpolated_EPIs = Interpolator(TimesWeCheck,EpiThroughTime)
for i=1:2^Splits
    Cell_Gatherings[:,1,i,:] = Interpolated_Cells[:,2+(i-1)*M:1+(i)*M]
    Cell_Gatherings[:,2,i,:] = Interpolated_EPIs[:,2+(i-1)*M:1+(i)*M]
end


Copy_String = string(1)
Current_Network_String = "/Gen_"*Generation_String*"_"*Copy_String*".jld"
Loaded_Data = load(Load_Data_String_Load*Current_Network_String)

CArray = Loaded_Data["CArray"]
ParameterMatrix = Loaded_Data["Network"]
FixPoint_Positions = Array{Float64}(undef,0)
Counter_Cycle = 0
XPositionTol = 0
Full_TimeSpan = 200
sol = 0
Sampling_Amounts = 8
Transformed_Data =  zeros(Float32,TimeChecks,2^Splits,3,Full_TimeSpan,Sampling_Amounts)
Saved_Orbits_All = 0
for q=1:Sampling_Amounts
    Saved_Orbits_All = zeros(Float16,TimeChecks,2^Splits,Full_TimeSpan,M)
    for t = 1:TimeChecks
        Counter_Cycle = 0
        FixPoint_Positions = Array{Float64}(undef,0)
        VectorTest = []
        for k = 1:2^Splits
            PositionArray = Cell_Gatherings[t,:,k,:]
            
            Parameters = (ParameterMatrix, CArray) #We have to update the splits every rerun, rest stays same
            tspan = (Float32(0.0),Full_TimeSpan) #How long ODE will run
            
            prob = SDEProblem(Evolution_NoEpi,σ_Genes,PositionArray,tspan,Parameters) #ODEdefining
            
            
            
            
            sol = solve(prob,SRA3(),dtmin=0.1,force_dtmin=true)
            if false 
                aba = plot(sol.t,sol[1,1,:])
                for j=2:40
                    aba = plot!(sol.t,sol[1,j,:],legend=false,title=TimesWeCheck[t])
                end
                if t> 1 && (k==1) 
                    
                    display(aba)
                end
            end
            for time=1:Full_TimeSpan
                Saved_Orbits_All[t,k,time,:] = sol(time)[1,:]
            end
            
            
            
            
            XVector = sol[1,:,end]
            EpiVector = sol[2,:,end]
            SumOfDerivates = sum(abs.(tanh.(40*(InvSqr*ParameterMatrix*XVector.+EpiVector.+CArray[:])) .- XVector))
            DerivativeTol = 0.1
            XPositionTol = 0.1
            
        end
            
    
    
    end

    for i=1:TimeChecks  
        for tau=1:2^Splits
            Tested_Transform = transform(Orbital_Model,Saved_Orbits_All[i,tau,:,:]')        
            Transformed_Data[i,tau,:,:,q] = Tested_Transform
        end
    end
    
end
## Save somehow the data/orbits of these frozen versions somewhere


# Plot this for the PCA as their axis

Multi_Colours = [:cyan,:blue,:green,:orange,:red,:brown,:black,:maroon1,:purple,:darkmagenta,:lightcyan4,:olive,:gold4,:lightsalmon,:navajowhite,:pink]




PCADim1 = 1
PCADim2 = 2





function PCA_Individual_Colour_Plotting(PcaDim1,PcaDim2,Time_Index_To_Plot,Colour_Index)
    Colour_Counter = 0
    for Colouring =Colour_Index:Colour_Index
        Plotting_Something = false
        plot([0])
        Colour_Index = Final_Epigenetic_Factors[1,1,1,1]+Final_Epigenetic_Factors[1,1,1,2]*2+Final_Epigenetic_Factors[1,1,1,3]*4+Final_Epigenetic_Factors[1,1,1,4]*8+1
    
    
        if Colour_Index == Colouring
            Colour_Counter = mod(Colour_Counter,16) + 1
            Plotting_Something = true
            PCAPlots_TimeFilter = plot!(Transformed_Data[Time_Index_To_Plot,1,PcaDim1,:,1],Transformed_Data[Time_Index_To_Plot,1,PcaDim2,:,1],c=Multi_Colours[Colour_Counter],legend=false,opacity=0.2)
            PCAPlots_TimeFilter = scatter!((Transformed_Data[Time_Index_To_Plot,1,PcaDim1,end,1],Transformed_Data[Time_Index_To_Plot,1,PcaDim2,end,1]),c=Multi_Colours[Colour_Counter])

        end
        for q=1:Sampling_Amounts
            for p=1:2^Splits
                for t =1:1
                    Colour_Index = Final_Epigenetic_Factors[t,1,p,1]+Final_Epigenetic_Factors[t,1,p,2]*2+Final_Epigenetic_Factors[t,1,p,3]*4+Final_Epigenetic_Factors[t,1,p,4]*8+1
    
                    if Colour_Index == Colouring
                        Colour_Counter = mod(Colour_Counter,16) + 1
                        Plotting_Something = true
                        PCAPlots_TimeFilter = plot!(Transformed_Data[Time_Index_To_Plot,p,PcaDim1,:,q],Transformed_Data[Time_Index_To_Plot,p,PcaDim2,:,q],c=Multi_Colours[Colour_Counter],legend=false,opacity=0.2,title="Final Type Index $(Colouring) Time = $(TimesWeCheck[Time_Index_To_Plot])")
                        #PCAPlots_TimeFilter = scatter!((Transformed_Data[Time_Index_To_Plot,p,PcaDim1,end,q],Transformed_Data[Time_Index_To_Plot,p,PcaDim2,end,q]),c=Multi_Colours[Colour_Index])

                    end
                end
            end
        end
        if Plotting_Something
            display(PCAPlots_TimeFilter)
        end
        
        xlims!((-3,2))
        ylims!((-4,1))

    end
end






for t in [1,4,5,6,7]
    PCA_Individual_Colour_Plotting(PCADim1,PCADim2,t,3)
    PCA_Individual_Colour_Plotting(PCADim1,PCADim2,t,4)

end



In [ ]:
using ForwardDiff
import ForwardDiff.jacobian

Fine_Graining = 50
TimesWeCheck = [10,20,30,40,45,50,55,60,65,70,75,80,85,90,95,100,110,120,140,160,180,200,250,300,2000]
TimeChecks = length(TimesWeCheck)
Time_Index = 3
Cell = 1
Cell_Gatherings = zeros(TimeChecks,2,2^Splits,M)

Interpolated_Cells = Interpolator(TimesWeCheck,Orbit_Array[1])
Interpolated_EPIs = Interpolator(TimesWeCheck,EpiThroughTime)
for i=1:2^Splits
    Cell_Gatherings[:,1,i,:] = Interpolated_Cells[:,2+(i-1)*M:1+(i)*M]
    Cell_Gatherings[:,2,i,:] = Interpolated_EPIs[:,2+(i-1)*M:1+(i)*M]
end

Interpolated_Epi = Interpolator([i for i=1:Length_Orbits],EpiThroughTime)

Interpolated_Epi_Reshaped = reshape(Interpolated_Epi[:,2:end], (Length_Orbits,M,2^Splits))

Interpolated_Orbits_Reshaped = reshape(Interpolated_Orbits[1][2:end,:]', (Length_Orbits,M,2^Splits));

function Gene_ODE(u)
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    Epi_Factors = Parameters[3]
    #vector = Vector{Float16}([])
    #append!(vector,tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1])
    vector = zero.(u)
    for c=1:M
        vector[c] = tanh(40*(InvSqr*ParameterMatrix[c,:]'*u[:]+Epi_Factors[c]+CArray[c])) - u[c]
    end

    return vector
end  
Amount_Lyapunov_Steps = 200
Lyapunov_Mean = zeros(TimeChecks,Amount_Lyapunov_Steps)
Jacobian_Sampling_Points_Multiplied = 0
DeltaX_ThroughTime = zeros(Amount_Lyapunov_Steps,M)

Parameters = [ParameterMatrix,CArray,Interpolated_Epi_Reshaped[Time_Index,:,Cell]]

TimesWeCheck = [10,30,50,70,90,120,150,200,300,500,1000,3000]
function Gene_ODE(u)
    ParameterMatrix = Parameters[1]
    CArray = Parameters[2]
    Epi_Factors = Parameters[3]
    #vector = Vector{Float16}([])
    #append!(vector,tanh(40*(InvSqr*ParameterMatrix[1,:]'*u[:]+Epi_Factors[1]+CArray[1])) - u[1])
    vector = zero.(u)
    for c=1:M
        vector[c] = tanh(40*(InvSqr*ParameterMatrix[c,:]'*u[:]+Epi_Factors[c]+CArray[c])) - u[c]
    end

    return vector
end  
Delta_T = 10^-3
D0 = 10^-6
Amount_Lyapunov_Steps = 500000
Norm_Saved = zeros(length(TimesWeCheck),Amount_Lyapunov_Steps)
Moving_Average = zeros(length(TimesWeCheck),Amount_Lyapunov_Steps)
Cell = 2

for t=1:length(TimesWeCheck)
    Time_Index=t
    Delta_X = D0*ones(M)/sqrt(M)
    Parameters = [ParameterMatrix,CArray,Interpolated_Epi_Reshaped[TimesWeCheck[Time_Index],:,Cell]]
    x0 = Interpolated_Orbits[1][2+(Cell-1)*M:1+Cell*M,TimesWeCheck[Time_Index]]
    
    for j=1:Amount_Lyapunov_Steps
        Stepx0 = Gene_ODE(x0)
        StepDeltax0 = Gene_ODE(x0+Delta_X)
    
    
        Delta_X = Delta_X + (StepDeltax0 - Stepx0)*Delta_T
        x0 += Stepx0*Delta_T
        Norm_Saved[t,j] = log(sqrt(sum(Delta_X.^2))/D0)
        if j==1
            Moving_Average[t,j] = Norm_Saved[t,j]
        else
            Moving_Average[t,j] = (Moving_Average[t,j-1]*(j-1) + Norm_Saved[t,j])/j
        end
        Delta_X /= sqrt(sum(Delta_X.^2))/D0
    end
    Start_Pont = 100
    TimeToString = TimesWeCheck[Time_Index]
    ada = plot([j*Delta_T for j=Start_Pont:Amount_Lyapunov_Steps],Moving_Average[t,Start_Pont:end],title="Time $TimeToString Cell = $Cell")
    ada = hline!((0,0))
    display(ada)
end



In [ ]:
plot(TimesWeCheck,Moving_Average[:,end],xaxis=:log10)
hline!((0,0))